# A workbook to sample near-ancestors from BAli-Phy Runs

## Quick-Start:
- Have all Bali-Phy results saved in an easy-to-access working directory.
  - Ensure that this notebook is NOT in the same working directory as the .ipynb checkpoints folder interferes with this workbook.
- Make sure to change all <mark> highlighted </mark> cells.

# Introduction/Background/READ ME

This python notebooks walks through the steps of sampling ancestral nodes for a consensus tree with a desired posterior probability.

### REQUIRED
- Bali-Phy bp-analyze outputs of a set of MCMC runs that have reached convergence

### Change log
Created by Patrick Finneran during the Spring of 2019
 
Updates James VanAntwerp made to this workflow (~2020-2021-ish):
- Added automatic creation of the Ancestor_Nodes folder
- Added an error handler for MacOS's .DS_Store, which interferes with the parsing of *_Descendants.fasta files
  - Otherwise, be sure to remove .DS_Store from any directory with a *_Descendants.fasta file in it, as that will cause the program to crash.
    
Updates Nathaniel Pascual during the Spring of 2024:
- Formatted into python notebook
- Replaced deprecated MafftCommandline with Subprocess.check_output
- addressed issue with Sequences_list not be being defined
- Created environment for other users to implement
- added ete4 integration to navigate trees and identify extent sequences (for Descendants input) for a desired node
- Removed 'SimplifyTree Function'
- (next steps) - Fix typos; automate descendants.fasta file generation
  - Just sample all nodes?


# Import Dependencies

In [2]:
from dendropy import Tree
import os
from tqdm import tqdm
from statistics import mode
#from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO
import operator
import seaborn as sns
import pandas as pd
from math import ceil
import pickle
import sys
import subprocess
from Bio import SeqIO
import PyQt6
from ete4 import Tree as eteTree #note this change to avoid clashing with Dendropy's Tree
from ete4 import PhyloTree
import matplotlib.pyplot as plt

## Details on Installing Dependencies

### Start by installing Qt (needed to use the PyQt6 package):
#### For Macs, use brew https://formulae.brew.sh/formula/qt:
```
brew install qt
```
#### For windows and others, follow their website:
https://www.qt.io/download-qt-installer-oss?hsCtaTracking=99d9dd4f-5681-48d2-b096-470725510d34%7C074ddad0-fdef-4e53-8aa8-5e8a876d6ab4

### install wget (if you don't have it)

### Then, create a conda environment using the following line using the provided .yaml file:
```
conda env create -f bali_ancestors.yml
```
You can find this .yml file in this folder: https://github.com/WoldringLabMSU/Evo-Seq-ML/blob/main/Sequence%20Reconstruction/BaliAncestor.yml  

### ... or individually install these packages using the following lines. (Version Information Included below)
```
conda create –name bali_ancestors
```

### Install each package independently
```
pip install DendroPy #these can be combined if desired
pip install tqdm
pip install biopython
pip install seaborn
pip install https://github.com/etetoolkit/ete/archive/ete4.zip
pip install PyQt6 #Note, this requires installing Qt
```
### mafft is available as a conda package, OR download from their website: https://mafft.cbrc.jp/alignment/software/source.html
```
conda install bioconda::mafft
```

### If you get the following error when initially trying out ete,
```
ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' 
(/usr/local/anaconda3/envs/bali_ancestors/lib/python3.10/site-packages/charset_normalizer/constant.py)
```
#### Install chardet:
```
pip install chardet
```

### Finally, run ipykernel to use the environment with JupyterLab:
```
python -m ipykernel install --user --name=bali_ancestors
```

# Extract the results from BAli-Phy's bp-analyze

## <mark> input sequences from MSA (Just paste your sequences within the quotes)</mark>
- This is the "P1.ancestors.fasta" MSA generated by BAli-Phy after running bp-analyze for a set of runs.
- Note: Remove {} => {} nodes at the end.

In [19]:
fasta_txt = """
>XP_033538655
FQTFEIPQQVTGSRGDIILGKAMINAWRRDGILQVSMDESQRSVADKAFATSQRYFDIPYDRKASCVNDQSFSGYIASGEELTDNIADYSEIFTVTKDLPQSDPRVKAKWPCHGPCPWPDTRMKSSIKTYMNHLGDSGDKILQLIACGLELPDINELTKYTRDGWHHLRVLRFPARDKTNGKGKDGRGIGSHTDYGLLVIASQDSVGGLFIRPPYKDEKYANWEKSVAGAYENDHRWVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNNDQRFAFAYFHEPNFQATLEALPGFESDENSRGIHYGTHFTNMSMRNYPQRITAKRIRAENRMAVL
>WP_123667040
MFDPCVFTLPATVTGSAADRALGRELLAAWRSDGLFLLTLDPSAAPAVDHALTAARSFFALPASSKRCYTDDRSYSGYAGSGEEMTAGQADHCEVFTVCKDVPDDDPRVLAGLPCHGPVPWPGLHHRRTITALMRGLGDVGDRVLALVALGLSRDPSVFLDLTRDGWHHLRSLRYPATSEETGHGLGVHTDYGMLVLTVQDDAGGLYVRPPVPGEPRGRNWLPEESTAGVYEDVEPWHHVTPVPGTITAFPGDILQFLTGGDLLSTPHQAKLSARERHSIAYFHEPSFDAEFTPLDGGDPLHYGTHFTQMFLRCYPDRPTTLRILREDRTALLR
>WP_206715722
MIELETFQLPQSVSGREADIALGLTMVRAWRRDGIFQVRMSPAQVEKSQRAFELSRHYFRQPLEVKARCVSDLTYSGYIASGQEITASEADLSEVFTVCKDVPLTDPRVQSRWPCHGPGPWPDEAWRQGMQAHADELGSVGERLLQLIALGLGLDIDALTTLTHDGWHHMRVLRFPARSATTTRGIGAHTDYGMLVIAAQDDVGGLYVRPPVEGEKRPRNWLPHESSAGMYEHDEPWTYVKPVPGVLTVFPGDILQFLTRGYLLSTPHKVVLNTRERFALAYFHEPQFEACVRPLSAPTRDEYIHYGTHFTNMFMRSYPDRITTQRILDESRLTTLSWLRQEAVLR
>KAF7559752
VAQLDTFIVPETVSGTTADQTMGKNIVQALQRDGIFQVKMTPLQEILYQNAAAASRGFFARPHDQKAACVDPHSYSGYVASGEEITDGIADYSEIFTVTPDLPAKDKRVQQRWPCHGPVPWPDQDMKSSVDRYMRYLGKAGEKLLQLIELGLRVPKGSITDYTDDGWHHMRILRFPGKDNTNGKGKAGRGIGSHTDYGMLVMAAQDEVGGLFVRPPIQGEAHANWQRSAAGHNENDSKWTYVPPTPGVFTVFPGDMMQYMTNSVLPATPHKVGLNTRERFAFAYFHEPNFKSVVKPLPGYNKGQAPAEGIHYGTHFTNMCLRNYPDRITTKKLLAED
>TMH32900
MSDLQTFHVPDTVTGVPSDIELAHRMIQAWRRDGIFQVATDEVQSRKTEQAFEASRRFFRLPMAAKAQCVSDLTYTGYIASGEEVTAGEADYSEIYTVCKDVPLDDPRVQAQWPCHGPAPWPDEAYRQAMTVFMDQLGAIGEQLLKLTALGLGLADIDSLSKLTQDGWHHMRVLRFPTASSKTARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGETRKRNWLPGESSAGMYENEAPWTFVKPVPNVLTVFPGDILQFMTGGQLLSTPHKVRLNTRERFALAYFHEPNFEACARPLFEPESDEHIHYGTHFTNMFTRCYPERITTRRIEREDRWSVLAGLKE
>WP_197386210
MTDLTTFHLPDRIINTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFESKSRHVSPLTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPSADYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMNTFTRLTDDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADAAAAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPTLSEVA
>SCN93509
NLQTFILPDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFSKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMHRYMKSLGGVGETLLQLTELGLGVPQGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGGNDEKLANWESSAAGFKEDDERWVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGYDIGQEPKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>WP_166054208
MTELRTFRLPETVSGSDDDVELGRRLIAAWQADGILQIATDATQDRTTAAAMTASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLEDSRVVDRWPCHGPVPWPDEDYRDAMREFMAMLGDAGEKILKLTALGFGVDIDTFTKLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDEVGGLYVRPPIDGEERPRNWLPDESAAGCFQDAEGWTFVEPVEHVFTVFPGDILQFLTGGTLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDPHGEDAIHYGTHFTNMFMRCYPDRVTTRRILEEDRLTVLSELRS
>WP_253992635
MAELETFQLPQSVSGSESDVALGLTMIRAWRRDGIFQLRMSPTQAEKARRAFESSRRFFRQTLDAKARCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDPRVQSNWPCHGPAPWPDEAYRQDMKEHTEELGSAGERLLRLVALGLGLEVDTLTALTRDGWHHMRVLRFPARSPMATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRHRNWLPGESSAGMYEHEEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLREPSREEFIHYGSHFTNMFIRCYPDRVTTQRILDEDRLTTLAWLREEAVQRTMPVEE
>WP_239309439
MNELQTFHLPEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVGEKLLKLTALGLELDDTDALTRLTADGWHHMRVLRFPALSKESRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDGFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLSGVSADCIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLPLLRD
>KAF7521891
QLKTFTLPEKATGSPRDVEMGKAMINAWREDGILQVSMSQKQQDLFDKASAASKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWLDSEMKTTIQQYMDSLGNSGETLLQMIEYGLSLDPHTLTSLTKDGWHHLRVLRFPQINKTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPIAKLYDGQPPDEEIHYGTHFTNMFMRNYPDRVTTERILREDRLKLLD
>WP_250739343
MGELATFRVPARITGTTADIMLGGAMVRAWQRDGIFQVAADGRQDARTRAALAASRRFFARPAAEKAAYVNDTSYSGYIASGEEVTDGRPDASEIFTITPDIPVAAAAGLPCHGPVPWPSPGYRAAMEAYLTGVGGLGERLLRLVALGLGLGPAGIDRLAGLTRGGWHHMRVLRFPAAERTAERGIGAHTDYGLLVIAAQDEVGGLYVRPPVPGEERPRNWLPGESTAGFAENRPGWTPVRPVARVLTVFPGDILQFMTGGALLSTPHRVRLADRERYALAYFHEPRFGARIRPLDPARAQRGGDGGEGASVIHYGEHFTRMFMRCYPQRATTRRIESQGLLRHL
>WP_013423727
MAGLQTFQLPDSVDGTPLDLQLAQELVRAWRTDGILQVATDPAQSRRTHAALESSRRFFSQPAAVKASCISDLTFSGYIASGEERTAGEPDYSEIFTVCPDVPLDDPRVRAQWPCHGPVPWPDDEYERAMTAFMAGLGVIGEKLLRLAALGLGLDLDTLTSLTRDGWHHLRVLRFPARSARTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVDGERRTRNWLPTESMAGRYEDDEPWTFVEPVPGVVTVFPGDMLQLITDGLLLSTPHKVRLNTRARYALAYFHEPRFQARIRSVSGENDGDSLHYGSHFTRMFMRSYPERVTTRRILAEDRLSRL
>XP_049228299
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHENTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGSHFTNMFMRNYPQRITADRMKAENRMALLDSPDLRWKDTPA
>WP_062029603
LRTFRLPVAVDGTDADRALGRALVAAWQSDGIFQVHATPAQEEATERALQASRAFCRRPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTAYIHYGTHFTRMFMRCYPERVTTTRIETDGRLKVLDRLRE
>XP_049164345
LQTFCLPERVTGSQSDRTMGKALVDAWQKDGILQIAMTPRQEALHSQAQQNSQEFFSRPHAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLELGDAKVKAQWPCHGRCPWPDAAMRDVMKEYMNELGNCGETMLRLLEYGLEIPEGTLTQQTDDGWHHLRILRFPPHNKTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGEKYANWEKSAAGMAEEDEKWTYVPPVPGVFTVFPGDMMQYMSNSTLPSTSHKVGLNTRDRFAFAYFHEPNFKTTCRPLPGYDAGQTPREGIHYGTHFTNMFLRNYPDRITTSRMNRENRYKILK
>KKY15282
NLKTFELPETLTGGASDQALGRSMIDAWRHDGILQISMDPINRKLADAAFICSKKFFDLPYKTKAACVDDQSFAGYIASGEELTDNIADYSEIFTVTKDLAATDARVQQQWPCHGPCPWPFKQMRTVMDAYMQYLGDSGEKMLQLIAWGLGLADGNALTRYTQDGWHHMRVLRFPERNNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEVVANWRESAAGLAEEDDKWVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNNRERFAFAYFHEPNFSSVMKPLEGFDAGQEPREGIHYGTHFTNMFMRNYPKRITADRIHKEDRMKLLD
>WP_097259396
APLRTFRLPAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATEKALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTARIETEGRLKVLDRLREEA
>KAH0536166
VDLETFNLPQQVSGSAADLTMARSMIAAFRKEGIFQIQLNPSQAATLSLAFGVAKTFFRQPYSEKAKCVDDQSFAGYIGSGEELTDGVRDYPEIFTVIKDLPPTDSRVRRRWPCHGPCPWPNDAYAATMKEYMSLLSESGEKLLQLLTLGLGLRDLNSLTRLTKDGWHHMRVLRFPELDHTNGKGKPGRGIGSHTDYGLLVLSAQDEVGGLFVRRPIAGEEWKNWEKSVAGFKEGDEKWSYILPVPNVLTVMAGDMLQYLTSSLIRSTPHKVGLNSRARNVFAYFHEPNFSALLKPLPECASDSPDRQGVHYGTHFTNMCMRNYPERITAKRVLAENRMEIL
>KAH7113358
LQTFTLSSEDMGSAADLSMGKALVAAWQQDGILQISMDAEQQTLYKAANEASKRFFRKPLHQKAACIDSQSYSGYIASGEELTDGLADYSEIFTVTKDLDLDDPRVSAKWPCHGRCPWPDADLRDPVKKYMDSLGQTGEKLLRLIELGINVPEGSLTRITKDGWHHMRILRFPASNDINGKGGDGRGIGSHTDYGLLVIAAADDVGALFVRPPYKDESVANWERSAAGMKEDSEGWVFIPPAENVFTYMTNSVLPSTLHKVGLNMKERFAFAYFHEPSFQAVIKPLPGYEAGQSPEEGIQYGKHFTNMFMRNYPERITTHRLVDEGRYKLLD
>UKZ86338
LETFSLPEKVSGSVCDKAIADAMISAWRRDGILQIDMSKAQRRLYQAANAASRRFFSRSYVQKQACVDDMSYSGYIASGEEITHGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQSMKNVMNSYMNDLAVSGEKLLQLIEMGLDVPTGSLTRYTDDGWHHMRILRFPQRDRTNGKGKNGRGIGSHTDYGLLVIAAQDDVGGLFVRPPRQDEQFANWEKTSAGLRENDAGWVYVPPTAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGYNAGQSPMGGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLGSEQLRDENDT
>OLN91671
VAQLETFTLPERVTGSPGDRAMGKALVDAWRRDGILQVSMNEKQQAIFDEATKASKTFFAKPHAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYDMKQPMTTYMNYLGESGEKLLQLVELGLNVPAGSLTNLTQDGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEHYANWEKSSAGMAEDDDRWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPDRITTTRMMAENRLPLLQ
>WP_007379180
LRTFRLPTAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTARIETEGRLKVLDRLRE
>KAH7178219
IPTILNASANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGALMIRPPYSDEKLANWESSAAGFREDDERWVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGYDVGQEPKEGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>ETS06778
LETFTLPERVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRVYQAASATSRRFFAQPHARKEACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWLDQEMKAVMNKYMEDLAVSGEKLLQLIEMGLDIPTGSLTKYTADGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRDTERFSNWEKSSAGLRENEAGWVYVPPVTGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGYNAGQSPLEGIHYGTHFTNMFLRNYPDRITTARLQEEGRYRLLESKELRDD
>WP_116173588
MTTLRTFQLPAEVSGSIWDIALGVEMIRTWQSDGIFQVATGTVANAVTAAALRESRRFFAQPLEAKTPLVSDLTYAGYVASGEEVTAGERDYSEIFTVCKDVPLGDPRVRAGWPCHGPVPWPGPVYRDVMTALMSQFGLVGDKVLKLAALGLGLSDMAALTSLATDGWHHMRVLRFPARSAATARGIGAHTDYGMLVIATQDDVGNGLYVRPPVIGEKRPRNWLDHESSAGAFEREAPWNLVSPVPSTFTVFPGDILQFLTNGALLSTPHKVKLAERERYSIAYFHEPDFAASLRPLTGGLPDEVIHYGTHFTSMFMRCYPDRLTTRRILAEDRLQVLDSMRESTLT
>KAH0541234
LTTFALPQEVVGSRSDLALGQRMIDTWRKDGIFQIFLTPAQGATMRRAFKASKGFFRRPYEEKAKYVDDQSFSGYIASGEELTDGVADYSEIFTVTKDLPVTDPRVQAKWPCHGPCPWPDTNYATTMKNLMSLLGESGEKLLKLTALGLGLDDPNALTNLTEDGWHHMRVLRFPQVDRTNGKGKAGRGIGSHTDYGLLVIAGQDEVGGLFVRPPVEGEKLHNWEKSAAGFQENDEKWSYVPPVENVLTVFPGDMMQYLTRDHLPSTPHKVGLNTRERYAFAYFHEPNFSSVVRPLRDYGHEAKNDKPGIHYGTHFTNMFMRNYPERITAKRLMAENRMAILERI
>WP_253508315
MTTLHTFHLPESISATPEDIALGKAMIEVWRRDGIFQVAATPAQNRAMLQAMKASQGFFRLQPRNKARRISDLTYSGYIASGEEVTAGQADYSEIFTVCKDLPLDDVRVKNGWPCHGPVPWPNKNYRLRMHTFMDALGDIGERLLTLVALGLELEDRHTFTDLTRDGWHHLRVLRFPAGSSSTARGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLEKESSAGRFEDQDPWTFVQPMPRVFTVFPGDLLQFMTDGYLLSTPHKVKLNTRERFALAYFHEPNFEAIARPVRGQTRGSFIHYGEHFTRMFMRCYPERITTRRILKENRLAHLELMKRHA
>OQD69102
QLKTFTLPEKATGSPTDIEMGKAMINAWREDGILQVSMNPKQQDLFDKASATSKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEPRVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGTSGETLLKMIEYGLNLEPETLTSLTKDGWHHLRVLRFPQNNKTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIPPVAKLYDGQPPDETIHYGTHFTNMFMRNYPDRVTTERMLREDRLKLLD
>MBW0496253
LETFDLPDYVTGSTSDQALGKALIKAWKRDGIFQISMKRDQREIYKNAEKASKKFFAKPHNEKAICVDSQSYSGYISSGEEITDGIADYSEIFTVTKDLPLDDPRVLRKWPCHGSCPWPDAEMRDPMKLYMSYLSDSGEKLLALTELGLGVPKGSLLHYTQDGWHHMRILRFPQVNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPYEGETYANWEKSAAGMKESDEGWLYVPPVADTFTVFPGDMMQYQTGSFLPSTPHKVGLNTRERFAFAYFHEPNFNAVLKPLVGFNVRQKPQEGIHYGTHFTNMFMRNYPERITTQRLISEDRYDLLK
>WP_030474037
MTELRTFRLPETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHDLTLEEKSRFVSDLTYAGYIASGEEVTDGVADYSEIFTVCKDVPLDDSRVVGRWPCHGPVPWPDERYRDAMREFMAVLGEAGEKILVLTALGFGHDADAFTRLTADGWHHMRVLRFPTRSGAVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEQRPRNWLPDESAAGCFQEADGWTYVEPVPHVFTVFPGDVLQFLTGGRLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDPHGDDVIHYGTHFTNMFMRCYPDRVTTRRIVDEDRLTVLSDLRSGTRHSRTG
>XP_044663983
ELDTFELPVAVTGSDADRILGRRLVEAWRKDGIIQVAFPENLRVLSEAVKQSKQYFLQDHATKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPNDAYAHAMRKLMDFKAGIGERLLQLTALGLDLKDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEGRGIGSHTDYGLLVIASQDEVGGLFVRPPSADETYSNWKKSSAGLHENDDKWLYIAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCRRLPEFKTGTAEDEEAVHYGTHFSRMFLRNYPDRITAHRMREENRMDLLPMLK
>XP_024751012
LETFTLPEKVSGSVSDKAVADAMIGAWRRDGILQIGMTRSQQRIYQAATAASRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQDMKAVMNKYMEDLAVSGEKLLQLIEMGLDIPTGSLTKYTDDGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRESERFSNWEKSSAGLRENESGWVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGFNAGQSPVEGIHYGTHFTNMFLRNYPDRVTTARLHEEGRYQLLESEELRDDDDA
>PHH66231
QLETFVLPEKVSGSVSDQAMARSMIDAWRRDGILQIGMGQPQRRLYQSAESASKKFFQRSAKDKQACVDSCSYSGYIASGEEITDGVADYSEIFTVTKDLSHSDKRVASNWPCHGPCPWPDQDMKSVMSRYMADLGTSGEKLLQLIELGLEVPAGSLTGYTDDGWHHMRILRFPARHATNGKGKEGRGIGSHTDYGLLVIAAQDHVGGLFVRPPNKEDKFANWQQSSAGMKEDDSGWVYVPPTPGVFTVFPGDMLQYMTNNVLLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGYNGGQAPMEGIHYGTHFTDMFLRNYPERITTKRMEAESRYQLLESKQLRDER
>WP_165984892
MTLLQTFRVPDLVTGTGYDDRLGRAMIRAWKVDGVFRVAVPGPQRHALRQALAAGRRFFARPDSLKSRCVSDLTYSGHAPATTTVGFPDAAPETVPESFTVCKDVPLQDARVLARWPCHGPVPWPDTEYRRAMKSCLPGLCSLGDRLLRLAALGLGLADMERLTALTFDGWHHARMLRHPPAGTGARTGYGLLLITAQEEQGGLAVRPPVKGEKRYRNWLAEESTSGMYQSEPPWIEAGSEPGHGTLTVLPGDLMQYLTGGELLATPHRTLPGERERVSLTYCHEPNFAACLRPLTGDADAARPGYLHYGSHFTQVFTRHYPERITTRRIVAEERLAVLDDLR
>WP_184965276
MMELTTFSLPEAVTGTRADLELGRTMVAAWRRDGIFQVAATTAQHETALAAFEASREFFALPLTEKSSYVSDLSYSGYVASGEEVTAGETDYSEIFTVCPDVPVHDARVRERWPCHGPVPWPGPHYQRRMEAFTAALGSIGERLLRLAALGLGLGGPDTLTGHTAGGWHHMRVLRFPPSSARTARGIGAHTDYGLLVIAAQDEVGGLYVRPPVEGEKRSRNWLEAESTAGMYENEPPWIFVRPVPRVFTVFPGDILQFLTGGYLLSTPHKVALADRERFAMAYFHEPSFGRSLEDPCGGPPLHYGTHFTNMFMRCYPGRAATRRIHDEGRLATLRE
>WP_219656666
MKKLSTFTLPETVSDSPAHRALGTQIVANWRKDGIIEVRRDAIQEQETQRAFEENQKFFARPLDEKRRYVDDLTFSGYVASGEEKKSSAKDFPEVFTIFPDISVSDERVRQEWPCHGPVPWPSEDFQSAMTAYIHTCSSIGHRLMQLTALGMGLPLDTFTSLCRRPWNYMRVLQFPAASVENSRGIGAHTDYDFFVISSQDLEKQGLFIRPPVEGEHRFRNWLPDESMRGVYETQEPWDLVEANPGVFTVFPGDMMQLVTDGYLLSTPHKVALYTSVRRAMPVFIGPDFDVRLRPHNKPDDAGFHYGEHVTNMHVSMYKDRSTSRRIVEKELL
>KAH7350224
NLQTFVLPAKVIDAEEHRKLGQAIVAAFQRDGILQIAMGPEQQALYHAANVASREFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLDLDDARVRQKWPCHGRCPWPDAALRDPMKAYMDSLGQNGEKMLRLIEFGLNVPAGQLTDLTQDGWHHMRILRFPATNDTNGKGKEGRGIGSHTDYGLLVLAAADDVGALFIRPPRQSESFANWSTSAAGFKEEDDGWVFVPPMDNVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLEGYDAGQEPRGGVHYGKHFTDMFLRNYPERITTQRLEMENRYELLSRKDLR
>WP_125810284
MTHLQTFHIPQTVGATDADSRLGHDLIGAWQTDGILQVATEPAQDRVITDAIEASARFFRLPMPVKAGHVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIAADDPRVRHRWPCHGPVPWPDPGYARRMRALMDQFGSIGQRLLRLIALGLDLADLDALTGLTTDGWHHMRVLRFPALSEQTSRGIGAHTDYGLLVIAAQDDVGGLFIRPPVDGERRNRNWLDTESSAGRYENEEPWTYVTPQPSVLTVFPGDIMQFLTGGRLLSTPHKVKLNTRERFALAYFHEPGFNTTIRPFADPAGDDSIHYGTHFTNMFVRCYPDRVTTRRIEAEDRLSILRDLR
>TQN63987
VAQLQTFTLPERVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFDEATAASKTFFAKPHEEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDMSESRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESGEKLLQLAELGLNVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGENYANWEKSSAGLKEDDDGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPERITTKRLHAENRLTLLQ
>WP_008068240
MITLKTFKLPAKVTGSTADITLGEQLIRAWQTDGIFQIKTRVEQDRKTQAAIAVSKRFFKKPQTFKSGCINDSAYSGYIASGEEITAGQADYSEIFTICKDLPESDPRVRNGWPCHGPVPWPDSVYHKRMKAFMDELGAEGEKLLKLTALGMNLPISTFTDLTRDGWHHMRVLRFPTETSKSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPEESSAGLFEHEEPWTFVTPTPRVWTVFPGDIMQFMTNGYLLSTPHKVKLNTRERFACAYFHEPGFEASARPLFEPGSQERIHYGEHFTHMFMRCYPDRITTRRIHEQNRLARLEELKARDKRA
>XP_035328441
VAQLETFTLPGRVTGSPGDRAMGKALVDAWRRDGILQVSMNDKQQHIFNEATASSKAFFAKPHDEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESGEKLLQLAELGLNVPAGSLTNLTQDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGMAEEDDRWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGSHFTNMFLRNYPERITTKRMHAENRLALLEKEELRAHS
>KAF6807323
VAQLETFTLPAKITGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKDPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESGEKLLQLTELGLGVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEHYANWEKSAAGMKEDDDRWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPERITTKRMMAENRLEKLQ
>KAG9196060
VGQLQTFELPETLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFGLDHKKKAVCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKNEDADGWVYVPPVSDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMAIL
>KAF2851311
VGSLQTFELPESLSGGAADKALGKSMIDAWKRDGILQISMDPINRKLAEAAFLTSKKFFSLDYQKKAACVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVKQKWPCHGPNPWPFGQMKTIMQAYMDYLGESGEKMLQLIALGLGMKDRNALTRYTQDGWHHMRVLRFPQANKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPIEGENYANWKESSAGKHQETDGWVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPQRITAARMKAEGRMALL
>XP_042995244
LQTFTLPDTFTGGQEDAELGRAIVRAWQQDGILQIAMTPAQQAQYKAADEASKRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLPLDEERVLGMWPCHGRCPWPDAAFQQPMERYMDGLGRSGELLLKLTELGLGVPEGSLVQYTRDGWHHMRVLRFPANNTTNGKGKDGRGIGSHTDYGLLVLAAADNVGALFIRPPYQNESFANWSESAAGFKEDDEGWVFVPPAENVFTVFPGDMMQFLTNSALPSTPHKVGLNTEERFAFAYFHEPSFQAVIKPLPGYDAGQEPREGVHYGKHFTDMFMRNYPGRITTQRLVEEGRYAMLQ
>WP_112230130
MTELRTFRLPETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEAAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPPDDSRVVGKWPCHGPVPWPDDSYRDAMREFMAMLGDSGERLLKLTALGFGLDIDTFTRLTADGWHHMRVLRFPPRSQAVDRGIGAHTDYGMLVLAAQDDVGGLHVRPPIEGEERPRNWLPDESAAGCFQDAEGWTFVEPVQHVFTVFPGDILQFLTGGRLLSTPHKVLLNARERFTMAYFHEPNFEAVVRPLDDPHGDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRS
>QBZ65430
VGQLETFTLPESVRGSRGDREMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESGEKLLQLTEYGLGVPEGSLTQYTNDGWHHMRILRSPQNNKTNGKGKEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQFANWEKSSAGMKEDDDRWVYVPPVPNVFTVFPGDIMQYITNSHLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGYDAGQSPKEGIHYGTHFTNMFMRNYESRITTQRLLGEDRYSLL
>WP_080427502
MKNLQIFSLPKNVTGSAADIALGKAMIDVWRSDGIFAVGTTPDQDEIADRAIASGKRFFKKSLPEKKMHISELTYSGYVAYAEEQTAGEPDFSEVFTVCKDVSLADERVKEQWPCHGPVPWPDKEYEESMLDFMGRFGGVVGEELLKLTALGLGLEINALTDLSVDGWHHMRVLRFPQATSEEDRGLGAHTDYGMLVIAVQRDAGGLDVDIRLPVNQNGEREKRNKNWLSGQSTAGENESDPNWWRISLVPGVFVVFPGDLLQYITGDYLVSTSHKVTLHKKIERFPIAYFHEPNFGCSVRPLLGENEPESFFYGKHFTDMFMRCYPERSTTQRIKKLNLCKNLESIRQTVPTKKS
>WP_157246697
MTELRTFDVPEAVTGSRSDVTLGRTMVETWRTDGVFQVTATTAQQEAALAAFEASWAFFALPREVKSRFVSDLTYSGYAASDEEAGGETDCCESFTIFPDVPVCDLRVHARWPGHGPVPWPGSIYRRRMTAFTAGLGALGDQVLRLVALGLQMDAPGRLTGLTTHGWHQLRAVRFPPRIGRGTRGIGAHTDYGLLVITTQDEVGGMYVRPPVDGEKRPRNWLPGESTDGMYAGEEPWVYVRPRPRTFTVHPGDLLQLLTGGLLPATPNRADLADAERHAMTYFHEPGFGSLLEDPYGGEPLHYGTHVTNVLMRRHPDRPATRRILREGRLTALR
>WP_086663953
MTELRTFRLPETVSGSDDDVELGRRLIAAWQADGILQIATDATQDRTTAAAMAASRRFFHGLSIEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCEDVPLDDSRVVDKWPCHGPVPWPDDLYRDAMREFMVMLGESGEKILKLTALGFGLDLDTFTRLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAAGWTFVEPVEHVFTVFPGDILQFLTGGKLLSTPHKVLLNNRERFTMAYFHEPNFEATMRPLDDPHGDDTIHYGTHFTNMFMRCYPDRVTTQRIFEEDRLTVLSELRS
>VFJ88583
MADLQNFHLPESISGTQSDRDLGREMIQAWRTDGAFRIAMSGIQSKKSEDAFAASRRFFRGSIEFKSQYLDDLTYSGYAASGEEITAGERDYPEVFTICRDIPVEDARVRAHWPCHGPVPWPDAEYRESLQTYLNELGSVGDRLLQLVALGLALKDMDALLALAKDGWHHLRALRYPVASQESNRGLGAHTDYGLIVITDEDDVGGLYIRPPVEGEKGSRNWLADESTAGMYENEEDWRFAKPVPNTFTVLPGDILQFITGNHILATIHKVTLSTRERFSMAYFHEPDFNARVYPLTHPSGEDPLFYGEHFTNMFMRCYPERTTTRRIVDENRLSVLTDLKN
>KAF2835959
TYVLPKVVEGTPGDKALAKSMIDAWHKDGILQVAMDPVDQILADNAFNASKQFFARPCREKEKYVDDQSFSGYIASGEEITDGVADYSEIFTVTKDLPLSDPRVQQKWPCHGPCPWPSEEMQNTMNAYMAYLGDGGEKILQLIALGLNIPIDSFSKFTRDGWHHMRVLRFPQIDKTNGKGKTGRGIGSHTDYGLLVIAAQEHIGGLYIRPPYDGEKVANWETSAAGLKEDDEGWVYVPPVPGVFTVFPGDMMQYVTDSYLPSTPHKVGLNTRERFALAYFHEPNFSAVMRPLKRPEEEGIHYGTHFTNMFMRNYPTRVTAKRMLEEGRMDMLKK
>XP_047764044
ELQTFMLPEEVTGSDADRALGKELVAAWRRDGILQIQFPPRLQVLSDAIKQSKSFFAQQHDAKAKCVDSQSFAGYIASGEEITDGIADYSEIFTVTKDLTSSDPRVQSKWPCHGPNPWPSKTYEQAITALMDYKGRIGEKLLHLTALGLNLWNPWALNQLTQDGWHHMRVLRFPPTNQTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFIRPPIKGEKYENWKNSAAGAHESDEKWVYVPPVENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCKRLPEFRDGSEAEKEEVHYGTHFTNMFMRNYPDRITAKRVRDEGRMDLLEGIK
>WP_064051710
MTDLTTFHLPERITNTEAHRELGQAMVKAWRTNGIFQIALSKPQEQATDEAFAESRQFFSQDFETKSRHVSALTYSGYVASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPGAAYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMDTFTRLTDDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADATAAPIHYGTHFTNMFMRCYPKRITTQRITETGLLDKLPTLSE
>WP_223745345
MAELETFRLPHVVSGSEADVALGLAMIRAWRRDGIFQIRMSPAQADKARDAFELSRRFFRQPLEEKQRCVSDSTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVHSGWPCHGPAPWPDEAYRLGMTAHTEELGSAGERLLRLTALGLGLDIDALTALTKDGWHHMRVLRFPARSPATTRGIGAHTDYGMLVIAAQDDVGGLWVRPPVEGEPRKRNWLQGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLASRERFAMAYFHEPHFEACVRPLKDPTRDEFIHYGTHFTHMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQR
>XP_043022746
LETFKLPDRVTGSASDKAMGQALVKAWKRDGIFQIAMNSSQREIYKYAEQASKRFFGKPHNEKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSGEKLLALTELGLGVPKGALLQYTQDGWHHMRILRFPPTNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGESYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGYNNGQKPQEGIHYGTHFTNMFMRNYPDRITTQRLQAEDRYKLL
>WP_046703800
MTELQTFVLPPTVDGSDVDKALGQALIAAWQDDGIFQIQATPKQEAVTERALEFSRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVIDRWPCHGPAPWPSEQYADAMKSYMGTVGDIGERLLRLVALGLGLDDMDHFTKLTADGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLPDESMAGRYENEEPWTFVTPVPCVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGGADEFIHYGTHFTNMFMRCYPERTATARIEKEDRLAVLERLRKEA
>NET46310
LGERIVAALQRDSLIQIKKNYQEDLLTAQALLQNRKFNQLPQDEKIKFVSDTTYTGYVFSGEEKKGGQYDQVEVYTVFPHFNQDDERVKKLRLPCHGEALYPSVDYKNAIESYMKATGNMCDRLIEIIGLGLGLEPNHYLSKMVRDGFHHARVLRFKSQKEAGGVSNGISAHSDYGFLVAASQDDVGGLWVRPIPNGEKRGRNYLEEETTIGAYHDQPGWAFVTPMPCVWTVFVGDMMELATNRKLKANIHKVILNKEKERFAIAYFHEPKFNQTFEAVNSKGKKKEIVYGKHLTNMYCKAYPNRSTTKRIQSEELCQNIDRLPKIS
>XP_014547936
VAQLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGVADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDNRMKSPMLNYMADMGSSGDKVLQLIELGLDVPQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEPGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGFNGSQEPADGIHYGTHFTNMALRNYPTRITTTKLLEEGRCEML
>WP_141641058
MADLVTFQLPQSVSGSESDVALGLTMIRAWRRDGIFQLRMSEAQLEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKEHTDELGSTGERLLRLVALGLGLEVGALTALTRDGWHHMRVLRFPARSTTATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTPGPDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTL
>WP_163989172
MSDLQTFRLPESVSGQPSDIELADQMIRAWRRDGIFQVAMSAEQDGKTGRAFEASRRFFRMPLEAKARCVSELTYSGYIASGEEVTAGEADSSEIFTVCKDVSLEDARVHGRWPCHGPAPWPDEDYRLSMKAFTDELGSVGERLLELTALGLGLEIDALTRLTRDGWHHMRVLRFPALSSRPARGIGAHTDYGLLVIAAQDDVGGLWVRPPVEGEKRNRNWLATESSAGMYENEEPWTFVKPVPKVLTVFPGDILQFITHGQLLSTPHKVRLNTRERYSMAYFHEPAFNTCVRPLLGPPSEEFIHYGTHFTNMFMRCYPDRVTTRRILDEHRLTTLAWLREEA
>KAF2730203
LQTFELPKEITASPEDFKLGRAIVDAWRKDGIIQIKMDPSHAKTVRKAFEASRNFFSLSREEKAKCVDSQSFAGYIASGEEITNGVSDYSEIFTVTKDLPPSDPRVQAKWPCHGPCPWPNAGMRDTVKEYMDYLGSNGERLLALTALGLELPDPNALTNLASDGWHHLRILRFPKTGDENGAGTKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYPDEQFKNWEKSTAGMAENDDRWVYVPPEDGVFTVFPGDMMQLITNNHLPSTPHKVGMNSRERYAFAYFHEPNFCANIKSLDGAHDEQMHYGTHFTNMFMRNYPERITAKRMKEENRMDVLDK
>WP_176434236
MAELETFQLPQSVSGSESDVALGLSLVRAWRRDGIFQLGMSPTQADKARRAFESSRRFFRQPLDAKSRCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRHDMKEHTDELGSTGERLLRLVALGLGLEVDTLTALTRDGWHHMRVLRFPARSPMATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRLRNWLPGESSAGMYEHEEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLREPTRDEFIHYGSHFTNMFMRCYPDRVTTQRILDEDRLTTL
>KAI4945826
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELTQSDPRVQQKWPCHGPNPWPFTQMKTVMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKVNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKETAAGKHEDADGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGFQAGQEPTEGIHYGTHFTNMFMRNYPQRITADRMKAEDRMALLD
>KAG7410234
KLQTFILPEEVSGSEEDKKLGKAIVDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEELTDSIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMMNPMERYMNSLGESGEILLKLTELGLGVPEGSLTGYTKDGWHHIRILRFPAINKTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFSNWEKSAAGFKEDDDGWLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGYNAGQEPKEGVHYGKHFTDMFMRNYPERITTQRLLDEGR
>WP_013230840
MRTFELPVSVGGTTIDRILGKQLVRTWEDDGIFQVAATAEQARITGEALAESRRFFAGSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTEGWPCHGPVPWPDDGYRTAMEAHLREVGEIGEKLLKLVALGLGVEIDRLTRLTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRLRNWLPEESTAGMYQDEPPWHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLGDRERYTLAYFHEPAFGVTVRPLAGGDESIHYGTHFTNMFTRCYPDRVTTKRIHEEGRMRLL
>KAH6669699
NLQTFVLPPQVTGTAEDKKLGNAIVAAFQQDGILQIAMGPEQQALYKAANAASKEFFSKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLDLDDARVKQKWPCHGRCPWPDAALRDPMKAYMDSLGDNGEKMLRLIELGLGVPQGQLTDLTEDGWHHMRILRFPATNDTNGKGKDGRGIGSHTDYGLLVLAAADDVGALFIRPPRQSESFANWSTSAAGFKEDDDGWVFVPPMENVFTVFPGDMMQYMTNSALPSTLHKVGLNSRERFAFAYFHEPSFQAVIKPLEGYDAGQEPREGVHYGKHFTDMFMRNYPDRITTQRLEIENR
>WP_037268971
MADLRTFGLPDRVVGTDSDSTLAHQMIDTWQTDGIFQIACDDRQAAKVTEALRESRRFFASATDEKAKCVSPFSYAGYIASGEEVTAGEADYSEIFTVCQDVPPDDARVQANWPCHGPVPWPSEQYQRTMQAYMDEMGLIGEKLLRLIALGLDLSEIDTLTRLAEGGWHHMRVLRFPARNSLTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLDSESSAGMYENEEPWTFVTPVADTLTVFPGDILQFLTDGVLLSTPHKVRLNDRERYTMAYFHEPNFDAGIRPLNGSDAFIHYGTHFTNMFMRCYPDRITTRAILEQDRLSVLARLAERA
>KAI2481640
VGNLQTFELPESLNGSVADRALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGMDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALYDPRVQQQWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPETNKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDTDGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPAEGIHYGSHFTNMFMRNYPERITADRMRKEGRLALL
>CAG9985177
FETFVLPEKVDGSPSNHALGKAMVQAWRRDGILQVAMSGQQQQVYREAQRISRRFFRRPHHQKATCVDAQSYAGYIASGEEITDGIPDYSEIFTVIKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLAANGERMLQLIEFGLHVPPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGEQYANWEKSSAGLLEDESGWVFVPPVPGVFTVFPGDMMQYMTDSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGYNRAQPPVDGIHYGTHFTNMFRRNYPDRLTTRRLEEEGR
>KAG8410572
QLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKAKPPQKQACVNDCSYAGYIASGEEITAGIADYSEIFTVTKDLHPKDPRVLEKWPCHGSCPWLDKRMKSPMLNYMADMGSSGDKVLRLIELGLDVPQGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWRQSVAGMKEDEPGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAVVRPLPGYNASQEPVDGIHYGTHFTNMALRNYPTRMATKKLLEEGRYAMLSKEELR
>VFK66213
MVDFRSFHLPESISGTQSDVDLGREMIRAWKTNGAFRIVMSEIQRKSSEDAFAANRQFFRGSIEFKSQYTNDLTYSGYTASGEEVTAGEKDYPEVFTICRDIPVDDARMRADWPCHGPVPWPNAEYRTSLQTYLNELGSVGDRLLQLVALGLELNDIDTLLALAKDGWHHLRALRYPVASQESRHGLGAHTDYGLIVITDEDNVGGLYIWPPVEGEKRNRNWLATESTAGTHENEKYWIFAKPAPNAFTVFPGDILQFITDNHILATIHKVLLNTRERFSMAYFHEPEFNACVYPLSNPSDKDYLFYGEHFTSMFMRCYPDGATTRRIMDENRLSVLTDLKERS
>CRG83534
LQTFELPDQVTGNQDDIVMGRALIDTWRRDGILQIAMNGKQEKLLKDAFTASKQFFGKPHAEKAACVDSQSYSGYIASGEEVTNGIADYSEIFTVTKTFDLSEPKVKEHWPCHGPCPWPDAEMKDPMQLYLADLGETGEKLLHLIEYGLGLERSSLTKYTKDGWHHMRILRFPANNANGKGPAGRGIGSHTDYGLLVIAAQDDVGGLFIRAPSKDEKTENWQTSAAGFREDDKGWIYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMSPLKGFDVGQSPKEGIHYGTHFTNMFMRNYPDRITTDRMIRENRRALLD
>PSK59305
SLQTFTLPEKITGSRGDRALGRTMVDTWRRDGILQISINHTNRKLADVAFESSKKYFGMSHRDKAKCVDDQSFAGYIASGEEITDNIADYSDIFTVTKDLPATDPRVVQKWPFHGPTPWPNAQMKTVMQAYMDYLGESGEKMLELIAYGLGLADGDALKKYTQDGWHHMRILRFPETHNTNGKGKEGRGIGSHTDYGLLVIAAQDAVGGLFIRPPYEGENYANWKDSAAGMKEDDDKWVYVPPVADTFTVFPGDMLQYATGNYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGYNAGQETTEGIHYGTHFTNMFMRNYPERITAHRMRSENRMEKLK
>WP_149852286
MTELRTFRLPDTVTGSEADLTLANALVRAWRADGIFQIAAEPTQYCAVADAMLASKRFFRLPRHVKARHVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPAHDRRVREAWPCHGPVPWPDRFYRNAMTALMSELGTMGEKLLRLIAFGLRLDDLDALTRLTRDGWHHMRVLRFPSATADTARGIGAHTDYGMLVIAAQDEVGGLYVRPPVPGEHRNRNWLPEESTAGLFEHEEPWTFVRPVPNTLTVFPGDILQFLTGGDLLSTPHKVRLNTRERFTMAYFHEPNFAATVRPLADPTGGEAIHYGTHFTNMFTRCYPGRITTRRIIEEDRLAVLARLR
>RYP78928
NLQTFELPRQVLGTEDDKNLGKSLVSAWQQDGILQITMNPDQQALYKAANAASQAFFKKPYAQKAACVDSQSFAGYIASGEELTDGIADYSEIFTVTKDLDLGDPRVRGKWPCHGRCPWPDADFRDPLTAYMDSLGESGEKLLQLIELGLDVPQGSLTNLTRDGWHHMRVLRFPATNDTNGKGKDGRGIGSHTDYGLLVLAAADEVGALFIRPPRKSESFANWEASAAGFKEDDEGWVFVPPMENVFTVFPGKSSKDLALWSRLHTDKVGSWLNTKERFAFAYFHEPSFQAIIRPLKGYNGGQEPREGVHYGKHFTDMFMRNYPERITTQRLVSEGRYKLLEE
>WP_057617985
LQTFRLPAAVRGTHNDRALGRALVAAWQSDGIFQIRATRTQQEATERALAESRTFCRRALKEKARHVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPVPWPSPGYARAMKDYMTVVGEIGGRLLRLTELGLGLDDPDRFTRLTTDGWHHMRVLRFPPADATSERGIGAHTDYGLLVIAAQDDVGGLHIRPPVPGEQRGRNWLPGESMAGRYEHEEPWTYVTPVAAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERHTLAYFHEPAFDAVARPLGAPDSAEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLRE
>KAH8785419
LETFDLPDRVTGSLSDKAMGQALIKAWKRDGIFQISMNSSQREIYKYAENASKGFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLGLAEPRVAKKWPCHGPCPWPDAEMRDPMKTYMNYLGDSGEKLLALTELGLGVPRGSLLQYTQDGWHHMRILRFPPTNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGETYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGYNNGQKPQEGIHYGTHFTNMFMRNYPDRITTQRLHAEDR
>MBW0498692
LETFDLPDYVTGSASDQALGKALIKAWKRDGIFQISMQRDQREIYKSAEKASKKFFAKPHNEKAACIDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDDPRVLKKWPCHGLCPWPDAEMRDPMKLYMNYLSESGEKLLALTELGLGVPKGSLLHYTQDGWHHMRVLRFPQTTKTNGKGKVGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGETYANWEQSAAGMKESDQGWVYVPPVADTFTVFPGDMMQYQTGSFLPSTPHKVGLNTRERFAFAYFHEPNFNAVLKPLVGYNVGQKPQEGIHYGTHFTNMFMRNYPERITTQRLNSEGRYKLL
>WP_026468744
MRTFELPDSVSGTTVDRILGRQLVRTWETDGIFQIAATAEQERITEEALAASRRFFGRSLPEKAKFVSDLTYSGYIASGEEVTAGEADFSEIFTVCPDVPLTDTRVTEGWPCHGPVPWPDDGYRTAMRAYLRQVGETGEKLLRLVALGLEVEIDRLTRLTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDEVGGLFIRPPVPGEPRRRNWLPEESTAGMYQDDEPWHFVEPEPRVFTVFPGDILQFLTGGALLSTPHKVRLATRERYTLAYFHEPSFRTTVRPLDGGDDRIHYGTHFTNMFTRCYPDRVTTKRIHEEGRLDRLRAISR
>KAI1845470
LQTFCLPERVTGSQSDRTMGKALVDAWQKDGILQIAMTPRQEALHSQAQQNSQEFFSRPHAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLELGDAKVKAQWPCHGRCPWPDAAMRDVMKEYMSELGNCGETMLRLLEYGLEIPEGTLTQQTDDGWHHLRILRFPPHNKTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGEKYANWEKSAAGMAEEDEKWTYVPPVPGVFTVFPGDMMQYMSNSTLPSTPHKVGLNTRDRFAFAYFHEPNFKTTCRPLPGYNAGQTPREGIHYGTHFTNMFLRNYPDRITTSRMNRENRYEILK
>KKO98130
LETFILPERVSGSVSDKAIADAMIGAWRRDGILQIGMSKPQQRLYQAANAASRRFFAQPHALKQACVDDKSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTRYMNDLAESGEKLLQLIEMGLDIPRGSLTRYTNDGWHHMRVLRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFSNWEKSSAGLRENEAGWVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGHNAGQSPIEGIHYGTHFTNMFLRNYPDRVTTARLREEGRYRLLE
>KAI3572988
KLQTFILPEEVSGSEEDKKLGKAIVDAWRKDGILQIAMKPEQEGLYKAANLASKRFFSKPHAQKAACIDSQTYSGYIASGEEMTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMNSLGESGEILLKLTELGLGVPEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFSNWEKSAAGFKEDDEGWLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGYNAGQEPKEGVHYGKHFTDMFMRNYPERITTQRLLEEGR
>XP_038783030
HLQTFELPESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTKYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKNEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDSPNLRWKDTPASG
>OZG71672
MSNLQTFRLPTSISTTSSNIKLGRDMIQAWRQDGIFQINTEAEQKQITKRAFEASKRFFSRPLTQKAACISQYSFSGYVSRGEEITAGKVDQSEIFTICPDIPKDDWRAKAQWPCHGPVPWPDSEYELAMKSFMGGLGSIGDKLLRLIALGLGLSDLNALADLAQNGWHHMRVLRFAAAGTDASSRGIGAHTDYGMLVIAAQEEVPGLYIRPPVEGEKRNRNWLTTESTAGMYGDKQPWNLVMPMPGVLTVFPGDILQFLTDSYLLSTPHKVELDAVRERYALAYFHEPGFETIVRPLNGKSGESIHYGTHFTNMFMRCYPEREATRRIISNGSLALLEAMR
>VFK55271
MSDLKTFYLPESVNGTALDIDLARKMIQAWRTDGAFQVAMTKIQREKSENAFAVNKWFFHMPNEFKSQFINDITYSGYTASGEELTADEKDYSEVFSIFKDIPIDDVRTQAHWPCHGPVPWPNVEYKKSIQAYMDELNSLGNDLLKLIALGLKLDNIDALTNLAKDGWCHIRALRYPEISQESERGIGSHTDYGLLVIINQNGVDGLYIRPPIEGESRNRNWLSTESTAGLYENEEPWIPVKPVSGVFIVLPGDIMQFITSSYLISTLHKIKLNTMERFSMAYFHDPDFNSCVRPLGNSEEYIFYGEYFTKIVMKSYPERDVTRRIMNENRLSVLATLK
>WP_073633445
LQTFHLPKSITGTQSDIDTAREIIQAWRTDGIFQVATNTIQDRKTESAFEASRRFFRMPMKFKSQCISDLNYGGYIASGEEVTAGKSDYSEIYTICKDIPLNDARVQAQWPCHGPMPWPDQEYHQSMKVFMDELGLIGEKLLKLTALGLGLDDINALTKLTRDGWHHMRVLRFPTLSQKSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESMAGMYENDDPWTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNMRERFALAYFHEPNFDACVRPLFDPSSDEHIHYGTHFTNMFMRSYPDRITTRRIINEDRLSILARL
>WP_184884329
MTQLRTFTVPASVTGQDSDARLAKDMIRAWQEDGVFQVATRPRQDALVAAAYAESGKFFARPQGFKAGHVSDLSYSGYTASGEEVTAGRADSAEIFTVCKDLPDDDPRVRDGWPCHGPVPWPGEDYRRAMTVLMHDLTSLGTRLLRLTALGLGLADLDVFTRLTHDGWHHMRVLRLPAVSPGDDRGTGGHTDHGLLVIASHDDTGGLYVRPPVPGERPGHDRPAGESTAGAYEEDKKDEDDDGWAYVPPVPRVLTVFPGDIMRFMTGGALPTTPHKVPPAPHDRHALAYFHEPGFQSVARPLTARTTPGDQDHIHYGTHVTAIFMRGHPRRVTTARIHAEGRLAILEQLR
>PHH77656
LETFTIPAEVTGSVSDKAMAKDMISAWRKDGILQIGMTHLQQRLFQDALSHSKRFFRRPLVEKRACVDSRSYSGYIASGEEVTDGVRDYSEIFTVTKDLPLSDQRVMESWPCHGPCPWPDNGFKEVISRYIAELGGCGQRLLQLIELGLGVQAGSLTRYTTDGWHHMRVLRFPARHLTNGKGKSGRGIGSHTDYGLLVIGSQDHIGGLFVRPPKEGENFVNWEQSAAGAKENDDGWMYVPPEQGTLTVFPGDIMQYLTNHVLRSTPHKVGLNTAERFALAYFHEPDFGSVVRPLAGYNGGQEPVEGFSYGTHFTNMCLRNYPDRITTRRL
>WP_229496330
MTDLQTFDLPEIITGSQSDIHLAHKMIQAWRTDGIFQVATNAIQARKTQNAFEASRRFFRMPLDFKSQCVSNLTYSGYIASGEEITAGESDYSEIFTICKDVPPDDMRVQAQWPCHGSVPWPDENYHQSMKAFMDELGIIGEKLLKLIALGLEIDVDTLTKLTKNGWHHMRVLRFPALSEKSTRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWLFNESSAGMYENDEPWTFVKPVPRVLTVFPGDILQFLTNNYLLSTPHKVRLNTRERFALAYFHEPNFEACVRSLFDLSSDDYIHYGMHFTNMFMRCYPYRITTRCILDEDRLSVLELLKN
>PNP54593
LETFTLPEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHTLKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLLHTDERVARGWPCHGPCPWLDQHMKAVMTKYMNDLAVSGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFSNWEKSSAGLRENEVGWVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGYNAGQSPIEGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLE
>WP_037692315
MTELQTFVLPPTVDGSDVDKELGQALIAAWQDDGIFQIQATPEQEAVTERALEASRGFFGRPFREKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVIDKWPCHGPAPWPSEQYADAMKSYMGAVGDIGERLLRLVALGLGLDDMDHFTKLTADGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLPDESMAGRYENEEPWTFVTPVPCVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGGADEFIHYGTHFTNMFMRCYPERTATARIEKEDRLAVLEQLRKEA
>KAG4418218
LRTFTLPDTVTSNSTSIHMAKAMIKEWRETGIFQIQMTPAQQAMLSSCFETSKNFFRSSHDEKAKHVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPETDVRVQAKWPCHGPTPWPSEEYKKQMDQLMKLLGDSGEKLLKLVGLGLGLKDPDALLNLTEDGWHHMRILRFPHVDKTNGKGKQGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIQGESIKNWETSAAGLNENDDKWLFVPPVPNTLTVFPGDMMQLITNDYLPSTPHKVGLNYDERYAFAYFHEPNFSSVIKTLPEFQSETEKVEKSEAIHYGTHFTDMFMRNYPERITADRIRDEGRLDIL
>User_Sequence
MTNLQTFELPTEVTGCAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLSVGDQRVKAGWPCHGPVPWPNNTYQKSMKTFMEELGLAGERLLKLTALGFELPINTFTDLTRDGWHHMRVLRFPPQTSTLSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDEPWTFVTPTPGVWTVFPGDILQFMTGGQLLSTPHKVKLNTRERFACAYFHEPNFEASAYPLFEPSANERIHYGEHFTNMFMRCYPDRITTQRINKENRLAHLEDLKKYSDTRATGS
>WP_003274237
MTGLTTFHLPERIIHSEAHRQLGQDMVAAWRADGIFQIALSTPEQHTTDEAFAQSRRFFELDFETKCRHVSAFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFGERLLQLIALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLAAESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADADTDVAPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSE
>XP_031038650
LENSRPFILPEEVSGSEEDKKLGKAIVDAWRKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASDEEMTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMNSLGESGEILLKLTELDLGVPEGSLTDYTKDGWHHMRILRFPVINKTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFSNWEKSAAGFKEDDEGWLFVPPAENVFTVFPGDMMQYLTNSFLPYTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGYNAGQEPKEGVHYGKHFTDMFMRNYPERITTQRLLEEGR
>WP_105968964
MTALQTFQLPLTVEGGDADRLLGRALVAAWQADGIFQVEATPDQDVATGRALDASRAFVGRPLKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPATDTRVVDKWPCHGPAPWPSGEYAAAMKDYMAVVGDIGHRLLQLTALGLGLDDMDHFTRLTDDGWHHMRVLRFPPADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLPEESMAGRYENEEPWTFVTPAPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLEPADADEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLSVLERLRKEA
>XP_013947392
QLEACTLPERVSGSVSDKAIADAMIGAWRRDGILQIDMSKTQQRLYQAANAASRRFFSRPPAQKRACVDHMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDQRVAQGWPCHGPCPWPDQSMKNAMKSYMSDLAVSGEKLLQLIEMGLDVPTGSLTRYTDDGWHHMRVLRFPHRDRTNGKGKKGRGIGSHTDYGLLVIAAQDEVGGLFVRPPRQDEQFANWEKTSAGLRENDAGWMYVPPAAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGYNAGQSPIGGIHYGTHFTNMFLRNYPDRVTTARLREEGR
>WP_138930184
MTDLTTFHLPERITNTEAHRELGQAMVKAWRTNGIFQIALSKPQEQATDEAFAESRQFFSQDFETKSRHVSALTYSGYVASREEVTAGETDYSEIFTICPDIGLEDARVREDLPCHGPVPWPGAAYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMDTFTRLTDDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADATAAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPTLSELA
>NEO55874
LGERIVAALQRDSLIQIKKNYQEDLLTARALLQNKKFNRLPQDEKIKFVSDTTYTGYVFSGEEKKGGRYDQVEVYTVFPHFDRDNERVKTLRLPCHGEAIYPSIDYKDAIEAYMKATGNMCDQLIEIIGLGLGLEPNHYLSEMVRDGFHHARVLRFKSQKEAGGASNGISAHSDYGFLIAASQDDVGGLWVRPVVNGEKRGRNYLEEETTIGAYDDKPGWAFVTPQESVWTVFVGDMMELATQRKLKANIHKVKLNEEKERFAIAYFHEPKFNQTFEAVNSRGKKKEIVYGKHLTNMYVKAYPNRSTTKRIESEELCKNIDRLPKIS
>KAI4941455
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELALTDPRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHEDTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGSHFTNMFMRNYPQRITADRMKAENRMALLD
>WP_196425794
MTALRTFELPDSVSGTTVDRILGRQLVRTWETDGIFQVAATGEQDRTTGEALAASRRFFARSLPEKAKFVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDVPLADTRVAEGWPCHGPVPWPDEDYRAAMKNYLHEVGEIGEKLLKLVALGLGADLDQLTRLTRNGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRNRNWLPEESTAGMYQDEEPWHFVEPEPRVFTVFPGDILQFLTGGRLLSTPHKVRLATRERYTLAYFHEPSFGATVRPLSSEDGDDEYIHYGTHFTNMFTRCYPDRVTTQRIHQEERLVTLDRLRARS
>KAG0154123
QLKTFTLPETATGSPSDVELGKAMINAWREDGILQVSMSPRQQALFENASAASKRFFAMPPNQKAACVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDVDMRTPIQQYMDSLGKSGETLLQMIEYGLSLHPDTLTSLTKDGWHHLRILRFPQNNKTNGRGKKGRGIGSHTDYGLLVIAAQDEVGGLFIRPPADDEKLENWKNSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVVSPVAKLYDGQPPVEKIHYGTHFTNMFMRNYPDRITTERIIKEDRLQLLD
>WP_253986668
MTDLQTFPLPDCVTGVPTDLELADRMVQAWRRDGIFQVAMNPSQEQQTERAFQASRRFFRMSLEAKSRCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDLPVTDARVRAQWPCHGPAPWPDDAYRQDMTAFTDALGSIGERLLRLTALGLGLEVDALTRLTRDGWHHMRVLRFPALSPTSSRGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMFEHEEPWTFVKPVPQVLTVFPGDILQFLTGGYLLSTPHKVRLNTRERYSVAYFHEPAFDACIRPLMGPRGEKAIHYGTHFTNMFMRCYPDRVTTRRILDENRLSTLARLRDEA
>XP_023450082
ELDTFELPVAVTGSDADRLLGRRLIDAWRKDGIIQVAFPENLRVLSEAVKQSKQYFLQDHATKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPNDAYAHAMRKLMDFKAGIGERLLQLTALGLDLKDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEGRGIGSHTDYGLLVIASQDEVGGLFVRPPSADETYSNWKKSSAGLHENDDKWLYIAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCRRLPEFKTGTPEDEEAVHYGTHFSNMFLRNYPDRVTAHRMREENRMDLLPMLK
>WP_181286820
MTLLQTFELPEDVHADQGNLWLGQRLIQAWRKDGIFQIRTEPAQDKKIQGAMAASKAFFKQSAQEKSRHVSDLSYAGYVASGEEMTAGEFDQSEIFTICKDLPQHDDRVKGGWPCHGPVPWPNDAYRDSMNAYLAELDLIGEKLLKLVALGLGLEIHTLTNLSQDGWHHMRVLRFPSESSKTCRGIGAHTDYGMLVIATQDDVGGLYIRPPVKGEHRNRNWLSAESTAGLFENDEPWTFVTPQPSVLTVFPGDILQFMTDGYLLSTPHKVKLNTRERFAMAYFHEPSFKASVRPLLKADDGARIHYGEHFTQMFMRCYPDRVTTQRIHKENRLRHLEEIR
>XP_042995097
QLETFSLPETVSGTLSDRATAKAMIDAWRRDGIFQIATSPSQQRLYDRATAASRAFFKRPPAQKRACVNDSSYAGYVASGEEVTAGVADYSEIFTVTKELKSSEYRVRNKWPCHGPCPWPDDGMRAAMTGYMADWDRSGGKVLQMVELGLAVPPGSLTKYTRDGWHHMRVLRFPPRHCTNGKGKAGRGIGSHTDYGLLVLAAQDEVGGLFVRPPRRGESFANWDESSAGMMEDEPGWVYVPPAPGVFTVFPGDMMQYVTNGFLRSTPHKVGLNARERFAFAYFHEPNFRAVVRPLPGYDAGQQPVDGVHYGTHFTDMALRNYPRRLSTTKLVREGRYAMLRGDSLRSDMGAV
>WP_221866169
MKKLSTFTLPETVSDSPAHRALGTQIVASWRKDGIIEVRRDAIQEQETQRAFEENQKFFARPLDEKRRYVDDLTFSGYVASGEEKKSSAKDFPEVFTIFPDISVSDERVRQEWPCHGPVPWPSEDFQSAMTAYIYTCSSIGHRLMQLTALGMGLPLDTFTSLCRRPWNYMRVLQFPAASVENTRGIGAHTDYDFFVISSQDLEKQGLFVRPPVEGEHRFRNWLPDESMRGVYETQEPWDLVEANPGVFTVFPGDMMQLVTDGYLLSTPHKVALFTSVRRAMPVFIGPDFDVRLRPHNKPDDAGFHYGEHVTNMHVSMYKDRSTSRRIVEKELLS
>WP_086762515
MTELQTFVLPSTVDGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVAGKWPCHGPAPWPSQSYADAMKAYMGTVGDIGEKLLRLVALGLGLDDMDHFTKLTDDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTFVTPVPSVFTVFPGDIMQFITGDILLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGAEDEFIHYGTHFTNMFMRCYPERTATARIENEDRLTVLERLRK
>QYC97872
MTNLQTFELPTEVTGCAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLSVGDQRVKAGWPCHGPVPWPNNTYQKSMKTFMEELGLAGERLLKLTALGFELPINTFTDLTRDGWHHMRVLRFPPQTSTLSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDEPWTFVTPTPGVWTVFPGDILQFMTGGQLLSTPHKVKLNTRERFACAYFHEPNFEASANERIHYGEHFTNMFMRCYPDRITTQRINKENRLAHLEDLKKYSDTRATGS
>VUC22500
FETFVLPEKVDGSPSNHALGKAMVEAWRRDGILQVTMTGQQQQVYREAQTISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGVPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQTYMDDLATNGEKMLQLIEFGLRVPSESLTKYTRDGWHHLRMLRFPPCNRTNGRGKKGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGELYDNWEKSSAGLLEDESGWVFVPPEPGVFTVFPGDMMQYMTNSVLSATHHKVGLNTRERFAFAYFHEPDFRAVVRPLSGYNRAQPPVDGIHYGTHFTNMFWRNYPDRLTTRRLEEDGRYSLL
>WP_248161456
MTDLITFALPEAVTDSKADRQLGRALIDAWRKDGIFQISIHDEQDQKVKRAMAASKAFFKLPREEKSRLVSSLTYAGYVASGEEVTAGEQDQSEIFTICKDLPLEDQRVVDGWPCHGPVPWPNDEYRDSMKAFLEELGDTGEKLLKLVALGLGLEINALTQLSQDGWHHMRVLRFPAESAQTCRGIGAHTDYGMLVIATQDEVGGLYIRPPVDGEHRKRNWLPDESTAGLFENDEPWTFVKPVPRVFTVFPGDILQFMTDGLLLSTPHKVKLNSRERFAMAYFHEPSFSVSAKPLFSQSCTERIHYGTHFTNMFMRCYPQRITTQCILKENRLSHLESI
>WP_057426137
MTNLQTFELPTEVIGSAADISLGRALIQAWQKDGILQIKTDSEQNRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLPVSDQRVKAGWPCHGPVPWPNNTYQKSMKAFMGELGLAGERLLKLTALGFELPINTFTDLTRNGWHHMRVLRFPPQTSTMSSGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDDPWTYVTPVQNVWTVFPGDILQFMTCGQLLSTPHKVRLNTRERFACAYFHEPNFEACAYQVFEPSGNERIHYGEHFTSMFMRCYPDRITTKRIHKDNRLAHFSNIPASS
>WP_063343083
MTVLQTFELPAQVTGTATDVTLGQKLIRAWRADGIFQIATDDKQDGTTLAAIAASKRFFKESQTFKSRCISDLTYSGYIASGEEITAGQADYSEIFTICKDLPESDPRVRNGWPCHGPVPWPDSVYHKRMKAFIDELGAAGEKLLKLTALGMNLPISTFTDLTRDGWHHMRVLRFPTETSKSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPEESSAGLFEHEEPWTFVTPTPRVWTVFPGDIMQFMTNGYLLSTPHKVKLNTRERFACAYFHEPGFEASARPLFEPGSQERIHYGEHFTHMFMRCYPDRITTRRIHEQNRLARLEELKARDKRA
>WP_123496614
MITLKTFKLPAKVTGSITDITLGEQLIRAWRTDGIFQIKTRVEQDRKTQAAIAVSKRFFKKPQTFKSGCINDSAYSGYIASGEEITAGQADYSEIFTICKDLPESDPRVQKGWPCHGPVPWPDSVYHKRMKAFMDELGAAGEKLLKLTALGMNLPISTFTDLTRDGWHHMRVLRFPTETPKSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGERRNRNWLPEESSAGLFEHEEPWTFVTPTPRVWTVFPGDIMQFMTNGYLLSTPHKVKLNTHERFACAYFHEPGFEASARPLFEPGSQERIHYGEHFTHMFMRCYPDRITTRRIHEQNRLARLEELK
>KAH0494541
LETFTLPERVSGSVSDKAIADAMIDAWRRDGILQIGMTKSQQKVYQAASAASRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQEMKAVMNKYMEDLAVSGEKLLQLIEMGLDIPTGSLTKYTADGWHHMRVLRFPQNNRTNGKGRKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRETERFSNWEKSSAGLRENDAGWVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGYNAGQSPLGGIHYGTHFTNMFLRNYPDRITTARLQEEGRYQLLELKELRD
>XP_040811647
LKTFTLPEKATGSPRDIEMGKAMINAWREDGILQVSMSSKQQDLFDKASAASKRFFAMPPTQKASCVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSGETLLQMIEYGLSLDPQTLTSLTKDGWHHLRVLRFPQNNNTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKLYDGQPPDEKIHYGTHFTNMFMRNYPNRVTTERILREDRLNLLD
>XP_024667451
LQTFELPAQVSGTPKDVELGKALVDAWRSDGILQISMKDTKKSLMDEVFDASKGFFQKPHSEKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEPRVKNHWPCHGPCPWPDLEMKDPMEQYMEYLGQSGEKLLQLIECGLGVPPGSLTNYTSDGWHHMRILRFPAVSNTNGKGKEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPSADEATANWEESAAGFHEDDSGWVYVPPMPDVFTVFPGDMMQYLTDSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVLRPLPGYDGGQKPREGIHYGTHFTNMFMRNYPDRITSKRMIRENR
>WP_174475597
MAELETFQLPQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKDHTDELGSTGERLLRLVALGLGLEVGALTALTRDGWHHMRVLRFPARSATATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLMTPGPDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTL
>KAF3069441
LGRLETFTLPERVSGSVSDKAIADAMIGAWRRDGILQIGMSKPQQRLYQAANAASRRFFAQPHAMKQACVDDRSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTRYMNDLAESGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFSNWEKSSAGLRENEAGWVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGHNAGQSPIEGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLE
>MBU3055413
MTSLSTFQIPKTVENDDTCRQLGKEIIAALQADGIFQISLPMDCVETTVNAFAESKRFFDLPFHQKSTYISELTYSGYIAQCEEVTAGEKDYSEIFTVCPDFPSNDSRVRDGLPCHGPTPWPNSAFRDRTDAFMNMLGTFGERLLRLIALGLDEEIETFTRLTEDGWHHMRILRFPTAQSSKSARGIGAHTDYGLLVIAAQDDVGGLYIRPPIKGERRNRNWLTSESSAGMYEHDEGWTFVEPVPAVLTVFPGDFLQFLTGGRLLSTPHKVRLNTRERFAMAYFHEPNFDTWVRPLDPTDPAEPIHYGTHFTRMFMRCYPQRSTTRRIERENLLSRL
>GKT55711
VAQLETFTLPDRVTGSKGDRAMGKALVEAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMNYLGESGEKLLQLVELGLEVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGLKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQ
>WP_141514839
MTGLTTFHLPERILHSEAHRQLGRDMVAAWRADGIFQIALSTPQQHTTDQAFAQSRQFFELEFETKRRHVSEFTYSGYIASREEITAGEADYSEIFTICPDIGMDDIRVREGWPCHGPVPWPGAAYRDRMKDFTDMLGTFGERLLQLIALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLASESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADADANAAPIHYGTHFTNMFMRCYPQRITTRRIEEQGLLNRLPSLSE
>RYP13485
NLQTFELPRQVLGTEDDKNLGKSLVSAWQQDGILQITMNPDQQALYKAANAASQAFFKKPYAQKAACVDSQSFAGYIASGEELTDGIADYSEIFTVTKDLDLGDPRVRGKWPCHGRCPWPDADFRDPMTAYMDSLGESGEKLLQLIELGLDVPQGSLTNLTRDGWHHMRVLRFPATNDTNGKGKDGRGIGSHTDYGLLVLAAADEVGALFIRPPRKNESFANWEASAAGFKEDDEGWVFVPPMENVFTVFPGKSSKDLAIWSRLHTDKVGSWLNTKERFAFAYFHEPSFQAIIRPLKGYNFGQEPREGVHYGKHFTDMFMRNYPERITTQRLVNEGRYKLLEE
>WP_103354041
MRTFELPDSVGGRTVDRILGRQLVRTWEVDGIFQVAATAEQDRITREALAASRRFFARSLPEKVKFVSDLTYSGYIASGEEVTAGEADFSEIFTVCPDIPFGDPRVTGNWPCHGPVPWPDEGYRTAMTAHLREVGGIGERLLQLVALGLGVEIDRLTRLTREGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFVRPPVPGEHRNRNWLPAESTAGMYQDDEPWHFVEPEAGVFTVFPGDILQFLTGGRLLSTPHKVRLATRERYTLAYFHEPSFGTTVRPLAGGDERIHYGTHFTNMFTRCYPDRVTTRRIHREGRLAVLDRIR
>WP_053789018
MAGFSTFLVPERVKATAADQELGRRMVRAWQQDGIFQVRRDPDLEELTDAAMCAARAFFRLPLGLKHRCVNDLSYSGYIESGEELTAGVPDQPEIFTVTPHFDHGHRRVRKGMPCHGPVPWPGDPFRYAIEAYLCGVGELGERILRLVALGLGLGPSGARQLTGLTRGGWHHLRALHFPARTGESERGIGAHTDYGLLVIAAQDEVGGLYVRPPVEGEQRSRNWVAGESSAGRAEDAQDWRFVEPESGVLTVFPGDILEFLTRGVVEATAHRVRLAERERYALAYFHEPHFDARMRPLGGRRSEETLHYGRHFTDMFMRCYPERATTRRIIREDRLAAM
>ODA76862
LKTFALPDKVSGSAADRATARAMLDAWQGDGILQIAMTPLQKQLYRSADAASRRFFARPSEEKRACVDGQSYAGYVASGEEMTDGIADYSEIFTVTKDLSLADGRVVDGWPCHGPCPWPDGTMKAVIDRYMADLGASGEKLLQLIELGLEIPAGSLTGYTEDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIANQDHVGGLFVRPPQGGGFANWQQSAAGMKEDEPGWTYVPPSPGVYTVFPGDMLQYMTNNTLLSTPHKVGLNTRERFAFAYFHEPNFRSVIRPLPGHDGGQEPRQGVHYGTHFTNMCLRNYPDRVTTKRMVAEDRYPMLATSALRNDDATTGS
>KAF4302266
HLQTFELPESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFSMDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPSHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDGRGIGSHTDYGLLVIAAQDEVGGLFVRPPMEGESYANWKESAAGKHEDADGWVYVPPVPNVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITAERMRAEGRMALLD
>KAG7136030
IGQLQTFVLPETFTSSASDVKLGKAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFSKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPIQRYMDSLGESGEMILKLTELGLDLPEGSLTGLTRDGWHHLRVLRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFANWERSAAGFKEDDEGWVFVPPAENVFTVFPGDMMQFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDVGQEPREGIHYGKHFTDMFIRNYPQRITTQRLVEEGRYDML
>WP_071092598
MTDLTTFHLPERITNTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPGAAYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMDTFTRLTQDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLPSESTAGVYEHDDGWNFIKPMPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADAAVAPIHYGTHFTNMFMRCYPKRITTQRITENGLLDKLPTLSELA
>WP_013006004
MTELQTFVLPPVVDGSDVDRALGEALIAAWQADGIFQIQATPEQEAATEQALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDMAEEDARVVDRWPCHGPAPWPSRQYAEAMKTYMGAVGDIGERLLRLVALGLGLDDMEHFTKLTRDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLHIRPPVPGEARGRNWLPEESMAGRYENEEPWTYVTPVPSVFTVFPGDIMQFITGGILLSTPHKVRLADRERYTIAYFHEPAFQAVARPLEGGGPEEFIHYGTHFTHMFMRCYPDRTTTVRIENEDRLAVLERLRKEALT
>WP_020135205
APLRTFRLPAAVDGTDADRALGRELVAAWQSDGIFQVHATPAQQEATDRALEASRAFCRRPLEEKAGYVSDLTYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKDYMTAVGEVGHRLLRLTALGLGLGDPDHFTRLTADGWHHMRVLRFPPAGAGSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEA
>WP_055324506
MTGLTTFHLPERILHSEAHRQLGQDMVAAWRADGIFQIALSTPQQHTTDEAFAQSRRFFELDFETKRRHVSELTYSGYIASREEITAGEADYSEIFTICPDISMDDARVREGWPCHGPVPWPGAAYRDRMTDFTGMLGAFGERLLQLTALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLPSESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLMSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADADTDVAPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALGEVA
>PMD18135
LETFVLPQTVEDNDYCRAIAKAMIEAWRKAGIFQIQMTTYQQDMLRECFKTSKEYFQLPFEEKAKSVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPESDPRVQKHWPCHGPCPWPNQTYKTKMNQLMDLLGDSGEKLLKLTALGLGLKDPDTFLKLTDDGWHHMRILRFPQVDRTNGKGKKGRGIGSHTDYGLLVIAGQDDVGGLFVRPKVEGENTKNWESSAAGLNEDDEKWLFVPPVENTLTVFPGDMMQFLTNDFLPSTPHKVGLNTAERFAFAYFHEPNFNSVLEPLPGFGEKIHYGTHFTNMFMRNYPERVTAKRIHDENRLEVLDRL
>WP_216586771
MTELQTFVLPSTVGGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVAGKWPCHGPAPWPSQSYADAMKAYMGTVGDIGEKLLRLVALGLGLDDMDHFTKLTDDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTFVTPVPSVFTVFPGDIMQFITGDILLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGAEDEFIHYGTHFTNMFMRCYPERTATARIENEDRLTVLERLR
>WP_164021163
MTDLQTFCLPDSVSGQPQDIELANRMIRAWRSDGIFQVAMNPTLEGTTERAFESSRRFFRLPLETRSCCVSDLTYSGYIRSGEEVTAGEADYSEIFTVTQDVSLQDSRVRAQWPCHGPAPWPDADYRQGMTAFTDALGSVGERLLKLTALGLGLDLDALTRLTRDGWHHMRVLRFPAQSSKSARGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEPRNRNWLATESSAGMYENEEPWTYVKPVPKVLTVFPGDILQFITNGYLLSTPHQVRLNTRERYSMAYFHEPAFDACVRPLMGPPSEEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLSILAGLREAS
>TDZ53310
VAQLETFTLPDRVTDTPGHRAMGKALVDAWRRDGILQVSMNEKQQAIFQEATQASKTFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPSPWPDYDMKQPMATYMDYLGESGEKLLQLAELGLNVPAGSLTDLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEHYANWEKSAAGMKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPEEGIHYGTHFTNMFLRNYPERITTKRMYAENRLEKLQ
>KAH6975750
LQTFTLSGEDMGSAADLSMGKALVAAWQQDGILQIAMDPEQQTLYKAANEASKRFFQKPFNQKAACIDSQSYSGYIASGEELTDGLADYSEIFTVTKDLDVDDPRVSAKWPCHGHCPWPDADLRDPVKKYMDSLWQTGEKLLRLIELGIDVPEGSLTRVTKDGWHHMRILRFPAINDINGKGDDRRGIGSHTDYGLLVIAAADDVGALFVRPPYKDEFVANWERSAAGMKEDSDGWVFIPPAENVFTVFPGDMMQYMTNSTLPSTLHKVGLNMKERLAFAYFHEPSFQAVIKPLPGYEAGQSPEEGIHYGKHFTNMCMRNYPERITTHRLVDEDRYKLLD
>VFK18546
MVDFRSFHLPESISGTQSDVDLGREMIRAWKTNGAFRIVMSEIQRKSSEDAFAANRQFFRGSIEFKSQYANDLTYSGYTASGEEVTAGEKDYPEVFTICRDIPVDDARMRADWPCHGPVPWPNTEYRTSLQTYLNELGSVGDRLLQLVALGLELNDIDTLLALAKDGWHHLRALRYPVASQESRHGLGAHTDYGLIVITDEDNVGGLYIWPPVEGEKSNRNWLATESTAGTHENEKYWIFAKPAPNAFTVFPGDTLQFITDNHILATIHKVLLNTRERFSMAYFHEPDFNACVYPLSNPSDKDYLFYGEHFTNMFMRCYPNGATTRRIMDENRLSVLTDLKN
>XP_008100791
VAQLETFTLPDRVTGSKGDIAMGKALVEAWRRDGILQVSMNDKQRFIFNDATAASKKFFAKTHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPVPWPDYEMKQPMTTYMDYLGESGEKLLQLTELGLNVPSGSLTNLTDDGWHHMRILRFPPTHNTNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEKFANWEKSSAGLKEEDEGWHYVPPVPNVFTVFPGDVMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDAGQSPREGIHYGTHFTNMFLRNYPERVTTKRMNAENRLDLLQKDELRSKGS
>WP_017528495
MTELTTFTLPEAVTGSHADHQLGRKLIEAWRTDGIFQVGTHDEQDQKVQRAMAASKAFFKQPRQDKSRWVSELTYAGYVASGEEVTAGEQDQSEIFTLCKDLPLDDQRVKDGWPCHGPVPWPNDEYRDSMKALLAELDVTGEKLLKLVALGLELEINALTALSRDGWHHMRVLRFPASSAHTHRGIGAHTDYGMLVIATQDEVGGLYIRPPVEGEQRKRNWLPGESTAGLFENDEPWTFVKPVPRVFTVFPGDILQFMTHGYLLSTPHKVTLNTRERFAMAYFHEPAFNASARPLLDPLNGERIHYGTHFTNMFMRCYPERVTTERIKRQDRLRHLEK
>KKY39969
LETFDLPDRVTGSASDKAMGQALIKAWKRDGIFQIAMNSSQREIYKYAETASKRFFAKPHAEKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVTKKWPCHGPCPWPDAEMRDPMKTYMNYLGDSGDKLLALTELGLGVPQGSLLQYTQDGWHHMRILRFPPTNKTNGKGKKGRGIGSHTDYGLLVIAAQDNVGGLFIRPPYQDESYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLDGFNNGQKPQEGIHYGTHFTNMFMRNYPDRITTQRLHAEDRYKSL
>TKW52418
VAQLETFTLPERVTGGKGDRAMGKALVDAWRRDGILQVSMNDEQQAIFNEATAASKAYFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMSEPRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESGEKLLQLVELGLDVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGLKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQ
>XP_020126245
NLKTFELPEVLTGGASDQALGRDMINAWRKDGILQIGMDPINRKLADAAFLCSKKFFGLPYEKKAACLDDQSFAGYIASGEELTDNIADYSEIFTVTKDLPATDARVRQKWPSHGPCPWPFQQMQTVMQSYMDYLGQSGEKMLQLIAWGLGLKDGDALTKYTQDGWHHMRILRFPERNNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEEVANWRKSAAGMSEDDDKWVYVPPVPDVFTVFPGDMMQYVTDSFLPSTPHKVGLNTRDRFAFAYFHEPNFSAVMKPLEGFDSGQEPREGIHYGTHFTNMFMRNYPERITAKRMRSENRMKLLE
>KIL86278
NLQTFVLPEKVSGSAEDKKLGKAIIDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPHAQKSACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLDLDEPRVVAKWPCHGRCPWPDYEMQNPMERYMNSLGESGETLLKLTELGLDVPEGSLADYTKDGWHHMRILRFPAMNKTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQKGDDFANWEASAAGFKEDADGWLFVPPAKNVFTVFPGDMMQYLTNSFLPSTPHKVGLNHRERFAFAYFHEPSFQATIKPLPGYDAGQEPKEGVHYGKHFTDMFMRNYPQRITTQRLLDEGR
>CRL20948
LKTFTLPEKATGSPRDVEMGKAMINAWREDGILQVSMSSKQQDLFDKASAASKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWLDSDMKTTIQQYMDSLGNSGETLLQMIEYGLSLDPGTLTSLTKDGWHHLRVLRFPQNNNTNGKGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKLYDGQPPDETIHYGTHFTNMFMRNYPDRVTTDRILREDRLKLLD
>KAI4918591
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDHKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELALTDPRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHEDTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFQAGQEPTEGIHYGSHFTNMFMRNYPQRITADRMKAENRMALLDSPDLRWKDTPA
>WP_212670581
LQTFRLPTVVRDTDGDRALGRALVAAWQRDGIFQVHATLDQQAATERALAASRAFCRRPLAEKTAHVSDLSYSGYVASGEEETAGQRDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKDYMKVVGEIGHRLLRLTALGLGLDDPDRFTRLTTDGWHHMRVLRFPPADATSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHDEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLAARERYTLAYFHEPAFNAVARPLDTAGSAEHIHYGTHFTRMFMRCYPQRVTTARIEAEGRLKVLDRLRE
>WP_167977648
MTELRTFRLPEIITGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVGKWPCHGPVPWPDDLYRDAMREFMAMLGDAGERILKLTALGFGLDIDTFTKLTVDGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPEESAAGCFQDSEGWTFVEPVQHVITVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDEPDGDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRS
>WP_007510697
MADLRTFQLPDFVDGTKSDIQLAGELIDAWRAGGILQVAAEPEQDQRTKAAIESSGRFFGMPTEFKRDCISGLTYSGYIALGEERTAGEADYSEIFTVCPDVRLDDPRVRAQWPCHGPVPWPDDEYEHDMLAFMAELGSIGERLLRLTALGLGIDVKGLTDLTAGGWHHMRVLRFPARSEHASRGIGAHTDYGLLVIAAQDDVGGLYIRPPVDGEQRSRNWLPGESMAGRYENEEPWTFVPPVPSVFTVFPGDMLQLITDGFLLSTPHKVRLNTRERFALAYFHEPNFQARVRPLSGGGDGDYIHYGRHFTSMFMRSYPERVTTRRILEEDRLSRL
>OPB42620
LETFTLPEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRIYQAANAASRRFFAQPHALKQACVDDKSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTKYMNDLAESGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFSNWEKSSAGLRENEAGWVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGHNAGQSPFEGIHYGTHFTNMFLRNYPDRVTTARLQQEGRYRLLE
>PK2
MTNLQTFELPTEVTGCAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLSVGDQRVKAGWPCHGPVPWPNNTYQKSMKTFMEELGLAGERLLKLTALGFELPINTFTDLTRDGWHHMRVLRFPPQTSTLSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDEPWTFVTPTPGVWTVFPGDILQFMTGGQLLSTPHKVKLNTRERFACAYFHEPNFEASAYPLFEPSANERIHYGEHFTNMFMRCYPDRITTQRINKENRLAHLEDLKKYSDTRATGS
>KAH8752860
LRTFELPREVTVNEENSNIGKALIEAWRKDGILQIRMTPEQSDTLRQCFKTSKEFFSLPAEEKATYVDDQSFAGYIASGEEITDGIADYSEIFTITKDLGQSDPRVQNKWPCHGPCPWPNESYKDKMNHLMDILGESGEKLLKLTALGLGLANPNALTEMTEDGWHHMRILRFPQVDKTNGKGKTGRGIGSHTDYGLLVIAGQDDVGGLFVRPRVEGESLKNWQTSAAGFQEDDDKWIYVPPVENVLTVFPGDMMQFLTNDYLPSTPHKVGLNTRERSAFAYFHEPTFSTALKPFAEYQDTVYYGTHFTNMFMRNYPERVTAKRMIAENRMDILEDL
>XP_049254215
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHEDTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFQAGQEPTEGIHYGKHFTNMFMRNYPQRITADRMKAENRMALLD
>XP_007695071
LQTFELPESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEARDPISGIHYGTHFTNMFMRNYPTRVTADRIRAEDRMALLE
>OQE43453
LKTFTLPEKAVGSPGDIQMGKDMINAWREDGILQVSMNSKQQDLFNKAFAASKQFFAKSPKQKAVCVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDIDMKAPIQQYMDSLGNSGETLLKMIEYGLSLDPGTLTSLTKDGWHHLRTLRFPQNNNTNGRGKDGRGIGSHTDYGLLVIAGQDDVGGLFIRPPAADEKLENWKKSAAGFREDDDRWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVVSPVAKLYDGKPPDETIHYGTHFTNMFMRNYPDRVTTERILGEDRLKLLD
>WP_059210412
LQTFRLPAAVRDTHDDRALGRALVAAWQSDGIFQIRATRTQQEATERALAESRTFCRRALNEKARHVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPVPWPSAGYARAMKDYMTVVGEIGGRLLRLTALGLGLDDPDRFTRLTTDGWHHMRVLRFPPADATSQRGIGAHTDYGLLVIAAQDDVGGLHIRPPVPGERRGRNWLPGESMAGRYEHEEPWTYVTPVAAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERHTLAYFHEPAFDAVARPLDAPDSAEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLRE
>WP_030327021
LRTFRLPAVVDGTDADRALGRALIAAWQSDGIFQIHATPAQQEATERALEASRAFCRRPLAEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKGYMTAVGEVGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSERGIGAHTDYGLLVVAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLAERERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEA
>KAF9634910
VGNLQTFELPEVLTGSASDQALGRSMIDAWRKDGILQISMDPINRKLADAAFICSKKFFDLPYKTKASCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPPTDPRVQQKWPCHGPCPWPFKQMRTVMNTYMDYLGESGEKMLQLIAWGLGLKDGNALTKYTQDGWHHMRVLRFPERNNVNGKGKDGRGIGSHTDYGLLVIAAQNDVGGLFIRPPMEGENYANWRKSSAGMAEDDDKWTYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGFDAGQEPREGIHYGTHFTNMFMRNYPERITANRMRSENRMKLL
>CAF9922273
LKTFVLPGTVTGESRDIELGKAIVRAFQTDGILQITMTPEQWKIYSEAEMASKRFFNKSHIQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLALDDVRVLDKWPCHGPTPWPDVEFRNPIKEYMTYLGKSGEKLLDLLELGLNVPKGSLSNLTEDGWHHLRVLRFPATDKTNGKGEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPYRDESYANWKDSAAGKFEHDDKWVYVPPQEGVFTVFPGDMMQYMTNSALPSTPHKVGLNTGERFAFAYFHEPSFQSTIKPLPGYDAGQEPVEGVHYGTHFTNMFLRNYPDRITSERMAEENRYDLLK
>WP_260470499
LRTFRLPDAVDGSDADRVLGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMNAYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRGRNWLPGESMAGRHEHEEPWTYVIPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTDYIHYGTHFTQMFMRCYPGRVTTARIETEGRLKVLDRLRE
>WP_167441344
MTELRTFELPDSVSGTTVDRILGRQLVRTWETDGIFQVAATAEQERITQEALTESRRFFGKSVPEKAKFVSDLTYSGYIASGEEVTAGEADFSEIFTVCPDVPLTDTRVTEGWPCHGPVPWPDDGYRTAMRAYLREVGEAGEKLLRLVALGLEVEIDRLTRLTRDGWHHLRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLPEESTAGMYQDDEPWHFVEPEPRVFTVFPGDILQFLTGGRLLSTPHKVRLATRERYTLAYFHEPSFSTTVRPLDGRDDRIHYGTHFTNMFTRCYPDRVTTQRIHEEGRLDRLRALSQ
>KAI1523220
NLQTFELPESLNGSVADRALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGMDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALYDPRVQQQWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPETNKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDTDGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPAEGIHYGSHFTNMFMRNYPERITADRMRNEGRLALLD
>WP_204767818
MTELRTFRLPETVTGSDDDVELGRRLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHGLSLEEKSQLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDNDYRDAMREFMAMLGEAGEKILKLTALGFGLDLDTFTRLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDEVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTFVEPVQHVFTVFPGDILQFLTGGALLSTPHKVLLNTRERFTMAYFHEPNFEAVIRPLDDPHGDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRS
>KIM35080
LATFTLPHEVVGNQSDIALGRKMVDTWRKDGIFQILSTPAQDETVRDAFKASKGYFSRPHEEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLPVTEPRVQMKWPCHGPCPWPYTNYATTMKKLMSHLGDSGEKLLKLTALGLGLDDPDALTSLTKDGWHHLRVLRFPQVDRTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFVRRPIEGENLHNWEKSAAGFKENDRNWTYVPPVDRVFTVFPGDMMQYLTRDYLPSTPHKVGLNTRQRYAFAYFHEPNFNAVVKPLRDYDHLGASDQPGIHYGTHFTNMFMRNYPERITAKRVVAENRMAILDEL
>WP_208776875
LQTFSSPHPVTGTDADRDLGRALVRAWQTDGIFQIRATTAQQAATEQALAASRAFCARPLREKATYVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPADDPRVIERWPCHGPTPWPSAAYAHAMKRYMALVGELGQRLLRLTALGLGLADPDHFTRLTAAGWHHMRVLRFPPATAASARGIGSHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHDEPWTYVTPVPAVFTVFPGDIMQFMTDGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLEPAAPADHIHYGTHFTSMFRRCYPDRITTTRMDADSALKTLTRLHE
>XP_009656748
IGQLQTFVLPETFTSSASDVKLGKAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFSKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPIQRYMDSLGESGEMILKLTELGLDLPEGSLTGLTRDGWHHLRVLRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFANWERSAAGFKEDDEGWVFVPPAENVFTFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDVGQEPREGIHYGKHFTDMFIRNYPQRITTQRLVEEGRYDML
>WP_254036614
MAELETFRLPHVVSGSEADVSLGLAMIRAWRRDGIFQIRMSPTQADKARHAFELSRRFFRQPLEEKQRCVSDSTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVHSSWPCHGPAPWPDEAYRLGMTAHTEELGSAGERLLRLAALGLGLDIDTLTSLTKDGWHHMRVLRFPARSPATTRGIGAHTDYGMLVIAAQDDVGGLWVRPPVEGEKRHRNWLQGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLAPRERFAMAYFHEPHFEACVRPLKNPTRDEFIHYGTHFTHMFMRCYPDRVTTQRILDEDRLTTLAWLRQEAVQR
>PVH92012
LGELTTFVLPETVTGSAEDVHLGRELVKTWRRDGILQIAFPKDLEVLNYEIRESKRFFALDYAEKAKCVDDQNFAGYIASGEELTDGIADYSEIFTVLKDLPESDHRVKEKWPCHGPVPWPSSVYRDAMTRYMDYQNTTGEKLLKLTSLGLGLQDPETLNRLTRDGWHHMRVLRFPAAHATNGKGKAGRGIGSHTDYGLLVIASQDDVGGLFIRKPRAGERYANWKESSAGKYEDDKHWTYVPPMDNVFTVFPGDMMQYLTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAYIEPLAEFGGSGEPIHYGTHFTNMFMRNYPERITAERIRKENRLEKLAELK
>OQE35386
FKTFTLPENVTGSPTDVQMGKDMINAWREDGILQVSMNPRQQDLCDKAFTASKRFFAKPPSEKAACVDTQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVQAKWPCHGPCPWPDIDMKTPMQSYIDSLGNSGETLLQLIEYGLDLDPSTLTSLTKDGWHHLRALRLPEMYNTNGRGKEGRGIGSHTDYGLLVIAAQDEVGGLFVRPPKTDEKPENWKKSAAGFREDDEGWVYVPPVPGVFGVFPGDMMQFITNSYLPSTPHKVGLNTRECFAFAYFHEPSFQATISPVGKLYNGQPPDEKIHYGTHFTNMFMRNYPDRVTTKRIISEDRLKLLD
>XP_040650479
LKTFTLPEKVTGCPGDIQMGKDLVNAWREDGILQIAMDPKQQALFDKAFAASKNFFALPPKQKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVKAKWPCHGPCPWTDNEMKTTVQQYMDSLGTSGERLLGLIEYGLNLEPKTLTGLTQDGWHHLRILRFPQTTNTNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRRPFDDEKLENWKKSAAGFQENDERWDYVPPVPGVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERYAFAYFHEPSFQAEIKPIPQLYDGNPPDEKVHYGTHFTNMFMRNYPDRITSERIIREDRMKILD
>KAH9219826
NLRTFVLPDTVNANDESTSMAKAMIKEWRETGIFQIQMTPAQQAMLTECFETSKEFFRRSHEEKSKHVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPSSDIRVQEKWPCHGPTPWPSKLYKDQMDQLMKLLGDSGEKLLKLVGLGLGLKDPDALLNLTEDGWHHMRILRFPHVEKTNGKGKKGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIQGEHIKNWKSSAAGLNENDDKWLFVPPVPNTLTVFPGDMMQLITNDYLPSTPHKVGLNYDERYAFAYFHEPNFSSVIKTLPEFQDEAEKVDKSEMIHYGTHFTDMFMRNYPERITADRIREEGRLDILAGLKK
>KAI5368897
VGHLQTFELPESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLAPSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKNEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALL
>KAI3400106
LETFDLPGRVTGSASDKAMGQAIIKAWKRDGIFQIAMNSSQREIYKYAEKASKGFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKTYMNYLGESGEKLLALAELGLGVPKGSLLQYTQDGWHHMRVLRFPPINKTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGENYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGYNNGQKPQEGIHYGTHFTNMFMRNYPERITTQRLLAEDRYKIL
>KJK79972
VAQLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPGDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSGDKVLQLIELGLDVPQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEEGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGFNGSQEPADGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKEK
>AKN93304
VNKIITFQIPPTIDVIQSNIDLGFYIIDAWRRDGIIQIALDDTQDAITQNAFKASQGYFALPHSEKSSHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPADDARVQHQWPCHGPVPWPNKGFQRDMALFTDQLGAIGEKLLRLVALGLHSLGRLDDIDGLNRLTLDGWHHMRVLRFPCATLGNQSRGISAHTDYGMLVIAAQDDVGGLYIRPPVEGEVRRRNWLSEESTAGMYENEEPWTFVQPTPRVVTAFPGDIMQFLTGGYLLSTPHKVKLNTRERYALAYFHEPNFKAVIEPLRPEPEDGRIHYGSHFTDMFMRSYPQRITTQRILRENRL
>NER04273
FETFDIRAVKNLKEQRELGERIVAALQRDSLIQIKKNYQEDLLTAQALLQNKKFNRLPQDEKIKFVSDTTYTGYVFSGEEKKGGRYDQVEVYTVFPHFDRDNERVKTLRLPCHGEAIYPSVEYKDAIEAYMKVTGNMCDRLIEIIGLGLGLEPNHYLSEMVRDGFHHARVLRFKSQKEAKGASNGISAHSDYGFLIAASQDDVGGLWVRPIVNGEKRGRNYLEEETTIGAYDEQPGWAFVTPMESVWTVFVGDMMELATQRKLKANIHKVKLNEERERFAIAYFHEPKFNQTFEAVNSRGKKKEVVYGKHLTNMYTKAYPNRSTTKRIESEELCKNIDRLPKIS
>KAH6843611
HLQTFELPESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKNEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMRPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLNSPNLRWKDTPAMG
>WP_257558168
MSGFATFQVPDRVTGTAADEALGRAMTAAWQRDGIFQIAADAGQRARTREAMAASRRFFARPPAEKAACVNDTSYSGYIASGEEVTAGERDASEIFTITPDLTDGPGLPCHGPVPWPDPGYRAAMEAYLAGVGGLGERLLRLVALGLGPGLEGGPGPAGMDRLAGLTRGGWHHMRVLRFPAASRTTERGIGAHTDYGLLVIAAQDEVGGLFVRPPIAGERRPRNWLPGESTAGFAQDSPGWTYVTPVPGVLTVFPGDILQFLTGGALLSTPHKVRLADRERYTLAYFHEPRFGARIRPLRPDPGDDTVIHYGEHFTRMFMRCYPQRATTRRIEAEGLLRRLGAVR
>OAA80456
LDTFCLPKKITGSAYDRSLARAMIEAWRRDGIFQVAMSPQQQRIYQNANAASRRFFRRTSAEKCRCVDEKSYAGYIASGEEVTDGVADYSEIFTVTKDLGENDPRVAGQWPCHGPCPWPDEDMKSNMTAYTKDLSQSGDQLLEMIELGLHVPQGSLKRYTDDGWHHMRVLRFPHKDRTNGKGKKGRGIGSHTDYGLLVMAAQDEVGGLFVRPPQQGEHYANWEQSAAGLKEDESGWVYVPPTPGVFTVFPGDMMQYMTNSFLKSTPHKVGLNTRERFAFAYFHEPNFASVIKPLPGYDGGVDASEGIHYGTHFTNMCLRNYPERITTKKLLREDRCSML
>XP_030980121
QLETFTLPDSVSGNRGDRAMGKALVEAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESGEKLLQLAEYGLGVPEGSLTQYTNDGWHHMRVLRFPQNNKTNGKGKEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQYANWEKSAAGMKEEDDRWVYVPPMPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPCFQAVAKPLPGYEAGQSPKEGIHYGTHFTNMFMRNYGSRITTQRLLAEDRYSLL
>KAH0525549
LGRLETFILPERVSGSVSDKAIADAMIGAWRRDGILQIGMSKPQQRLYQAANAASRRFFAQPHALKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKVIMTRYMKDLAESGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFSNWEKSSAGLRENEAGWVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGHNAGQSPIEGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLE
>CAA9960346
NLQTFELPETLNGSVADRALGKRMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLSMEDGNALTRYTQDGWHHMRILRFPQTNKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPAEGIHYGSHFTNMFMRNYPKRITADRMKDEGRLALLD
>RCJ18531
MTDLQTFDLPKSITGSQSDIDLAHQMIQAWRTDGIFQVATNAIQTRKTENAFEASKRFFRMPLDFKSQCISNLTYSGYIASGEEITAGESDYSEIFTICKDVRLDDVRVQAQWPCHGSVPWPDNNYHQNMKAFMDELGIMGEKLLKLVALGLELDDIDALTKLTRDGWHHMRVLRFPALSEKSTRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWLSDESSAGMYENDQPWTFVKPVPKVLTVFPGDILQFMTHNYLLSTPHKVRLNTRERFALAYFHEPNFQACVRPLFDSSNDDYIHYGTHFTNMFMRCYPYRITTRRILDEDRLSVLELLRN
>WP_193874723
MTDLKTLHLPQTITGTQSDIDLAHEMIQAWRTDGIVQVATNAIQHQKTERAFDASKRFFRMPLNFKSQCISDLTYSGYIASGEEVTDGQADYSEIFTVCKDVLPDDPRVQKQWPCHGPVPWPDQNYQQSMTAFMDELGCIGNVLLKLLALGLGLNNINALTELTDDGWHHMRVLRFPAASAHSARGIGAHTDYGLLVIAAQDDVGGLYVRPPVAGETRNRNWLPTESSAGMYENEEPWTFVKPVPKVLTVFPGDILQFMTHGYLLSTPHKVRLNTRERFALAYFHEPKFSRCVRPLFWPLGNDHIDYGTHFTNMFMRSYPDRITTRRIIAEDRLTNWTRFAQSPRERVAG
>RFU77259
LETFTLPEKVSRSVSDKAIADAMIRAWRRDGILQIGMSKAQQRLYQTANYASKRFFSRPYAQKQACIDALSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDKDMKTVMKRYMADLTVSGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQRDRTNGKGRTGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHRDERFANWEKSSAGLREDDAGWVYVPPVAGTFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRSVLRPLPGYNTGQSPIGGIHYGTHFTNMFLRNYPDRVTTARLREEGRYLLLGSEELRDDDD
>XP_022481658
VAQLETFTLPERVTGSPGDCAMGKALVDAWRRDGILQVSMNDKQQHIFNEATTSSKAFFAKPHDEKAACVDSQSFSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGDSGEKLLQLAELGLNVPAGSLTNLTRDGWHHMRILRQVIITGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGMAEEDDRWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPEEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLE
>XP_046049950
NLQTFILPEKVSNNEANRKLGKALVKAWQKDGILQISMTPEQHALYKSANYASRRFFSKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQNLGGVGETLLQLTELGLDVPEGSLTNYTEDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGALMIRPPYSDEKLANWESSAAGFREDDERWVFVPPAENVFTYITNSVLPSTPHKVGLNLKERFAFAYFHEPSFQAVVKPVPGYDVGQEPEDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>WP_054082735
MTNLQTFELPTEVTGCAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLSVGDQRVKAGWPCHGPVPWPNNTYQKSMKTFMEELGLAGERLLKLTALGFELPINTFTDLTRDGWHHMRVLRFPPQTSTLSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDEPWTFVTPTPGVWTVFPGDILQFMTGGQLLSTPHKVKLNTRERFACAYFHEPNFEASAYPLFEPSANERIHYGEHFTNMFMRCYPDRITTQRINKENRLAHL
>KKA30863
LETFTLPSTVTGSLSDRILGQRLISAWRRDGILQVRTSAAQQAVVDAAFAASKVFFREPHATKARLVDRWSYGGYIGSGEEITDGIADYSEVYTVTKDIPESDKRFMMPCHGRQPWPNSQFANVVSRYQQQLGRAGDVVLGLVELGLNIPRGSLTGLAEDGWHHLRILRFPHCNNTNGKGKAGRGIGSHTDYGMLVIASQDEVGGLFVRPPRDDEKTQNWRTSAAGIKENESGWVFVPPEDNVFTVFPGKDLPGDMLQYMTGHYLPSTPHKVGLNTAERFAFAYFHEPHFDRVIKPLPGFNNGQQPVEGIHYGTHFTNMFKRNYPDRVTTLQLNKDGR
>SMD26909
MADLRTFGLPDRVVGTDSDITLAHQMIDTWQADGIFQIACDDRQAAKVTEALMESRRFFASATEEKAKCVSPLSYAGYIASGEEVTAGEADYSEIFTVCQDIPPDDVRVQADWPCHGPVPWPSEGYQRTMRAYMDEMGLIGEKLLRLIALGLDLPEIDALTRLAEGGWHHMRTLRFPARNSLTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLDSESSAGMYENEEPWTFVTPVAGTLTVFPGDILQFLTDGVLLSTPHKVRLNDRERYTMAYFHEPNFDAGIRPLSGSDAFIHYGTHFTNMFMRCYPDRITTRAILEADRLSVLARLAERASAR
>WP_062237234
LRTFRLPSAVDGTDADRALGRELVAAWQSDGIFQVHATPAQQEATERALEASRAFCRKPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIRADDPRVIDRWPCHGPAPWPSPGYARAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTAYIHYGTHFTRMFMRCYPERVTTTRIETDGRLKVLDRLRE
>TKX19899
SLQTFTLPEKITGSRGDRALGRTMVDAWRRNGILQISMDHTNRKLADVAFENSKKYFGMSHRDKAKCVDDQSFAGYIASGEEITDNIADYSEIFTATKDLPLTDPRVVQKWPCHGPTPWPNAQMKTVMQAYMDYLGESGEKMLELIAYGLGLADGDALKKYTQDGWHHMRILRFPETHNTNGKGKEGRGIGSHTDYGLLVIAAQDAVGGLFIRPPYEGEKYANWKDSAAGMKEDDDKWVYVPPVADTFTVFPGDMLQYATGNYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGYNAGQEPTEGIHYGTHFTNMFMRNYPERITAHRMRSENRMEKLK
>AKF86718
MIRAWRRDGIFQIRMSPAQADKARHAFELSRRFFRQPLEDKQRCVSDSTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVHSGWPCHGPAPWPDEAYRLGMTAHTEELGSAGERLLRLAALGLGLDIDALTALTKDGWHHMRVLRFPARSPATTRGIGAHTDYGMLVIAAQDDVGGLWVRPPVEGEQRKRNWLQGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLASRERFAMAYFHEPHFEACVRPLKDPTRDEFIHYGTHFTHMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQR
>RCI10733
QLQTFTLPLKVTGSLSDRAMAEAMMSAWQRDGILQISKTRLQQRLYQDALASSRHFFDRPLAEKQACVDSRSYSGYVASGEEITDGIRDYSEIFTVTKDLPLSDERVLDGWPCHGPCPWPNSDFKEVISRYMKELGACGERLLQLIELGLDVPAGSLTRYTTDGWHHARGKSGRGIGSHTDYGLLVMGSQDEVGGLFVRPPREGETFMNWQKSAAGAMEDDDGWMYVPPVNGTITVFPGDIMQYLTNHVLRSTPHKVGLNTRQRFSFAYFHEPNFSSTVKPLSGYDVNQEPVDGINYGTHFTKMCLRNYPDRITTRRL
>ARV63120
LGERIVAALQRDSLIQIQKNYQEDLLTAQALLQNRKFNQLPFDGKIKFVSDTTYTGYVFSGEEKKGGQYDQVEVYTVFPHFDKDDELVKTLRLPCHGEAVYPSVEYKNAIEAYMKATGNMCDQLIEIIGLGLGLEPNHYLSKMVRDGFHHARVLRFKSQKEAGGSSNGISAHSDYGFLIAASQDDVGGLWVRPITNGEKRGRNYLEEETTIGAYNDQPGWAFVTPTASVWTVFVGDMMELATDRKLKANLHKVILNEEKERFAIAYFHEPKFNQTFLSVNSKGQKKEIVYGKHLTNMYTKAYPNRSTTKRIESEELIKNIDRLPKIS
>EXF73961
VAQLETFTLPERVTGSPGDRAMGKALVDAWRRDGILQVSMNDKQQHIFNEATASSKAFFSKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESGEKLLQLAELGLNVPVGSLTNLTQDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGMAEEDDRWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLEREELRAHS
>XP_044716689
LETFVLPEKVSGSVSDQAMAKAMIDAWRRDGILQVTMSQLQRQRYQSADASSRRFFRRPAGEKQACVDRRSYAGYIASGEEITDGVADYSEIFTVTKDLSPRDKRVAHGWPCHGPCPWPDSGMKAVMGRYMADLGASGNKLLELIELGLGVPTGSLTRYTDDGWHHMRVLRFPARHNTNGKGKDGRGIGAHTDYGLLVMAAQDDVGGLFVYPPQRGGRPVNWRQGAAGDEENEPGWVYVPPTPGTLTVFPGDMLQYMTNGVLRSTPHKVGLNTRERFAFAYFHEPNFRAVVKPLPGYSGGQEPAEGIHYGTHFTNMCRRNYPDRITTRRLVAEGRYRMLDSEK
>WP_109640897
MTELRTFRLPETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEAAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPPDDSRVVGKWPCHGPVPWPDDSYRDAMREFMAMLGDSGERLLKLTALGFGLDIDTFTRLTADGWHHMRVLRFPPRTEAVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIDGEERPRNWLPDESAAGCFQDAEGWTFVEPVQHVFTVFPGDILQFLTGGRLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDPHGDDVIHYGTHFTSMFMRCYPDRVTTRRILDEDRLTVLSELRS
>WP_030440504
MTDLQTFHLPTSVARTRATVDLGHDLIRAWQSDGIFQVAADPAQRETTRTAIAASWRFFRAPLEVKAQHVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLDDARVEARWPCHGPVPWPDDDYRARMRAFMGMLGSIGERLLRLVALGLDLPDIDALTRYTADGWHHMRVLRFPARSEQTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLASESSAGMYENEEPWTFVAPVPDVFTVFPGDILQFLTGGLLLSTPHKVRLNTRERFAMAYFHEPSFVSTVRPLFGPANDETIHYGTHFTNMFVRCYPDRITTRRILAEDRLALLTALAE
>QPG95136
NFETFVLPDKILGTLSDTSMAKAMINAWRKDGILQLAMSPRQERLFSQASKASKKFFQKTPAQKHACVNDSSFCGYVASGEEITSGIADYSEIFTVSKNLQPEDHRVLAEWPCHGPCPWPDKLMERDIMDYVSELGSNGEKLLQMVELGLDVPPGSLTKYTQDGWHHMRALRFPSRDRTNGKGAAGRGIGSHTDYGLLVVAAQDEVGGLFVRPPGQAEPCKHERSDTAVSRGDDSGWVYVAPTPGVFTVFPGDMMQYVTNSFLTSTPHKVGLNVCERFAFAYFHEPNFKAVVKPLPGFDAGQKPAGGIHYGTYFTNMTLRNYPDRISTRKLLDEGRCSMLEQDELRSDMAA
>MBW4538205
MTDLQTFHLPASITGTQSDVVLARKLIQAWRTDGIFQVAINAMQERKSESAFASSKRFFRMPQEVKSQCISDLTYSGYIALGKEITAGQADYSEIFTVCQDVPLDDARVQAHLPCHGPAPWPDEEYQQSMKAFMDELGTMGQRLLQLTALGLELDDINTLTNLTENGWHHLRVLRFPAASQESARGIGAHTDYGLLVIAAQDDVGGLYVRPPVAGEKRKRNWLATESSAGLYEHEELWTFVKPVPKVVTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFEPSSDEYIHYGRHFTNMFMRSYPDRITTRRLVDEDRLSNLAPVKNNALRRLT
>GJN75845
QLETFVLPEAVTGSVADVAMGRSLVAAWQRDGILQVCMSPLQRRLYEAADAQSRRFFARPQAEKQACVDSQSYSGYVASGEEITDGIADYSEIFTVTKDLPSSDPRVQSRWPCHGPCPWPDQKMKQVISRYIADLGTAGQKLLALIELGLDVPTGSLTQYTNDGWHHMRVLRFPARHKTNGKGKAGRGIGSHTDYGLLVMAAQDDVGGLFVRPPQGDSFANWQQSAAGVKEDDAGWVYVPPTPGVFTVFPGDMLQYMTNNALLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGYNAGQAPTEGIHYGTHFTNMCLRNYPDRVTTKRLAAEDRYQML
>WP_211770529
MTTLRTFQLPTEVTGSIWDISLGVELVRTWQSDGIFQVATGGVANSITAAALKESRRFFALPPAEKANLVSDLTYAGYVASGEEVTAGEQDFSEIFTVCRDIPGTDPRVRAGWPCHGPVPWPDTIYRDVMAALMSQFGLVGDKILKLTALGLGLPDMNALIDLATNGWHHMRVLRFPARSAATNRGIGAHTDYGMLVIATQDDVGDGLYVRPPVVGEKRPRNWLDHESSAGAFENEAPWHLVDPVPSTFTVFPGDILQFLTGGELLSTPHKVKLAPRERYSIAYFHEPDFTAALRPFRGGDPDDAIHYGTHFTSMFMRCYPDRLTTRRILADNRLAVLTQLRD
>KAH0427588
VAQLETFTLPERVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQSIFNEATASSKAFFAKPPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVRAKWPCHGPAPWPDYEMKQPMTTYMEYLGESGEKLLQLTELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSAAGMNEEDEKWHYIPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQD
>WP_243340464
MGELATFRVPARITGTTADTMLGGAMVRAWQRDGIFQVAADGRQEARTRAALAASRRFFARPAAEKAACVNDTSYSGYTASGEEVTAGRPDASEIFTITPDIPGIPDIPDGAAPGLPCHGPVPWPDPDYRAAMEAYLAGVGGLGERLLRLVALGLGLGPAGIDRLAGLTRGGWHHMRVLRFPAAERTAERGIGAHTDYGLLVIAAQDEVGRLFVRPPVAGEERPSNWLPGESTAGFAEHRPGWTPVRPVARVLTVFPGDILQFMTGGALLSTPHRVRLAERERYALAYFHEARFGARIRPLDPARSQQGGDGGTAPAVIHYGEHFTRMFMRCYPQRATTRRIEAQGLLRHL
>KND88345
LETFVLPEQVSGSVSDQAMASAMVDAWRRDGILQVSMSQLQRRLYQAANVASKRFFGRPQGAKNASVDSRSYAGYVASGEEITDGLADYSEIFTVTKDLSPRDRRVARRWPCHGPCPWPDKDMKAVVGRYMADLGASGETILRLIELGLGVPTGSLTRYTDDGWHHMRVLRFPSRHRTNGKGKDGRGIGSHTDYGLLVLAAQDEVGGLFVRPPQQGEKFNNWQKSAAGAKENESGWIYVPPVPGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGYNGGQAPVEGIHYGKHFTNMALRNYPDRVTTKRLVAEDRYRML
>WP_239333142
MNELQTFHLPEFIDGTESDLALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSGLTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPAPWPDAEYQRSMRVFMDELGRVGEKLLKLTALGLELDDTDALTRLTADGWHHMRVLRFPALSKESRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDGFLLSTPHKVRLNTRERFALAYFHEPNFAARVRPLSGTSADCIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLSLLRD
>WP_086746405
MTELQTFALPSTVDGSDTDKALGQALIAAWQADGIFQVQATPEQEAVTERALGASRGFFERPFKEKAGRVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVIDKWPCHGPAPWPSEQYADAMKDYMGSVGDIGERLLRLVALGLGLDDMDHFTKLTEDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTFVTPVPAVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGGPDEFIHYGTHFTNMFMRCYPERTATARMEKEDRLAVLERLRK
>WP_118884900
MTDLTTFNLPERITNTEAHRELGRAMVKAWRTDGIFQIALSTPQEHATDEAFAESRRFFALDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDLRVREDWPCHGPVPWPGTAYRERMKAFMGMLGTFGERLLQLTALGLGLNDMETFTRLTHDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGMYEHDDGWTFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADAAVAPIHYGTHFTNMFMRCYPKRITTRRIEEKGLLGKLPTLSEVA
>WP_141208136
LRTFRLPAAVDGTEADRALGRALVAAWQSDGIFQVHATPAQQEATDRALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDVPADDPRVIDQWPCHGPVPWPSPGYARAMKGYMTAVGEMGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPPGTADPTEYLHYGTHFTRMFMRCYPERVTTARIETEGRLKVLDRLRE
>CRK46671
QLQTFVLPETFTSSASDVKLGKAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFSKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPIQRYMDSLGESGEMILKLTELGLDLPEGSLTGLTRDGWHHLRVLRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFANWERSAAGFKEDDEGWVFVPPAENVFTVFPGKLLFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDVGQEPREGIHYGKHFTDMFIRNYPQRITTQRLVEEGRYDMLGEDSRTY
>OQE30276
KTFTLPEKATGSPGDVEMGKALINAWREDGILQIAMNPKQQDLFNKAFAASKRFFALPPNVKANCVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVADKWPCHGPCPWPDIDTKAPIQEYMDSLGSSGETLLQLIEHGLSLEPNTLTSLTKDGWHHLRTLRFPQNNKTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPYSDEKLENWKSSAAGFREHDDRWTYVPPVPGVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERYAFAYFHEPSFQAEISPIARLYDGKPPDEKIHYGTHFTNMFMRNYPDRVTTERILKEDRLKLLD
>WP_015351455
MIELETFQLPQSVSGREADIALGLTMVRAWRRDGIFQVRMSPAQAEKSQRAFELSRHFFRQSLETKARCVSDLTYSGYIASGQELTASEADLSEVFTVCRDVPLTDPRVQSKWPCHGPGPWPDESWRQGMQAHAEELGSVGERLLRLIALGLGLDIDALTTLTHDGWHHMRVLRFPARSPTTTRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRPRNWLPHESSAGMYEHDEPWTYVKPVPGVLTVFPGDILQFLTRGYLLSTPHKVVLNTRERFALAYFHEPQFEACVRPLSAPTRDEYIHYGTHFTNMFMRSYPDRVTTQRILDESRLTTLSWLRQEAVLR
>WP_057414127
MTNLKTFELPTEVIGSAADISLGRALIQAWQKDGILQIKTDSEQNRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLPVSDQRVKAGWPCHGPVPWPNNTYQKSMKAFMGELGLAGERLLKLTALGFELPINTFTDLTRNGWHHMRVLRFPPQTSTMSSGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDDPWTFVTPVQNVWTVFPGDILQFMTCGQLLSTPHKVRLNTRERFACAYFHEPNFEACAYQVFEPSGNERIHYGEHFTSMFMRCYPDRITTQRIHKDNRLAHFK
>WP_103337982
MRTFELPVSVGGTTVDRILGKQLVRTWEDDGIFQVAATAEQDRITGEALAASRRFFARSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTGGWPCHGPVPWPDEGYRTAMKAHLREVGEIGEKLLKLVALGLGVEIDTLTRLTRDGWHHMRVLRFPARSTTTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLPEESTAGMYQDDEPWHFVEPEPAVFTVFPGDIMQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGVTVRPLAGGDEGIHYGTHFTNMFTRCYPDRVTTKRIHEEDRMRLL
>WP_209639274
MADLRTFGLPGCIEGSAPDVRLAQEMIDAWRTDGIFQIACDRWQARKIQDAFEASRKFFALAAGMKSRCVSPLSYAGYIASGEEVTAGEADYSEIFTVCQDISLDDPRVRGNWPCHGPVPWPSAEYRRTMQAYMLEVGMMGEKLLRLIALGLDLPGINTLTRLTDNGWHHMRVLRFPARSSQTERGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLESESSAGMYENDEPWNLVRPVADTLTVFPGDILQFLTGGALLSTPHKVRLNDRERYTMAYFHEPNFDAGMRPLLGSDQDSYIHYGTHFTNMFMRCYPDRITTRGIVEADRLSVLAALAERASVGA
>KAH6610050
LETFTLPEKVSGSVSDKAIADAMIGAWRRDGILQIDMSRAQQRLYQAANMASRRFFSRPHAQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDADMKTVMNRYTNDLAVSGEKLLQLIEMGLDIPTGSLTRYTKDGWHHMRILRFPQRDRTNGKGKNGRGIGSHTDYGLLVIAAQDDVGGLFVRPPRQDERFANWEKSSAGLRENDAGWVYVPPVAGTFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGYNAGQSPIGGIHYGTHFTNMFLRNYPDRVTTARLREEGRTLLLGSEELRDEDD
>WP_239376700
MNELQTFHLPEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVGEKLLKLTALGLELDDTDALTRLTADGWHHMRVLRFPALSKESRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDDFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLSGVSADCIHYGTHFTNMFMRSYPARITTQRI
>UPL03496
LQTFVLPDKVTGGDLDKKLGKAIVQAWQKDGILQIAMKESQQALYDGANQASQRFFKKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLPLDESRVKEKWPCHGRCPWPDADLQIPMKQYMDSLGESGEVLLQLTELGLDVPEGSLTGLTRDGWHHMRILRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVALFVRPPQQSDEFANWNKSAAGFKEDDDGWMFVPPAKNVFTVFPDMMQYLTNSTLPSTPHKVGLNVRERFAFAYFHEPSFQAVVKPLPGYDVGQEPKEGVHYGKHFTNMFMRNYPQRITTQRLIDEGRYNLLEN
>EFQ89844
LQTFELPETLNGSVADRALGKRMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPQTNKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPAEGIHYGSHFTNMFMRNYPKRITADRMKDEGRLALLD
>KAH8589804
LKTFVLPETVDGQEENKAMADAMIHEWRNTGIFQIQMTPSQQALLTDCFSTSKEFFHHSFEEKSKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLKQSDSRVQNRWPCHGPTPWPTETYEAKMKQLMALLGDSGEKLLKLTALGLGLVNPNALLDMTEDGWHHMRILRFPQVDKTNGKGKQGRGIGSHTDYGLLVIAAQDDVGGLFIRPKIEGENTKNWKESAAGKHENDDKWLFVPPVPNTLTVFPGDMMQFLTNDFLPSTPHKVGLNYKERYAFAYFHEPNFSTVVETLPEFREKIHYGTHFTNMFMRNYPERITAERIHKEGRMGILEEL
>WP_215150000
MSGFATFQVPDRVTGTAADEALGRAMTAAWQRDGIFQIAADAGQRARTREAMAASRRFFARPPAEKAACVNDTSYSGYIASGEEVTAGERDASEIFTITPDLTDGTGLPCHGPVPWPDPGYRAAMEAYLAGVGGLGERLLRLVALGLGPGLDGGPGPAGMDRLAGLTRGGWHHMRVLRFPAASRTTERGIGAHTDYGLLVIAAQDEVGGLFVRPPIAGERRPRNWLPGESTAGFAQDSPGWTYVTPVPGVLTVFPGDILQFLTGGALLSTPHKVRLADRERYTLAYFHEPRFDAGIRPLRPDPGEDTVIHYGEHFTRMFMRCYPQRATTRRIEAEGLLRRLGAVR
>XP_007693478
LQTFELPESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNQKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDLRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFANWRESAAGKHEDTDGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVLKPLPGFEVREDSIGGIHYGTHFTKMFMRNYPTRVTADRIRAEERMALLE
>WP_189259271
MTELRTFRLPETVTGSDDDVELGRRLIAAWQADGILQIATDAAQDRKTEDAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVADKWPCHGPVPWPDNDYRDAMREFMAMLGEAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTFVEPVQHVFTVFPGDILQFLTGGALLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDPHGDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRS
>XP_001884879
LTTFTLPSSPHIAPTPENRALGQQMVQTWRKDGIFQIQLTPDQASALRAAFVLSKSYFKRPHHEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPDTLYESAMKRLMAVLGDSGEKLLQLTALGLNLANPNVFTDLTDDGWHHMRVLRFPQADKTNGKGKPGRGIGAHTDYGLLVIAGQDDVGGLFVRPPTDSGETSRNWEKYSGGLKEDQQESWSFVPPVENVLTVFPGDMMQYMTNDYLPSTPHKVVLNTRERYAFAYFHEPNFSSVVQPLPEYGRDQGKDVQVEDKGIHYGTHFTEMFMRNYPERVTAKRMKAENRLDIL
>XP_016595579
QLKTFTLPDKATGSPSDVQLGKAMIDAWRQDGILQVSMSPKQQDLFDKAFTASKRFFAMPPKQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVEAKWPCHGPCPWPDTDMRTPIQKYMESLGNSGETLLKMIEYGLDLDPKTLTSLTEDGWHHLRILRFPQNNNTNGKGKDGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAEDEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAEISPVAKLYDGQPPDEKIHYGTHFTNMFMRNYPDRITTDRIIKEDRLKLLD
>WP_174461383
MAELETFQLPQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEAKARCVSDSTYSGYVASGEELTAGEADSSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKAHTDELGSTGERLLRLVALGLGLEVGALTALTRDGWHHMRVLRFPARSPTATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEMRRRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTPGPDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLRE
>WP_152789356
MTALQTFRLPSAIEGGEADKTLGRALIAAWQSDGIFQIEATPEQEAATQRALNASRAFFSRPLPEKAKHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEGDARVVEGWPCHGPAPWPSKHYEGAMKDYMGVVGDIGHRLLRLTALGLGLDDTEHFTRLTDDGWHHMRVLRFPAADATSERGIGSHTDYGLLVIAAQDDVGGLYIRPPVPGEERGRNWLPEESMAGRFENEEPWTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTLAYFHEPSFQAVARPLDDDGDGEDGEDGEFIHYGTHFTNMFMRCYPERTTTARIEAEGRLDTLEALRK
>RYO28243
LQTFELPESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKNEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLD
>KAH8662548
ETFVLPDRVTGTFGDRTLGKVIIDTWRRDGILQIASDREGQQLQTHAYEASRQFFSKPQQEKASCVDSQSYAGYIASGEEITDGIADFSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPDHGMREPVTKYMEWLGTQGEKLLKLIELGLGIPEGSLSRYTNDGWHHSRILRFPPVFPSKDNGNEGRGIGSHTDYGLLVIASQDHVGGLFIRPPHEQENYANWEKSTAGFRQGDDGWYFIPPVPNVHTVFLGDMMQYLTNNKLTATLHKVGLNTRERFAFAYFHEPSFQAVLKPLTTHDLERGANGSGVDNIHYGTHFTNMFMRNYPDRITTRRIIEEGRYEKLSEDK
>CAF1459828
IANLKTFHIPPVVVGNAADIKLGKSMVLEWQTEGMFQVACTPDVYTSSQLAFEASREFFARPTSEKLSYVSDLTFAGYIASGEELTAGEADSSEIFTLVKDLPLTDKRVQDGWPGHGPVPWPSSTFKEALQKYLRELGSIGDRLLQLISLGLGLQNLNALKSIANDGWHHARVLRYPPTSSQSSRGTGSHTDYGFLVIITQDDAGGLSIRPPINGEIRNRNWLPSESSDGMYEDSEEWNYVKPVRSTLVVFPGDMFQLLTDGTLLSTPHKVHLTQKERYSLAYFHEPNFQQIIKPFNNADSMDFVHYGEHVTNMFLRGYPERITSKRINHENRMDTLVKLKTKS
>KLO82560
NLQTFILPDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFSKPYAQKAVCVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMHRYMQSLGGVGETLLQLTELGLGVPQGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGGNDEKLANWESSAAGFKEDDERWVFVPPAENVFTYITNSALPSTPHKVGLNLKERFAFAYFHEPSFQAVVKPLPGYDVGQEPKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>XP_018164639
VAQLETFTLPERVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFDEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMSEPRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESGEKLLQLAELGLNVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGLKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPQEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQ
>OAR04029
LDTFYLPNKVTGSAYDRSLARAMIEAWRRDGIFQVAMSPQQQRIYQNANAASRRFFRRPSAEKHRCVDGNSYAGYVASGEEVTDGVADYSEIFTVTKDLDENDPRVAGQWPCHGPCPWPDEDMKSNMTAYTQDLSQSGDRLLEMVELGLHVPQGSLKRYTDDGWHHMRVLRFPHRDRTNGKGKKGRGIGSHTDYGLLVMAAQDEVGGLFVRPPHQGEHYANWEKSAAGLKEDESGWVYMPPTPGVFTVFPGDMMQYMTSSFLKSTPHKVGLNTRERFAFAYFHEPNFASVIRPLPGYDGGGVGASEGIHYGTHFTNMCLRNYPERITTKKLLREDRCSML
>EOD48827
VGHLNTFTLPETLSGSPADKALGRSMIDAWKKDGILQISMDPINRRLADAAFLCSKKFFSMPYKEKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLQQSDPRVQQKWPCHGPCPWAFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMKDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPKEGIHYGTHFTNMFMRNYPERITAERMRQEGRMALL
>WP_024712742
MNKIITFQLPPTIDVIQSNIDLGFYIIDAWRRDGIIQIALDDTQDSITQNAFKASQGYFALPHSEKSSHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPADDARVQHQWPCHGPVPWPNKGFQREMALFTDQLGAIGEKLLRLVALGLHSLGRLDDIDGLNRLTLDGWHHMRVLRFPCATLGNQSRGISAHTDYGMLVIAAQDDVGGLYIRPPVEGEVRRRNWLSEESTAGMYENEEPWTFVQPTPRVVTAFPGDIMQFLTGGYLLSTPHKVKLNTRERYALAYFHEPNFKAVIEPLRPEPEDGRIHYGSHFTDMFMRSYPQRITTQRILRENRL
>WP_206784437
MRTFELPASVGGTTADRILGRQLVRTWEDDGIFQVAATPEQDRVTGEALAESRRFFARSLPEKAKFVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPRTDTRVTEGWPCHGPVPWPDDGYRTAMEAHLRQVGEIGEKLLRLVALGLEVEIDRLTELTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLPDESTAGMYQDEEPWHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGTTVRPLAGGDETIHYGRHFTNMFTRCYPDRVTTKRILAENRLRLL
>KAF4505756
QLETFVLPEQVSGSVSDQAMAKAMIRAWRRDGILQVAMSQPQRQLYQSADASSRRFFRRPPGEKRACVDRRSYAGYVASGEEVTDGVADYSEIFTVTKDLPPKDKRVAGGWPCHGPCPWPDREMKSVLGRYTADLGASGTKLLQLIELGLDVPAGSLTKYTNDGWHHMRVLRFPPRHNTNGKGKDGRGIGSHTDYGLLVIAAQDEVGGLFVYPPHRGEQFSNWRQSAAGAKENEPGWVYVPPTPGTLTVFPGDMMQYMTNGMLQSTPHKVGLNTRERFAFAYFHEPNFRSTVRPLPGFNGGQEPLEGIHYGTHFTNMCLRNYPDRITTRRLVAEGRYRMLGCEELR
>KAF4469781
QLQTFVLPDNVTGGDLDRKLGKAIVKAWQQDGILQIAMKESQQGLYKAANQASQRFFKKPYAQKSACIDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVVEKWPCHGRCPWPDADLQNPMKQYMDSLGESGETLLKLTELGLDVPEGSLTRLTQDGWHHMRILRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPQKSDEYANWNKSAAGFKEDDDGWMFVPPAENVFTVFPGDMMQYLTNSVLPSTPHKVGLNVSERFAFAYFHEPSFQAVIKPLPGHDAGQEPKEGVHYGKHFTDMFMRNYPQRITTQRLIDEGRYNLLE
>XP_044649230
SLETFDLPDRVTGSPGDKAMGQAIIKAWKRDGIFQIAMNSNQREIYKYAENASKKFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEHRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSGEKLLALTELGLGVPKGSLLQYTQDGWHHMRVLRFPPTNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGENYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGYNNGQKPQEGIHYGTHFTNMFMRNYPDRITTQRLLAEDR
>KID94921
VAQLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDKHMKSPMLSYMADMGSSGDKVLQLIELGLDVPQGSLTKYTDDGWHHMRVLRFPARHRTNGKGRAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEPGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNIRERFAFAYFHEPNFRAMVRPLPGYNASQEPADGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEVLSKEK
>WP_120050161
LRTFRLPAAVDGTDADRALGRELVAAWQSDGIFQVHATPAQQQATERALEASRAFCRRPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTAYIHYGTHFTRMFLRCYPERITTTRIETEGRLKVLDRLRE
>XP_014551369
LQTFEFPESLSGNVADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLTLSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPIQGETFANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAKEDPISGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLE
>KAF5706270
NLHTFILPDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEEITDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDNEMRNPMQRYMQSLGGVGETLLQLTELGLGVPKGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGGKYSTDEKLANWESSAAGFKEDDERWVFVPPAENVFTVFPGDMMQYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGYDVGQEPKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>WP_184858281
MTTLRTFQLPAEVTGSIWDIALGVELVRTWQSDGILQVATGSVANAVSAAALRESRRFFARPLPDKAALVSDLTYAGYVASGEEVTAGEQDFSEIFTVCKDVPVTDPRVRAGWPCHGPVPWPDPVYRDVMTALMSQFGLIGDKILKLTALGLGLADMNALTTLATDGWHHMRVLRFPARSEASTRGIGAHTDYGMLVIATQDEVGNGLFVRPPVPGEKRPRNWLDHESSAGAFENEEPWRLVDPVPGTFTVFPGDILQFLTSGELLSTPHKVKLADRERYSVAYFHEPDFPAALRPLAGGEPEEVIHYGSHFTSMFMRCYPERVTTRRILAEDRLEILARMRE
>WP_176174048
MRTFELPDSVSGTTVDRILGRQLIRTWEEDGILQVAATAEQASATGEALAESRRFFARSLPEKAKFVSDLTYSGYIASGEEMTAGEADYSEIFTVCPDIPLTDTRVSEGWPCHGPVPWPDEAYRTAMEAHLREVGGIGEKLLQLVALGLGVEIDRLTRLTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVAGEPRRRNWLPEESTAGMYQDEEPWHYVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPAFGVTVRPLAGGDETIHYGTHFTNMFTRCYPDRVTTKRILAEGRLRLLD
>WP_019362686
MTVLQTFELPETVTGSDSDKSLGQSLIAAWRKDGIFQVATTAPQHARTEAAMAASRRFFKMPMAFKSACVSDLTYSGYIASGEEITAGEADGSEIFTVCKDVPERDERVREGWPCHGPAPWPDENYQNSMRAFMDELGHIGEKLLQLAALGLQLPHINTLSDLTRDGWHHMRVLRFPTASSRSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSSESSAGMFENQEPWTFVKPVPKVLTVFPGDILQFMTDGYLLSTPHKVKLNTRERYALAYFHEPNFEAVARPLFRPGSDERIHYGDHFTHMFMRSYPERITTRRIHQEDRLARLREMQQRSLTPAT
>WP_197010885
MSDPHTFRLPAVVTPSCAGLGRELVQSLRTGGILHVRVDAVPAQKIVTAMTASRRFFSMPLAYKARHVSDLSFSGYLASRDPSAEGEDDHAETFTVCKDVPYGDARVRRGWPCHGPVPWPDPEYRLAMRAFMDVAGGVGERVLRLVALGLGWSADTLTRLTADGWHTLRVARFPARPARAAGPVGAHTDHGLLTVVAQSGPCGMYVRPPVPGEKRPRNWVEGESTAGLFEDEEPWHPVRHRPDLLSVFPGDLMQLLTDGRLPATPHKTAPAEDERHTLAYSHEPGFETSLTPIGVDGDPLHYGTHFTAVFMRRHPDRPVTRRILDEGRLEVLERLR
>WP_245822033
MTGLRTFRLPTTVTGSDDDAELGRQLIAAWQADGILQIATDASQDRTTAEAMAASRRFFHGLTLEEKSRFVSDLTYSGYIASGEEVTDGIADYSEIFTICKDVPLDDSRVAGKWPCHGPVPWPDESYRDAMREFMTLLGGAGERVLQLTALGFGLDLGTFTRLTADGWHHMRVLRFPTRSESDRGIGAHTDYGLLVLAAQDDVGGLFVRPPIEGEDRPRNWLADESAAGCFQDAHGWNFVAPVDRVLTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFDAVVRPLDDPDGDTIHYGSHFTAMFMRCYPDRVTTRRIIEEDRLKVLSELRS
>WP_097251030
MTELQTFVLPPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIYTVCPDIPEDDARAVDKWPCHGPAPWPSEQYAGAMKDYMGAVGDIGERLLRLVALGLGLDDMDHFTKLTEDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLADESMAGRYENEEPWTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGSAPDEFIHYGTHFTNMFMRCYPDRAATARIENEDRLAVLARLRKEA
>XP_029744816
QLETFTLPESVTGNRGDRAMGKALVEAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVRAKWPCHGPCPWPDAEMRDPMKKYMDYLGESGEKLLQLTEYGLGVPEGSLTQYTNDGWHHMRVLRFPQNNKTNGKGKEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADENYANWEKSSAGMKEEDDRWVYVPPVSNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGYDVGQSPREGIHYGTHFTNMFMRNYESRITTQRLLAEDRYSLL
>XP_035369290
LKTFELPEVLTGSASDQALGRSMIDAWRKDGILQISMDPINRKLADAAFICSKKFFDLPYKTKASCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPPTDPRVQQKWPCHGPCPWPFKQMRTVMNTYMDYLGESGEKMLQLIAWGLGLKDGNALTKYTQDGWHHMRVLRFPERNNVNGKGKDGRGIGSHTDYGLLVIAAQNDVGGLFIRPPMEGENYANWRKSSAGMAEDDDKWTYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGFDAGQEPREGIHYGTHFTNMFMRNYPERITANRMRSENRMKLLE
>EME41613
ELQTFVLPEKITGSDTDRALGNELVAAWRRDGILQIAFPPGLQVLSDAIKQSKSFFAQQHEAKAKCVDSQSFAGYIASGEEITDGIADYSEIFTVTKDLPASDLRVQSQWPCHGPNPWPSKSYSQAITALMDYKGRIGEKILHLTALGLNLRNPWALNQLTQDGWHHMRVLRFPPTNQTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFIRPPIEGERYENWKKSAAGAHEGDEKWVYVPPVENVFTVFPGDMMQFMTNSYLPSTPHKVGLNIRERFAFAYFHEPNFNAVCKKLPEFRDGSEAEKDEVHYGTHFTNMFMRNYPDRITAKRMVDEDRMGLLEGMK
>WP_050879374
MKKLRTFTLPDTVSDSPAHRALGILIIACWRKDGIIEVRRDEIQEQETRRAFEENQKFFARPLDEKRRYVDDLTFSGYVASGEEKKSSAKDFPEVFTIFPDISVSDERVRQEWPCHGPVPWPSEDFKNAMTAYIYTCSSIGHRLMQLTALGMGLPLDTFTRLCRRPWNYMRVLQFPAASVENTRGIGAHTDYDFFVISSQDLEKQGLFVRPPVEGEHRFRNWLPDESMRGVYETQEPWDLVEANPGVFTVFPGDMMQLVTDGYLLSTPHKVALFTSVRRAMPVFIGPDFDVRLRPHNKPDDAGFHYGEHVTNMHVSMYKDRSTSRRIVEKELLS
>KIM92818
HLQTFDLPEQVTGTATDRELGKAIINAWRRDGILQITMDPKQAALLEEAVVASKRFFRKPHKEKAACIDSQSYSGYIASGEEVTDGIADYSEIFTVTKDLDLAEPRVKAKWPCHGRCPWPDIEMKAPMQAYIDYLGKSGEKMLQLIEYGLEVEPGSLTKYTKDGWHHMRILRHVLYKFPATNNTTDEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGEEYANWERSTAGLKEDDDGWVYVPPVPGVFTAFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLKGFDAGQLPKEGIHYGTHFTNMFLRNYPDRITTERLIAENRQSLL
>KJZ78591
LETFVLPEQVSGSVSDRAMAKAMINAWRRTGILQVGMSQLQRQLYESADTSSRRFFRRPAMEKRACVDSRSYAGYVASGEEITNGVADYSEIFTVTKDLSHRDNRVLHGWPCHGPCPWPDREMKAVMRRYMADLGASGDKLLRLIELGLDVPEGSLTRYTHDGWHHMRVLRFPSRNNTNGKGKDGRGIGSHTDYGLLVMAAQDDVGGLFVLPPREGRQFANWQQSAAGLKESEPGWVYVPPVSGAFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVIKPLPGFQGGQQPLEGIHYGAHFTNMCLRNYPDRITTKRLEAENRYRLLD
>KLU82627
LDTFELPAEVRGTPGDRAIGKAMVESWRRNGILQIAMSPAQDAVYKAAQEASERFFAKPPNEKQACIDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVLNKWPCHGPCPWPDAEMREPMRRYMDYLNESGEKLLQLTEYGLGVPEGSLTNLTKDGWHHMRILRFPPTNQTNGKGKEGRGIGSHTDYGLFVIAAQDNVGGLFVRPPYDGEKYANWEKSSAGMKQDDDGWVYVPPVPGTFTVFPGDMMQYMTNSHLPSTPHKVGLNTAERFAFAYFHEPSFQAVCKPLPGYEAGQEPCEGIHYGTHFTNMFLRNYPDRITTQRLLAEGRQHKLSD
>KAF2212808
LETFDLPVAVTGSDADRLLGLQLVDAWRKDGIVQIAFPKDLRVISDALKQSKKYFIEDHETKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVVNGWPCHGPNPWPSDTYAHAMLKLMDFKASIGDRLLQLISLGLDLEDQWALSKLTQDGWHHMRVLRFPATHATNGKGKEGRGIGSHTDYGLLVIASQDEVGGLFIRPPTADETYSNWKTSTAGLHENDDRWLYVAPQENVFTVFPGDMMQFITNSYLPSTPHKVALNTRERFAFAYFHEPNFNAVCRRLPEFKTGTPEDEEAVHYGTHFTNMFMRNYNTRITANRMREEKRMDMLPQL
>WP_174485597
MADLVTFQLPQSVSGSESDVALGLTMIRAWRRDGIFQLRMSEAQLEKTQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKEHTDELGSTGERLLRLVALGLGLEVGALTALTRDGWHHMRVLRFPARSTTATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTPGPDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLRE
>KAI4923174
VGNLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHEDTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFQAGQEPTEGIHYGSHFTNMFMRNYPQRITADRMKAENRMALL
>WP_207390922
MTNLQTFYLPKSITATQSNIDLGHQMIQSWRTDGIFQIATSTIQDRKVQNAFEASRRFFRMPLGFKSQRISDLTYSGYIASGEEITAGESDYSEIFTLCKDVPLDDVCVQKQWPCHGPVPWPNHDYQQNMTIFMDELGLIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRVLRFPPISKKSARGIGAHTDYGLLVIAAQDDAGGLYVRPPVEGEQRNRNWRSDESSAGMYENEEPWTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSNPSSNEYIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLRN
>KAG4431558
NLRTFVLPDTVNANDESTNMAKAMIKEWRETGIFQIQMTPPQQAMLTECFETSKKFFRRSHEEKSKHVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPSSDARVQEKWPCHGPTPWPSKVYKDQMDQLMKLLGDSGEKLLKLVGLGLGLKDPDALLNLTEDGWHHMRILRFPHVEKTNGKGKKGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIQGESIKNWQSSAAGLNENDDKWLFVPPVPNTLTVFPGDMMQLITNDYLPSTPHKVGLNYDERYAFAYFHEPNFSSVIKTLPEFQDKAENFDKSEMIHYGTHFTDMFMRNYPERITADRIREEGRLDILAGLKE
>CAH0016877
FETFVLPEKVDGSPSNHALGKAMVEAWRRDGILQVTMSGQQQQVYREAQRISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGIPDYSEIFTVTKDLPLDDQRVINGWPCHGPCPWPDSDMANVIQRYMDDLAANGERMLQLIELGLHVPPRSLTKYTGDGWHHLRMLRFPPCDRTNGRGKEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGERYANWEKSSAGLLEDESGWVFVPPVPGVFTVFPGDMMQYMTNSVLLATPHKVGLNTRERFAFAYFYEPDFRAVVRPLPGYNRAQPPVDGIHYGTHFTNMFWRNYPDRLTSRRLEEEGR
>PPJ58082
ELDTFELPVAVTGSDADRLLGRRLIEAWRKDGIIQVAFPENLRVLSEAVKQSKQYFLQDHATKAKCVDSNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPNDAYAHAMRKLMDFKAGIGERLLQLTALGLDLKDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEGRGIGSHTDYGLLVIASQDEVGGLFVRPPSADEEYSNWKKSSAGLHENDDKWLYIAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCRRLPEFKTDTPEDEEAVHYGTHFSKMFLRNYPDRVTAHRMREENRMDMLPLLK
>WP_053745397
MTELQTFQLPQAVEGSDADRVLGQALVAAWQADGIFQIQATPEQDAATGRALDASRAFVGRPLKEKAQHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPGDDPRVLDRWPCHGPAPWPSEDYAAAMKSYMAIVGDIGHRLLQLTALGLGLDDMEYFTRLTEDGWHHMRVLRFPAADATSERGIGSHTDYGLLVIAAQDDVGGLYIRPPVPGETRGRNWLPEESMAGRYENEDPWTFVTPAPSVFTVFPGDIMQFITGGALLSTPHKVRLADRERFTMAYFHEPSFQAVARPLDAPETDEFIHYGTHFTNMFMRCYPERSTTARIEQEDRLVVLERLRK
>XP_001270369
DLETFELPAVVTGSLADVDLGKALIAAWRRDGILQVAMEDSQKGLLGETFDASKNFFRKPHREKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLELSEPRVKSHWPCHGPCPWPDDEMKTPMQRYMDYLGQSGEKLLQLVEYGLGVPSGALTNYTQDGWHHMRILRFPAVNAANGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPAVDETTANWEQSAAGLWEDDDGWTYVAPVPNVFTVFPGDMMQYLTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVITPLEGYDGGQNPREGIHYGTHFTNMFMRNYPDRITSQRIIRENRKALLAD
>PHH72215
LETFVLPEKVSGSVSDQAMAKSMINAWRRDGILQIGMGQSQRRLYRSADAASRKFFQRTAKDKQACVDSCSYSGYIASGEEITDGVADYSEIFTVTKDLSHSDKRVANNWPCHGPCPWPDPDMKSVMSRYMADLGTSGEKLLQLIELGLEVPAGSLTGYTDDGWHHMRILRFPPRHATNGKGKDGRGIGSHTDYGLLVIAAQDHVGGLFVRPPNKEDKFANWQQSSAGMKEDDSGWVYVPPMPDVFTVFPGDMLQYMTNNVLLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGYNGGQAPMEGIHYGTHFTDMFLRNYPERITTKRMEAESRYQLLK
>KZL84523
VAQLETFTLPDRVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKTFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESGEKLLQLAELGLDVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGLKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQ
>WP_062048822
LQTFRLPAAVRGTHSDRALGRALVAAWQSDGIFQIRATRTQQEVTDRALAASRTFCRRALNEKARHVSDLSYSGYVASNEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPVPWPSPGYARAMKDYMTVVGEIGGRLLRLTALGLGLDDPDRFTRLTTDGWHHMRVLRFPPADATSQRGIGAHTDYGLLVIAAQDDVGGLHIRPPVPGEQRGRNWLPGESMAGRYEHEEPWTYVTPVAAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERHTLAYFHEPAFDAVARPLDAPDSAAYIHYGTHFTQMFMRCYPERVTTARIEAEGRLKVLDRLRE
>XP_014080001
LQTFELPESLSGSVADRALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYHKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFANWKESAAGKHEDADGWVYVPPVSDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEARQDPISGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLE
>XP_049196726
NLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELALTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKTTAAGKHEDTDGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFQAGQEPTEGIHYGKHFTNMFMRNYPQRITADRMKAENRMALLDSPELRWKDTPA
>WP_086841583
MRTFELPVSVSGTTVDRILGRQLVRTWEDDGIFQVAATAEQARVTGEALAESRRFFARSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTKGLPCHGPVPWPDDGYRTAMEAHLREVGEIGEKLLKLVALGLGVEIDRLTRLTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLAEESTAGMYQDDEPWHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGTTVRPLAGGDETIHYGTHFTNMFTRCYPDRVTTKRILAENRLRLLK
>KAI8307980
LETFTLPERVTGSKGDIAMGKALVDAWRRDGILQVSMNDMQQAIFNEATASSKAFFAKPPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMEYLGESGEKLLQLTELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSAAGMKEDDEKWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQ
>OPG07481
LRTFRLPAAVDGTDADRALGQELVAAWQADGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMNGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTARIETEGRLKVLDRLRE
>QHB56733
MTGLTTFHLPERIIHSEAHRQLGQDMVAAWRADGIFQIALSTPEQHTTDEAFAQSQRFFELDFETKCRHVSAFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFGERLLQLIALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLAAESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDTWVEPLKADADTDVAPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSEVA
>WP_247311730
MTDLTTFHLPDRIINTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSPLTYSGYIASREEVTAGEADYSEIFTICPDIGLDDARVREDLPCHGPVPWPSADYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMNTFTRLTDDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADATAAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPALSEVA
>XP_043175567
VGHLQTFELPESLSGSAADKALGNSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFGLNYAKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEADQEPVEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALL
>KAI6304082
QLETFTLPESVRGSRGDREMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDNQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESGEKLLQLTEYGLGVPEGSLTQYTNDGFPQNNKTNGKGKEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQFANWEKSSAGMKEDDDRWVYVPPVPNVFTVFPGDIMQYITNSHLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGYDAGQSPKEGIHYGTHFTNMFMRNYESRITTQRLLGEDRYSLL
>WP_028861955
MTDLTTFHLPERITNTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVRENLPCHGPVPWPGAAYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMDTFTRLTQDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLPSESTAGVYEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAVAYFHEPNFDAWVEPLEADAAVAPIHYGTHFTNMFMRCYPKRITTRRIMENGLLDKLPTLSELA
>WP_030204587
MGELTTFRMPACGAGTAADAALGRAMVAAWRRDGIFQVAADAGGRARTEDALAASRRFFARPYPEKAALVSDVSYSGYIASGEELTAGRPDRSEVFTVTPDLPPTHPRVRAGWPAHGPVPWPDSGYRAAMEAYLAVVGELGERLLRLVALGLRLGERGAAALAGLTRGGWHHMRVLRFPARTVAGGSGIGAHTDYGLLVIAAQDEVGGLYVRPPVPGEDRPCNWRPGESTAGRWEREPGWCFVPPVPGVFTVFPGDMMQFLTAELAGGPLLSTPHRVALAERERYALAYFHEPAFDAVLRPLRPAGRRAGPGAEEIHYGAHFTRMFMRCYPDRVTTRRILAEDRLARL
>WP_063849526
MTNLQTFYLPKSISATQSNVDLGHQMIQAWRTDGIFQVATNTIQDGKIQNAFEVSRRFFRMPLGLKSQYISDITYSGYIASGEEITAGKSDYSEIFTICKDVPLDDVCVQKQWPCHGPVPWHNQNYQQNMTTFMDELGFMGQKLLKLVALGLGVDMNTFDFTLNGWHHMRVLRFPAVSEKSARGIGAHTDYGLLVIAVQDDVGGLYVRPPVEGEKRNKNWRSDESSAGMYENEEPWTFVKPIPRVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLSNPSSNEYIHYGTHFTNMFMRCYPDRITTRRIHDEDRLSVLAVLRN
>USP78727
VGQLQTFELPETLSGSAADKALGKSMIDAWRRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQRWPCHGPCPWPFSQMKTVMQAYMDYLGESGEKMLQLIAWGLGLRDGNALTRCTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDAEGWVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLSGFQAGQEPKEGIHYGTHFTNMFMRNYPKRITADRMRAEERMDLL
>CAG9936525
FKTFVLPEKVDGSPSNHALGKAMVEAWRRDGILQVAMSGQQQQVYWEAQRISRRFFRRPYQQKAACVDAQSYAGYIASGEEITDGIPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLAANGERMLQLIEFGLHVPPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGERYANWEKSSAGLLEDESGWVFVPPVPGVFTIFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLQGYNRAQPPVDGIHYGTHFTNMFWRNYPDRLTTRRLEEEGR
>XP_007718836
VGHLQTFELPESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFGLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLTLSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPIQGETFANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAREDPISGIHYGTHFTNMFMRNYPTRITADRIRAEKRMALL
>TLD11728
VGQLETFTLPESVKGTRGDRAMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESGEKLLQLTEYGLGVPEGSLTQYTNDGWHHMRILRFPQNNKTNGKGKEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQFANWEKSSAGMKEEDDRWVYVPPVSNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGYEAGQSPKEGIHYGTHFTNMFMRNYESRITTQRLLAEDRYSLL
>XP_046022954
VGNLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELTQSDPRVQQKWPCHGPNPWPFTQMKTVMQAYMDYLGESGEKMLSLIAWGLGLEDGNALTRYTQDGWHHMRVLRFPQTDKVNGKGKAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYANWKETAAGKHEDADGWVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGFQAGQEPTEGIHYGTHFTNMFMRNYPQRITADRMKAEDRMALL
>KKF94376
DLETFDLPTSVSGSREDQMLGQRLIAAFRRDGILQIRIPSMQQNIVDDAFVASKAFFRQPHAEKAALVDPYSYGGYIASGEEITDGIADYSEIYTVTKDVAPTDPRFKLPCHGQQPWPNKTFKDVVNRYMKQLGNTGDIMLGLAELGLNIPRGSLTKLSQDGWHHLRVLRFPQNNKTNGKGKDGRGIGSHTDYGMLVIASQNDVGGLFIRPPYDDEKTQNWAKSAAGVKENDAGWVFVPPVSNVFTVFPGDMLQYMTNHYLPSTPHKVGLNTTERFAFAYFHEPHFDSVVKPLPGYNGGQEPTEGIHYGTHFTNMFKRNYPDRVTTQQLLKDGREVML
>KAH7212114
NLQTFILPDKVSDTKANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGALMIRPPYSDENFANWESSAAGFREDDERWVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGYDVGQEPKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>CAH0035662
LQTFILPAQVTGTESDKKLGEELVAALQQDGILQVAMTSKQQALYKAANDASKRFFQKPYSQKAACIDSQSYAGYIASGEELTDGLADYSEIFTVTKDLELDDPRVLARWPCHGRCPWPDADLRNPMRAYMDSLGENGEKLLALLEQGLGIPSGALTNMTQDGWHHMRVLRFPATNDTNGKGKDGRGIGSHTDYGLLVIAAADEVGALFIRPPRENESFANWNTSAAGFKEDDEGWVFVPPVEGVFTVFPGDMMQYMTNSALPSTPHKVGLNIDERFAFAYFHEPNFQAVVKPLPGYDAGQEPRGGIHYGKHFTDMFIRNYPERITTERLQSEGRYKLLE
>KAH7065587
HLQTFELPETLTGSPGDKALGRSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFRLPYEDKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMKDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPQRITASRMREEGRMALLD
>WP_003091748
MRTFELPDSVVGTTVDRILGRQLVRTWEDDGIFQVAATAAQARITAEALAASRRFFAKSLPEKAEFVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDIRVTGGWPCHGPVPWPDEGYRTAMTAHLKEVGAIGEKVLQLVALGLGAEIDRLTRLTREGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEHRNRNWLPAESTAGMYQDAEPWHFVEPEAGVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGATVRPLAGGDEAIHYGTHFTNMFTRCYPDRVTTRRIEREGRLAVLDRIR
>WP_132836977
LRTFRLPAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLNEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYAHAMKGYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVTPLPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTADPTEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLRE
>XP_046094133
LQTFTLPSEDMGLAADLSMGKALVAAWQQDGILQISIDAEQQTLYKAANKASKHFFQKPFHQKAACIDSQSYSNYITSGEELTDSLADYSEIFTVTKDLDLDNPRVSAKWPCHGRCPWPDADLRDPVKKYMDSLGQTGEKLLQLIELGINVPEGSLTHTTKDGWHHMRILRFHTTNDINGKDEGGRGIGSHTDYGLLVIAAADDVGALFVRPPYKDESVANWERSAAGMKEDSDGWVFIPPAENFLGDIMQYMTNSVLPSTLHKVGLNMKERFAFAYFHEPSFQIVINPLPGYEAGQSPEEGIHYGKHFTNMFMRNYPERITTHRLVDEGRYKLLD
>QKD49327
NLQTFILPDKVSDTEANRKLGEALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINKTNGKGKEGRGIGSHTDYGLLVIAAADDVGGNDENFANWESSAAGFREDDERWVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGYDVGQEPKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLE
>MBW4490662
MTDLQTFSLPESVTGTQADIEIARQMIQVWQSDGIFQIALNLTQDQTTKNAFETSQRFFRLPLKSKSQCISDLSYSGYIAVGEEVTAGEADHSEIFTICKDIPFHDVSVQTQWPCHGPVPWPDQNYQQGMKTYMDELGAIGEKMLRLVALGLELNDINALTNLTKDGWHHMRVLRFPALSESATKGIGAHTDYGLLVIASQDDRGGLYIRPPVPGEQRQRNWLSTESSAGMYENDDPWTFVKPVPSVLTVFPGDILQFITNGHLLSTPHKVKLNPRERFALAYFHEPNFNAEIRPLLDADSDASIHYGTHFTNMFMRSYPERITTRRIVDEDRLSRLA
>WP_190877710
TNLQISFPTFDLDLRQATTTQEKKNLGKEIVKALQQDSLIQIKKNEQEDLLTANALMQNKLFFSRTLEEKKAYTSDLTYTGYVYSGEEKKGGQYDLVELYTVFPDIQEEQDFDIPCHGPAIYPTEDYKQAITTYMKATGDMCDILLELIGLGLNYDDPNFLKDIAKKGFHHSRNLRFKSQKETSGASNGISAHSDYGFLIAASQDDVGGLYVRPIQNNEYRGRNWLDDETTIGAYNNDPGWAFVTPKPSVWTVFVGDMLELATQGSLRANLHKVKLHPEKERFAIAYFHEPRFNQKFTSINSRGNTKQFTYGEHLTKMYTKAYPTRPTTTRILQQNLMTNL
>PSN67428
DLKTFELPPFVTGSTQDIALGKAMIDAWQKDGILQIRMDPQNKYLAERAFKESQRFFSLSSSEKKDCVDNQSFAGYIASGEEITDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPNDKMQQTMQGYMDYLGTSGERMLQLIALGLELPALNSLTNLTRDGWHHLRILRFPKTGDENGKGVKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKFQNWKKSTAGMAENDDRWVYVPPEDGVFTVFPGDMMQFITNNRLPSTPHKVGMNIRERFAFAYFHEPNFNAVLKPLEGSSDERVHYGTHFTNMFMRNYPERITATRMHTERRLDSLESMK
>WP_164312020
MTELRTFPLPPTVEGSDADRLLGQALVAAWQVDGIFQIQATPDQDAATGRALDASRAFAGLPLKEKAQYVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVLDRWPCHGPVPWPSVEYAAAMKDYMIVVGDIGRRLLQLTALGLGLDDMDHFTRLTDDGWHHMRVLRFPSADTTSERGIGSHTDYGLLVIAVQDEVGGLHIRPPVPGETRGRNWLPEESMAGRHENEDPWTFVPPTPSVFTVFPGDIMQFITGGSLLSTPHKVRLADRERFAMAYFHEPSFQAVARPLGAPEADDLIHYGTHFTNMFMRCYPDRSTTARIEREDRLAVLERLRK
>OXV10832
DLETFDIPSEVTGSPGDIAMGHALVDAWRRDGILQVAVDKSQKKLLQDTFASSKRFFSKPHYEKASCVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLDLSDPRVQQKWPCHGPCPWPDIDMKLPMTRYMDYLNTIGERLLKLIECGLNLGPNALTQYTQDGWHHMRILRFPATNNTNGKGKDGRGIGSHTDYGLLVIASQDEVGGLFIRPPYEDEKFENWKSSTAGFREHDDRWVYVPPVPDVFTVFTGDMMQYLTNSYLPSTPHKVGLNTRERYAFAYFHEPSFQAVINPLPEMHSGKSQDDGIHYGTHFTNMFMRNYPERVTSKRLIRENRVSLLD
>PTB78996
LETFTLPESVSGSVSDKAIADAMIRAWRRDGILQIGMTKSQQRTYQAAYAGSRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQDMKAVMNRYMEDLAVSGEKLLQLIEMGLGISTGSLTKYTDDGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRESERFSNWEQSSAGLRENEAGWVFVPPVAGTFTVFPGDMMQYMTNNYLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGFNAGQSPAGGIHYGTHFTNMFLRNYPDRITTTRLQEEGRYQLLQSEELRDDDD
>WP_096562523
MTDLQTFHLPKSITGTQSDIDTAREIIQAWRTDGIFQVATNTIQDRKTESAFEASRRFFRMPMKFKSQCISDLNYGGYIASGEEVTAGKSDYSEIYTICKDIPLNDARVQAQWPCHGPMPWPDQEYHQSMKVFMDELGLIGEKLLKLTALGLGLDDINALTKLTRDGWHHMRVLRFPTLSQKSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESMAGMYENDDPWTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFDPSSDEHIHYGTHFTNMFMRCYADRITTRRIINEDRLSILARLENKTLGRLT
>OTA08591
LETFTLPERVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRVYQAASAASRRFFAQPHARKEACVDGSSYSGYIASGEEITGGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQKMKAVMNKYMEDLAVSGEKLLQLIEMGLDIPTGSLTKYTADGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRETERFSNWEKSSAGLRENEAGWVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGYNAGQPPLEGIHYGTHFTNMFLRNYPDRITTARLQEEGRYQLLESKELRDD
>KKY23838
QLETFTLPSQVTGTPGDRALGKALIDAWRRDGILQIQMQPEQIKVAEEAFQTSKTFFKKPSNEKASCVDSQNYSGYIASGEEVTDGIKDYSEIFTVTKDLDLAERRVRSKWPCHGPVPWPDIEMKEPVQRYMDSLGQSGEKMLQLIEYGLDVPPHSLTKYTNDGWHHLRILRFPANSRTNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIENENFSNWETTTAGLKEEDEGWLYVPPVPGVFTVFPGDMMQYITGSYLPSTPHKVGLNTQDRFAFAYFHEPSFQAVLKPLPGYDIGQTPREGVHYGTHFTNMFLRNYPDRITTQRLVADDRMALLE
>NMO14212
MTDLQTFALPDRVTGMSSERELADRMIRAWRRDGIFQLEMNPMQEQHAERAFEASRRFFRMPLQLKAGCVSDLTYSGYIASGEEVTAGEADFSEIFTLCKDVPLTDARVRAQWPCHGPTPWPDDSYRQAMTAFTEVLGSVGERLLRLTALGLGLEVDALTRLTRDGWHHMRVLRFPALSPASSRGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMFEHQEPWTFVKPVPKVLTVFPGDILQFLTGGYLLSTPHKVRLNTRERYSMAYFHEPAFDACVRPLLGPSGDDFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLATLARLRD
>KAI1854398
LQTFCLPERVTGSQSDRTMGKALVDAWQKDGILQIAMTPRQEALHSQAQQNSQEFFSRPHAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLELGDAKVKAQWPCHGRCPWPDAAMRDVMKEYMNELGNCGETMLRLLEYGLEIPEGTLTQQTDDGWHHLRILRFPPHNKTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQDEKYANWERSAAGMAEEDEKWTYVPPVPGVFTVFPGDMMQYMSNSTLPSTPHKVGLNTRDRFAFAYFHEPNFKTTCRPLPGYDAGQTPREGIHYGTHFTNMFLRNYPDRITTSRMNRENRYEILK
>WP_111876543
MTELTTFQIPSAVGDTHSDIDLGLDIIQAWRRDGIIQIALDDAQDARTRQAFQASRDYFALPPQEKSRHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPEDDPRVQQQWPCHGPAPWPGAGFQRDMTLFMEQLGAIGEKLLHLTALGLRSLGRLDDIHAIKRLTHDGWHHMRVLRFPCATPGNQGRGISSHTDYGLLVIAAQDDVGGLYIRPPVEGEARRRNWLPGESAAGMYENEDPWTFVRPTPRVVTAFPGDILQFLTGGYLLSTPHKVKLNTRERFALAYFHEPHFEASIQPLRPQPGDGQIHYGSHFTNMFMRSYPQRITTQRILREDRL
>WP_131746854
MTDLQTFRLPEAVSGTESDIHLAREMIQAWRSDGIYQVATDPAQDRKTLEALEASRRFFGMPMDFKARCISDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQARWPGHGPAPWPDREYQQSMQVFMDELGSVGERLLKLTALGLELDDTDALTKLAVDGWHHMRVLRFPARSEQTSRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEQRNRNWLENESAAGMYENEQPWTFVKPVPSVLTVFPGDILQFLTNGFLLSTPHKVRLNTRERFALAYFHEPNFEAYLRPLSGLGTEYIRYGTHFTNMFMRSYPERITTRRILAEDRLSFFVAAR
>WP_254381577
MGELATFRVPARITGTTADIMLGGAMVRAWQRDGIFQVTADGRQEARTRSALAASRRFFARPAAEKAACVNDTSYSGYIASGEEVTAGRPDASEIFTITPDVPAAAAGGLPCHGPVPWPSPGYRAAMEAYLAGVGGLGERLLRLVALGLGLGPAGIDRLAGPTRGGWHHMRVLRFPAAGRTTERGIGAHTDYGLLVIAAQDQVGGLFVRPPVAGERRPRNWLPGESTAGFAEDRPGWTPVRPVPRVLTVFPGDILQFMTGGALLSTPHRVQLADRERYALAYFHEPRFGARIRPLPGTGEGAGETTGEGGAVIHYGEHFTRMFMRCYPERATTRRIEEQGLLRALAGAR
>KID83261
VAQLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSGDKVLQLIELGLDVPQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEPGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNIRERFAFAYFHEPNFRAMVKPLPGYNASQEPADGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKEK
>XP_038801042
LKTFKLPLVVRGSLDDIELGKAMIRTWQSDGILQIKMDEQNKRVAEEAFKSSKRFFSLPPSAKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVRAKWPCHGPCPWPNSAMQTTMQNYMDSLGSSGERMLQLIALGLGLPDMNTFTELTRDGWHHLRILRFPKTGDENGKGMKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYEDEKFQNWQKSTAGMAENDDRWVYVPPEEGVFTVFPGDMMQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLRPLGKSSEDSVHYGAHFTNMFMRNYPDRVTAKRMRAEDRMKLLESPELREY
>XP_049125671
VAQLETFTLPARVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPSPWPDYEMKQPMTTYMDYLGESGEILLQLAELGLGVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSSAGLKEDEEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVTKPLPGYDVGQSPQEGIHYGTHFTNMFMRNYPNRITTKRMHAENRLTLLQ
>WP_146358578
MTELRTFRLPDTVTGSDSDIALGRQLIEAWQADGILQIATDAVQDQKTEAAMQASRSFFHDLPFEEKSRLVSDLTYAGYIASGEEVTDGVADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPSESYHQAMQTFMDSLGEHGEKILQLTALGFGLDIDTFTRLTVDGWHHMRVLRFPEVKEDSDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQEAEGWTFVKPVPNVLTVFPGDIMQFLTGGALLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDEPAGDDVIHYGTHFTNMFMRCYPDRVTTQRIHAEDRLSVLAELR
>PLN86625
LQTFELPAQVTGTPKDVELGKALIDAWRSDGILQISMTDTKKSLMDEVFAASKGFFRKPHSEKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEKRVRDHWPCHGPCPWPDLEFKDPMEQYMKYLGQSGEKLLQLIECGLGVPPGSLTNYTNDGWHHMRILRFPAVNSTNGKGKEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPSADEATANWEQSAAGFHEDDSGWVYVPPMPDVFTVFPGDMMQYLTDSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVLRPLPGYDGGQKPREGIHYGTHFTNMFMRNYPDRITSKRMIREDRQALL
>KAF4450525
NLQTFTLPDKVTTNEEHRKLGKAIVEAWQKDGIFQIAMTSEQLAQYKSANFASKKFFGKPYAQKAACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLDLDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGDVGETLLQLTELGLNVPEGSLTNYTQDGWHHMRILRFPAINRTNGKGKDGRGIGSHTDYGLLVIAAADDVGALMIRPPYEDEKLANWETSAAGYKEDDARWMFVPPAENVFTVFPGDMMQYMTNSALPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGYDAGQEPKEGIHYGKHFTDMFMRNYPQRITTQRMNDEGRYQLLE
>WP_059078688
MTELQTFVLPPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATEQALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDMAEEDARVVDRWPCHGPAPWPSQQYAEAMKTYMGAVGDIGERLLRLVALGLGLDDMEHFTKLTRDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLHIRPPVPGEARGRNWLPEESMAGRYENEEPWTYVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPAFQAVARPLEGGGPEEFIHYGTHFTHMFMRCYPDRTTTVRIENEDRLAVLERLRKEA
>WP_239394497
MNELQTFHLPEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVGEKLLRLTALGLELDDTDALTRLTADGWHHMRVLRFPALSKESRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDSFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLSGVSADCIHYGTHFTNMFMRSYPARITTQRILAEDRLSVL
>WP_152166644
MTELQTFVLPSTVDGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIHEDDARVAGKWPCHGPAPWPSRQYADAMKDYMGTVGDIGERLLRLVALGLGLDDMDHFTKLTEDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGENRGRNWLPGESMAGRYENEEPWTFVTPVPSVFTVFPGDIMQFITGDTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLEGGGEDEFIHYGTHFTNMFMRCYPERTATARIENEDRLTVLERLRKEALTS
>CAH0048186
FETFVLPEKVDGSPSNHALGKAMVEAWRRDGILQVAMTGQQQQVYLEAQRVSRSFFLRPHRQKAACVDAQSYAGYIASGEEITDGVPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMASVIQRYMDDLATNGEKMLQLIELGLHVPPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGEQYANWEKSSAGLLEDESGWVFVPPVPGVFTVFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGYNRAQPPVDGIHYGTHFTNMFWRNYPDRLTTRRLEEEGR
>OMP85163
NLKTFELPETLTGGASDQALGRSMIDAWRHDGILQISMDPINRKLADAAFICSKKFFDLPYKTKAACVDDQSFAGYIASGEELTDNIADYSEIFTVTKDLALTDARVQQQWPCHGPCPWPFKQMRTVMDAYMQYLGDSGEKMLQLIAWGLGLADGNALTRYTQDGWHHMRVLRFPERNNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEVVANWRESAAGLAEEDDKWVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNNRERFAFAYFHEPNFSSVMKPLEGFDAGQEPREGIHYGTHFTNMFMRNYPKRITADRIRKEDRMKLLD
>RAM52958
MTNLQTFYLPKSITATQSNIDLGHQMIQSWRTDGIFQIATNTIQDRKVQNAFEASRKFFRMPLGFKSQHLSDLTYSGYIASGEEITAGESDYSEIFTLCKDVPLDDVCVQKQWPCHGPVPWPNHDYQQSMTTFMDELGLIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRILRFPPVSEKSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEQRNRNWRSDESSAGMYENEEPWTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFKACVRPLSNPSSNEFIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLRNQA
>KAH7306032
LETFAMPDHVTGSEGDRTMGKAMIDCWRRDGILQIATSPEGQRLQKEAYGASKAFFGKPHNEKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPFTDARVQAKWPCHGPCPWPDGEMKEAIGNYIQWLGGEGEKLIRLIELGLEIPEGSLSNYTRDGWHHCRVLRFPATHLTNKKGKEGRGIGSHTDYGLLVIASQDDVGGLFIRPPYEDEKFANWEKSSAGLHENDDRWVYVPPIPNVHTVFPGDMMQFLTQNYLPSTPHKVGLNKRERFAVAYFHEPSFQSVIKPLEQYAKDKDASEGVHYGTHFTNMFMRNYPERITTQRLKRENRYEKLKE
>XP_024697818
LQTFELPAQVTGTPTDVELGKALVDAWRSDGILQISMKDTKKSLMDEVFDASKGFFRKPHSEKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEPRVRNHWPCHGPCPWPDLEMKDPMEQYMEYLGQSGEKLLQLIECGLGVPPGSLTKYTNDGWHHMRILRFPAVSNTNGKGKEGRGIGSHTDYGLLVIAAQDEVGGSGLFIRPPSADEATANWEESAAGFHEDDSGWVYVPPMPDVFTVFPGDMMQYLTDSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVLRPLPGYDGGQKPREGIHYGTHFTNMFMRNYPDRITSKRMIREDR
>WP_005474170
MTELQTFVLPPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATERALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIYTVCPDIPEDDARVVDKWPCHGPAPWPSEQYADAMKGYMGAVGDIGERLLRLVALGLGLDDMDHFTKLTEDGWHHMRVLRFPRADATSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLADESMAGRYENEEPWTFVTPVSAVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLDGGGQDEFIHYGTHFTNMFMRCYPDRAATARIENEERLAVLERLRKEA
>RDA91345
LETFSLPIKVTGSIQERAMAKAMMSAWQKNGILQINMTRLQQRLYRNALASSRRFFDRPLAEKQACVDSRSYSGYVASGEEMTDGIRDYSEIFTVTKDLPLSDKRVLEGCPCHGPCPWPDNDFKEAISRYMEELGACGQRLLQLIELGLDVAAGSLTRYTTDGWHHARVLRFPARHMTNGKGKSGRGIGSHTDYGLLVMGSQDEVGGLFVRPPRKGEMFMNWKKSAAGAMEDDDGWMYVPPEEGTITVFPGDIMQYLTNHVLRSTPHKVGLNTRQRFALAYFHEPNFSSTVKPLFGSNGGNEEPVEGINYGTHFTNMCLRNYPDRITTRRL
>WP_181080955
MTTLHTFHLPESISATPEDIALGKAMIEVWRRDGIFQVAATPAQNRAMLQAMKASQGFFRLQSRNKARRISDLTYSGYIASGEEVTAGQADYSEIFTVCKDLPLDDVRVKNGWPCHGPVPWPNKNYRLRMHTFMEALGDIGERLLTLVALGLELEDRHTFTDLTRDGWHHLRVLRFPAGSSSTARGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLEKESSAGRFEDQDPWTFVQPMPRVFTVFPGDLLQFMTDGYLLSTPHKVKLNTRERFALAYFHEPNFEAIARPVWGQTRGSFIHYGEHFTRMFMRCYPERITTRRILRENRLAHLELMKRHADS
>WP_055523095
MTELQTFPLPPTVEGGDADRLLGQALIAAWQADGIFQIQATPDQDAATERALAASRAFVGRPLKEKTQYVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVHDKWPCHGPAPWPSDQYAAAMKDYMAVVGDIGHRLLQLTALGLGLDDMEHFTRLTDDGWHHMRVLRFPPADATSERGIGSHTDYGLLVIAAQDEVGGLYIRPPVPGETRGRNWLPDESMAGRYENEDPWTFVAPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLEPSETDEYIHYGTHFTNMFMRCYPDRSTTARIEREDRSAVLERLRKEA
>XP_013958481
LETFILPEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHASKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDQRVAQGWPCHGPCPWLDQDMKAIMSNYMDDLAVSGEKLLQLIEMGLDVPTGSLTRYTKDGWHHMRVLRFPQNNRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENEQFSNWEKSSAGLRENEAGWVYVPPVDGTFTVFPGDMMQYMTNNLLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGYNAGQSPIEGIHYGTHFTNMFLRNYPDRVTTARLREEGRYQLLGLEELR
>KAH7551596
VGHLQTFELPESLSGSVADRALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYHKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFANWKESAAGKHEDADGWVYVPPVSDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEARQDPISGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLESPK
>KAE8847895
VGNLQTFELPETLNGSVADRALGKRMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLGMEDGNALTRYTQDGWHHMRILRFPQTNKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPAEGIHYGSHFTNMFMRNYPKRITADRMKDEGRLALL
>WP_064046801
MTGLTTFHLPERILHSEAHRQLGRDMVAAWRADGIFQIALSTPQQHTTDQAFAQSRQFFELDFETKRRHVSEFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTDMLGTFGERLLQLIALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYARPPIAGERRNRNWLASESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADADAEPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLNRLPALSE
>KAI1000636
LKTFTLPTRIEGNADDHAMAKSMIDEWRQTGIFQIKIPEAQRELLRDCFKVSQQFFSAPYAEKAKLVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPISDSRVKNAWPCHGPCPWPNALFQNKISQLISHLGESGEKLLKLTALGLGLRDPDTLVKLTEDGWHHMRVLRFPHVDMTNGKGKTGRGIGSHTDYGLLVIAAQDDVGGLFVRPRIEGESIKNWQSSAAGLNETDDKWLFVPPVPDTFTVFPGDMMQHMTNDFLPSTPHKVGLNVAERKAFAYFHEPNFSSVIEPLPEFRERIHYGTHFTNMFMRNYPERITAKRINQERRMDTLDQL
>WP_174453320
MAELETFQLPQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKDHTDELGSTGERLLRLVALGLGLEVGALTALTRDGWHHMRVLRFPARSATATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLMTPGPDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLRE
>WP_203005384
MADLRTFQLPDFVDGGGSNIKLADELIQAWRTDGILQVATDPTQDLKTKDAIESSRRFFGMPAEFKRGCVSDLTYSGYIASGEEQTAGESDYSEIFTVCPDVPLDDPRVLARRPCHGPAPWPDGEYRRAMLAFMAELGTVGEKLLKLTALGLGVGIDALTDLTKDGWHHMRVLRFPARSEHTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVHGERRSRNWLPGQSMAGRYENEDPWTYVKPVPSVFTVFPGDMLQLITDGFLLSTPHKVILNTRERFALAYFHEPNFQARVRPLSGGGDGDHIHYGRHFTSMFMRSYPERITTRRIVEEDRLSLL
>WP_071780333
MTNLQTFYLPESISATQSNVDLGHQMIQAWRTDGIFQVATNTIQDGKIQNAFEVSRRFFRMPLGLKSQYISDLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVCVQKQWPCHGPVPWHNQNYQQNITTFMDELGFMGQKLLKLVALGLGVDMNTFDFTLNGWHHMRVLRFPAVSEKSARGIGAHTDYGLLVIAVQDDVGGLYVRPPVEGEKRNKNWRSDESSAGMYENEEPWTFVKPIPRVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLSNPSSNEYIHYGTHFTNMFMRCYPDRITTRRIHDEDRLSVLAVLRN
>WP_248542106
MPHVVSGSEADVALGLAMIRAWRRDGIFQIRMSPAQADKARHAFELSRRFFRQPLEEKQRCVSDSTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSGWPCHGPAPWPDEAYRLGMKAHTEELGSAGERLLRLAALGLGLDIGALTALTKDGWHHMRVLRFPARSAATTRGIGAHTDYGMLVIAAQDDVGGLWVRPPVEGEQRKRNWLPGESSAGMYEHDEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLASRERFAMAYFHEPHFEACVRPLKDPTRDEFIHYGTHFTHMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQR
>GJN79908
QLETFVLPEAITGSVADVAMGRSLVAAWQRDGILQVCMSPLQRRLYEAADAQSRRFFTRPQAEKQACVDSHSYSGYVASGEEITDGIADYSEIFTVTKDLPSSDPRVLSRWPCHGPCPWPDQKMKQVISRYIADLGTAGQKLLALIELGLDVPTGSLTQYTNDGWHHMRVLRFPARHKTNGKGKAGRGIGSHTDYGLLVMAAQDDVGGLFVRPPQGDSFANWQQSAAGVKEDDAGWVYVPPTPGVFTVFPGDMLQYMTNNALLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGYNAGQAPTEGIHYGTHFTNMCLRNYPDRVTTKRLAAEDRYQMLS
>WP_234064780
MAELETFRLPESVSGSDADITLALSMIQAWRKDGIFQVQTTNAQELKLQRAFEASRRFFRQPLAVKARCVSDLTYSGYIASGEELTAGEADYSEVFTVCKDVPLTDARAQARWPCHGPAPWPEEGYRRAMKEHMEELGAVGEKLLRLVALGLGLDDLNALTRLTREGWHHMRVLRFPARSPTATRGIGAHTDYGLLVIAAQDDVGGLYVRPPIEGESRPRNWLPGESSAGMYEHDERWTYVKPTPGVLTVFPGDILQFLTRGYLLSTPHRVRLNTRERFAMAYFHEPHFEACVRPLTHPTGDEFIHYGSHFTNMFMRSYPDRVTTQRILDEDRLTILAWLRQEAVLR
>KAI7784882
LETFNLPDRVTGSASDKAMGQALVKAWKRDGIFQISMNSNQREIYKYAEHASKRFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSGEKLLALTELGLGVPRGSLLQYTQDGWHHMRILRFPPTNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGESYANWEKSAAGMKEDDEGWVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGYNNGQKPQEGIHYGTHFTNMFMRNYPDRITTQRLQAEDR
>GFP54281
LETFTLPEKVSGSVCDKAIADAMISAWRRDGILQIDMSKAQRRLYQAANAASRRFFSRPYVQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQSMKNVMNSYMNDLAVSGEKLLQLIEMGLDVPTGSLTRYTDDGWHHMRILRFPHTDRTNGKGKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPRQDEQFANWEKTSAGQRENDAGWVYVPPTAGTFTVFPGDMMQYMTNSFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGYNAGQSPMGGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLGSEQLRDEND
>XP_008020756
VGNLQTFELPETLSGSPVDRALGRSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFNLDYKKKSACLDDQSFAGYIASGEELTDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESGEKMLQLIAWGLGLNDGNALIKYTQDGWHHMRILRFPAANNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSSVMKPLPGFEAGQEPVDGIHYGTHFTNMFMRNYPKRITADRMRAEGRMALL
>WP_112276902
MTELRTFRLPEIITGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMEASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDNGYRDAMQDFMAMLGEAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTFVEPVQHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDPHADVNSVIHYGTHFTNMFMRCYPDRITTQRILEEDRLTVLSELRS
>XP_036579499
VAQLETFTLPDRVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESGEKLLQLAELGLQVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSAAGMKEDDEGWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPDRITTKRMMAENRLSKLE
>KAF5011898
QLQTFVLPEKVTGGDLDRKLGKAIVKAWQQDGILQIAMKESQQGLYKAANQASQRFFKKPYAQKSACIDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVVEKWPCHGRCPWPDADLQDPMKQYMDSLGESGETLLKLTELGLDVPEGSLTRLTQDGWHHMRILRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPQKSDEYANWNKSAAGFKEDDDGWMFVPPAENVFTVFPGDMMQYLTNSVLPSTPHKVGLNVSERFAFAYFHEPSFQAVIKPLPGYDAGQEPKEGVHYGKHFTDMFMRNYPQRITTQRLIDEGRYNLLE
>KAH6991545
LQTFVLPEKVSGSVEDKNLGKTIIDAWKKDGILQIAMKPEQQGLYKNANLASKRFFSKPYAQKSACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMERYMNSLRESGETLLQLTELGLDVPAGSLTDYTKDGWHHMRILRFPAINKTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQEGDDFANWEKSAAGFKEDSEGWLFVPPAENVFTVFPGDMMQYLTNSYLPSTPHKVGLNHRERFAFAYFHEPSFQAVINPLPGYDAGQEPKEGVHYGKHFTDMFMRNYPQRITTQRLIDEGRYDLLKQESLYT
>WP_191133867
MTDLQTFHLPASITGIQSDVVLARELIQAWRTDGIFQVATNAIQQRKTESAFASSQRFFRMSQEFKSQCISDLTYSGYIASGKEITAGQADYSEIFTVCQDVPLHDARVQAHLPCHGPAPWPDEEYQQSMKAFMDELGTIGQRLLQLTALGLELDDMNTLIDLTENGWHHLRVLRFPAASQDSARGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRKRNWLATESSAGLYEHEDPWTFVKPVPKVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFEPSSDEYIHYGRHFTSMFMRSYPDRITTRRLVNEDRLSNLAPVKN
>KGO67238
QLKTFTLPDKATGSPSDVQLGKAMIDAWRQDGILQVSMSPKQQDLFDKAFTASKRFFAMPPKQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVEAKWPCHGPCPWPDTDMRTPIQKYMESLGNSGETLLKMTEYGLDLDPKTLTSLTEDGWHHLHSLKITTPTEKGKDGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAEDEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAEVSPVAKLYDGQPPDEKIHYGTHFTNMFMRNYPDRITTDRIIKEDRLKLLD
>WP_164413328
MTELRTFPLPPTVEGSDADRLLGQALVAAWQVDGIFQIQATPDQDAAIGRALDASRAFVGLPLKEKAQFVSDLTYSGYAASCEEETAGEKDGSELFTVCPDIPEDDPRFLDRWPCHGPTPWPSVEYAAAMKDYMVVVGDIGHRLLQLTALGLGLDDMEHFTRLTNDGWHHMRVLRFPAADTTSEHGIGSHTDYGLLVIEVQDEVGGLYIRPPVPGETRGRNWLPGESMAGRHENEDPWTFVPPTPSVFTVFPGDIMQFITGGTLLSTPNKVRLADRERFTVAYIHEPSFQAVARPLDAPEADDFIHYGTHFTNMLMRCCPDHWTTARIEREDRLAVLERLRK
>WP_206728402
MSELQTFVLPDAVTGQPSDIELAGQMIRAWRRDGIFQVAMSDEQEIKTERALEASRRFFRLPLEAKAKCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVSLEDPRVYGKWPCHGPAPWPDEDYRQRMAAFTGELGSVGERLLELTALGLGLEPDALTRLTRDGWHHMRVLRFPALTSRTARGIGAHTDYGLLVIAAQDDVGGLLIRPPVEGEQRNRNWLPGESTAGMYENEEPWTFVTPVPKVLTVFPGDILQFLTNGHLLSTPHKVRLNTRERYSMAYFHEPAFDTCIRPLLGPPSRELIHYGTHFTNMFMRCYPDRVTTRRILEESRLTTLAWLREAS
>XP_001890626
LTTFTLPSSPHIAPTPENRALGHQMVQTWRKDGIFQIQLTPDQASALRAAFVLSKSYFKRPHHEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPDTLYESAMKRLMAVLGDSGEKLLQLTALGLNLANPNVFTDLTDDGWHHMRVLRFPEADKTNGKGKPGRGIGAHTDYGLLVIAGQDDVGGLFVRPPTDGDTSRNWEKYSGGLKEDQQESWSFVPPVENVLTVFPGDMMQYMTNDYLPSTPHKVVLNTRERYAFAYFHEPNFSSVVQPLPEYSRDQGKDVQVKDKGIQYGTHFTEMFMRNYPERVTAKRMKAENRLDIL
>WP_091288634
MADLQTFSLPVSVTGTNAEKILGKQMIRVWQEDGIFQVVAEPEQDQKTLEAFQASKRFFESPLAHKSGFVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPEDDPRVEAGWPCHGPVPWPDEAYQKTMRSYMDELGAMGENLLKLVALGLGLEIDALTKLTVDGWHHMRVLRFPPMSTATSRGIGAHTDYGLLVIAAQDEVGGLFIRPPVEGEHRNRNWLEDESMAGMYENEEPWHFVEPVPSVFTVFPGDILQFLTGGELLSTPHKVRLNTRERFTMAYFHEPNFEACVRPLSGSATEDEYIHYGTHFTNMFTRCYPDRVTTARIEAEDRLSTLAGLRE
>WP_033210435
MTDLRTFQVPDAVTGTDADSSLGREMISAWQTGGIFQVATTPEMRSAIRGAYTASKRFFSRPREWKSRHVSDVTYSGYISSGEEVTAGEADYSETFTVCKDIQPDDVRCTESWPCHGPVPWPDQEYRRAMRAYMDGLGAVGHKLLRLTALGLGLEDIDAFDKLTDDGWHHMRVLRFPAETASNAARGIGAHTDYGLLVIATQDDVGGLYIRPPVVGEQRNRNWLREESAAGVNETDEAWRYVEPVPNVLTVFPGDIMQFMTGGGLLATPHKVRLAQRERFAVAYFHEPNFQAVARPLGESDRDEHLHYGTHFTNMFMRCYPERVTTRRIHDENRLAVLEELRVKA
>XP_018140447
LETFVLPKEVSGSLSDRAMAKAMVNAWRRDGILQVAMSPLQQQIYAQAKAASKNFFKKTPKEKHACVNDSSYAGYIASGEEITGGIADYSEIFTVTKDLNHNDQRVMENWPCHGPCPWSEDSMKTAMGDYMADLGSSGDKLLQLIELGLDVPPGSLIKYTQDGWHHMRVLRFPPRHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQKGEEFANWKKSVAGMKEDESGWVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAIVKPLPGYNDGQQPGNGIHYGTHFTNMALRNYPDRTATKKLLNDGRYDML
>ATY65176
LNTYCLPDKVTGSASDRLMAEAMVQSWRRDGILQIAMSPQQQSIYQNANAASRRFFQRRSAQKQQCVNEKSYAGYVASGEEITDGVADYSEIFTVTKDLAEDDHRVAGKWPCHGPCPWPNEDMKLNMTAYTDDLSRSGDQLLEMIELGLRVPRGSLKRYTEDGWHHMRVLRFPQRDQTNGKGKKGRGIGSHTDYGLLVMAAQDEVGGLFVRPPHQSEHYANWEKSAAGMREDDSGWVYVPPTPGVFTVFPGDMMQYMTNSYLKSTPHKVGLNTRERFAFAYFHEPNFASVLKPLPGYESGAASDAEGIHYGTHFTNMCLRNYPDRVTTKKLLQEGRCKML
>POS74919
LETFDLPERVTGSPGDKAMGQAIIKAWKRDGIFQISMESSQREIYKYAEDASKKFFAKPHNEKAACIDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGNSGEKLLALTELGLGVPKGSLLQYTQDGWHHMRVLRFPPTNKTNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGENYANWEKSAAGMKEDDEGWVYVPPVANTFTVFPGDMMQYMTSSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVIKPLEGYNNGQKSQEGIHYGTHFTNMFMRNYPDRITTQRLHAEDR
>KAF4582743
LETFTIPAEVTGSASDKAMAKDMIRAWRKDGILQIGMTHFQQRLFQDALSHSKRFFRRPLVEKRACVDSCSYSGYVASGEEVTDGVRDYSEIFTVTKDLPLSDRRVMESWPCHGPCPWPDNGFKEVISRYIAELGGSGQRLLQLIELGLGVPAGSLTRYTTDGWHHMRVLRFPARHLTNGKGKSGRGIGSHTDYGLLVIGSQDHIGGLFVRPPREGENFVNWEKSAAGAKEDDDGWMYVPPEQGTLTVFPGDIMQYLTNHALRSTPHKVGLNTAERFAFAYFHEPDFGSVVRPLVGYDGGQEPAEGFNYGTHFTNMCLRNYPDRITTRTL
>TWU76274
KLETFVLPKRVSGCLSERAMAKAMVNAWRRDGILQIAMSSLQQQTYAEANAASRRFFKKTLAQKQACVNDSSYAGYIASGEEITGGTADYSEIFTVTKDLQPSDPRVKEQWPCHGPCPWLDTRMKSFMTDYMADLRSSGDKLLELIEMGLDVPPGSLTKCTQDGWHHMRVLRFPARHQANGKGKAGRGIGSHTDYGLLVIASQDEVGGLFVRPPKEGEQYANWRQSAAGMKEDESGWVYVPPTPGVFTVFPGDMMQYMTNGFLHSTPHKVGLNIRERFAFAYFHEPNFRTVVKPLPGYNASQEPVDGIHYGTHFTNMAMRNYPDRMTTQKLLEDGRYEMLSKEELRDYS
>RMZ70101
VGLLQTFELPETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKIVMQAYMDYLGESGEKMLQLIAWGLGMKDGNALTKYTQDGWHHMRILRFPETNNVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEDETYANWKESAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPVEGIHYGTHFTNMFMRNYPERITADRIRAEGRMALL
>KAH7236631
NLQTFVLPERVSGSAEDKKLGKAIIDAWKKDGILQIAMKPEQEGLYKAANLASKRFFSKPYAQKSACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLDLDEPRVVAKWPCHGRCPWPDYEMQNPMKRYMDSLGESGETLLKLTELGLDVPEGSLTDYTKDGWHHMRILRFPAMNNTNGKGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQKGDDFANWEASAAGFKEDAEGWSFVPPAKNVFTVFPGDMMQYLTNSFLPSTPHKVGLNHRERFAFAYFHEPSFQATIKPIHGYNAGQEPKEGVHYGKHFTDMFMRNYPQRITTQRLLDEGR
>KAH0592203
VAQLETFVLPERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQTCVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPGDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSGDKVLQLIELGLDVPQGSLTKYTDDGWHHMRVLSSHFLGIRFPARHRTNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEEGWLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGFNGSQEPADGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKEK
>PFH60078
LETFNLPSKVTGSILDKTIAKHMISAWRRDGILQIGMTRFQQRLYQDALASSRRFFSRPIVEKQACVDSRSYSGYIASCEEITDGVPDYSEIFTVTKDLPLSDKRVVDEWPCHGPCPWPDNGFKEVMNRYIAELGACGQRILQLIELGLDVPSGSLTRYTTDGWHHMRALRFPARNRTNGQGKSGRGIGSHTDYGLLVVGSQDNVGGLFVRPPRDGESFVNWQKSAAGALEDEDGWMYVPPEQGTFTVFPGDIMQYLTNHVLRSTPHKVGLNTQERFAFAYFHEPNFSSTVKPLAGYDGKQEPAEGIHYGTHFTNMFLRNYPDRITTKRILERSR
>POR33302
QLETFILPEQVSGSVSDQAMATAMIDAWRRDGILQVGMSQLQRRLFQAANAASRRFFARPQGDKQACVDARSYAGYVASGEEMTDGVADYSEIFTVTKDLSPRDRRVARRWPCHGPCPWPDKDMKAVVGRYMADLGASGEKILRLIELGLGVPTGSLTKHTNDGWHHMRALRFPPRHRTNGKGKDGRGIGSHTDYGMLVLAAQDEVGGLFVRPPQQGERFTNWQQSAAGAKEDESGWVYVPPVPGVFTVFPGDMLQYMTNSVLRSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGYNGGQAPVEGIHYGTHFTNMALRNYPDRVTTRRLVAEDRYRML
>WP_131118892
MTNLQTFYLPKSITATQSNIDLGHQMIQSWRTDGIFQIATSTIQDRKVQNAFEASRRFFRMPLGFKSQRISDLTYSGYIASGEEITAGESDYSEIFTLCKDVPLDDVCVQKQWPCHGPVPWPNHDYQQNMTIFMDELGLIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRVLRFPPISKKSARGIGAHTDYGLLVIAAQDDAGGLYVRPPVEGEQRNRNWRSDESSAGMYENEEPWTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSNPSSNEYIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLR
>MBE8521782
MRTFELPVSVGGTTVDRILGRQLVRTWEDDGIFQVAATAEQARITGEALAESRRFFARSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDPRVTEGRPCHGPVPWPDEGYRTAMEAHLREVGQIGEKLLKLVALGLGVEIDTLTRLTRDGWHHMRVLRFPARSATTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLAEESTAGMYQDDEPWHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGTTVRPLAGDDETIHYGTHFTNMFTRCYPDRVTTKRILSENRLRLLE
>WP_206263246
MQTFHLPESITGTQSDKYLAHQMIEAWRTDGIFQIAINTIQQQKTSAAFDSSRKFFHMPLQFKQQCISDLTYGGYIACGEEITAGEFDYSEIFTICKDVPPDHPCVQAHWPCHGPMPWPCCNYQQTMNAFMDELNLIGNKLLLLTALGLELDDINTLTRLTNNGWHHMRVLRFPPLSTKQTRGIGAHTDYGLLVIAIQDHVGGLYIRPPVEGESRNRNWLPNESSAGMFENEEPWIFVKPRKSVLTVFPGDILQFLTNGYLLSTPHQVRLNTSERFSLAYFHEPNFNAFIRPLGTSSTDEYIHYGTHFTNMFIRCYPNRITTRRILEEDRLSVLTDLNKSSST
>KAH8173892
MGMDLISAWRRDGIIQVAMTEAQQKTYQAADASSRGFFARKPAEKTKCVDPDSYSGYVGSGEEITNGIADYSEIFTVTKDLKEDDPRAVAKWPCHGPCPWPDQSMKQSIDKYMGDLGDNGHKILQLIELGLDVPEGSLTDYSNDGWHHMRILRFPAQNATNGKGKAGRGIGSHTDYGLLVIAAQDEVGGLFIRPPHKDEKFANWNSSAAGHKEDDAGWVYVPPVPGVFTVFPGDMMQYMTGSFLPSTPHKVGLNTRERFAFAYFHEPNFRSMVRPLPAYATGEETEDAIHYGTHFTNMCLRNYPDRITTARLLEEGRYNMLD
>WP_020407952
MKHLQTFQLPQSVTTSSTDIQLARDMIHAWRQDGIFQVAVNSEQHMKTERAFHASKRFFRMPLEKKSRCVSDLSFSGYVACGEEITAGKVDHSEIFTVCPDIPDSDWRVKEGWPCHGPVPWPDQDYELAMKAFMKSLGEIGDNLLRLVAIGLGLGDINHLTDVTRNGWHHMRVLRFASAGAQASSRGIGAHTDYGLLVIAAQEDIPGLYVRPPVRGEKRCRNWLPEESSAGMYNEIGPWNLVSPMPEVLTVFPGDILQFLTNGYLLSTPHKVELHPSKERYALAYFHEPDFAAKITPLNAKEGESIHYGTHFTNMFMRCYPDRSATRRITENGGLAVLSDMR
>RDA83884
HLETFRLPIKVTGSISDRAMAKAMMSAWRRDGILQINMTSLQQRLYQDALASSKRFFDLPLAEKQSCVDSRSYSGYVASGEEITDGIRDYSEIFTVTKDLPLSDERVLDGWPCHGPCPWPSNDFKEVIGRYMEELGASGEKLLQLIEIGLDVPAGSLTRYTTDGWHHARVLRFPARHMTNGKGKSGRGIGSHTDYGLLVMGSQDEVGGLFVRPPREGERLINWQKSAAGAMEDDNGWMYVPPVDGTITVFPGDIMQYLTNHVLRSTPHKVGLNTRQRFAFAYFHEPNFSSTVKPLSGYDGNQEPVEGIHYGTHFTNMCLRNYPDRITTRRLLEGNQI
>KAH7160531
LKTFTLSSEDMGSAADLSMSKALVAAWQQDGILQISMDAEQQALYKAANEASKRFFRKPLHQKAACINGQSYSGYIASGEELTDGLADYSEIFTVTKDLDLDDPRVSAKWPCHGRCPWPDAGLRDPVKKYMDSLGQTGEKLLRLIELGLNVPEGSLTRITKDGWHHMRILRFPANNDINGNGGGGRGIGSHTDYGLLVIAAADDVGALFVRPPHKDESVANWERSAAGMKEDSEGWVFIPPAETVFTFPGDMMQYMTNSVLPSTLHKVGLNTKERFAFAYFHEPSFQAVIKPLPGYEAGQSPDEGIHYGKHFTNMFMRNYPERITTHRLVDEGRYKLLD
>XP_038742924
VAQLETFTLPDRITGSKGDVAMGKGLVDAFRRDGILQVSMNDKQQAIFNEATAASKAFFAKPPPEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESGEKLLQLIELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEHYANWEKSAAGMKEDDDLWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMMAENRLEKL
>WP_197345905
MTDLTTFHLPERITNTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVRENLPCHGPVPWPGAAYRDRMKAFMGMLGTFGERLLQLIALGLDLDDMDTFTRLTQDGWHHMRVLRFPTVQSSENARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVYEHDDGWNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEADAAVAPIHYGTHFTNMFMRCYPKRITTRRITENGLLDKL
>OAA32211
MGQLETFVLPDKVSDTLSHRAMAKAMINAWRNDGILQVAMSEAQQRRYEEANAISKKFFGRKPSEKQACLNDTNVGGYIASGEEITAGVADYSEIFTVIKDLDSSDERVRNQWPCHGPCPWPDKEMGRVIKEYMAGLGAAGERLLQLIELGLEVPPGALTKYTQDGWHHMRILRFPARDKTNGKGKDGRGIGSHTDYGLLVIAAQDDVGGLFIRAPQGENFANWDQSVAGLNEEAAGWTFVPPVPGVFTVFPGDMMQYMTNNFLPSTPHKVGLNVAERFAFAYFHEPNFKAVMKQLPGYEGENVSPESIHYGKHFTNMALRAYPDRASIKKLNEEGRYAMLDTSELRSD
>WP_045317146
MTELRTFRLPEIVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMEASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDQWPCHGPVPWPDNLYRDAMRDFMAMLGESGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWNLVEPVQHVFTVFPGDILQFLTGGQLLSTPHKVLLNNRERFTMAYFHEPNFEAVVRPLDDPHGDDVIHYGTHFTNMFMRCYPDRVTTRRIENEDRLTVLSELRS
>PHH93207
QLETFVLPERVSGSVSDQAMAKAMIAAWRRDGILQVGMSKLQRQLYQSAEASSRRFFRRPQEEKQACVDSRSYAGYIASGEEMTDGVADYSEIFTVTKDLPARDKRVVDRWPCHGPCPWPDREMKAVMGRYLTELNAGGQKLLQLIELGLGVPAGSLTRYTTDGWHHMRVLRFPPRHKTNGKGKAGRGIGSHTDYGMLVIAGQDDVGGLFVLPPRQSERCANWQQSAAGLKENEPGWVYVPPESGVFTVFPGDMLQYMTNSSLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGYNGGQEPVKGIHYGTHFTNMSLRNYPDRITTRRLLAEDRYRML
>WP_163861341
MAELETFQLPQSVSGSESDVALGLSLIRAWRRDGIFQLGMSPTQADKARRAFESSRRFFRQPLDAKSRCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRHDMKEHTDELGSTGERLLRLVALGLGLEVDTLTALTRDGWHHMRVLRFPARSPMATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRLRNWLPGESSAGMYEHEEPWTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLREPTRDEFIHYGSHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLRADAVQRTAPVEE
>QBG97949
MNKIITFQIPPTIDVIQSNIDLGLDIIDAWRRDGIIQIALDDTQDAITQNAFKASQGYFALPHSEKSSHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPADDARVQQQWPCHGPVPWPNKGFQRDMALFTDQLGAIGEKLLRLVALGLHSLGRLDDIDGLNQLTRDGWHHMRVLRFPRATLGNQSRGISSHTDYGMLVIAAQDDVGGLYIRPPVEGEVRRRNWLSEESAAGMYENEEPWTFVQPTPRVVTAFPGDIMQFLTGGYLLSTPHKVKLNTRERYALAYFHEPNFKAVIEPLRPEPEDGRIHYGSHFTNMFMRSYPQRITTQRILRENRL
>XP_018663437
LETFTLPEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPRPQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDQRVAQGWPCHGPCPWADQSMKNAMNSYMVDLAISGEKLLQLIEMGLDVPTGSLTRYTDDGWHHMRVLRFPHKDRTNGKGKKGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQDEQFANWEKTSAGLRENDAGWMYVPPAAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGYNAGQSPIGGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLASEQLRDEDDV
>WP_153525272
MSDIPVLELPESLTGSPADRALAAGMIDAWRTHGIFYLATGPAEQEIVRAAMSESRRFFALPFAEKARYVNDTTYSGYIASGEEVTAGEADHSEIFTVCKDIAGDDERSLSAWPAHGPVPWPGQAYERAMKAFMTQLGSMGERLLALAALGLGLDDPSALNALTADGWHHMRVLRFPAASDAGERGIGAHTDYGLLVIAAQDSAGGLWIRPPVAGEHRNRNWLGHESTAGMFENDEPWTYVPPVPGTLTVFPGDILQLLTGGHLLSTPHKVRLAADRERYSMAYFHEPAFQSWVEPLRHAFPDDGGAGADPENRLHYGTHFTRMFLRCYPDRMTTLRLLATGGLERLARLRE
>WP_039570089
MTGLTTFHLPERIIHSEAHRQLGQDMVAAWRADGIFQIALSTPQQHTTDEAFAQSRRFFELDFETKRRHVSAFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFGERLLQLIALGLGLDDMETFTRLTRDGWHHMRVLRFPTVQSSENARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLAAESTAGMFEHDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKADADMDVAPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSEVA
>KAH8894156
ETFVLPDKLSGCEGDRALGRAMIRAWRRDGIFQVATAAEAQHIRNDATATSHRFFRKPHEEKAACVDSQSFAGYIGSGEEITDGVADYSEIFTVVKDLPLRDPRVRARWPCHGPCPWPDVEMKESIERYMRYLGSEGEKILRLTEYGMSLPETTLTRYTVDGWHHMRMLRFPATHMTNGKGENGRGIGSHTDYGMLVMASQDDVGGLFVRPPRKDEHFANWKKSAAGLREKDSGWVYIPPVPNVHTVFLGDMMQYLTSNYLMATPHKVGLNTRERFVFAYFHEPSFQAVLKPVHGYASGLSGDSEAIHYGTHFTNMFLRNYPDRITTKRLLSEGRFSKLQE
>XP_028492418
IGQLQTFVLPETFTSSASDVKLGRAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFKKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPVQRYMDSLGESGEMILKLTELGLDLPEGSLTGLTRDGWHHLRVLRFPATHATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFANWERSAAGFKEDDEGWVFVPPAENVFTVFPGDMMQFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDVGQEPKEGIHYGKHFTDMFMRNYPQRITTQRLVEEGRYDMLDE
>XP_002562422
LKTFTLPEKATGSPGDVEMGKALINAWREDGILQIAMNPKQQDLFNKAFAASKRFFALPPNVKANCVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVAAKWPCHGPCPWPDIDTKAPIQEYMDSLGSSGETLLQLIEHGLSLEPKTLTSLTKDGWHHLRTLRFPQNNKTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPYSDEKLENWKSSAAGFREHDDRWTYVPPVPGVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERYAFAYFHEPSFQAEISPIAKLYDGKPPDEKNHYGTHFTNMFMRNYPDRVTTERILKEDRLKLLD
>WP_116665714
MTDLITFALPEAVTDSKADRQLGRALIDAWRKDGIFQISIHDEQDQKVKRAMAASKAFFKLPREEKSRLVSSLTYAGYVASGEEVTAGEQDQSEIFTICKDLPLEDQRVVDGWPCHGPVPWPNDEYRDSMKAFLEELGDTGEKLLKLVALGLGLEINALTQLSQDGWHHMRVLRFPAESAQTCRGIGAHTDYGMLVIATQDEVGGLYIRPPVDGEHRKRNWLPDESTAGLFENDEPWTFVKPVPRVFTVFPGDILQFMTDGLLLSTPHKVKLNSRERFAMAYFHEPSFSVSAKPLLSQSRTERIHYGTHFTNMFMRCYPQRITTQCILKDNRLSHLEGI
>KAH0558849
LTTFALPQEVVGSQSDLSLGQRMIDTWRKDGIFQIFLTPAQGAMLRRAFKSSKGFFRRPYEEKAKHVDDRSFSGYIASGEELTDGVADYSEIFTVTKDLPPTDPRVQAKWPCHGPCPWPDANYATTMKDLMTLLGESGEKLLKLTALGLGLGDPNALTKLTEDGWHHMRVLRFPQVDRTNGKGKAGRGIGSHTDYGLLVIAGQDDVGGLFIRRPMDGEKLHNWEKSAAGFKENDENWTYVPPVENTLTVFPVCSHILGDMMQYLTRDHLPSTPHKVGLNTRERYAFAYFHEPNFSSVVKPLRDYGHEASNDKPGIHYGTHFTNMFMRNYPERITAKRVKAENRMQILKDI
>KAF9690909
LKTFELPIFVSGSSKDLELGKAMIKAWQRDGILQIKMDDGNKRIAKEAFKSSKTFFSLPTSEKTKCVDDQSFAGYIASGEELTDGVADYSEIFTVTKDLPQSDPRVQAKWPCHGSCPWPTPSMQTTMQNYMDNLGSSGERILQLIALGLGLPNMNTLNELTRDGWHHLRILRFPKTGDENGKGLKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYADEQFQNWQKSTAGMAENDDRWVYVPPEDGVFTVFPGDMLQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLKPLGGSSENSVHYGTHFTNMFMRNYPDRITAKRMQREGRLGSLQNPELREY
>WP_245695288
LRTFRLPAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLTYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMHDYMTAVGELGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTYVIPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDSADPTEYIHYGTHFTQMFMRCYPERVTTARIETEGRLKVLDRLREEALKR
>XP_040655762
LKTFALPDKVSGSAADRATARAMLDAWQGDGILQIAMTPLQKQLYRSADAASRRFFARPSKEKRACVDGQSYAGYVASGEEMTDGIADYSEIFTVTKDLSLADGRVADGWPCHGPCPWPDGTMKAVIDRYMADLGASGEKLLQLIELGLEIPAGSLTGYTEDGWHHMRVLRFPARHRTNGKGKAGRGIGSHTDYGLLVIANQDHVGGLFVRPPQGGGFANWQQSAAGMKEDEPGWTYVPPSPGVYTVFPGDMLQYMTNNRLLSTPHKVGLNTRERFAFAYFHEPNFRSVIRPLPGHDGGQEPRQGVHYGTHFTNMCLRNYPDRVTTKRMVAEDRFPMLATSALRNDDATTGS
>WP_006381705
MTELQTFSLPPTVEGGDADRELGRALIAAWQADGIFQIQATPEQDAATGRALDASRAFVSRPLKEKAQHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPADDPRVLDRWPCHGPAPWPSDQYAAAMKDYMDVVGDIGQRLLRLTALGLGLDDMEHFTRLTDDGWHHMRVLRFPPADATSERGIGSHTDYGLLVIAAQDEVGGLYIRPPVPGEARGRNWLPDESMAGRFENEDPWTFVAPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLGAPEGDEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLAVLERLRK
>WP_124232900
FTLPAIVTGSDEDRQLGREMIEVWRRDGIFEVQLTPEQATIADAAIASSKRFFKEPVEKKKTFLNDTSYSGYIASKEEITAGEEDYSEIFTVYKDLPLSDARVAAGWPCHGPVPWPDDVYQKTVTDHTNAMGDIVDRLLQLASLGLGLALNQLKSLTEDGWHHMRILRFQKAVPESDRGIGSHTDYGFLVIAVQKDAEGLDIRPPLAADGNREVRKKNWLEGQSTAGEYQSDPHWVTMPLTPDVVTVFPGDLLQYLTGDHLISTPHKIVLHPDSERYALAYFHEPNFKQSVRPLMGQNEPESITYGEHFTNVFMRCYPDRPTTLRITEENRQKN
>WP_054596572
MTDLQTFKLPDTVTGLTADISLGVKLIHAWQTDGIFQIETEEEQDYKTQAAITASKDFFKKKMEFKSSRISDLTYSGYIASREEVTAGKADYSEIFTICKDLPETDPHVEAGWPCHGPVPWPDDTYRISMKAFMDELGLAGEKLLKLTALGLKLPITTFTDLTRDGWHHMRVLRFPTSTSESSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGLFEHEEPWTFVKPTPKVWTVFPGDILQFMTNGRLLSTPHKVKLNTRERFACAYFHEPNFEAVARPLFDPDSKERIHYGEHFTHMFMRCYPKRITTQRIHAEKRLRHLEDLKKSETR
>WP_253971230
MAELETFRLPESVSGSDADITLALSMIQAWRKDGIFQVQTTNAQELKVQRAFEASRRFFRQPMAAKARCVSDLTYSGYIASGEELTAGEADHSEVFTVCKDVPLTDARAQARWPCHGPAPWPDEAYRRAMKDHLEGLGAIGDKLLQLVALGLGLDDMHVLTRLTREGWHHMRVLRFPARSPTATRGIGAHTDYGLLVIAAQDEVGGLYVRPPIEGETRPRNWLPGESSAGRYEDDERWTYVKPVPGVLTVFPGDILQFLTRGYLLSTPHRVRLNTRERFAMAYFHEPHFEACVRPLTHPAGNEFIHYGSHFTHMFMRSYPDRVTTQRILDEDRLTVLAWLRHEAVQRTT
>PCD21150
KLQTFILPEEVSGSDEDKKLGKAIVDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEELTDGIADYSETFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMKSLGESGEILLKLTELGLGVPEGSLTDYTKDGWHHMRILRFPVINKTNGRGKEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFSNWEKSAAGFKEDDDGWLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGYNAGQEPKEEVHYGKHFTDMFIRNYPERITTQRLLGEGR
>WP_239346593
MNELQTFHLPEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVGEKLLKLTALGLELDDTDALTRLTADGWHHMRVLRFPALSKESRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDSFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLSGVSADCIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLPLLR
>EQB53597
VAQLETFTLPERVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMEYLGESGEKLLQLTELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYANWEKSAAGMKEDDEKWHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGYDVGQSPKEGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQD
>KNG49275
VGHLQTFELPESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFGLEYAKKAYCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESGEKMLQLIAWGLGLKDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKETAAGKHEDADGWVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEAGQEPTEGIHYGTHFTNMFMRNYPERITAKRMRAEGRMSLL
>XP_003000034
IGQLQTFVLPETFTSSASDVKLGRAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFKKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPVQRYMDSLGESGEMILKLTELGLDLPEGSLTGLTRDGWHHLRVLRFPATYATNGKGKDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFANWERSAAGFKEDDEGWVFVPPAENLLVFRKGDMMQFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDVGQEPKEGIHYGKHFTDMFMRNYPQRITTQRLVEEGRYDMLDE
>KUM66907
LKTFTLSEKATGSPTDLEMGKAMINAWREDGILQVSMNPKQQDLFDKASATSKRFFAMTPTQKAACVDTQSYAGYIASGEQITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSGETLLKMIEYGLNLEPETLTSLTKDGWHHLRVLRFPQNNKTNGRGKEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLENWKKSAAGFREDDERWVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIPPVAKLYDGQPPDETIHYGTHFTNMFMRNYPDRVTTERMLREDRLKLLD
>WP_216670419
VSNLQIFHLPESITGIQSDIDIARQMIQAWRRDGIFHVAVNKIQERKSERTFAASRRFFGMPLESKSQFISDLTYSGYIASGEEVTAGESDYSEIFTVCKDVPLNDRRVQAQWPCHGPAPWPDEDYQQSMKAYMDELGSIGEKLLKLTALGLELDDINALTELTKDGWHHMRVLRFPALSQKSTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPTESMGGMYENEEPWILVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFAIAYFHEPNFEACVRPLFAPSSDEHIHYGSHFTNMFMRCYPDRITTRRIIDENRLSILGVLKNEGLRRLTTAKKA
>Node74
MTELTTFSLPEEAVTGSRADLALGRTMVAAWRTDGIFQVAATTAQQETALAAFEASREFFALPLEEKSRYVSDLTYSGYVASGEEVTAGETDYSEIFTVCPDVPVHDARVRARWPCHGPVPWPGPHYRRRMTAFTAALGSIIGERLLRLVALGLGLDAGDPDTLTGLTADGWHHMRVLRFPPRSARTTARGIGAHTDYGLLVIAAQDEVGGLYVRPPVEGEKRSRNWLPAESTAGMYENEEPWIIFVRPVPRVFTVFPGDILQFLTGGYLLSTPHKVALADRERHAMAYFHEPSFGRSLGEDDPCGGGGGCGGEGGGGPLHYGTHFTNMFMRCYPDRPATRRILDEGRLATLRE
>Node68
MTELRTFRLPSETVSGSDDDVELGRRLIAAWQADGILQIATDATQDRTTAAAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDDSYRDAMREFMAMLGEAAGEKILKLTALGFGLADIDTFTRLTADGWHHMRVLRFPTRSETVDDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAAEGWTTFVEPVEHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLGDDDPHGGGDDHGDDDGDDDTIHYGTHFTNMFMRCYPDRVTTRRILEEDRLTVLSELRSSE
>Node9
NLQTFVLPTAQVTGTEEDKKLGKAIVAAFQQDGILQIAMGPEQQALYKAANAASKEFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLDLDDARVRQKWPCHGRCPWPDAALRDPMKAYMDSLGENNGEKMLRLIELGLNVPQQGQLTDLTQDGWHHMRILRFPATNDTNGKGKDDGRGIGSHTDYGLLVLAAADDVGALFIRPPRQSESFFANWSTSAAGFKEDDDGWVVFVPPMENVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLEGRYDDAAGGQQEPPPPRREPREPGVHYGKHFTDMFMRNYPERITTQRLEMENRYELLEE
>Node22
LETFDLPSDYVTGSASDQALGKALIKAWKRDGIFQISMQRDQREIYKSAEKASKKFFAKPHNEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDDPRVLKKWPCHGPCPWPDAEMRDPMKLYMNYLSESSGEKLLALTELGLGVPQKGSLLHYTQDGWHHMRILRFPQTNKTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGETYYANWEKSAAGMKESDEGWVVYVPPVADTFTVFPGDMMQYQTGSFLPSTPHKVGLNTRERFAFAYFHEPNFNAVLKPLVGQYNNVVGGQQKPPPPQQKPQEPGIHYGTHFTNMFMRNYPERITTQRLISEDRYKLL
>Node125
LQTFCLPSERVTGSQSDRTMGKALVDAWQKDGILQIAMTPRQEALHSQAQQNSQEFFSRPHAEKAACVDSQSYSGYIAASGEEITDGIADYSEIFTVTKDLELGDAKVKAQWPCHGRCPWPDAAMRDVMKEYMNELGNCCGETMLRLLEYGLEIPEGTLTQQTDDGWHHLRILRFPPHNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGEKYAANWEKSAAGMAEEDEKWTTYVPPVPGVFTVFPGDMMQYMSNSTLPSTPHKVGLNTRDRFAFAYFHEPNFKTTCRPLPGPRYDAPDAAGGQTPPPPRRTPREGIHYGTHFTNMFLRNYPDRITTSRMNRENRYEILKK
>Node73
MTELRTFRVPTDSVTGTDADSRLGREMIRAWQTDGIFQVATTPEQRAAIRAAYAASRRFFARPQAFKARHVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDIPADDARVTEGWPCHGPVPWPDEEYRRAMKAYMSGLGSLLGHRLLRLTALGLGLAHDDMDAFTRLTHDGWHHMRVLRFPAASASNEERGTGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLAEESTAGMYENEEEPWTTYVEPVPGVLTVFPGDIMQFMTGGALLATPHKVRPADRERYALAYFHEPNFQAVARPLGRADPDPPPDDDPDDDDAGDPDYIHYGTHFTNMFMRCYPERVTTRRIHAEGRLAVLEELRE
>Node47
MTELQTFVLPSPTVDGSDVDKALGQALIAAWQDDGIFQIQATPEQEAVTERALEASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVIDKWPCHGPAPWPSEQYADAMKSYMGAVGDIIGERLLRLVALGLGLDADDMDHFTKLTADGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLPDESMAGRYENEEPWTTFVTPVPCVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGGGGADDGGDDEGGGGAEFIHYGTHFTNMFMRCYPERTATARIEKEDRLAVLERLRKEAER
>Node112
KLQTFILPSEEVSGSEEDKKLGKAIVDAWRKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEEMTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMNSLGESSGEILLKLTELGLGVPKEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFFSNWEKSAAGFKEDDEGWLLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGKYNNAAGGQQEPPPPKKEPKEPGVHYGKHFTDMFMRNYPERITTQRLLEEGR
>Node65
MTDLTTFHLPTERITNTEAHRELGQAMVKAWRTNGIFQIALSKPQEQATDEAFAESRQFFSQDFETKSRHVSALTYSGYVASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLQLIALGLDLDDDDMDTFTRLTDDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEAADATTTTAAATAAATAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPTLSELAP
>Node85
MKKLSTFTLPSEETVSDSPAHRALGTQIVASWRKDGIIEVRRDAIQEQETQRAFEENQKFFARPLDEKRRYVDDLTFSGYVASGEEKKSSAKDFPEVFTIFPDISVSDERVRQEEWPCHGPVPWPSEDFQSAMTAYIYTCSSIIGHRLMQLTALGMGLPPLDTFTSLCRRPWNYMRVLQFPAASVENNTRGIGAHTDYDFFVISSQDLEKQGLFVRPPVEGEHRFRNWLPDESMRGVYETQQEPWDLVEANPGVFTVFPGDMMQLVTDGYLLSTPHKVALFTSVRRAMPVFIGPDFDVRLRPHNNKKPDDDDFHYGEHVTNMHVSMYKDRSTSRRIVEKELLS
>Node62
MITLKTFKLPSAKVTGSTADITLGEQLIRAWQTDGIFQIKTRVEQDRKTQAAIAVSKRFFKKPQTFKSGCINDSAYSGYIASGEEITAGQADYSEIFTICKDLPESDPRVRNGWPCHGPVPWPDSVYHKRMKAFMDELGAAAGEKLLKLTALGMNLPRISTFTDLTRDGWHHMRVLRFPTETSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPEESSAGLFEHEEPWTTFVTPTPRVWTVFPGDIMQFMTNGYLLSTPHKVKLNTRERFACAYFHEPGFEASARPLFEEPGSSSQQGSQEGSQQERIHYGEHFTHMFMRCYPDRITTRRIHEQNRLARLEELKARDKRAEELK
>Node38
NLQTFELPTRQVLGTEDDKNLGKSLVSAWQQDGILQITMNPDQQALYKAANAASQAFFKKPYAQKAACVDSQSFAGYIASGEELTDGIADYSEIFTVTKDLDLGDPRVRGKWPCHGRCPWPDADFRDPMTAYMDSLGESSGEKLLQLIELGLDVPRQGSLTNLTRDGWHHMRVLRFPATNDTNGKGKDDGRGIGSHTDYGLLVLAAADEVGALFIRPPRKNESFFANWEASAAGFKEDDEGWVVFVPPMENVFTVFPGKSSKDLAIWSRLHTDKVGSWLNTKERFAFAYFHEPSFQAIIRPLKGRYNNGGGGQQEPPPPRREPREPGVHYGKHFTDMFMRNYPERITTQRLVSEGRYKLLEEE
>Node56
APLRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLTDTTTTTTDTATEDTADPEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEAD
>Node84
MTELQTFVLPSSTVDGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVAGKWPCHGPAPWPSQSYADAMKAYMGTVGDIIGEKLLRLVALGLGLDDDDMDHFTKLTDDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGDILLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGAEGEDDEGDDEEGGAEEFIHYGTHFTNMFMRCYPERTATARIENEDRLTVLERLRKER
>Node42
MVDFRSFHLPTESISGTQSDVDLGREMIRAWKTNGAFRIVMSEIQRKSSEDAFAANRQFFRGSIEFKSQYTNDLTYSGYTASGEEVTAGEKDYPEVFTICRDIPVDDARMRADWPCHGPVPWPNAEYRTSLQTYLNELGSVVGDRLLQLVALGLELNKDDIDTLLALAKDGWHHLRALRYPVASQESSRHGLGAHTDYGLIVITDEDNVGGLYIWPPVEGEKRNRNWLATESTAGTHENEKYWIIFAKPAPNAFTVFPGDILQFITDNHILATIHKVLLNTRERFSMAYFHEPDFNACVYPLSNNPSDDDKKSDKDSDKKDYLFYGEHFTNMFMRCYPDGATTRRIMDENRLSVLTDLKNTD
>Node119
LKTFTLPSEKATGSPRDVEMGKAMINAWREDGILQVSMSSKQQDLFDKASAASKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSSGETLLQMIEYGLSLDKPQTLTSLTKDGWHHLRVLRFPQNNNTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKALYYDGGQQPPPPPDDPPDEPKIHYGTHFTNMFMRNYPDRVTTERILREDRLKLLDD
>Node14
NLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELALTDPRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESSGEKMLSLIAWGLGLEQDDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYYANWKTTAAGKHEDTDGWVVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFQQAAGGQQEPPPPTTEPTEEPETEGIHYGSHFTNMFMRNYPQRITADRMKAENRMALLDSPDLRWKDTPAD
>Node69
MTELRTFELPTDSVSGTTVDRILGRQLVRTWETDGIFQVAATAEQERITQEALAASRRFFGRSLPEKAKFVSDLTYSGYIASGEEVTAGEADFSEIFTVCPDVPLTDTRVTEGWPCHGPVPWPDDGYRTAMRAYLREVGETTGEKLLRLVALGLEVEEIDRLTRLTRDGWHHMRVLRFPARSATTTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLPEESTAGMYQDDEPWHHFVEPEPRVFTVFPGDILQFLTGGRLLSTPHKVRLATRERYTLAYFHEPSFSTTVRPLGGGDDDGDRIHYGTHFTNMFTRCYPDRVTTKRIHEEGRLDRLRALSRDR
>Node13
VGNLQTFELPSESLNGSVADRALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGMDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALYDPRVQQQWPCHGPCPWPFTQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPETNKVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDTDGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGAFEEAAGGQQEPPPPAAEPAEEPAAGIHYGSHFTNMFMRNYPERITADRMRNEGRLALLDD
>Node15
VGHLQTFELPSESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYAANWKETAAGKNEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEEAAGGQQEPPPPTTEPTEPTGIHYGTHFTNMFMRNYPERITADRMRAEGRMALL
>Node23
LGERIVAALQRDSLIQIKKNYQEDLLTAQALLQNKKFNRLPQDEKIKFVSDTTYTGYVFSGEEKKGGRYDQVEVYTVFPHFDRDNERVKTLRLPCHGEAIYPSVDYKDAIEAYMKATGNMMCDQLIEIIGLGLGLERPPNHYLSEMVRDGFHHARVLRFKSQKAAGAASNGISAHSDYGFLIAASQDDVGGLWVRPIVNGEKRGRNYLEEETTIGAYDDQPGWAAFVTPMESVWTVFVGDMMELATQRKLKANIHKVKLNEEKERFAIAYFHEPKFNQTFEAVNSSRGKKKKKGKKKGKKKEIVYGKHLTNMYTKAYPNRSTTKRIESEELCKNIDRISKIS
>Node116
HLQTFELPSESLSGSASDKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFALDHKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTIMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKETAAGKNEDAADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFEEAAGGQEPPPPTTEPTEPEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDSPNLRWKDTPADSG
>Node6
LGRLETFILPSERVSGSVSDKAIADAMIGAWRRDGILQIGMSKPQQRLYQAANAASRRFFAQPHALKQACVDDKSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTRYMNDLAESSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGIHNNAAGGQQSPPPPPIISPIEPGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLEE
>Node86
MTDLQTFDLPSESITGSQSDIDLAHQMIQAWRTDGIFQVATNAIQTRKTQNAFEASRRFFRMPLDFKSQCISNLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVRVQAQWPCHGSVPWPDQNYHQSMKAFMDELGIIIGEKLLKLVALGLELDKDDIDALTKLTKNGWHHMRVLRFPALSEKSSTRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWLSNESSAGMYENDEPWTTFVKPVPKVLTVFPGDILQFMTNNYLLSTPHKVRLNTRERFALAYFHEPNFQACVRPLFDDSSSSSDDSSDDSSDDDYIHYGTHFTNMFMRCYPYRITTRRILDEDRLSVLELLKNEL
>Node46
MADLRTFGLPTDRVVGTDSDITLAHQMIDTWQTDGIFQIACDDRQAAKVTEALRESRRFFASATEEKAKCVSPLSYAGYIASGEEVTAGEADYSEIFTVCQDIPPDDARVQANWPCHGPVPWPSEQYQRTMQAYMDEMGLIIGEKLLRLIALGLDLPEEEIDTLTRLAEGGWHHMRVLRFPARNSLTTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLDSESSAGMYENEEPWTTFVTPVADTLTVFPGDILQFLTDGVLLSTPHKVRLNDRERYTMAYFHEPNFDAGIRPLSGGSDAFIHYGTHFTNMFMRCYPDRITTRAILEADRLSVLARLAERAAR
>Node40
SLQTFTLPSEKITGSRGDRALGRTMVDAWRRDGILQISMDHTNRKLADVAFESSKKYFGMSHRDKAKCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPLTDPRVVQKWPCHGPTPWPNAQMKTVMQAYMDYLGESSGEKMLELIAYGLGLAQDDGDALKKYTQDGWHHMRILRFPETHNTNGKGKEEGRGIGSHTDYGLLVIAAQDAVGGLFIRPPYEGEKYYANWKDSAAGMKEDDDKWVVYVPPVADTFTVFPGDMLQYATGNYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPYNNAAGGQQEPPPPTTEPTEPGIHYGTHFTNMFMRNYPERITAHRMRSENRMEKLKK
>Node24
LGERIVAALQRDSLIQIKKNYQEDLLTAQALLQNRKFNQLPQDEKIKFVSDTTYTGYVFSGEEKKGGQYDQVEVYTVFPHFDRDDERVKTLRLPCHGEAIYPSVDYKNAIEAYMKATGNMMCDQLIEIIGLGLGLEPPNHYLSKMVRDGFHHARVLRFKSQKAAGAASNGISAHSDYGFLIAASQDDVGGLWVRPIVNGEKRGRNYLEEETTIGAYNDQPGWAAFVTPMPSVWTVFVGDMMELATNRKLKANIHKVILNEEKERFAIAYFHEPKFNQTFEAVNSSKGKKKKKGKKKGKKKEIVYGKHLTNMYTKAYPNRSTTKRIESEELCKNIDRISKIS
>Node113
VAQLETFTLPSERVTGSPGDRAMGKALVDAWRRDGILQVSMNDKQQHIFNEATASSKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTQDGWHHMRILRFPPTHNTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGMAEEDDRWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGPYDDVVGGQQSPPPPKKSPKEPEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLEEREELRAHS
>Node59
MTELRTFRLPSETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEAAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPPDDSRVVGKWPCHGPVPWPDDSYRDAMREFMAMLGDSSGERLLKLTALGFGLDADIDTFTRLTADGWHHMRVLRFPPRSEAVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGRLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDDPHGGGDDHGDDDGDDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node64
MTDLQTFHLPTESVAGTDADVQLGHDLIRAWQTDGIFQVAADPAQDQTTQEAIAASRRFFRLPLEVKARHVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLDDARVRARWPCHGPVPWPDEAYRRRMRAFMDMLGSIIGERLLRLVALGLDLPADDIDALTRLTADGWHHMRVLRFPARSEQTTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLASESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFESTVRPLSGGPPAGGGGDDAGDEAGDDETIHYGTHFTNMFVRCYPDRITTRRILAEDRLSVLAALREAA
>Node83
MTTLRTFQLPSAEVTGSIWDIALGVELVRTWQSDGIFQVATGSVANAVTAAALRESRRFFALPLAEKAALVSDLTYAGYVASGEEVTAGEQDFSEIFTVCKDVPVTDPRVRAGWPCHGPVPWPDPVYRDVMTALMSQFGLVVGDKILKLTALGLGLSDMNALTSLATDGWHHMRVLRFPARSAATTTRGIGAHTDYGMLVIATQDDVGNGLYVRPPVVGEKRPRNWLDHESSAGAFENEEAPWHLVDPVPSTFTVFPGDILQFLTGGELLSTPHKVKLADRERYSIAYFHEPDFAAALRPLGGAGGGGDGAGGDEDGGEPEVIHYGTHFTSMFMRCYPDRLTTRRILAEDRLAVLAARMREEARR
>Node90
MGELATFRVPTARITGTTADIMLGGAMVRAWQRDGIFQVAADGRQEARTRAALAASRRFFARPAAEKAACVNDTSYSGYIASGEEVTAGRPDASEIFTITPDIPAAAGLPCHGPVPWPSPGYRAAMEAYLAGVGGLLGERLLRLVALGLGLGRGPAGIDRLAGLTRGGWHHMRVLRFPAAERTAAERGIGAHTDYGLLVIAAQDEVGGLFVRPPVAGEERPRNWLPGESTAGFAENRPGWTTPVRPVARVLTVFPGDILQFMTGGALLSTPHRVRLADRERYALAYFHEPRFGARIRPLPARAQRGGDEGGGAAAVIHYGEHFTRMFMRCYPQRATTRRIEAQGLLRHL
>Node80
MIELETFQLPRQSVSGREADIALGLTMVRAWRRDGIFQVRMSPAQAEKSQRAFELSRHFFRQPLETKARCVSDLTYSGYIASGQELTASEADLSEVFTVCKDVPLTDPRVQSRWPCHGPGPWPDEAWRQGMQAHAEELGSVVGERLLRLIALGLGLHDIDALTTLTHDGWHHMRVLRFPARSPTTTTRGIGAHTDYGMLVIAAQDDVGGLYVRPPVEGEKRPRNWLPHESSAGMYEHDEPWTTYVKPVPGVLTVFPGDILQFLTRGYLLSTPHKVVLNTRERFALAYFHEPQFEACVRPLSAAPTRRRDDTRDETRDDEYIHYGTHFTNMFMRSYPDRVTTQRILDESRLTTLSWLRQEAVLRSW
>Node49
MTNLQTFELPSTEVIGSAADISLGRALIQAWQKDGILQIKTDSEQNRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLPVSDQRVKAGWPCHGPVPWPNNTYQKSMKAFMGELGLAAGERLLKLTALGFELPRINTFTDLTRNGWHHMRVLRFPPQTSTMMSSGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDDPWTTFVTPVQNVWTVFPGDILQFMTCGQLLSTPHKVRLNTRERFACAYFHEPNFEACAYQVFEEPSGGGNNSGNESGNNERIHYGEHFTSMFMRCYPDRITTQRIHKDNRLAHFS
>Node57
MRTFELPTVSVGGTTVDRILGKQLVRTWEDDGIFQVAATAEQARITGEALAESRRFFARSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTEGWPCHGPVPWPDEGYRTAMEAHLREVGEIIGEKLLKLVALGLGVEREIDRLTRLTRDGWHHMRVLRFPARSATTEERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLPEESTAGMYQDDEPWHHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLAADRERYTLAYFHEPSFGVTVRPLADEESIHYGTHFTNMFTRCYPDRVTTKRIHEEGRMRLL
>Node37
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPHAQKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQDMKTVMNRYMNDLAVSSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQRDRTNGKGKKGGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQDERFFANWEKSSAGLRENDAGWVVYVPPVAGTFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGIYNNAAGGQQSPPPPIISPIGPGIHYGTHFTNMFLRNYPDRVTTARLREEGRYLLLGSEELRDEDDG
>Node95
MSGFATFQVPTDRVTGTAADEALGRAMTAAWQRDGIFQIAADAGQRARTREAMAASRRFFARPPAEKAACVNDTSYSGYIASGEEVTAGERDASEIFTITPDLTDGPGLPCHGPVPWPDPGYRAAMEAYLAGVGGLLGERLLRLVALGLGPGLEGGPGRGPAGMDRLAGLTRGGWHHMRVLRFPAASRTTTERGIGAHTDYGLLVIAAQDEVGGLFVRPPIAGERRPRNWLPGESTAGFAQDSPGWTTYVTPVPGVLTVFPGDILQFLTGGALLSTPHKVRLADRERYTLAYFHEPRFGARIRPLRPDDTTVIHYGEHFTRMFMRCYPQRATTRRIEAEGLLRRLGAVR
>Node99
LQTFELPNESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFFANWKESAAGKHEDAADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGFEREAARREDPPPPIIDPISDPDGIHYGTHFTNMFMRNYPTRVTADRIRAEERMALLEE
>Node79
MTELRTFRLPSETVTGSDDDVELGRRLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDNDYRDAMREFMAMLGEAAGEKILKLTALGFGLDADIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGALLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDDPHGGGDDHGDDDGDDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node39
NLQTFILPTDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFSKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMHRYMQSLGGVVGETLLQLTELGLGVPQQGSLTNYTQDGWHHLRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVIAAADDVGGNDEKLLANWESSAAGFKEDDERWVVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGKYDDVVGGQQEPPPPKKEPKDPGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLEE
>Node105
VAQLETFTLPSERVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFDEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMSEPRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESSGEKLLQLAELGLNVPDAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGLKEDDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGQYDPDVVGGQSPPPPQQSPQESPGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQQ
>Node25
LDTFCLPSNKVTGSAYDRSLARAMIEAWRRDGIFQVAMSPQQQRIYQNANAASRRFFRRPSAEKHRCVDEKSYAGYVASGEEVTDGVADYSEIFTVTKDLGENDPRVAGQWPCHGPCPWPDEDMKSNMTAYTQDLSQSSGDQLLEMIELGLHVPDQGSLKRYTDDGWHHMRVLRFPHRDRTNGKGKKKGRGIGSHTDYGLLVMAAQDEVGGLFVRPPHQGEHYYANWEKSAAGLKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLKSTPHKVGLNTRERFAFAYFHEPNFASVIKPLPGYDDGGGGVDAAASDASEAGIHYGTHFTNMCLRNYPERITTKKLLREDRCSML
>Node45
MNKIITFQIPIPTIDVIQSNIDLGFYIIDAWRRDGIIQIALDDTQDAITQNAFKASQGYFALPHSEKSSHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPADDARVQHQWPCHGPVPWPNKGFQRDMALFTDQLGAIIGEKLLRLVALGLHSLGRLDLDDIDGLNRLTLDGWHHMRVLRFPCATLGNQQSRGISAHTDYGMLVIAAQDDVGGLYIRPPVEGEVRRRNWLSEESTAGMYENEEPWTTFVQPTPRVVTAFPGDIMQFLTGGYLLSTPHKVKLNTRERYALAYFHEPNFKAVIEPLRPPEPEEEDDPEDGPEDDGRIHYGSHFTDMFMRSYPQRITTQRILRENRL
>Node122
QLETFVLPSEQVSGSVSDQAMAKAMIDAWRRDGILQVAMSQLQRQLYQSADASSRRFFRRPAGEKQACVDRRSYAGYVASGEEITDGVADYSEIFTVTKDLSPRDKRVAHGWPCHGPCPWPDREMKAVMGRYMADLGASSGTKLLQLIELGLGVPNAGSLTRYTNDGWHHMRVLRFPPRHNTNGKGKDDGRGIGSHTDYGLLVIAAQDDVGGLFVYPPQRGERFFANWRQSAAGAKENEPGWVVYVPPTPGTLTVFPGDMLQYMTNGVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNGGGGQQEPPPPVLEPLEEPEVGIHYGTHFTNMCLRNYPDRITTRRLVAEGRYRMLDSEELRD
>Node61
LRTFRLPTAAVDGTDADRALGRELVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLDADDPDHFTRLTADGWHHMRVLRFPPADAGSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTTPTTDTTPTADAADPAYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEDR
>Node76
MTDPRTFRLPSATVTGSAADRALGRELVAAWRTDGIFQVAADPAQAQAVQAAMTASRRFFALPLAEKARYVSDLSYSGYVASGEEVTAGEADHSEIFTVCKDVPADDARVRAGWPCHGPVPWPGPHYRRAMTAFMSALGSIIGERVLRLVALGLGLDDPDTLTRLTADGWHHLRVLRFPARSAQTAARGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRSRNWLPEESTAGMYEDEEEPWHHHVRPVPGVLTVFPGDILQFLTGGHLLSTPHKVALADRERHSMAYFHEPSFEASLTPLDDDPAGGGGGGGGDGDPPLHYGTHFTNMFMRCYPDRPTTRRILEEGRLAVLARLRALR
>Node111
QLETFTLPNESVTGNRGDRAMGKALVEAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESSGEKLLQLTEYGLGVPNEGSLTQYTNDGWHHMRVLRFPQNNKTNGKGKEEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQYYANWEKSSAGMKEEDDRWVVYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGKYDDAAGGQQSPPPPKKSPKEPGIHYGTHFTNMFMRNYESRITTQRLLAEDRYSLL
>Node114
VGNLKTFELPSEVLTGSASDQALGRSMIDAWRKDGILQISMDPINRKLADAAFICSKKFFDLPYKTKASCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPPTDPRVQQKWPCHGPCPWPFKQMRTVMNTYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTKYTQDGWHHMRVLRFPERNNVNGKGKDDGRGIGSHTDYGLLVIAAQNDVGGLFIRPPMEGENYYANWRKSSAGMAEDDDKWTTYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGRFDDAAGGQQEPPPPRREPREPGIHYGTHFTNMFMRNYPERITANRMRSENRMKLLEE
>Node124
ELQTFVLPSEEVTGSDADRALGKELVAAWRRDGILQIAAFPPGLQVLSDAIKQSKSFFAQQHEAKAKCVDSQSFAGYIASGEEITDGIADYSEIFTVTKDLPSSDPRVQSKWPCHGPNPWPSKTYEQAITALMDYKGRIIGEKLLHLTALGLNLRQNNPWALNQLTQDGWHHMRVLRFPPTNQTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFIRPPIEGEKYYENWKKSAAGAHESDEKWVVYVPPVENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCKRLPEFRRDDGGSEAAAKEEAKEEEVHYGTHFTNMFMRNYPDRITAKRMRDEDRMDLLEGMKE
>Node71
MADLVTFQLPSQSVSGSESDVALGLTMIRAWRRDGIFQLRMSEAQLEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKEHTDELGSTTGERLLRLVALGLGLREVGALTALTRDGWHHMRVLRFPARSTTAATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTTPGPPPDDGPDEGPDDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLREAW
>Node12
NLRTFVLPNDTVNANDESTSMAKAMIKEWRETGIFQIQMTPAQQAMLTECFETSKEFFRRSHEEKSKHVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPSSDVRVQEKWPCHGPTPWPSKVYKDQMDQLMKLLGDSSGEKLLKLVGLGLGLKEDDPDALLNLTEDGWHHMRILRFPHVEKTNGKGKKKGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIQGESIIKNWQSSAAGLNENDDKWLLFVPPVPNTLTVFPGDMMQLITNDYLPSTPHKVGLNYDERYAFAYFHEPNFSSVIKTLPEFDEAEKVDKSEMIHYGTHFTDMFMRNYPERITADRIREEGRLDILKK
>Node107
ELDTFELPSVAVTGSDADRLLGRRLIEAWRKDGIIQVAAFPENLRVLSEAVKQSKQYFLQDHATKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPNDAYAHAMRKLMDFKAGIIGERLLQLTALGLDLKDDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEGGRGIGSHTDYGLLVIASQDEVGGLFVRPPSADETYYSNWKKSSAGLHENDDKWLLYIAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNNTRERFAFAYFHEPNFNAVCRRLPEPPEFKKTTGGTPEDEEEEEEEEEEAVHYGTHFSNMFLRNYPDRVTAHRMREENRMDLLPMLKP
>Node30
LETFVLPSEKVSGSVSDQAMAKSMIDAWRRDGILQIGMGQPQRRLYQSADAASRKFFQRSAKDKQACVDSCSYSGYIASGEEITDGVADYSEIFTVTKDLSHSDKRVANNWPCHGPCPWPDQDMKSVMSRYMADLGTSSGEKLLQLIELGLEVPDAGSLTGYTDDGWHHMRILRFPARHATNGKGKEGGRGIGSHTDYGLLVIAAQDHVGGLFVRPPNKEDKFFANWQQSSAGMKEDDSGWVVYVPPTPGVFTVFPGDMLQYMTNNVLLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGEYNNGGGGQQAPPPPMMAPMEPGIHYGTHFTDMFLRNYPERITTKRMEAESRYQLLESKQLRER
>Node51
LQTFRLPTAAVRGTHNDRALGRALVAAWQSDGIFQIRATRTQQEATERALAESRTFCRRALNEKARHVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPVPWPSPGYARAMKDYMTVVGEIIGGRLLRLTALGLGLDTDDPDRFTRLTTDGWHHMRVLRFPPADATSSQRGIGAHTDYGLLVIAAQDDVGGLHIRPPVPGEQRGRNWLPGESMAGRYEHEEPWTTYVTPVAAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERHTLAYFHEPAFDAVARPLDDPDSASAADSAAEAPPDAEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLREDR
>Node126
NLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESSGEKMLSLIAWGLGLEDDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYYANWKTTAAGKHEDTDGWVVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFQQAAGGQEPPPPPTTEPTEPGIHYGKHFTNMFMRNYPQRITADRMKAENRMALLDSPELRWKDTPAD
>Node43
MTGLTTFHLPSERIIHSEAHRQLGQDMVAAWRADGIFQIALSTPEQHTTDEAFAQSRRFFELDFETKCRHVSAFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFFGERLLQLIALGLGLDRDDMETFTRLTRDGWHHMRVLRFPTVQSSENNARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLAAESTAGMFEHDDGWTTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKAADADTDDDVVADVAADVPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSEVAP
>Node88
MTDLTTFHLPTDRIINTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSPLTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPSADYRDRMKAFMGMLGTFFGERLLQLIALGLDLDDDDMNTFTRLTDDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEAADADAAAAAAAAAAAAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPTLSEVAP
>Node34
VGQLETFTLPSESVRGSRGDREMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESSGEKLLQLTEYGLGVPNEGSLTQYTNDGWHHMRILRFPQNNKTNGKGKEEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQFFANWEKSSAGMKEDDDRWVVYVPPVPNVFTVFPGDIMQYITNSHLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGKYDDAAGGQQSPPPPKKSPKEPGIHYGTHFTNMFMRNYESRITTQRLLGEDRYSLL
>Node1
FETFVLPSEKVDGSPSNHALGKAMVEAWRRDGILQVAMSGQQQQVYREAQQRISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGIPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLAANNGERMLQLIEFGLHVPGPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGERYANWEKSSAGLLEDEESGWVFVPPVPGVFTVFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGVYNNRRAAQQPPPPPVVPPVDPVDGIHYGTHFTNMFWRNYPDRLTTRRLEEEGR
>Node123
VGNLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELTQSDPRVQQKWPCHGPNPWPFTQMKTVMQAYMDYLGESSGEKMLSLIAWGLGLEQDDGNALTRYTQDGWHHMRVLRFPQTDKVNGKGKAAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYYANWKETAAGKHEDADGWVVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGEFQQAAGGQQEPPPPTTEPTEPGIHYGTHFTNMFMRNYPQRITADRMKAEDRMALLDD
>Node20
LETFTLPSTSVTGSRSDRMLGQRLIAAWRRDGILQIRTSAAQQAIVDAAFAASKAFFRQPHAEKAALVDPYSYGGYIASGEEITDGIADYSEIYTVTKDIPPSDPRFMLPCHGQQPWPNSQFKDVVNRYMQQLGNTTGDIMLGLVELGLNIPQRGSLTKLSQDGWHHLRILRFPQNNKTNGKGKDGGRGIGSHTDYGMLVIASQDDVGGLFIRPPYDDEKTTQNWAKSAAGIKENDSGWVVFVPPVSNVFTVFPGDMLQYMTNHYLPSTPHKVGLNTTERFAFAYFHEPHFDSVVKPLPGPYNNGGGGQQEPPPPTVEPTEEPEVGIHYGTHFTNMFKRNYPDRVTTQQLLKDGR
>Node58
MRTFELPTDSVGGTTVDRILGRQLVRTWEDDGIFQVAATAEQDRITAEALAASRRFFARSLPEKAKFVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTGGWPCHGPVPWPDEGYRTAMTAHLREVGAIIGEKLLQLVALGLGVEEIDRLTRLTREGWHHMRVLRFPARSATTTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEHRNRNWLPAESTAGMYQDDEPWHHFVEPEAGVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGATVRPLGDDEGERIHYGTHFTNMFTRCYPDRVTTRRIHREGRLAVLDRIRD
>Node109
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRIYQAATAASRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQDMKAVMNKYMEDLAVSSGEKLLQLIEMGLDIPTGSLTKYTDDGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRESERFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGPPVFNPPNAAGGQQSPPPVVPPVEPVVEGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYQLLESEELRDDDDE
>Node54
MTDLQTFHLPTKSITGTQSDIDTAREIIQAWRTDGIFQVATNTIQDRKTESAFEASRRFFRMPMKFKSQCISDLNYGGYIASGEEVTAGKSDYSEIYTICKDIPLNDARVQAQWPCHGPMPWPDQEYHQSMKVFMDELGLIIGEKLLKLTALGLGLDRDDINALTKLTRDGWHHMRVLRFPTLSQKSSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESMAGMYENDDPWTTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFDDPSSSSDDSSDESSDDEHIHYGTHFTNMFMRCYPDRITTRRIINEDRLSILARLENKTLGRLTAR
>Node28
LETFTLPSERVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRVYQAASAASRRFFAQPHARKEACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQEMKAVMNKYMEDLAVSSGEKLLQLIEMGLDIPATGSLTKYTADGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRETERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGPYNNAAGGQSPPPPLLSPLEPGIHYGTHFTNMFLRNYPDRITTARLQEEGRYQLLESKELRDDE
>Node21
QLQTFTLPTEQVTGTPGDRALGKALIDAWRRDGILQIAMDPKQQKLLEEAFAASKRFFRKPHKEKAACVDSQSYSGYIASGEEVTDGIADYSEIFTVTKDLDLAEPRVKAKWPCHGPCPWPDIEMKDPMQRYMDYLGQSSGEKMLQLIEYGLDVPQPGSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSTAGLKEDDDGWVVYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLPGKADDAAGGQQSPPPPKKSPKEPGIHYGTHFTNMFLRNYPDRITTQRLIAENRMALLEE
>Node93
MTDLQTFALPMDSVTGMPSDLELADRMIRAWRRDGIFQVAMNPAQEQQTERAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLTDARVRAQWPCHGPAPWPDDAYRQAMTAFTDALGSVVGERLLRLTALGLGLERVDALTRLTRDGWHHMRVLRFPALSPTSSSRGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMFEHEEPWTTFVKPVPKVLTVFPGDILQFLTGGYLLSTPHKVRLNTRERYSMAYFHEPAFDACVRPLMGGPSGGGEESGEESGSEEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLSTLARLRDEAAR
>Node17
VALETFVLPSERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSSGDKVLQLIELGLDVPDQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAGGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEEPGWLLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNIRERFAFAYFHEPNFRAMVKPLPGAYNNAASSQEPPPPAEPADPGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKES
>Node50
MTELQTFVLPSPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATEQALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDMAEEDARVVDRWPCHGPAPWPSQQYAEAMKTYMGAVGDIIGERLLRLVALGLGLDRDDMEHFTKLTRDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLHIRPPVPGEARGRNWLPEESMAGRYENEEPWTTYVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPAFQAVARPLGGGGPEEGGEEEEGGGPEFIHYGTHFTHMFMRCYPDRTTTVRIENEDRLAVLERLRKEAER
>Node26
NLKTFELPGETLTGGASDQALGRSMIDAWRHDGILQISMDPINRKLADAAFICSKKFFDLPYKTKAACVDDQSFAGYIASGEELTDNIADYSEIFTVTKDLAATDARVQQQWPCHGPCPWPFKQMRTVMDAYMQYLGDSSGEKMLQLIAWGLGLAQDDGNALTRYTQDGWHHMRVLRFPERNNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEVVVANWRESAAGLAEEDDKWVVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNNRERFAFAYFHEPNFSSVMKPLEGRFDDAAGGQQEPPPPRREPREPGIHYGTHFTNMFMRNYPKRITADRIRKEDRMKLLDD
>Node108
LQTFELPTAQVTGTPKDVELGKALVDAWRSDGILQISMKDTKKSLMDEVFDASKGFFRKPHSEKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEPRVRNHWPCHGPCPWPDLEMKDPMEQYMEYLGQSSGEKLLQLIECGLGVPNPGSLTNYTNDGWHHMRILRFPAVSNTNGKGKEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPSADEATTANWEESAAGFHEDDSGWVVYVPPMPDVFTVFPGDMMQYLTDSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVLRPLPGRYDDGGGGQQKPPPPRRKPREPGIHYGTHFTNMFMRNYPDRITSKRMIREDR
>Node55
MTELQTFVLPSPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATERALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIYTVCPDIPEDDARVVDKWPCHGPAPWPSEQYADAMKDYMGAVGDIIGERLLRLVALGLGLDEDDMDHFTKLTEDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLADESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGGGGPDDGGDDEGGGGPEFIHYGTHFTNMFMRCYPDRAATARIENEDRLAVLERLRKEAER
>Node91
MTTLHTFHLPTESISATPEDIALGKAMIEVWRRDGIFQVAATPAQNRAMLQAMKASQGFFRLQSRNKARRISDLTYSGYIASGEEVTAGQADYSEIFTVCKDLPLDDVRVKNGWPCHGPVPWPNKNYRLRMHTFMDALGDIIGERLLTLVALGLELERDDRHTFTDLTRDGWHHLRVLRFPAGSSSTTARGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLEKESSAGRFEDQDPWTTFVQPMPRVFTVFPGDLLQFMTDGYLLSTPHKVKLNTRERFALAYFHEPNFEAIARPVWGGQTRRRGGTRGSRFIHYGEHFTRMFMRCYPERITTRRILKENRLAHLELMKRHAELMK
>Node102
VGHLQTFELPSESLSGSVADRALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYHKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFFANWKESAAGKHEDADGWVVYVPPVSDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGIFEEAARRQQDPPPPIIDPISPGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLEE
>Node66
MTGLTTFHLPSERILHSEAHRQLGRDMVAAWRADGIFQIALSTPQQHTTDQAFAQSRQFFELDFETKRRHVSEFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTDMLGTFFGERLLQLIALGLGLDRDDMETFTRLTRDGWHHMRVLRFPTVQSSENNARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLASESTAGMFEHDDGWTTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKAADADADDDAAADAAADPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLNRLPALSEP
>Node70
MAELETFQLPSQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKDHTDELGSTTGERLLRLVALGLGLREVGALTALTRDGWHHMRVLRFPARSATAATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLMTTPGPPPDDGPDEGPDDEYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLREAW
>Node78
MADLRTFQLPTDFVDGTKSDIQLAGELIQAWRTDGILQVATDPAQDQKTKAAIESSRRFFGMPAEFKRGCISDLTYSGYIASGEERTAGEADYSEIFTVCPDVPLDDPRVRAQWPCHGPVPWPDDEYERAMLAFMAELGSIIGEKLLKLTALGLGIRDIDALTDLTRDGWHHMRVLRFPARSEHTTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVDGERRSRNWLPGESMAGRYENEEPWTTFVKPVPSVFTVFPGDMLQLITDGFLLSTPHKVRLNTRERFALAYFHEPNFQARVRPLSGGGGDDDGGGDGDGGDDYIHYGRHFTSMFMRSYPERITTRRILEEDRLSRL
>Node52
MTNLQTFYLPTKSISATQSNVDLGHQMIQAWRTDGIFQVATNTIQDGKIQNAFEVSRRFFRMPLGLKSQYISDLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVCVQKQWPCHGPVPWHNQNYQQNMTTFMDELGFMMGQKLLKLVALGLGVDMNTFDFTLNGWHHMRVLRFPAVSEKSSARGIGAHTDYGLLVIAVQDDVGGLYVRPPVEGEKRNKNWRSDESSAGMYENEEPWTTFVKPIPRVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLSNNPSSSSNNSSNESSNNEYIHYGTHFTNMFMRCYPDRITTRRIHDEDRLSVLAVLRNAV
>Node2
QLETFVLPSEAVTGSVADVAMGRSLVAAWQRDGILQVCMSPLQRRLYEAADAQSRRFFARPQAEKQACVDSQSYSGYVASGEEITDGIADYSEIFTVTKDLPSSDPRVQSRWPCHGPCPWPDQKMKQVISRYIADLGTAAGQKLLALIELGLDVPNTGSLTQYTNDGWHHMRVLRFPARHKTNGKGKAAGRGIGSHTDYGLLVMAAQDDVGGLFVRPPQGDSFFANWQQSAAGVKEDDAGWVVYVPPTPGVFTVFPGDMLQYMTNNALLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNAAGGQQAPPPPTTAPTEAPTTGIHYGTHFTNMCLRNYPDRVTTKRLAAEDRYQML
>Node16
LETFDLPSDRVTGSASDKAMGQALIKAWKRDGIFQISMNSSQREIYKYAENASKRFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSSGEKLLALTELGLGVPQRGSLLQYTQDGWHHMRILRFPPTNKTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGESYYANWEKSAAGMKEDDEGWVVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGQYNGNNNGGQQKPPPPPQQKPQEPEGIHYGTHFTNMFMRNYPDRITTQRLHAEDR
>Node41
LETFTLPSEKVSGSVCDKAIADAMISAWRRDGILQIDMSKAQRRLYQAANAASRRFFSRPYVQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQSMKNVMNSYMNDLAVSSGEKLLQLIEMGLDVPDTGSLTRYTDDGWHHMRILRFPHRDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPRQDEQFFANWEKTSAGLRENDAGWVVYVPPTAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGMYNNAAGGQQSPPPPMMSPMGSPMMGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLGSEQLRDENDTG
>Node100
VGNLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPVFEAEAAGGQQEPPPPPVVEPVEEPEVGIHYGTHFTNMFMRNYPKRITADRMRAEGRMALL
>Node4
QLQTFVLPGDKVTGGDLDRKLGKAIVKAWQQDGILQIAMKESQQGLYKAANQASQRFFKKPYAQKSACIDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVVEKWPCHGRCPWPDADLQNPMKQYMDSLGESSGETLLKLTELGLDVPQEGSLTRLTQDGWHHMRILRFPATHATNGKGKDDGRGIGSHTDYGLLVLAAADDVGALFVRPPQKSDEYYANWNKSAAGFKEDDDGWMMFVPPAENVFTVFPGDMMQYLTNSVLPSTPHKVGLNVSERFAFAYFHEPSFQAVIKPLPGKYDDAAGGQQEPPPPKKEPKEEPEKGVHYGKHFTDMFMRNYPQRITTQRLIDEGRYNLLEE
>Node75
MTDLQTFHLPTASITGTQSDVVLARELIQAWRTDGIFQVATNAIQERKTESAFASSKRFFRMPQEFKSQCISDLTYSGYIASGKEITAGQADYSEIFTVCQDVPLDDARVQAHLPCHGPAPWPDEEYQQSMKAFMDELGTIIGQRLLQLTALGLELDEDDINTLTNLTENGWHHLRVLRFPAASQESSARGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRKRNWLATESSAGLYEHEEPWTTFVKPVPKVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFEEPSSSSDDSSDESSDDEYIHYGRHFTNMFMRSYPDRITTRRLVNEDRLSNLAPVKNAP
>Node103
QLKTFTLPSDKATGSPSDVQLGKAMIDAWRQDGILQVSMSPKQQDLFDKAFTASKRFFAMPPKQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVEAKWPCHGPCPWPDTDMRTPIQKYMESLGNSSGETLLKMIEYGLDLDEPKTLTSLTEDGWHHLRILRFPQNNNTNGKGKDDGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAEDEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAEISPVAKALYYDGGQQPPPPPDDPPDEPKIHYGTHFTNMFMRNYPDRITTDRIIKEDRLKLLDD
>Node67
MTELRTFPLPSPTVEGSDADRLLGQALVAAWQVDGIFQIQATPDQDAATGRALDASRAFVGLPLKEKAQYVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVLDRWPCHGPVPWPSVEYAAAMKDYMVVVGDIIGHRLLQLTALGLGLDDDDMEHFTRLTDDGWHHMRVLRFPAADTTSSERGIGSHTDYGLLVIAVQDEVGGLYIRPPVPGETRGRNWLPEESMAGRHENEDPWTTFVPPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLAEAAADDAADDDDAPEADFIHYGTHFTNMFMRCYPDRSTTARIEREDRLAVLERLRKER
>Node81
MSDLQTFALPQDSVTGQPSDIELADQMIRAWRRDGIFQVAMSPEQEGKTERAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVSLEDARVHGRWPCHGPAPWPDEDYRQSMTAFTDELGSVVGERLLELTALGLGLERIDALTRLTRDGWHHMRVLRFPALSSRSAARGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPKVLTVFPGDILQFITNGHLLSTPHKVRLNTRERYSMAYFHEPAFDTCVRPLLGGPPSSSEEPSEEPSEEEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTTLAWLREEAAW
>Node97
LTTFTLPSSPHIAPTPENRHRALGQQMVQTWRKDGIFQIQLTPDQASALRAAFVLSKSYFKRPHHEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPDTLYESAMKRLMAVLGDSSGEKLLQLTALGLNLADNNPNVFTDLTDDGWHHMRVLRFPQADKTNGKGKPPGRGIGAHTDYGLLVIAGQDDVGGLFVRPPTDGETSSRNWEKYSGGLKEDQQESWSSFVPPVENVLTVFPGDMMQYMTNDYLPSTPHKVVLNTRERYAFAYFHEPNFSSVVQPLPEYGGRRDDQGKDVQVEKKKDDGKDKKGIHYGTHFTEMFMRNYPERVTAKRMKAENRLDIL
>Node121
LETFDLPSDRVTGSPGDKAMGQAIIKAWKRDGIFQIAMNSSQREIYKYAENASKKFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSSGEKLLALTELGLGVPQKGSLLQYTQDGWHHMRVLRFPPTNKTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGENYYANWEKSAAGMKEDDEGWVVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGQYNNNNGGQQKPPPPQQKPQEPGIHYGTHFTNMFMRNYPDRITTQRLHAEDR
>Node31
LETFTIPSAEVTGSVSDKAMAKDMISAWRKDGILQIGMTHLQQRLFQDALSHSKRFFRRPLVEKRACVDSRSYSGYIASGEEVTDGVRDYSEIFTVTKDLPLSDRRVMESWPCHGPCPWPDNGFKEVISRYIAELGGCCGQRLLQLIELGLGVPTAGSLTRYTTDGWHHMRVLRFPARHLTNGKGKSSGRGIGSHTDYGLLVIGSQDHIGGLFVRPPREGENFFVNWEKSAAGAKEDDDGWMMYVPPEQGTLTVFPGDIMQYLTNHVLRSTPHKVGLNTAERFAFAYFHEPDFGSVVRPLAGAYDDGGGGQQEPPPPVVEPVEPGFNYGTHFTNMCLRNYPDRITTRRL
>Node104
QLETFVLPSKRVSGSLSDRAMAKAMVNAWRRDGILQIAMSPLQQQIYAQANAASRRFFKKTPAQKQACVNDSSYAGYIASGEEITGGIADYSEIFTVTKDLQPSDPRVMEKWPCHGPCPWLDKRMKSAMTDYMADLGSSSGDKLLQLIELGLDVPQPGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGEQFFANWKQSVAGMKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAVVKPLPGVYNANAASSQQEPPPPVVEPVDEPVVGIHYGTHFTNMALRNYPDRMTTKKLLEDGRYEMLSKEELRS
>Node18
NLQTFVLPSEKVSGSAEDKKLGKAIIDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKSACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLDLDEPRVVAKWPCHGRCPWPDYEMQNPMERYMNSLGESSGETLLKLTELGLDVPKEGSLTDYTKDGWHHMRILRFPAMNKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQKGDDFFANWEASAAGFKEDAEGWLLFVPPAKNVFTVFPGDMMQYLTNSFLPSTPHKVGLNHRERFAFAYFHEPSFQATIKPLPGKYDDAAGGQQEPPPPKKEPKEPGVHYGKHFTDMFMRNYPQRITTQRLLDEGR
>Node19
VAQLETFVLPSERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPGDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSSGDKVLQLIELGLDVPDQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFFANWKQSVAGMKEDEEGWLLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGAFNNGGSSQQEPPPPAAEPADPGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKES
>Node33
QLETFVLPSEQVSGSVSDQAMATAMIDAWRRDGILQVGMSQLQRRLYQAANAASRRFFARPQGEKQACVDSRSYAGYVASGEEITDGVADYSEIFTVTKDLSPRDRRVARRWPCHGPCPWPDKDMKAVVGRYMADLGASSGEKILRLIELGLGVPNTGSLTRYTNDGWHHMRVLRFPPRHRTNGKGKDDGRGIGSHTDYGLLVLAAQDEVGGLFVRPPQQGERFFTNWQQSAAGAKENESGWVVYVPPVPGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLEGEYNNGGGGQQAPPPPVVAPVEAPVVGIHYGTHFTNMALRNYPDRVTTKRLVAEDRYRML
>Node82
MTNLQTFYLPTKSITATQSNIDLGHQMIQSWRTDGIFQIATSTIQDRKVQNAFEASRRFFRMPLGFKSQRISDLTYSGYIASGEEITAGESDYSEIFTLCKDVPLDDVCVQKQWPCHGPVPWPNHDYQQNMTIFMDELGLIIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRVLRFPPISKKSSARGIGAHTDYGLLVIAAQDDAGGLYVRPPVEGEQRNRNWRSDESSAGMYENEEPWTTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSNNPSSSSNNSSNESSNNEYIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLRNAL
>Node53
MRTFELPTVSVGGTTVDRILGRQLVRTWEDDGIFQVAATAEQARVTGEALAESRRFFARSLPEKAKLVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTEGWPCHGPVPWPDDGYRTAMEAHLREVGEIIGEKLLKLVALGLGVEIDRLTRLTRDGWHHMRVLRFPARSATTTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRRRNWLAEESTAGMYQDDDEPWHHFVEPEPAVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGTTVRPLGDDEGETIHYGTHFTNMFTRCYPDRVTTKRILAENRLRLLE
>Node10
LETFVLPSEKVSGSASDRAMGKDMIQAWRRDGILQVAMTPAQQRIYQAANAASRGFFARPHAQKAACVDPQSYSGYVASGEEITDGIADYSEIFTVTKDLPEDDPRVMNKWPCHGPCPWPDQDMKSSIDRYMADLGSSSGEKLLQLIELGLDVPEGSLTDYTNDGWHHMRILRFPARNRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQGEKYAANWEKSAAGHKEDDSGWVVYVPPTPGVFTVFPGDMMQYMTNSFLPSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNAAGGQQEPPPPPAAEPAEEPEGEGIHYGTHFTNMCLRNYPDRITTKRLLEEGRYKMLDD
>Node44
MTNLQTFQLPTQSVTTTSSDIQLGRDMIQAWRQDGIFQIATNSEQHQKTERAFEASKRFFRMPLEQKSRCVSDLSFSGYVASGEEITAGKVDHSEIFTVCPDIPKDDWRVKAQWPCHGPVPWPDQEYELAMKAFMKSLGSIIGDKLLRLVALGLGLGRDDINALTDLTRNGWHHMRVLRFAAAGASASSSRGIGAHTDYGLLVIAAQEDVPGLYIRPPVEGEKRNRNWLPEESSAGMYNDKEPWNNLVTPMPSVLTVFPGDILQFLTNGYLLSTPHKVELNPTRERYALAYFHEPNFEAVVRPLNGGKSSGEGSIHYGTHFTNMFMRCYPERSATRRIISNGSLAVLEEMRE
>Node36
LETFTLPSIKVTGSISDRAMAKAMMSAWQRDGILQINMTRLQQRLYQDALASSRRFFDRPLAEKQACVDSRSYSGYVASGEEITDGIRDYSEIFTVTKDLPLSDERVLDGWPCHGPCPWPDNDFKEVISRYMEELGACCGERLLQLIELGLDVPTAGSLTRYTTDGWHHARVLRFPARHMTNGKGKSGGRGIGSHTDYGLLVMGSQDEVGGLFVRPPREGETFMNWQKSAAGAMEDDDDGWMMYVPPVDGTITVFPGDIMQYLTNHVLRSTPHKVGLNTRQRFAFAYFHEPNFSSTVKPLSGVVYDEPDGGNNQQEPPPPPVVEPVEEPEPVEGINYGTHFTNMCLRNYPDRITTRRL
>Node8
LQTFTLSSSEDMGSAADLSMGKALVAAWQQDGILQISMDAEQQTLYKAANEASKRFFRKPLHQKAACIDSQSYSGYIASGEELTDGLADYSEIFTVTKDLDLDDPRVSAKWPCHGRCPWPDADLRDPVKKYMDSLGQTTGEKLLRLIELGINVPKEGSLTRITKDGWHHMRILRFPASNDINGKGGDGGRGIGSHTDYGLLVIAAADDVGALFVRPPYKDESVVANWERSAAGMKEDSEGWVVFIPPAENVFTVFPGDMMQYMTNSVLPSTLHKVGLNMKERFAFAYFHEPSFQAVIKPLPGEYEEAAGGQQSPPPPEESPEESPGIHYGKHFTNMFMRNYPERITTHRLVDEGRYKLLDD
>Node115
VAQLETFTLPSDRITGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPPAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLTELGLNVPAAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEHYYANWEKSAAGMKEDDDRWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDDVVGGQQSPPPPKKSPKESPKKGIHYGTHFTNMFLRNYPERITTKRMMAENRLEKLQQ
>Node77
MTDLTTFHLPTERITNTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVRENLPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLQLIALGLDLDQDDMDTFTRLTQDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVYEHDDGWNNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEAADAAAAAVAAVAAVPIHYGTHFTNMFMRCYPKRITTRRITENGLLDKLPTLSELAP
>Node35
LETFVLPTDKVSGTLSDRAMAKAMINAWRRDGILQIAMSPAQQRLYEQANAASKKFFKRTPAQKQACVNDSSYAGYIASGEEITAGIADYSEIFTVTKDLQPSDHRVRNKWPCHGPCPWPDKEMKRAITDYMADLGSSSGEKLLQLIELGLDVPQPGSLTKYTQDGWHHMRVLRFPARDRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFAANWDQSAAGMKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNNVRERFAFAYFHEPNFRAVVKPLPGPVYDDAAGGQQEPPPPPVPPVPDEPADGIHYGTHFTNMALRNYPDRISTKKLLEEGRYAMLDSDELRSDDMGAR
>Node60
MTELRTFRLPSEIVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMEASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDNLYRDAMRDFMAMLGEAAGEKILKLTALGFGLDADIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAAEGWTTFVEPVQHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDDPPHGGGDDHGDDDGDDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node32
LLETFVLPSEQVSGSVSDQAMAKAMIDAWRRDGILQVGMSQLQRQLYQSADASSRRFFRRPQGEKQACVDSRSYAGYVASGEEITDGVADYSEIFTVTKDLSPRDKRVAHGGWPCHGPCPWPDREMKAVMGRYMADLGASSGEKLLQLIELGLGVPAGSLTRYTNDGWHHMRVLRFPPRHNTNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFVLPPRQGERFFANWQQSAAGLKENEPGWVYVPPTSGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPVYNPNGGGGQEPPPPPVVPVEEPGIHYGTHFTNMCLRNYPDRITTRRLVAEDRYRMLDSEELRD
>Node72
MAELETFQLPSQSVSGSESDVALGLSLIRAWRRDGIFQLGMSPTQADKARRAFESSRRFFRQPLDAKSRCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRHDMKEHTDELGSTTGERLLRLVALGLGLREVDTLTALTRDGWHHMRVLRFPARSPMAATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRLRNWLPGESSAGMYEHEEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLREEPTRRRDDTRDETRDDEFIHYGSHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLRADAVQRAWTAPVEE
>Node106
QLETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPRAQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDQRVAQGWPCHGPCPWPDQSMKNAMNSYMNDLAVSSGEKLLQLIEMGLDVPDTGSLTRYTDDGWHHMRVLRFPHRDRTNGKGKKKGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQDEQFFANWEKTSAGLRENDAGWMMYVPPAAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGIYNNAAGGQQSPPPPIISPIGPGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLASEQLRDEDDVA
>Node11
ETFVLPSDKVTGSEGDRTLGKAMIDAWRRDGILQIATSPEGQQLQKDAYAASKRFFSKPHQEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPDAEMKEPIKRYMQWLGSEEGEKLLRLIELGLNIPQEGSLTRYTQDGWHHMRILRFPATHLTNGKGKEEGRGIGSHTDYGLLVIASQDDVGGLFIRPPHEDEKYAANWEKSAAGLRENDDGWVVYIPPVPNVHTVFLGDMMQYLTNNYLTATPHKVGLNTRERFAFAYFHEPSFQAVLKPLEGSYAASSGGQQDAGGSSDGSEGGIHYGTHFTNMFMRNYPDRITTQRLLEEGRYEKLQEE
>Node98
LKTFTLPSEKATGSPGDVEMGKALINAWREDGILQIAMNPKQQDLFNKAFAASKRFFALPPNVKANCVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLEEPRVAAKWPCHGPCPWPDIDTKAPIQEYMDSLGSSSGETLLQLIEHGLSLEKPKTLTSLTKDGWHHLRTLRFPQNNKTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPYSDEKLLENWKSSAAGFREHDDRWTTYVPPVPGVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERYAFAYFHEPSFQAEISPIAKALYYDGGKKPPPPPDDPPDEPKIHYGTHFTNMFMRNYPDRVTTERILKEDRLKLLDD
>Node89
MTDLITFALPSEAVTDSKADRQLGRALIDAWRKDGIFQISIHDEQDQKVKRAMAASKAFFKLPREEKSRLVSSLTYAGYVASGEEVTAGEQDQSEIFTICKDLPLEDQRVVDGWPCHGPVPWPNDEYRDSMKAFLEELGDTTGEKLLKLVALGLGLEQINALTQLSQDGWHHMRVLRFPAESAQTTCRGIGAHTDYGMLVIATQDEVGGLYIRPPVDGEHRKRNWLPDESTAGLFENDEPWTTFVKPVPRVFTVFPGDILQFMTDGLLLSTPHKVKLNSRERFAMAYFHEPSFSVSAKPLLSSQSCRRTTSRTESCTTERIHYGTHFTNMFMRCYPQRITTQCILKENRLSHLESIE
>Node7
LTTFALPSQEVVGSQSDLALGQRMIDTWRKDGIFQIFLTPAQGATLRRAFKASKGFFRRPYEEKAKHVDDQSFSGYIASGEELTDGVADYSEIFTVTKDLPVTDPRVQAKWPCHGPCPWPDTNYATTMKNLMSLLGESSGEKLLKLTALGLGLDEDDPNALTKLTEDGWHHMRVLRFPQVDRTNGKGKAAGRGIGSHTDYGLLVIAGQDDVGGLFVRPPVEGEKLLHNWEKSAAGFKENDENWTTYVPPVENVLTVFPGDMMQYLTRDHLPSTPHKVGLNTRERYAFAYFHEPNFSSVVKPLRDYGGHHEEAASNNNNDDSNDKNDKGIHYGTHFTNMFMRNYPERITAKRVKAENRMAILEEIE
>Node117
LKTFELPSLFVSGSSEDIELGKAMIKAWQRDGILQIKMDEQNKRVAEEAFKSSKRFFSLPTSEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNSSMQTTMQNYMDNLGSSSGERMLQLIALGLGLPRDDMNTLTELTRDGWHHLRILRFPKTGDENGKGMKKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYEDEKFFQNWQKSTAGMAENDDRWVVYVPPEDGVFTVFPGDMMQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLKPLGGSSSSEESSEDESVHYGTHFTNMFMRNYPDRITAKRMRAENRMKSLESPELREYE
>Node96
LRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMNGYMTAVGELLGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEEPWTYVIPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLTDTTTTTADTATETAADPEYIHYGTHFTQMFMRCYPERVTTARIETEGRLKVLDRLREDR
>Node118
LKTFALPSDKVSGSAADRATARAMLDAWQGDGILQIAMTPLQKQLYRSADAASRRFFARPSEEKRACVDGQSYAGYVASGEEMTDGIADYSEIFTVTKDLSLADGRVADGWPCHGPCPWPDGTMKAVIDRYMADLGASSGEKLLQLIELGLEIPEAGSLTGYTEDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIANQDHVGGLFVRPPQGGGFFANWQQSAAGMKEDEPGWTTYVPPSPGVYTVFPGDMLQYMTNNTLLSTPHKVGLNTRERFAFAYFHEPNFRSVIRPLPGRHDDGGGGQQEPPPPRREPRQEPRRGVHYGTHFTNMCLRNYPDRVTTKRMVAEDRYPMLALRNDDATTGS
>Node63
MKNLQTFSLPSENVTGSQADIQLGRAMIEVWRRDGIFQVGTTPEQDQIADRAIASSKRFFKKPLEEKKMHISDLTYSGYIASAEEVTAGEADYSEIFTVCKDLPLSDERVKAGWPCHGPVPWPDDEYQESMKDFMNELGDIIGEKLLKLTALGLGLEINALTDLTQDGWHHMRVLRFPQATSESDDRGIGAHTDYGMLVIAVQKDAGGLDIRPPVNANGEREKRNKNWLPGQSTAGEYESDPHWVTTISLVPSVFTVFPGDLLQYMTGDYLISTPHKVTLHTRIERFALAYFHEPNFKASVRPLLGGQQNEEEPPNEPENEPPESIHYGEHFTNMFMRCYPERPTTQRIKKENRLKNLESIREEIR
>Node92
MAELETFRLPSESVSGSDADITLALSMIQAWRKDGIFQVQTTNAQELKVQRAFEASRRFFRQPLAAKARCVSDLTYSGYIASGEELTAGEADYSEVFTVCKDVPLTDARAQARWPCHGPAPWPDEAYRRAMKEHMEELGAVVGEKLLRLVALGLGLDRDDLNALTRLTREGWHHMRVLRFPARSPTAATRGIGAHTDYGLLVIAAQDDVGGLYVRPPIEGESRPRNWLPGESSAGMYEHDERWTTYVKPVPGVLTVFPGDILQFLTRGYLLSTPHRVRLNTRERFAMAYFHEPHFEACVRPLTHHPTGGGDDTGDETGDDEFIHYGSHFTNMFMRSYPDRVTTQRILDEDRLTILAWLRQEAVQRAW
>Node29
KLQTFILPSEEVSGSEEDKKLGKAIVDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMNSLGESSGEILLKLTELGLGVPEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFFSNWEKSAAGFKEDDDGWLLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGPPKYNNAAGGQQEPPPPKKEPKEQEGVHYGKHFTDMFMRNYPERITTQRLLDEGR
>Node87
MNELQTFHLPTEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVVGEKLLKLTALGLELDADDTDALTRLTADGWHHMRVLRFPALSKESRRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTFVRPVPSVLTVFPGDILQFLTDSFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLGSGGVSAAAADACIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLPLLRP
>Node5
VAQLETFTLPSERVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMEYLGESSGEKLLQLTELGLNVPAAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSAAGMKEDDEKWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDDVVGGQQSPPPPKKSPKESPKKGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQQD
>Node120
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFGLDYAKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKETAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFEEAAGGQQEPPPPTTEPTEEPETGIHYGTHFTNMFMRNYPERITADRMRAEGRMALL
>Node110
IGQLQTFVLPSETFTSSASDVKLGRAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFKKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPVQRYMDSLGESSGEMILKLTELGLDLPREGSLTGLTRDGWHHLRVLRFPATHATNGKGKDDGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFFANWERSAAGFKEDDEGWVVFVPPAENVFTVFPGDMMQFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGEYDDVVGGQQEPPPPKKEPKEPKEGIHYGKHFTDMFMRNYPQRITTQRLVEEGRYDMLDE
>Node27
QLKTFTLPSEKATGSPTDIEMGKAMINAWREDGILQVSMNPKQQDLFDKASATSKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSSGETLLKMIEYGLNLEKPETLTSLTKDGWHHLRVLRFPQNNKTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIPPVAKALYYDGGQQPPPPPDDPPDEPTIHYGTHFTNMFMRNYPDRVTTERMLREDRLKLLDD
>Node161
MTELRTFRLPSETVTGSDDDVELGRQLIAAWQADGILQIATDATQDRTTAEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASSGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDDSYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVEHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDGDDDPHGGGGDDHGDDGDGDTIHYGTHFTNMFMRCYPDRVTTRRILEEDRLTVLSELRSSE
>Node195
LQTFCLPSERVTGSQSDRTMGKALVDAWQKDGILQIAMTPRQEALHSQAQQNSQEFFSRPHAEKAACVDSQSYSGYIAASGEEITDGIADYSEIFTVTKDLELGDAKVKAQWPCHGRCPWPDAAMRDVMKEYMNELGNCCGETMLRLLEYGLEIPEGTLTQQTDDGWHHLRILRFPPHNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQDEKYAANWEKSAAGMAEEDEKWTTYVPPVPGVFTVFPGDMMQYMSNSTLPSTPHKVGLNTRDRFAFAYFHEPNFKTTCRPLPGPRYDAPDAAGGQTPPPPRRTPREGIHYGTHFTNMFLRNYPDRITTSRMNRENRYEILKK
>Node165
MTELRTFRVPTDSVTGTDADSALGREMIRAWQTDGIFQVATTPEQRAAIRAAYAASRRFFARPQAWKARHVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDIPADDARVTEGWPCHGPVPWPDEEYRRAMKAYMSGLGSIIGHRLLRLTALGLGLDDDMDAFTRLTHDGWHHMRVLRFPAASASNEERGIGAHTDYGLLVIAAQDDVGGLYIRPPVPGEKRNRNWLAEESTAGMYENEEPWTTYVEPVPGVLTVFPGDIMQFMTGGALLATPHKVRLADRERYALAYFHEPNFQAVARPLADPPPDDDPDDEDAGDPEYIHYGTHFTNMFMRCYPERVTTRRIHAEGRLAVLEELREKAEE
>Node172
MKKLSTFTLPSEETVSDSPAHRALGTQIVASWRKDGIIEVRRDAIQEQETQRAFEENQKFFARPLDEKRRYVDDLTFSGYVASGEEKKSSAKDFPEVFTIFPDISVSDERVRQEEWPCHGPVPWPSEDFQSAMTAYIYTCSSIIGHRLMQLTALGMGLPPLDTFTRLCRRPWNYMRVLQFPAASVENNTRGIGAHTDYDFFVISSQDLEKQGLFVRPPVEGEHRFRNWLPDESMRGVYETQQEPWDLVEANPGVFTVFPGDMMQLVTDGYLLSTPHKVALFTSVRRAMPVFIGPDFDVRLRPHNNKKPDDDDFHYGEHVTNMHVSMYKDRSTSRRIVEKELLS
>Node158
MTTLQTFKLPSAKVTGSTADITLGQKLIRAWQTDGIFQIKTDDEQDRKTQAAIAASKRFFKKPQTFKSGCISDLTYSGYIASGEEITAGQADYSEIFTICKDLPESDPRVRNGWPCHGPVPWPDSVYHKRMKAFMDELGAAAGEKLLKLTALGMNLPRISTFTDLTRDGWHHMRVLRFPTETSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPEESSAGLFEHEEPWTTFVTPTPRVWTVFPGDIMQFMTNGYLLSTPHKVKLNTRERFACAYFHEPGFEASARPLFEEPGSSSQQGSQEGSQQERIHYGEHFTHMFMRCYPDRITTRRIHEQNRLARLEELKARDKRAEELK
>Node146
NLQTFVLPTAAQVTGTEEDKKLGKALVAAWQQDGILQIAMNPEQQALYKAANAASKRFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLDLDDPRVRAKWPCHGRCPWPDADLRDPMKAYMDSLGEESGEKLLQLIELGLNVPQGSLTNLTQDGWHHMRILRFPATNNDTNGKGKDDGRGIGSHTDYGLLVLAAADDVGAALFIRPPRQNESFFANWETSAAGFKEDDDEGWVVFVPPMENVFTVFPGDMMQYMTNSALPSTPHKVGLNNTRERFAFAYFHEPSFQAVIKPLPGRADPDAGGQQEPPPPRREPREPREGVHYGKHFTDMFMRNYPERITTQRLVSEGRYKLLEE
>Node171
MTELQTFVLPSSTVDGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVAGKWPCHGPAPWPSQQYADAMKDYMGTVGDIIGERLLRLVALGLGLDEDDMDHFTKLTEDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGDTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGGEGEDDEGDDEGGGGEEFIHYGTHFTNMFMRCYPERTATARIENEDRLTVLERLRKEAERLTS
>Node148
MADLQSFHLPTESISGTQSDVDLGREMIQAWRTDGAFRIAMSEIQRKKSEDAFAASRRFFRGSIEFKSQYLNDLTYSGYTASGEEVTAGEKDYPEVFTICRDIPVDDARVRAHWPCHGPVPWPNAEYRESLQTYLNELGSVVGDRLLQLVALGLELNKDDIDALLALAKDGWHHLRALRYPVASQESSNRGLGAHTDYGLIVITDEDDVGGLYIRPPVEGEKRNRNWLATESTAGMYENEEDWIIFAKPVPNAFTVFPGDILQFITDNHILATIHKVTLNTRERFSMAYFHEPDFNACVYPLSNNPSSGGEESGEDSDEEDYLFYGEHFTNMFMRCYPDRTTTRRIMDENRLSVLTDLKNTD
>Node192
QLKTFTLPSEKATGSPRDVEMGKAMINAWREDGILQVSMSSKQQDLFDKASAASKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEARVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSSGETLLQMIEYGLSLDKPQTLTSLTKDGWHHLRVLRFPQNNKTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKALYYDDGGQQPPPPPPDDPPDEPPDEKIHYGTHFTNMFMRNYPDRVTTERILREDRLKLLDD
>Node162
MTELRTTFELPTDDSVSGTTVDRILGRQLVRTWETDGIFQVAATAEQERITGEALAASRRFFARSLPEKAKFVSDLTYSGYIASSGEEVTAGEADYSEIFTVCPDVPLTDTRVTEGWPCHGPVPWPDEGYRTAMKAYLREVGEIIGEKLLKLVALGLGVIDRLTRLTRDGWHHMRVLRFPARSATTEERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRNRNWLPEESTAGMYQDEEPWHHFVEPEPRVFTVFPGDILQFLTGGRLLSTPHKVRLATRERYTLAYFHEPSFGATVRPLGDGGGGGGGGEGERIHYGTHFTNMFTRCYPDRVTTQRIHEEGRLAVLDRLRARSDR
>Node130
LETFILPSERVSGSVSDKAIADAMIGAWRRDGILQIGMSKPQQRLYQAANAASRRFFAQPHALKQACVDDKSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTRYMNDLAESSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGIHNNAAGGQQSPPPPPIISPIEPGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLEE
>Node151
MADLRTFGLPTDCVVGTDSDITLAQQMIDAWQTDGIFQIACDPRQAQKVQEAFEASRRFFALATEEKARCVSPLSYAGYIASGEEVTAGEADYSEIFTVCQDIPLDDARVQANWPCHGPVPWPSEQYRRTMQAYMDEMGLIIGEKLLRLIALGLDLPEEDIDTLTRLTENGWHHMRVLRFPARSSQTTARGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLDSESSAGMYENEEPWTTFVKPVADTLTVFPGDILQFLTGGALLSTPHKVRLNDRERYTMAYFHEPNFDAGIRPLSGGSDSSSDDDSDADSDDYIHYGTHFTNMFMRCYPDRITTRAILEADRLSVLARLAERAAR
>Node177
MAELETFRLPSHVVSGSEADVALGLAMIRAWRRDGIFQIRMSPAQADKARHAFELSRRFFRQPLEEKQRCVSDSTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSGWPCHGPAPWPDEAYRLGMKAHTEELGSAAGERLLRLAALGLGLKDIDALTALTKDGWHHMRVLRFPARSPATTTRGIGAHTDYGMLVIAAQDDVGGLWVRPPVEGEQRKRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLASRERFAMAYFHEPHFEACVRPLKDDPTRRRDDTRDETRDDEFIHYGTHFTHMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAW
>Node138
LGERIVAALQRDSLIQIKKNYQEDLLTAQALLQNKKFNRLPQDEKIKFVSDTTYTGYVFSGEEKKGGQYDQVEVYTVFPHFDRDDERVKTLRLPCHGEAIYPSVDYKNAIEAYMKATGNMMCDQLIEIIGLGLGLEPPNHYLSEMVRDGFHHARVLRFKSQKAAGAASNGISAHSDYGFLIAASQDDVGGLWVRPIVNGEKRGRNYLEEETTIGAYNDQPGWAAFVTPMPSVWTVFVGDMMELATQRKLKANIHKVKLNEEKERFAIAYFHEPKFNQTFEAVNSSRGKKKKKGKKKGKKKEIVYGKHLTNMYTKAYPNRSTTKRIESEELCKNIDRISKIS
>Node188
VAQLETFTLPSERVTGSPGDRAMGKALVDAWRRDGILQVSMNDKQQHIFNEATASSKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTQDGWHHMRILRFPPTHNTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGMAEEDDRWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGPYDDVVGGQQSPPPPKKSPKEPEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLEE
>Node170
MTTLRTFQLPSAEVTGSIWDIALGVELVRTWQSDGIFQVATGSVANAVTAAALRESRRFFALPLAEKAALVSDLTYAGYVASGEEVTAGEQDFSEIFTVCKDVPVTDPRVRAGWPCHGPVPWPDPVYRDVMTALMSQFGLIIGDKILKLTALGLGLADMNALTSLATDGWHHMRVLRFPARSAATTTRGIGAHTDYGMLVIATQDDVGNGLYVRPPVPGEKRPRNWLDHESSAGAFENEEEPWHLVDPVPGTFTVFPGDILQFLTGGELLSTPHKVKLADRERYSIAYFHEPDFAAALRPLGGAGGGGDGAGGDEAGGEPEVIHYGTHFTSMFMRCYPDRVTTRRILAEDRLAVLAARMREEARR
>Node176
MGELATFRVPTARITGTTADIMLGGAMVRAWQRDGIFQVAADGRQEARTRAALAASRRFFARPAAEKAACVNDTSYSGYIASGEEVTAGRPDASEIFTITPDIPAAAGLPCHGPVPWPSPGYRAAMEAYLAGVGGLLGERLLRLVALGLGLGRGPAGIDRLAGLTRGGWHHMRVLRFPAAGRTTTERGIGAHTDYGLLVIAAQDEVGGLFVRPPVAGERRPRNWLPGESTAGFAEDRPGWTTPVRPVPRVLTVFPGDILQFMTGGALLSTPHRVRLADRERYALAYFHEPRFGARIRPLDEGGGGGGGAAAGGGGGAVIHYGEHFTRMFMRCYPQRATTRRIEAQGLLRHLAGAR
>Node152
MTNLQTFELPSTEVTGSAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQEAMAASKQFCKEPLTFKSSCVSDLTYSGYVASGEEVTAGKPDFPEIFTVCKDLPVSDQRVKAGWPCHGPVPWPNNTYQKSMKAFMEELGLAAGERLLKLTALGFELPRINTFTDLTRDGWHHMRVLRFPPQTSTLSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHDEPWTTFVTPTPGVWTVFPGDILQFMTGGQLLSTPHKVKLNTRERFACAYFHEPNFEACAYPLFEEPSAAANNSANESANNERIHYGEHFTNMFMRCYPDRITTQRIHKENRLAHLEDLKKYSDTRATGSEDLK
>Node180
LQTFELPNESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPEANNNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFFANWKESAAGKHEDAADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGIFEPEAARREEDPPPPIDPISPDGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLEE
>Node147
NLQTFILPTDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVIAAADDVGGNDEKLAANWESSAAGFREDDERWVVFVPPAENVFTYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGKYDDVVGGQQEPPPPPKKEPKDEPKKDGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLEE
>Node182
VAQLETFTLPSERVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMSEPRVRAKWPCHGPAPWPDYDMKQPMTTYMDYLGESSGEKLLQLAELGLNVPDAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGLKEDDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGQYDPDVVGGQSPPPPQQSPQESPGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQQ
>Node139
LDTFCLPSDKVTGSASDRAMAKAMIEAWRRDGILQIAMSPQQQRIYQNANAASRRFFRRPSAQKQQCVDEKSYAGYVASGEEITDGVADYSEIFTVTKDLAEDDPRVAGKWPCHGPCPWPDEDMKSNMTAYTDDLSQSSGDQLLEMIELGLHVPDQGSLKRYTDDGWHHMRVLRFPQRDRTNGKGKKKGRGIGSHTDYGLLVMAAQDEVGGLFVRPPHQGEHYYANWEKSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMMQYMTNSFLKSTPHKVGLNTRERFAFAYFHEPNFASVIKPLPGYDDSSGGADAAAASAAEAGIHYGTHFTNMCLRNYPDRITTKKLLQEGRCKML
>Node150
MNKIITFQIPIPTIDVIQSNIDLGLDIIDAWRRDGIIQIALDDTQDAITQNAFKASQGYFALPHSEKSSHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPADDARVQQQWPCHGPVPWPNKGFQRDMALFTDQLGAIIGEKLLRLVALGLHSLGRLDRDDIDGLNRLTRDGWHHMRVLRFPCATLGNQQSRGISSHTDYGMLVIAAQDDVGGLYIRPPVEGEVRRRNWLSEESAAGMYENEEPWTTFVQPTPRVVTAFPGDIMQFLTGGYLLSTPHKVKLNTRERYALAYFHEPNFKAVIEPLRPPEPEEEDDPEDGPEDDGRIHYGSHFTNMFMRSYPQRITTQRILRENRL
>Node157
LRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLDADDPDHFTRLTADGWHHMRVLRFPPADAGSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTTPTTDTTPTADAADPAYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEDR
>Node134
LRTFVLPNDTVNANDESTSMAKAMIKEWRETGIFQIQMTPAQQAMLTECFETSKEFFRRSHEEKSKHVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPESDVRVQEKWPCHGPTPWPSETYKDQMDQLMKLLGDSSGEKLLKLVGLGLGLKEDDPDALLNLTEDGWHHMRILRFPHVDKTNGKGKQGGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIQGESIIKNWQSSAAGLNENDDKWLLFVPPVPNTLTVFPGDMMQLITNDYLPSTPHKVGLNYDERYAFAYFHEPNFSSVIKTLPEFDEAEKVDKSEMIHYGTHFTDMFMRNYPERITADRIREEGRLDIL
>Node184
ELDTFELPSVAVTGSDADRLLGRRLVEAWRKDGIIQVAAFPENLRVLSEAVKQSKQYFLQDHATKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPNDAYAHAMRKLMDFKAGIIGERLLQLTALGLDLKDDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEGGRGIGSHTDYGLLVIASQDEVGGLFVRPPSADETYYSNWKKSSAGLHENDDKWLLYIAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNNTRERFAFAYFHEPNFNAVCRRLPEPPEFKKTTGGTEEDEEEEEEEEEEAVHYGTHFSNMFLRNYPDRITAHRMREENRMDLLPMLKP
>Node153
LQTFRLPTAAVRGTHNDRALGRALVAAWQSDGIFQIRATRTQQEATERALAESRTFCRRALKEKARHVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPVPWPSPGYARAMKDYMTVVGEIIGGRLLRLTALGLGLDTDDPDRFTRLTTDGWHHMRVLRFPPADATSSERGIGAHTDYGLLVIAAQDDVGGLHIRPPVPGEQRGRNWLPGESMAGRYEHEEPWTTYVTPVAAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERHTLAYFHEPAFDAVARPLDDPDSASAADSAAEAPPDAEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLREDR
>Node149
MTGLTTFHLPSERIIHSEAHRQLGQDMVAAWRADGIFQIALSTPQQHTTDEAFAQSRRFFELDFETKRRHVSAFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFFGERLLQLIALGLGLDRDDMETFTRLTRDGWHHMRVLRFPTVQSSENNARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLAAESTAGMFEHDDDGWTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKAADADTDDDVVADVAADAVPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSEVAP
>Node174
MTDLTTFHLPTERITNTEAHRELGQAMVKAWRTDGIFQIALSKPQEQTTDEAFAESRQFFSQDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVREDLPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLQLIALGLDLDDDDMDTFTRLTDDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGVFEHDDGWNNFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLEAADADAAAAAAAAAAAAPIHYGTHFTNMFMRCYPKRITTQRITEKGLLDKLPTLSELAP
>Node143
VGQLETFTLPSESVKGSRGDRAMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESSGEKLLQLTEYGLGVPEGSLTQYTNDGWHHMRILRFPQNNKTNGKGKEEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQFFANWEKSSAGMKEEDDRWVYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGPYDPDAAGGQQSPPPPKKSPKESPGIHYGTHFTNMFMRNYESRITTQRLLAEDRYSLL
>Node127
FETFVLPSEKVDGSPSNHALGKAMVEAWRRDGILQVAMSGQQQQVYREAQQRISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGIPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLAANNGERMLQLIEFGLHVPGPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGEQYANWEKSSAGLLEDEESGWVFVPPVPGVFTVFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGPYNNRRAAQQPPPPPVVPPVDPVVGIHYGTHFTNMFWRNYPDRLTTRRLEEEGR
>Node141
LETFTLPSERVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRVYQAASAASRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQEMKAVMNKYMEDLAVSSGEKLLQLIEMGLDIPATGSLTKYTADGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRETERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGPYNNAAGGQSPPPPLLSPLEPGIHYGTHFTNMFLRNYPDRITTARLQEEGRYQLLESKELRDDE
>Node140
NLKTFELPGEVLTGGASDQALGRSMIDAWRKDGILQISMDPINRKLADAAFICSKKFFDLPYKTKAACVDDQSFAGYIASGEELTDNIADYSEIFTVTKDLPATDARVQQKWPCHGPCPWPFKQMRTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTKYTQDGWHHMRVLRFPERNNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEEVVANWRKSAAGMAEDDDKWVVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGRFDDAAGGQQEPPPPRREPREPGIHYGTHFTNMFMRNYPERITANRMRSENRMKLLEE
>Node185
LQTFELPTAQVTGTPKDVELGKALIDAWRSDGILQISMKDTKKSLMDEVFDASKGFFRKPHSEKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEPRVRNHWPCHGPCPWPDLEMKDPMEQYMEYLGQSSGEKLLQLIECGLGVPNPGSLTNYTNDGWHHMRILRFPAVNNTNGKGKEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPSADEATTANWEQSAAGFHEDDSGWVVYVPPMPDVFTVFPGDMMQYLTDSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVLRPLPGRYDDGGGGQQKPPPPRRKPREPGIHYGTHFTNMFMRNYPDRITSKRMIREDRQALL
>Node155
MTELQTFVLPSPVVDGSDVDKALGQALIAAWQADGIFQIQATPEQEAATERALEASRGFFGRPFEEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVVDKWPCHGPAPWPSEQYADAMKDYMGAVGDIIGERLLRLVALGLGLDEDDMDHFTKLTEDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVPGETRGRNWLPDESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGGGGPDDGGDDEGGGGPEFIHYGTHFTNMFMRCYPDRTATARIENEDRLAVLERLRKEAER
>Node135
VGNLQTFELPSESLNGSVADRALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPQTNKVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGAFEEAAGGQQEPPPPAAEPAEEPAAGIHYGSHFTNMFMRNYPERITADRMRDEGRLALLDD
>Node159
MTGLTTFHLPSERILHSEAHRQLGQDMVAAWRADGIFQIALSTPQQHTTDEAFAQSRRFFELDFETKRRHVSEFTYSGYIASREEITAGEADYSEIFTICPDIGMDDARVREGWPCHGPVPWPGAAYRDRMKDFTGMLGTFFGERLLQLIALGLGLDRDDMETFTRLTRDGWHHMRVLRFPTVQSSSENNARGIGAHTDYGLLVIAAQDDVGGLYVRPPIAGERRNRNWLASESTAGMFEEHDDGWTTFIKPEPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKKAADADDTDDDVVADVADAPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPALSEVAP
>Node163
MAELETFQLPSQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEDKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKEHTDELGSTTGERLLRLVALGLGLEREVGALTALTRDGWHHMRVLRFPARSATAATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTTPPGPPPDDGPDEPDDYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLREAW
>Node167
MADLQTFQLPTDSVDGTESDIQLAQELIQAWRTDGILQVATDPAQDQRTKAAIESSRRFFGMPAEFKASCISDLTYSGYIASGEERTAGEADYSEIFTVCPDVPLDDPRVRAQWPCHGPVPWPDDEYERAMTAFMAELGSIIGEKLLKLTALGLGLRDIDALTDLTRDGWHHMRVLRFPARSEHTTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVDGERRSRNWLPGESMAGRYENEEPWTTFVKPVPSVLTVFPGDMLQLITDGFLLSTPHKVRLNTRERFALAYFHEPNFQARVRPLSGGGGDDDGGGDGDGDDDYIHYGSHFTSMFMRSYPERVTTRRILAEDRLSRL
>Node181
VGHLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPVFEAEAAGGQQEPPPPPVVEPVEEPEVGIHYGTHFTNMFMRNYPERITADRMRAEGRMALL
>Node128
QLQTFVLPGDKVTGGDLDKKLGKAIVKAWQKDGILQIAMKESQQALYKAANQASQRFFKKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVVEKWPCHGRCPWPDADLQNPMKQYMDSLGESSGETLLKLTELGLDVPQEGSLTGLTQDGWHHMRILRFPATHATNGKGKDDGRGIGSHTDYGLLVLAAADDVGALFVRPPQQSDEFFANWNKSAAGFKEDDDGWMMFVPPAENVFTVFPGDMMQYLTNSVLPSTPHKVGLNVRERFAFAYFHEPSFQAVIKPLPGKYDDAAGGQQEPPPPKKEPKEEPEKGVHYGKHFTDMFMRNYPQRITTQRLIDEGRYNLLEE
>Node166
MTDLQTFHLPTESITGTQSDIDLAREMIQAWRTDGIFQVATNAIQDRKTESAFEASKRFFRMPMEFKSQCISDLTYSGYIASGEEVTAGQADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDQEYQQSMKAFMDELGSIIGQKLLKLTALGLGLDDDINALTKLTEDGWHHMRVLRFPAASQKSAARGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWLPTESSAGMYENEEPWTTFVKPVPKVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFEEPPSSSSDDSSDESSDDEYIHYGTHFTNMFMRSYPDRITTRRIINEDRLSNLARVKNAR
>Node160
MTELQTFQLPSPTVEGSDADRLLGQALVAAWQADGIFQIQATPDQDAATGRALDASRAFVGRPLKEKAQHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVLDRWPCHGPAPWPSEEYAAAMKDYMAVVGDIIGHRLLQLTALGLGLDDDMEHFTRLTDDGWHHMRVLRFPAADATSEERGIGSHTDYGLLVIAAQDEVGGLYIRPPVPGETRGRNWLPEESMAGRYENEDPWTTFVTPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLAEAAADDAADDEDAPEAEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLAVLERLRKER
>Node168
MTDLQTFALPQDSVTGQPSDIELADRMIRAWRRDGIFQVAMNPTQEGKTERAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVSLEDARVRAQWPCHGPAPWPDEDYRQSMTAFTDELGSVVGERLLKLTALGLGLERIDALTRLTRDGWHHMRVLRFPALSSKSAARGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPKVLTVFPGDILQFITNGYLLSTPHKVRLNTRERYSMAYFHEPAFDACVRPLMGGPPSSSEEPSEEPSEEEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLSTLAWLREEAAW
>Node193
LETFDLPSDRVTGSASDKAMGQALIKAWKRDGIFQISMNSSQREIYKYAENASKRFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKSYMNYLGDSSGEKLLALTELGLGVPKGSLLQYTQDGWHHMRILRFPPTNKTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGESYAANWEKSAAGMKEEDDEGWVYVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRDRFAFAYFHEPSFQAVVKPLEGQYNNNGGQQKPPPPQQKPQEQEGIHYGTHFTNMFMRNYPDRITTQRLHAEDR
>Node142
LETFTLPSAKVTGSISDKAMAKDMISAWRRDGILQIGMTRLQQRLYQDALASSRRFFSRPLVEKQACVDSRSYSGYIASGEEITDGVRDYSEIFTVTKDLPLSDKRVMDGWPCHGPCPWPDNGFKEVISRYIAELGACCGQRLLQLIELGLDVPTAGSLTRYTTDGWHHMRVLRFPARHRTNGKGKSSGRGIGSHTDYGLLVIGSQDNVGGLFVRPPREGESFFVNWQKSAAGAKEDDDGWMMYVPPEQGTFTVFPGDIMQYLTNHVLRSTPHKVGLNTRERFAFAYFHEPNFSSTVKPLAGAYDDGGGGQQEPPPPVVEPVEPGIHYGTHFTNMCLRNYPDRITTRRL
>Node136
NLQTFVLPSEKVSGSAEDKKLGKAIIDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKSACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMERYMNSLGESSGETLLKLTELGLDVPEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDDFFANWEKSAAGFKEDSEGWLLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNHRERFAFAYFHEPSFQAVIKPLPGKYDPDAAGGQQEPPPPPKKEPKEEPKGVHYGKHFTDMFMRNYPQRITTQRLLDEGRYDLLKK
>Node137
VAQLETFVLPSERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSSGDKVLQLIELGLDVPDQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFFANWKQSVAGMKEDEPGWLLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGAFNNGGSSQQEPPPPAAEPADPGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKES
>Node169
MTNLQTFYLPTKSITATQSNIDLGHQMIQSWRTDGIFQIATNTIQDRKVQNAFEASRRFFRMPLGFKSQHISDLTYSGYIASGEEITAGESDYSEIFTLCKDVPLDDVCVQKQWPCHGPVPWPNHDYQQNMTTFMDELGLIIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRVLRFPPVSEKSSARGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEQRNRNWRSDESSAGMYENEEPWTTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSNNPSSSSNNSSNESSNNEYIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLRNAL
>Node145
LETFTLPSIKVTGSISDRAMAKAMMSAWQRDGILQINMTRLQQRLYQDALASSRRFFDRPLAEKQACVDSRSYSGYVASGEEITDGIRDYSEIFTVTKDLPLSDKRVLDGWPCHGPCPWPDNDFKEVISRYMEELGACCGERLLQLIELGLDVPTAGSLTRYTTDGWHHARVLRFPARHMTNGKGKSGGRGIGSHTDYGLLVMGSQDEVGGLFVRPPREGETFMNWQKSAAGAMEDDDDGWMMYVPPEEGTITVFPGDIMQYLTNHVLRSTPHKVGLNTRQRFAFAYFHEPNFSSTVKPLSGVVYNEPNGGNNQQEPPPPPVVEPVEEPEPVEGINYGTHFTNMCLRNYPDRITTRRL
>Node144
LETFVLPTEKVSGTLSDRAMAKAMINAWRRDGILQIAMSPAQQRLYEQANAASRKFFKRTPAQKQACVNDSSYAGYIASGEEITAGIADYSEIFTVTKDLQPSDHRVRNKWPCHGPCPWPDKRMKSAMTDYMADLGSSSGEKLLQLIELGLDVPQPGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFAANWDQSAAGMKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNNVRERFAFAYFHEPNFRAVVKPLPGVVYDDAAGGQQEPPPPPVEPVPDEPVDGIHYGTHFTNMALRNYPDRISTKKLLEEGRYAMLDSDELRSDDMGAVR
>Node156
MTELRTFRLPSEIVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDDLYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDGDDDPHGGGGDDHGDDDDDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node194
LLETFVLPSEQVSGSVSDQAMAKAMIDAWRRDGILQVGMSQLQRQLYQSADASSRRFFRRPQGEKQACVDSRSYAGYVASGEEITDGVADYSEIFTVTKDLSPRDKRVAHGGWPCHGPCPWPDREMKAVMGRYMADLGASSGEKLLQLIELGLGVPAGSLTRYTNDGWHHMRVLRFPPRHNTNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFVLPPQQGERFFANWQQSAAGAKENEPGWVYVPPTPGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPVYNPNGGGGQEPPPPPVVPVEEPGIHYGTHFTNMCLRNYPDRITTRRLVAEDRYRMLDSEELRDR
>Node164
MAELETFQLPSQSVSGSESDVALGLTMIRAWRRDGIFQLRMSPTQAEKARRAFESSRRFFRQPLDAKARCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRQDMKEHTDELGSTTGERLLRLVALGLGLREVDTLTALTRDGWHHMRVLRFPARSPMAATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRHRNWLPGESSAGMYEHEEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLREEPTRRRDDTRDETRDDEFIHYGSHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAWTAPVEE
>Node183
QLETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPHAQKQACVDDMSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQSMKNVMNSYMNDLAVSSGEKLLQLIEMGLDVPDTGSLTRYTDDGWHHMRVLRFPHRDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQDEQFFANWEKTSAGLRENDAGWVVYVPPTAGTFTVFPGDMMQYMTNNFLKSTPHKVGLNVRERFAFAYFHEPNFRSVIKPLPGIYNNAAGGQQSPPPPIISPIGSPIIGIHYGTHFTNMFLRNYPDRVTTARLREEGRYHLLGSEQLRDEDDVG
>Node133
LETFVLPSDKVTGSEGDRTMGKAMIDAWRRDGILQIATSPEGQQLQKEAYAASKRFFSKPHQEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPDAEMKEPIKRYMQWLGSEEGEKLLRLIELGLNIPQEGSLTNYTQDGWHHMRILRFPATHLTNGKGKEEGRGIGSHTDYGLLVIASQDDVGGLFIRPPYEDEKYAANWEKSAAGLRENDDGWVVYVPPVPNVHTVFPGDMMQYLTNNYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLEGSYAAASGGQQDAAPPSSDASEPGIHYGTHFTNMFMRNYPDRITTQRLLEEGRYEKLQEE
>Node179
LKTFTLPSEKATGSPGDVQMGKAMINAWREDGILQVSMNPKQQDLFNKAFAASKRFFAKPPNQKAACVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDIDMKAPIQQYMDSLGNSSGETLLQMIEYGLSLDKPKTLTSLTKDGWHHLRTLRFPQNNNTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDDRWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKDLYYDGGKKPPPPPDDPPDEPDEKIHYGTHFTNMFMRNYPDRVTTERILKEDRLKLLDD
>Node175
MTDLTTFTLPSEAVTGSQADRQLGRALIEAWRKDGIFQISTHDEQDQKVQRAMAASKAFFKQPRQEKSRWVSDLTYAGYVASGEEVTAGEQDQSEIFTICKDLPLDDQRVKDGWPCHGPVPWPNDEYRDSMKAFLAELGDTTGEKLLKLVALGLGLEQINALTQLSQDGWHHMRVLRFPAESAQTTCRGIGAHTDYGMLVIATQDEVGGLYIRPPVEGEHRKRNWLPGESTAGLFENDEPWTTFVKPVPRVFTVFPGDILQFMTDGYLLSTPHKVKLNTRERFAMAYFHEPSFNASARPLLSSQSNNNGGSNGESNGGERIHYGTHFTNMFMRCYPERITTQRIHKENRLRHLEEIKE
>Node131
LTTFTLPSQEVVGSQSDIALGQRMIDTWRKDGIFQILLTPAQGATLRQAFKASKGFFSRPYEEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLPVTDPRVQAKWPCHGPCPWPDTNYATTMKKLMSLLGESSGEKLLKLTALGLGLDEDDPNALTKLTEDGWHHMRVLRFPQVDRTNGKGKAAGRGIGSHTDYGLLVIAGQDDVGGLFVRPPVEGEKLLHNWEKSAAGFKENDENWTTYVPPVENVLTVFPGDMMQYLTRDYLPSTPHKVGLNTRERYAFAYFHEPNFSSVVKPLRDYGGHHEEAASNSSSDDSSDKADKGIHYGTHFTNMFMRNYPERITAKRVKAENRMAILEELE
>Node190
DLKTFELPSPFVTGSSEDIALGKAMIDAWQKDGILQIKMDPQNKRVAEKAFKASKRFFSLPSSEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNSAMQTTMQGYMDYLGSSSGERMLQLIALGLGLPRDDLNSLTNLTRDGWHHLRILRFPKTGDENGKGMKKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKFFQNWQKSTAGMAENDDRWVVYVPPEDGVFTVFPGDMMQFITNNHLPSTPHKVGMNTRERFAFAYFHEPNFNAVLKPLEGSSSSDDSSDEDSVHYGTHFTNMFMRNYPERITAKRMRAENRMDSLESMKE
>Node178
LRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMNGYMTAVGELLGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADARSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEEPWTTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTDTTTTTADTATEDTADPTYIHYGTHFTRMFMRCYPERVTTARIETEGRLKVLDDRLREDRR
>Node191
LETFVLPSEKVSGSVADQAMARAMIDAWQRDGILQIGMSPLQRRLYQSADAASRRFFARPQAEKQACVDSQSYSGYVASGEEITDGIADYSEIFTVTKDLSHSDKRVANGWPCHGPCPWPDQTMKSVISRYMADLGASSGEKLLQLIELGLDVPAGSLTGYTNDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPQGDSFFANWQQSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMLQYMTNNVLLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNGGGGQQEPPPPPTTPPTEEPAGEGIHYGTHFTNMCLRNYPDRVTTKRLVAEDRYQMLELR
>Node187
KLQTFILPSEEVSGSEEDKKLGKAIVDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMKNPMERYMNSLGESSGEILLKLTELGLGVPEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFFSNWEKSAAGFKEDDEGWLLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGPPKYNNAAGGQQEPPPPKKEPKEQEGVHYGKHFTDMFMRNYPERITTQRLLDEGR
>Node129
VAQLETFTLPSERVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMEYLGESSGEKLLQLTELGLNVPAAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSAAGMKEDDEKWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDDVVGGQQSPPPPKKSPKEPKGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQQ
>Node186
IGQLQTFVLPSETFTSSASDVKLGKAIVAAWQKDGILQIAMKPNQQATYKAANAASKRFFKKSHAQKAACVDSQSYAGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVRAKWPCHGRCPWPDVEMQEPVQRYMDSLGESSGEMILKLTELGLDLPREGSLTGLTRDGWHHLRVLRFPATHHATNGKGKDGGRGIGSHTDYGLLVLAAADDVGALFVRPPEKDENFFANWERSAAGFKEDDDEGWVVFVPPAENVFTVFPGDMMQFLTNSVLPSTPHKVGLNIKERFAFAYFHEPSFQAVIKPLEGYDGDVVGGQEPPPPKKEPKEPGIHYGKHFTDMFMRNYPQRITTQRLVEEGRYDMLE
>Node245
LQTFVLPSERVTGSASDRAMGKALVDAWQKDGILQIAMTPEQQAIYSEAQAASKRFFSKPHAQKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLELDDARVLAKWPCHGPCPWPDAEMRDPMKEYMNYLGESSGEKLLQLLELGLNVPEEGSLTNLTEDGWHHLRILRFPATNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQDEKYYANWEKSAAGMAEDDDKWVVYVPPVPGVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQSTIKPLPGPYDDAAGGQQEPPPPRREPREEPERGIHYGTHFTNMFLRNYPDRITTQRMAEENRYELLKK
>Node226
MTELRTFSLPSETVTGSAADRALGREMVAAWRTDGIFQVAADPAQQQATQAAFEASRRFFALPLAEKARYVSDLTYSGYVASGEEVTAGEADYSEIFTVCPDVPVDDARVRAGWPCHGPVPWPGPHYRRAMTAFMSALGSIIGERLLRLVALGLGLDDPDTLTRLTADGWHHMRVLRFPARSARTAARGIGAHTDYGLLVIAAQDEVGGLYVRPPVEGEKRSRNWLPEESTAGMYENEEPWTFVRPVPGVFTVFPGDILQFLTGGYLLSTPHKVALAADRERHAMAYFHEPSFDATLRPLAGDDDPAGGGGGGGGGDGGDPPLHYGTHFTNMFMRCYPDRPTTRRILDEGRLAVLALRER
>Node217
MTNLQTFKLPSETVTGSTADISLGQKLIQAWQTDGIFQIKTDEEQDRKTQAAIAASKRFFKKPMTFKSSCISDLTYSGYIASGEEVTAGKADYSEIFTICKDLPESDPRVKAGWPCHGPVPWPDDTYQKSMKAFMDELGLAAGEKLLKLTALGLKLPRISTFTDLTRDGWHHMRVLRFPTETSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGLFEHEEPWTTFVKPTPKVWTVFPGDILQFMTNGYLLSTPHKVKLNTRERFACAYFHEPNFEAVARPLFEEPSSSSNNSSNESSNRERIHYGEHFTHMFMRCYPDRITTQRIHKENRLAHLEELKKSDTRAEKLK
>Node209
NLQTFVLPTAAQVTGTESDKKLGKALVAAWQQDGILQIAMNPEQQALYKAANAASKRFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDDLDLDDPRVLAKWPCHGRCPWPDADLRDPMKAYMDSLGEESGEKLLQLIELGLNVPQGSLTNLTQDGWHHMRILRFPATNDTNGKGKDDGRGIGSHTDYGLLVIAAADDVGALFIRPPRQNESFFANWETSAAGFKEDDDEGWVVFVPPVENVFTVFPGDMMQYMTNSALPSTPHKVGLNNTRERFAFAYFHEPSFQAVIKPLPGRADPDAAGGQQEPPPPPRREPREQEPREGIHYGKHFTDMFMRNYPERITTQRLVSEGRYKLLEE
>Node225
MTELQTFVLPSSTVDGSDTDKALGQALIAAWQADGIFQIQATPEQEAATERALDASRGFFGRPFKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVVDKWPCHGPAPWPSEQYADAMKDYMGTVGDIIGERLLRLVALGLGLDEDDMDHFTKLTEDGWHHMRVLRFPRADATSSERGIGSHTDYGLLVIAVQDDVGGLYIRPPVEGETRGRNWLPGESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTIAYFHEPSFQAVARPLGGGGPDDGGDDEGGGGPEFIHYGTHFTNMFMRCYPERTATARIENEDRLAVLERLRKEAER
>Node211
MSDLQTFHLPTESITGTQSDIDLARQMIQAWRTDGAFQVAMSKIQRKKSENAFAASRRFFRMPIEFKSQFINDLTYSGYTASGEEVTAGEKDYSEVFTICKDIPIDDVRVQAHWPCHGPVPWPNAEYKQSMQAYMDELGSIIGDKLLKLVALGLELDKDDIDALTKLAKDGWHHLRALRYPALSQESSTRGIGAHTDYGLLVITDQDDVGGLYIRPPVEGEKRNRNWLSTESTAGMYENEEPWIIFVKPVPSVFTVFPGDILQFITNNYLLSTLHKVRLNTRERFSMAYFHEPDFNACVRPLSNNPPSSSSSEESSEESSEEEYIFYGEHFTNMFMRCYPDRTTTRRIMDENRLSVLATLKNAT
>Node242
QLKTFTLPSEKATGSPSDVEMGKAMINAWREDGILQVSMSPKQQDLFDKASAASKRFFAMPPTQKAACVDTQSYAGYIASGEEITDGVADYSEIFTVTKDLPLDEPRVKAKWPCHGPCPWPDSDMKTTIQQYMDSLGNSSGETLLQMIEYGLSLDKPQTLTSLTKDGWHHLRVLRFPQNNKTNGRGKEEGRGIGSHTDYGLLVIAGQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKALYYDDGGQQPPPPPPDDPPDEPPDEKIHYGTHFTNMFMRNYPDRVTTERILREDRLKLLDD
>Node239
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLTSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTRYTQDGWHHMRVLRFPQDTKVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKETAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFEPEAAGGQQEPPPPTTEPTEEPTGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDSPNLRWKDTPADSG
>Node199
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHALKQACVDDKSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAIMTKYMNDLAESSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQNDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNVLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGIHNNAAGGQQSPPPPIISPIESPIIGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLEE
>Node213
MADLRTFHLPTDSVVGTDADIQLGRQMIRAWQTDGIFQVACDPAQDQKTQEAFEASRRFFALPTEEKARCVSDLTYAGYIASGEEVTAGEADYSEIFTVCKDIPLDDARVQAGWPCHGPVPWPSEAYRQAMQAYMDELGSIIGEKLLRLIALGLGLDDIDALTRLTDDGWHHMRVLRFPARSSQTTSRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLPSESSAGMYENEEPWTTFVKPVPSTLTVFPGDILQFLTGGALLSTPHKVRLNTRERYTMAYFHEPNFEACIRPLSSGGSSDSSSSSDDDSDESDDYIHYGTHFTNMFMRCYPDRITTRRILEEDRLSVLARLRERAAR
>Node204
TNLQISFPTFDLDTLRQATTTQEKKNLGKEIVKALQQDSLIQIKKNEQEDLLTAQALMQNKKFFRLPLEEKKKYVSDLTYTGYVFSGEEKKGGQYDQVEVYTVFPDFDRDDERVKTLRLPCHGPAIYPSEDYKQAITAYMKATGNMMCDRLLELIGLGLGLDRDDPNYLSDMARDGFHHARVLRFKSQKKTGAASNGISAHSDYGFLIAASQDDVGGLYVRPIVNGEKRGRNWLDEETTIGAYNDEPGWAAFVTPMPSVWTVFVGDMLELATQGKLRANLHKVKLNPEKERFAIAYFHEPKFNQTFTAINSSRGNNNTKGNTKGKGKQITYGKHLTNMYTKAYPNRSTTKRILQENLMTNLDRISKIS
>Node224
MTDLRTFQLPSESVTGSAADRALGAEMVRAWQTDGIFQVATGPAQQAVTQAAMAESRRFFALPLAEKARYVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDIPADDPRVRAGWPCHGPVPWPDEAYRRAMKAFMSQLGSIIGEKLLRLVALGLGLDADDMDALTRLTADGWHHMRVLRFPARSAATTARGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLDHESTAGMFENEEPWTTFVEPVPGTFTVFPGDILQFLTGGHLLSTPHKVRLADRERYSMAYFHEPNFQATVRPLAGAGGDPGPDDAPDPDEAGDPDEYIHYGTHFTNMFMRCYPDRVTTRRILAEDRLEVLARLREAR
>Node230
MSELATFRVPTARVTGTAADEALGRAMVRAWQRDGIFQVAADAGQEARTRAAMAASRRFFARPAAEKAACVNDTSYSGYIASGEEVTAGEPDASEIFTITPDLPAAAGLPCHGPVPWPDPGYRAAMEAYLAGVGGLLGERLLRLVALGLGLGRGPAGMDRLAGLTRGGWHHMRVLRFPAASRTTTERGIGAHTDYGLLVIAAQDEVGGLFVRPPVAGERRPRNWLPGESTAGFAEDSPGWTTYVTPVPGVLTVFPGDILQFLTGGALLSTPHRVRLADRERYALAYFHEPRFGARIRPLPPGGGDDGGDDADGGGDAVIHYGEHFTRMFMRCYPQRATTRRIEAEGLLRRLAAVR
>Node233
VGHLQTFELPNESLSGNAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFFANWKESAAGKHEDAADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGIIFEPEAARREEDPPPPIDPISPIGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLEE
>Node210
NLQTFILPTDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINNKTNGKGKEGGRGIGSHTDYGLLVIAAADDVGALMIRPPYSDENFAANWESSAAGFREDDERWVVFVPPAENVFTYITNSALPSTPHKVGLNNLRERFAFAYFHEPSFQAVVKPLPGPKYDPDVVGGQQEPKPPPKKEPKDKQKGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLEE
>Node234
VAQLETFTLPDDRVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGLKEDDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGQYDPDVGGQSPPPQQPPQESQSPQGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQQ
>Node212
MTELTTFQIPTSSVGDTQSDIDLGLDIIQAWRRDGIIQIALDDAQDARTQQAFKASRDYFALPHQEKSRHLSDLNYAGYIASGEEITAGKADYSEIYTVCKDLPEDDPRVQQQWPCHGPVPWPNAGFQRDMTLFMDQLGAIIGEKLLRLTALGLRSLGRLDRDDINALNRLTRDGWHHMRVLRFPCATGGNQQSRGISSHTDYGLLVIAAQDDVGGLYIRPPVEGEARRRNWLPGESAAGMYENEEPWTTFVRPTPRVVTAFPGDILQFLTGGYLLSTPHKVKLNTRERFALAYFHEPNFEAVIQPLRPPQPGGGDDPGDGPGDDGQIHYGSHFTNMFMRSYPQRITTQRILRENRL
>Node216
LRTFRLPAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLSEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLDADDPDHFTRLTADGWHHMRVLRFPPADAGSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEPWTTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDDDPTTTTDPTPTETADPTEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRRLREDR
>Node202
LKTFVLPNETVDGNEENKAMAKAMIEEWRKTGIFQIQMTPAQQAMLTECFKTSKEFFRHSYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDSRVQNKWPCHGPTPWPNETYKAKMNQLMSLLGDSSGEKLLKLTALGLGLKEDDPDALLNLTEDGWHHMRILRFPQVDKTNGKGKQGGRGIGSHTDYGLLVIAGQDDVGGLFIRPKIEGESIIKNWQSSAAGLNENDDKWLLFVPPVPNTLTVFPGDMMQFLTNDFLPSTPHKVGLNYAERYAFAYFHEPNFSSVIETLPEFEKIHYGTHFTNMFMRNYPERITAERIHEEGRMDILEELE
>Node235
ELETFELPSVAVTGSDADRLLGRQLVEAWRKDGIIQIAAFPKNLRVLSDAVKQSKQYFLQDHETKAKCVDPNSFAGYIASGEELTDGIADYSEIFTVTKEVDPGDPRVMSGWPCHGPNPWPSDTYAHAMRKLMDFKASIIGERLLQLTALGLDLKQDDRWALNKLTQDGWHHMRVLRFPATHATNGKGKEEGRGIGSHTDYGLLVIASQDEVGGLFIRPPTADETYYSNWKKSSAGLHENDDKWLLYVAPQENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCRRLPEFKKTTGGTPEDEEEEEEEEEAVHYGTHFTNMFMRNYPDRITANRMREENRMDLLPQLKP
>Node214
LQTFRLPTAAVRGTDADRALGRALVAAWQSDGIFQIHATPAQQAATERALAASRAFCRRPLKEKARHVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKDYMTVVGEIIGHRLLRLTALGLGLDTDDPDRFTRLTTDGWHHMRVLRFPPADATSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHEEPWTTYVTPVPAVFTVFPGDIMQFMTGGALLSTPHKVRLAARERYTLAYFHEPAFNAVARPLTTDSTSAADSAAEDAADSEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLREDR
>Node246
VGNLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQTDSRVQQKWPCHGPNPWPFTQMKTIMQAYMDYLGESSGEKMLSLIAWGLGLEQDDGNALTRYTQDGWHHMRVLRFPQTDKTNGKGKAAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYYANWKTTAAGKHEDTDGWVVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFQQAAGGQQEPPPPTTEPTEPGIHYGSHFTNMFMRNYPQRITADRMKAENRMALLDSPDLRWKDTPAD
>Node207
VGQLETFTLPSESVTGSRGDRAMGKALVDAWRRDGILQIAMNPKQQAMYDAAQAASKRFFAKPHEQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKKYMDYLGESSGEKLLQLTEYGLGVPEGSLTQYTNDGWHHMRILRFPQNNKTNGKGKEEGRGIGSHTDYGLLVIAAQDSVGGLFIRPPYADEQYYANWEKSSAGMKEEDDRWVYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTAERFAFAYFHEPSFQAVAKPLPGPYDPDAAGGQQSPPPPKKSPKESPGIHYGTHFTNMFMRNYESRITTQRLLAEDRYSLL
>Node197
FETFVLPSEKVDGSPSNHALGKAMVEAWRRDGILQVAMTGQQQQVYREAQRISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGVPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLATNNGEKMLQLIEFGLHVPGPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGEQYYANWEKSSAGLLEDESGWVVFVPPVPGVFTVFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGPYNPNRRAAQQPPPPPVVPPVDPDGIHYGTHFTNMFWRNYPDRLTTRRLEEEGR
>Node206
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMTKSQQRVYQAASAASRRFFAQPHARKQACVDGSSYSGYIASGEEITDGVADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQDMKAVMNKYMEDLAVSSGEKLLQLIEMGLDIPTGSLTKYTDDGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDSVGGLFVRPPRESERFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGPPVYNPPNAAGGQQSPPPVVPPVEPVGEGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYQLLESEELRDDDDE
>Node205
VGNLKTFELPSEVLTGSASDQALGRSMIDAWRKDGILQISMDPINRKLADAAFICSKKFFDLPYKTKASCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPLTDPRVQQKWPCHGPCPWPFKQMRTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTKYTQDGWHHMRVLRFPERNNVNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGENYYANWRKSAAGMAEDDDKWVVYVPPVPDVFTVFPGDMMQYVTNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLEGRFDDAAGGQQEPPPPRREPREPGIHYGTHFTNMFMRNYPERITANRMRSENRMKLLEE
>Node236
LQTFELPSAQVTGSPADVELGKALIDAWRRDGILQIAMEDTQKSLLDETFDASKNFFRKPHREKAACVDSQSYSGYIASAEEITDGIPDYSEIFTVTKDLDLSEPRVKSHWPCHGPCPWPDAEMKDPMQRYMDYLGQSSGEKLLQLIEYGLGVPQPGSLTNYTQDGWHHMRILRFPAVNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPSADETTTANWEQSAAGFREDDDGWVVYVPPVPNVFTVFPGDMMQYLTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQSVMRPLPGPYDDGGGGQQNPPPPRRPPREPGIHYGTHFTNMFMRNYPDRITSKRMIRENRKALLDD
>Node221
MAELETFQLPSQSVSGSESDVALGLTLIRAWRRDGIFQLRMSEAQSEKAQRAFESSRRFFRQPLEAKARCVSDSTYSGYVASGEELTAGEADYSEIFTVCKDVPLTDARVQARWPCHGPAPWPDETYRRDMKAHTDELGSTTGERLLRLVALGLGLEREVGALTALTRDGWHHMRVLRFPARSPTAATRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTTTPPGPPPDDGPDEPDDYIHYGSHFTNMFMRCYPDRITTQRILDEDRLTTLAWLREAW
>Node219
MTELQTFQLPGPTVEGGDADRLLGQALIAAWQADGIFQIQATPDQDAATGRALDASRAFVGRPLKEKAQHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVLDRWPCHGPAPWPSDQYAAAMKDYMAVVGDIIGHRLLQLTALGLGLDDDMEHFTRLTDDGWHHMRVLRFPPADATSEERGIGSHTDYGLLVIAAQDEVGGLYIRPPVPGETRGRNWLPDESMAGRYENEDPWTTFVAPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLAEAAADDEADDEDAPEAEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLAVLERLRKER
>Node222
MTDLQTFALPDDSVTGQPSDIELADRMIRAWRRDGIFQVAMNPAQEQKTERAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLTDARVRAQWPCHGPAPWPDEAYRQAMTAFTDELGSVVGERLLRLTALGLGLERIDALTRLTRDGWHHMRVLRFPALSSTSSSRGIGAHTDYGLLVIAAQDDVGGLLVRPPVEGEKRNRNWLPGESSAGMYEHEEPWTTFVKPVPKVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERYSMAYFHEPAFDACVRPLMGGPSSSSEESSEESGSEEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLSTLAWLREEAAW
>Node203
VALETFVLPSERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPLQKQACVNECSYAGYVASGEEITAGIADYSEIFTVTKDLHPEDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSSGDKVLQLIELGLDVPDQGSLTKYTDDGWHHMRVLRFPARHRTNGKGKAGGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFANWKQSVAGMKEDEEPGWLLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAMVKPLPGAYNNAASSQEPPPPAEPADPGIHYGTHFTNMALRNYPTRITTTKLLEEGRYEMLSKES
>Node223
MTNLQTFYLPTKSITATQSNIDLGHQMIQAWRTDGIFQVATNTIQDRKVQNAFEASRRFFRMPLGFKSQHISDLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVCVQKQWPCHGPVPWPNQNYQQNMTTFMDELGLIIGEKLLKLVALGLEVDMNTFDFTLNGWHHMRVLRFPAVSEKSSARGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWRSDESSAGMYENEEPWTTFVKPVPNVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSNNPSSSSNNSSNESSNNEYIHYGTHFTNMFMRCYPDRITTRRIQDEDRLSVLALLRNAL
>Node208
QLETFTLPSAKVTGSISDRAMAKAMISAWRRDGILQIGMTRLQQRLYQDALASSRRFFSRPLAEKQACVDSRSYSGYIASGEEITDGIRDYSEIFTVTKDLPLSDKRVMDGWPCHGPCPWPDNDFKEVISRYMAELGACCGERLLQLIELGLDVPTAGSLTRYTTDGWHHMRVLRFPARHRTNGKGKSSGRGIGSHTDYGLLVIGSQDEVGGLFVRPPREGESFFVNWQKSAAGAKEDDDGWMMYVPPEQGTFTVFPGDIMQYLTNHVLRSTPHKVGLNTRERFAFAYFHEPNFSSTVKPLAGVYDDGGGGQQEPPPPVVEPVEEPEVGIHYGTHFTNMCLRNYPDRITTRRL
>Node220
MTELRTFRLPSEETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRTTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDDSYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDGDDDPHGGGGDDHGDDPDGVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node244
QLETFVLPSEQVSGSVSDQAMAKAMIDAWRRDGILQVGMSQLQRRLYQSADAASRRFFARPQGEKQACVDSRSYAGYVASGEEITDGVADYSEIFTVTKDLSPRDKRVAHGWPCHGPCPWPDRDMKAVMGRYMADLGASSGEKLLQLIELGLGVPNAGSLTRYTNDGWHHMRVLRFPPRHRTNGKGKDDGRGIGSHTDYGLLVIAAQDDVGGLFVRPPQQGERFFANWQQSAAGAKENESGWVVYVPPTPGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNGGGGQQEPPPPPVVPPVEPGIHYGTHFTNMCLRNYPDRITTKRLVAEDRYRMLDSEELRD
>Node201
LETFVLPSEEKVTGSASDRAMGKAMIDAWRRDGILQIATSPEQQQIYKAAYAASKRFFSKPHAEKAACVDSQSYAGYIASSGEEITDGIADYSEIFTVTKDLPLSDPRVMAKKWPCHGPCPWPDAEMKDAINRYMQYLGSSSGEKLLQLIELGLNIPEGSLTNYTQDGWHHMRILRFPATNKTNGKGKEGGRGIGSHTDYGLLVIASQDDVGGLFIRPPYEDEKYAANWEKSAAGMKENDDSGWVYVPPVPNVFTVFPGDMMQYMTNNYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLPGPPSEYNPNAAGGQQEPPPPPSSEPSEPGIHYGTHFTNMFMRNYPDRITTQRLLEEGRYKLLEE
>Node232
LKTFTLPSEKATGSPSDVQMGKAMINAWREDGILQVSMNPKQQDLFDKAFAASKRFFAKPPNQKAACVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDIDMKTPIQQYMDSLGNSSGETLLQMIEYGLSLDKPKTLTSLTKDGWHHLRTLRFPQNNNTNGRGKEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNNTRERFAFAYFHEPSFQAVISPVAKDLYYDGGQQPPPPPDDPPDEPPDDEKIHYGTHFTNMFMRNYPDRVTTERIIKEDRLKLLDD
>Node229
MTNLQTFELPSESVTGSQADLQLGRALIQAWRKDGIFQISTHPEQDQKVQRAMAASKAFFKQPAQEKSRHVSDLTYAGYVASGEEVTAGEQDQSEIFTICKDLPLDDQRVKGGWPCHGPVPWPNDEYRDSMKAFLAELGDIIGEKLLKLVALGLGLEQINALTNLSQDGWHHMRVLRFPAESSKTTCRGIGAHTDYGMLVIATQDDVGGLYIRPPVEGEHRNRNWLPGESTAGLFENDEPWTTFVKPVPSVFTVFPGDILQFMTDGYLLSTPHKVKLNTRERFAMAYFHEPSFKASVRPLLSSQSNNNGGSNGESNGGERIHYGEHFTNMFMRCYPERITTQRIHKENRLRHLEEIKE
>Node200
LTTFTLPSQEVTGSQSDIALGQRMIETWRKDGIFQIQLTPAQAATLRQAFKASKSFFSRPYEEKAKHVDDQSFSGYIASGEELTDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPDTSYATTMKKLMSLLGESSGEKLLKLTALGLGLAEDDPNALTKLTEDGWHHMRVLRFPQVDKTNGKGKAGGRGIGSHTDYGLLVIAGQDDVGGLFVRPPIEGEKLLKNWEKSAAGFKENDENWTTYVPPVENVLTVFPGDMMQYLTNDYLPSTPHKVGLNTRERYAFAYFHEPNFSSVVKPLPEPYGGHHDDQQSNSSSDDSSDKNDKGIHYGTHFTNMFMRNYPERITAKRMKAENRMDILEELE
>Node240
DLQTFELPSQQVTGSPEDIALGKAMIDAWRKDGILQIKMDPSNKKVAEKAFKASKRFFSLPSEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNASMQTTMKAYMDYLGSSSGERLLQLIALGLGLPRDDPNALTNLTRDGWHHLRILRFPKTGDENGKGTKKGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKFFQNWEKSTAGMAENDDRWVVYVPPEDGVFTVFPGDMMQFITNNHLPSTPHKVGMNTRERFAFAYFHEPNFSAVLKPLDGSSSSDDSSDESSVHYGTHFTNMFMRNYPERITAKRMRAENRMDVLESLKE
>Node241
QLETFVLPSEKVSGSVSDQAMAKAMIDAWRRDGILQIGMSQLQRRLYQSADAASRRFFARPQAEKQACVDSRSYSGYIASGEEITDGVADYSEIFTVTKDLSHSDKRVANGWPCHGPCPWPDQDMKSVMSRYMADLGASSGEKLLQLIELGLDVPAGSLTGYTNDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDHVGGLFVRPPHQGDRFFANWQQSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMLQYMTNNVLLSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPGNNGGGGQQEPPPPPVVPPVEEPVPVEGIHYGTHFTNMCLRNYPDRITTKRLVAEDRYQMLDSEELRD
>Node238
NLQTFVLPSEKVSGSEEDKKLGKAIVDAWKKDGILQIAMKPEQQGLYKAANLASKRFFSKPYAQKAACIDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMERYMNSLGESSGETLLKLTELGLDVPEGSLTDYTKDGWHHMRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVLAAADDVGALLVRPPQQGDNFFANWEKSAAGFKEDDEGWLLFVPPAENVFTVFPGDMMQYLTNSFLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGKYDPDAAGGQQEPPPPPKKEPKEEPKGVHYGKHFTDMFMRNYPQRITTQRLLDEGRYDLLEE
>Node227
MNELQTFHLPTEFIDGTESDRALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPTPWPDAEYQRSMRVFMDELGRVVGEKLLKLTALGLELDADDTDALTRLTADGWHHMRVLRFPALSKESSRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEEPWTTFVRPVPSVLTVFPGDILQFLTDGFLLSTPHKVRLNTRERFALAYFHEPNFAARIRPLSGGVSAAAASADAACIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLPLLRDPL
>Node198
VAQLETFTLPSDRVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLTELGLNVPAAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSAAGMKEDDEKWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDDVVGGQQSPPPPKKSPKEPKKEGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQQ
>Node237
QLQTFVLPSEPEKVTGSESDRKLGKAIVAAWQKDGILQIAMKPSQQALYKAANAASKRFFKKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLPLDEPRVVAKWPCHGRCPWPDAEMQNPMQRYMDSLGESSGETLLKLTELGLDVPQEGSLTGLTQDGWHHMRILRFPATHHATNGKGKDGGRGIGSHTDYGLLVLAAADDVGALFVRPPQKDENFAANWEKSAAGFKEDDDEGWVVFVPPAENVFTVFPGDMMQYLTNSVLPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGKYDPDAAGGQQEPKQPPPKKEPKEEPEEGVHYGKHFTDMFMRNYPQRITTQRLVDEGRYNLLEE
>Node262
MTELRTFRLPSEETVTGSAADRALGREMVAAWRTDGIFQVAADPAQQQATQAAMAASRRFFALPLAEKARYVSDLTYSGYVASGEEVTAGEADYSEIFTVCKDVPADDARVRAGWPCHGPVPWPDPHYRRAMTAFMSALGSIIGERLLRLVALGLGLDDPDTLTRLTADGWHHMRVLRFPARSARTAARGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRSRNWLPEESTAGMYENEEEPWTFVRPVPGVFTVFPGDILQFLTGGHLLSTPHKVALADRERHTMAYFHEPSFEATLRPLAGDDDGPAGGGGGGAGGDGGDPPLHYGTHFTNMFMRCYPDRPTTRRILEEGRLAVLAARLREEARLR
>Node256
MTNLQTFELPSETVTGSAADISLGRALIQAWQKDGIFQIKTDSEQDRKTQAAMAASKRFFKKPMTFKSSCVSDLTYSGYIASGEEVTAGKADYSEIFTVCKDLPESDQRVKAGWPCHGPVPWPNDTYQKSMKAFMDELGLAAGEKLLKLTALGLELPRINTFTDLTRDGWHHMRVLRFPTETSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFEHEEPWTTFVKPTPKVWTVFPGDILQFMTNGYLLSTPHKVKLNTRERFACAYFHEPNFEAVARPLFEEPSSSSNNSSNESSNNERIHYGEHFTHMFMRCYPDRITTQRIHKENRLAHLEELKKRSDTRATEKLK
>Node254
MSDLQTFHLPTESITGTQSDIDLARQMIQAWRTDGIFQVATNKIQDRKSENAFAASRRFFRMPLEFKSQFISDLTYSGYIASGEEVTAGESDYSEIFTICKDVPLDDVRVQAQWPCHGPVPWPDQEYQQSMKAYMDELGSIIGEKLLKLVALGLELDKDDINALTKLTKDGWHHMRVLRFPALSQKSSTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESTAGMYENEEPWIIFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLFDDPPSSSSSDDSSDESSDDEYIHYGTHFTNMFMRCYPDRITTRRIIDENRLSVLAVLKNAN
>Node272
QLKTFTLPSEKATGSPSDVEMGKAMINAWREDGILQVSMSPKQQDLFDKASAASKRFFAMPPNQKAACVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDTDMKTPIQQYMDSLGNSSGETLLQMIEYGLSLDKPKTLTSLTKDGWHHLRILRFPQNNKTNGRGKEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKDLYYDDGGQQPPPPPDDPPDEPPDDEKIHYGTHFTNMFMRNYPDRVTTERIIKEDRLKLLDD
>Node249
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHALKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVARGWPCHGPCPWLDQHMKAVMTKYMNDLAVSSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVIRPLPGIYNNAAGGQQSPPPPIISPIESPIIGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYRLLEE
>Node252
MTNLQTFQLPTQSVTTTQSDIQLGREMIQAWRQDGIFQIATNPEQDQKTQQAFEASKRFFRMPLEEKSRCVSDLTYSGYVASGEEITAGKVDHSEIFTVCPDLPKDDARVKAQWPCHGPVPWPDEEYQQAMKAFMKALGSIIGDKLLKLVALGLGLDDDINALTDLTRDGWHHMRVLRFAAASSSASSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPEESSAGMYNNEEPWTTFVTPMPSVLTVFPGDILQFLTNGYLLSTPHKVKLNPTRERFALAYFHEPNFEAVVRPLNGGKKSSSSDDSSDESSSGESIHYGTHFTNMFMRCYPERSTTRRILSENRLAVLEEMKEEMK
>Node266
MGELSTFRVPTARVTGTAADQALGRAMVRAWQRDGIFQVAADAGQEARTEAALAASRRFFARPLAEKAACVNDLSYSGYIASGEEVTAGEPDQSEIFTVTPDLPAGHPRVRAGLPCHGPVPWPDPGYRAAMEAYLAGVGELLGERLLRLVALGLGLGPAGMDRLAGLTRGGWHHMRVLRFPARSSATTEERGIGAHTDYGLLVIAAQDEVGGLYVRPPVPGERRPRNWLPGESTAGRAEDEEPGWTTFVEPVPGVLTVFPGDILQFLTGGALLSTPHRVRLAADRERYALAYFHEPRFDARIRPLGAGGGGADGGAAEGPGAEVIHYGEHFTRMFMRCYPERATTRRILAEDRLAAL
>Node268
VGHLQTFELPSESLSGSAADKALGKSMIDAWRKDGILQISMDPINRKLADAAFLCSKKFFSLDYNKKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDFRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMEQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIQGETFFANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGIFEEAARRQQDPPPPIIDPISDPIIGIHYGTHFTNMFMRNYPTRITADRIRAEERMALLEE
>Node269
VAQLETFTLPSDRVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASSGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTDDGWHHMRILRFPPTHNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGLKEDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGQPPQYDVSPDVGGQSPPPPQQPPQESPQQGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQQ
>Node250
LKTFVLPNQTVEGNEENRAMAKAMIEEWRKTGIFQIQMTPAQQDMLRECFKTSKEFFSLPYEEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDSRVQNKWPCHGPCPWPNETYKTKMNQLMSLLGDSSGEKLLKLTALGLGLKDDPDALLKLTEDGWHHMRILRFPQVDKTNGKGKTGGRGIGSHTDYGLLVIAGQDDVGGLFVRPKIEGESIIKNWQSSAAGLNENDDKWLLFVPPVPNTLTVFPGDMMQFLTNDFLPSTPHKVGLNTAERYAFAYFHEPNFSSVIEPLPEFEKIHYGTHFTNMFMRNYPERITAKRIHEENRMDILEELE
>Node270
ELQTFVLPSEEVTGSDADRALGKELVEAWRRDGILQIAAFPKDLQVLSDAIKQSKRFFAQDHEAKAKCVDSQSFAGYIASGEEITDGIADYSEIFTVTKDLPSSDPRVQSKWPCHGPNPWPSKTYQQAMTALMDYKGSIIGEKLLQLTALGLNLKQDDPWALNKLTQDGWHHMRVLRFPATNATNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFIRPPIEGEKYYSNWKKSAAGMHENDEKWVVYVPPVENVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVCKRLPEFRGSSGGSASDADDEDDEDSAVHYGTHFTNMFMRNYPDRITAKRMREENRMDLLEELKE
>Node275
VGNLQTFELPSETLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKQKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKELAQSDPRVQQKWPCHGPNPWPFTQMKTVMQAYMDYLGESSGEKMLSLIAWGLGLEQDDGNALTRYTQDGWHHMRVLRFPQTDKVNGKGKAAGRGIGSHTDYGLLVIAAQDAVGGLFIRPPIEGEEYYANWKETAAGKHEDADGWVVYVPPTPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFQQAAGGQQEPPPPTTEPTEPGIHYGTHFTNMFMRNYPQRITADRMKAEDRMALLDSPDLRWKDTPAD
>Node264
MTDLTTFHLPTERITNTEAHRELGQAMVKAWRTDGIFQIALSTPQEHTTDEAFAESRRFFALDFETKSRHVSALTYSGYIASREEVTAGEADYSEIFTICPDIGLEDARVREDWPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLQLIALGLGLDDDMETFTRLTQDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGMYEHDDGWTTFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKAADAAAAAVVAAVAAAVPIHYGTHFTNMFMRCYPKRITTRRIEEKGLLDKLPTLSEVAP
>Node247
FETFVLPSEKVDGSPSNHALGKAMVEAWRRDGILQVAMTGQQQQVYREAQRISRRFFRRPHHQKAACVDAQSYAGYIASGEEITDGVPDYSEIFTVTKDLPLDDQRVMNGWPCHGPCPWPDSDMANVIQRYMDDLATNNGEKMLQLIEFGLHVPGPQSLTKYTGDGWHHLRMLRFPPCNRTNGRGKEEGRGIGSHTDYGLLIIAAQDDVGGLFIRPPSQGEQYYANWEKSSAGLLEDESGWVVFVPPVPGVFTVFPGDMMQYMTNSVLSATPHKVGLNTRERFAFAYFHEPDFRAVVRPLPGPYNPNRRAAQQPPPPPVVPPVDPDGIHYGTHFTNMFWRNYPDRLTTRRLEEEGRYSLL
>Node271
LQTFELPSEQVTGSPGDIAMGKALIDAWRRDGILQIAMDPKQQKLLDEAFAASKRFFAKPHKEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAHWPCHGPCPWPDAEMKDPMQRYMDYLGESSGEKLLQLIEYGLGVPQPGSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPSEDEKTTANWEKSAAGFREDDDGWVVYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLPGKADDAAGGQQSPPPPKKSPKEPGIHYGTHFTNMFMRNYPDRITTKRMIRENRRALLDD
>Node259
MAELETFQLPSQSVSGSESDVALGLTMIRAWRRDGIFQLRMSPAQAEKAQRAFESSRRFFRQPLEAKARCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRQDMKAHTDELGSTTGERLLRLVALGLGLREVDALTALTRDGWHHMRVLRFPARSPTAATRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVALNTRERFAMAYFHEPHFEACVRPLTEEPTRRRDDTRDETRDDEFIHYGSHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAW
>Node257
MTELQTFQLPGPTVEGGDADRLLGQALIAAWQADGIFQIQATPDQDAATGRALDASRAFVGRPLKEKAQHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVLDKWPCHGPAPWPSDQYAAAMKDYMAVVGDIIGHRLLQLTALGLGLDDDMEHFTRLTDDGWHHMRVLRFPPADATSEERGIGSHTDYGLLVIAAQDEVGGLYIRPPVPGETRGRNWLPDESMAGRYENEDPWTTFVAPTPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLPEPPADDEPDDEEPEEAEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLAVLERLRKEAER
>Node273
LETFDLPSDRVTGSASDKAMGQALIKAWKRDGIFQIAMNSSQREIYKYAEKASKRFFAKPHNEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVAKKWPCHGPCPWPDAEMRDPMKTYMNYLGESSGEKLLALTELGLGVPKGSLLQYTQDGWHHMRILRFPPTNKTNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGESYYANWEKSAAGMKEDDEGWVVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNNTRDRFAFAYFHEPSFQAVVKPLEGYNNNGGQQKPPPQQKPQEQEGIHYGTHFTNMFMRNYPDRITTQRLLAEDRYKLL
>Node251
QLETFVLPSERLSGSLSDRAMAKAMVRAWRRHGILQIAMSPSQQQIYAQANAASRDFFKATPPQKQACVNDCSYAGYIASGEEITAGIADYSEIFTVTKDLHPKDPRVLEKWPCHGPCPWLDKRMKSPMLNYMADMGSSSGDKVLQLIELGLDVPQQGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFFANWKQSVAGMKEDEPGWLLYVPPTPGVFTAFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAVVKPLPGPYNNAASSQQEPPPPVVEPVDPGIHYGTHFTNMALRNYPTRMTTKKLLEEGRYEMLSKEELRS
>Node260
MTDLQTFHLPTESITGTQSDIDLAHQMIQAWRTDGIFQVATNTIQDRKTQNAFEASRRFFRMPLEFKSQCISDLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVCVQAQWPCHGPVPWPDQNYQQSMKAFMDELGLIIGEKLLKLVALGLELDKDDINALTKLTKNGWHHMRVLRFPALSEKSTTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSNESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLSDDPSSSSSDDSSDESSDDYIHYGTHFTNMFMRCYPDRITTRRILDEDRLSVLALLKNAL
>Node255
MRTFELPTDSVGGTTVDRILGRQLVRTWEDDGIFQVAATAEQDRITGEALAASRRFFARSLPEKAKFVSDLTYSGYIASGEEVTAGEADYSEIFTVCPDIPLTDTRVTEGWPCHGPVPWPDEGYRTAMKAHLREVGEIIGEKLLKLVALGLGVEEIDRLTRLTRDGWHHMRVLRFPARSATTTERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRNRNWLPEESTAGMYQDDEPWHHFVEPEPGVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGATVRPLGDDEGERIHYGTHFTNMFTRCYPDRVTTKRIHEEGRLAVLDRLRD
>Node258
MTELRTFRLPSETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASSGEEVTDGIADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDDSYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSETVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGKLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDGDDDPHGGGGDDHGDDPDGVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node265
MTNLQTFELPSESVTGSQADIQLGRAMIQAWRKDGIFQIATTPEQDQKTQRAMAASKRFFKQPMEEKSRHVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDLPLDDQRVKAGWPCHGPVPWPNDEYQESMKAFMDELGDIIGEKLLKLTALGLGLEINALTDLTQDGWHHMRVLRFPAASSKTSSRGIGAHTDYGMLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESTAGMFENDEPWTTFVKPVPSVFTVFPGDILQFMTDGYLLSTPHKVKLNTRERFALAYFHEPNFKASVRPLLSLGGQQSSSSSSGGSSGESSGSRERIHYGEHFTNMFMRCYPERITTQRIHKENRLRHLEEIKEEIK
>Node267
LRTFRLPAAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALALEASRAFCRRPLKEKAGYVSDLSYSGYVASSGEEETAGERDGSEIFTVCPDIPADDPRVIDQQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLLDADPDHFTRLTADGWHHMRVLRFPPADAGSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEEHEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLTDADTTTTTDTATETADPTEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDDRRLREDRLR
>Node274
QLETFVLPSEKVSGSVSDQAMAKAMIDAWRRDGILQIGMSQLQRRLYQSADAASRRFFARPQAEKQACVDSRSYSGYIASGEEITDGVADYSEIFTVTKDLSHSDKRVANGWPCHGPCPWPDQDMKSVMSRYMADLGASSGEKLLQLIELGLDVPAGSLTRYTNDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQGERFFANWQQSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMLQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGVGNNGGGGQQEPPPPPVVPPVEEPVPVEGIHYGTHFTNMCLRNYPDRITTKRLVAEDRYQMLDSEELRD
>Node263
MNELQTFHLPTEFIDGTESDLALARAMVRAWRSDGIFQIATSPAQDQKTQAAIESSKRFFRLPMKAKSQCVSELTYSGYIASGEEVTAGEADYSEIFTVCPDIALDDARVQARWPCHGPAPWPDAEYQRSMRVFMDELGRVVGEKLLKLTALGLELDADDTDALTRLTADGWHHMRVLRFPALSKESSRRGIGAHTDYGLLVIAAQDDVGGLYIRPPVAGEKRNRNWLANESAAGMYENEEPWTTFVRPVPSVLTVFPGDILQFLTDGFLLSTPHKVRLNTRERFALAYFHEPNFAARVRPLSGGTSAAAASADACIHYGTHFTNMFMRSYPARITTQRILAEDRLSVLSLLRDSL
>Node248
VAQLETFTLPSDRVTGSKGDIAMGKALVDAWRRDGILQVSMNDKQQAIFNEATASSKAFFAKPPAEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSAAGMKEDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDDVVGGQQSPPPPKKSPKEPKKGIHYGTHFTNMFLRNYPERITTKRMMAENRLAKLQQ
>Node282
MTNLQTFELPSETVTGSQADISLGRALIQAWRKDGIFQIATTAEQDRKTQAAMAASKRFFKMPMTFKSSCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDLPESDQRVKAGWPCHGPVPWPNENYQKSMKAFMDELGHIIGEKLLKLTALGLELPRDINTLTDLTRDGWHHMRVLRFPTASSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFENEEPWTTFVKPVPKVFTVFPGDILQFMTNGYLLSTPHKVKLNNTRERFALAYFHEPNFEAVARPLFSFEEPPSSSSSDDSSDESSDRERIHYGEHFTHMFMRCYPERITTRRIHKENRLAHLEEMKKRSDTRATEEMK
>Node293
QLKTFTLPSEKATGSPSDVQMGKAMINAWREDGILQVSMSPKQQDLFDKAFAASKRFFAMPPNQKAACVDTQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDTDMKTPIQQYMDSLGNSSGETLLQMIEYGLSLDKPKTLTSLTKDGWHHLRILRFPQNNNNTNGRGKEEGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAADEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKLYYDGGQQPPPPPDDPPDEPDEKIHYGTHFTNMFMRNYPDRVTTERIIKEDRLKLLDD
>Node277
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHAQKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQDMKAVMNKYMNDLAVSSGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGIYNNAAGGQQSPPPPIISPIESPGIHYGTHFTNMFLRNYPDRVTTARLQEEGRYQLLESEELRDDDDE
>Node289
MGELSTFRMPTACVTGTAADQALGRAMVRAWQRDGIFQVAADAGQEARTEAALAASRRFFARPLAEKAACVSDLSYSGYIASGEEVTAGEPDRSEIFTVTPDLPATHPRVRAGWPCHGPVPWPDPGYRAAMEAYLAGVGELLGERLLRLVALGLGLGPAGMDALAGLTRGGWHHMRVLRFPARSSATTEERGIGAHTDYGLLVIAAQDEVGGLYVRPPVPGERRPRNWLPGESTAGRWENEEPGWTTFVEPVPGVFTVFPGDILQFLTGGALLSTPHRVRLAADRERYALAYFHEPSFDAVIRPLGAGGGGADGGAAEGPGAEVIHYGAHFTRMFMRCYPDRVTTRRILAEDRLAAL
>Node280
NLQTFILPTDKVSDTEANRKLGKALVEAWQKDGILQISMTPEQHSLYKSANYASRRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGGVVGETLLQLTELGLGVPEGSLTNYTQDGWHHLRILRFPAINKTNGKGKEEGRGIGSHTDYGLLVIAAADDVGALMIRPPYSDEKLLANWESSAAGFKEDDERWVVFVPPAENVFTVFPGDMMQYITNSALPSTPHKVGLNLRERFAFAYFHEPSFQAVVKPLPGKYDDVVGGQQEPPPPKKEPKDEPKKGIHYGKHFTNMFMRNYPQRITTQRLNDEGRYRLLEE
>Node278
LKTFVLPNQTVEGNEENRAIAKAMIEAWRKTGIFQIQMTPAQQDMLRECFKTSKEFFSLPYEEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVQNKWPCHGPCPWPNETYKTKMNQLMDLLGDSSGEKLLKLTALGLGLKDDPDALLKLTEDGWHHMRILRFPQVDKTNGKGKTGGRGIGSHTDYGLLVIAGQDDVGGLFVRPKIEGESIKKNWESSAAGLNENDDKWLLFVPPVENTLTVFPGDMMQFLTNDFLPSTPHKVGLNTAERYAFAYFHEPNFSSVLEPLPEFEKIHYGTHFTNMFMRNYPERITAKRIHEENRMDILEELE
>Node292
ELQTFVLPSEEVTGSAADIALGKELVEAWRRDGILQIAAFPKDLQVLSDAIKQSKRFFALDYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPVPWPSSTYQQAMTAYMDYQGSSIGEKLLQLTALGLGLQRDDPNALNKLTRDGWHHMRVLRFPATNATNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGEKYYANWKKSAAGMYENDEKWTTYVPPVENVFTVFPGDMMQFLTNSYLPSTPHKVGLNTRERFAFAYFHEPNFNAVIKPLPEFGGSSGGQSSSSDDSSDESSKAIHYGTHFTNMFMRNYPERITAKRMRKENRMDLLEELKE
>Node296
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESSGEKMLQLIAWGLGLKDDGNALTRYTQDGWHHMRVLRFPQTNKVNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKETAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNNTRERFAFAYFHEPNFSAVMKPLPGPTFEPEAAGGQQEPPPPTTEPTEEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDSPNLRWKDTPAD
>Node288
MTDLTTFHLPTERITNTEAHRQLGQEMVAAWRTDGIFQIALSTPQEHTTDEAFAESRRFFALDFETKSRHVSELTYSGYIASREEVTAGEADYSEIFTICPDIGLDDARVREGWPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLQLIALGLGLDDDMETFTRLTQDGWHHMRVLRFPTVQSSENNARGIGAHTDYGMLVIAAQDDVGGLYVRPPIEGERRNRNWLASESTAGMYEHDDGWTTFIKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVEPLKAADAAAAAVVAAVAAAVPIHYGTHFTNMFMRCYPKRITTRRIEEQGLLDRLPTLSEVAP
>Node285
MAELETFQLPSQSVSGSEADVALGLTMIRAWRRDGIFQIRMSPAQAEKAQRAFELSRRFFRQPLEAKARCVSDLTYSGYVASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRQGMKAHTEELGSAVGERLLRLVALGLGLDRDIDALTALTRDGWHHMRVLRFPARSPTTTTRGIGAHTDYGMLVIAAQDDVGGLYVRPPVEGEKRRRNWLPGESSAGMYEHDEPWTTFVKPVPDVLTVFPGDILQFLTRGYLLSTPHKVVLNTRERFAMAYFHEPHFEACVRPLTEEPTRRRDDTRDETRDDEFIHYGTHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAW
>Node283
MTELQTFQLPGPTVEGGDADRLLGQALIAAWQADGIFQIQATPDQDAATGRALDASRAFVGRPLKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDPRVVDKWPCHGPAPWPSDQYAAAMKDYMAVVGDIIGHRLLQLTALGLGLDDDMDHFTRLTDDGWHHMRVLRFPPADATSEERGIGSHTDYGLLVIAAQDDVGGLYIRPPVPGETRGRNWLPEESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERFTMAYFHEPSFQAVARPLPEPPADDEADDEDAADAEFIHYGTHFTNMFMRCYPDRSTTARIEQEDRLAVLERLRKEAER
>Node294
LETFDLPSDRVTGSASDKAMGKALIKAWKRDGIFQIAMNSDQREIYKSAEKASKRFFAKPHNEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVLKKWPCHGPCPWPDAEMRDPMKRYMNYLGESSGEKLLALTELGLGVPQKGSLLHYTQDGWHHMRILRFPPTNKTNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGESYYANWEKSAAGMKEDDEGWVVYVPPVADTFTVFPGDMMQYMTDSFLPSTPHKVGLNTRERFAFAYFHEPSFQAVVKPLEGQYNNAAGGQQKPPPPQQKPQEKPQQGIHYGTHFTNMFMRNYPDRITTQRLLAEDRYKLL
>Node279
QLETFVLPSERVSGSLSDRAMAKAMVNAWRRDGILQIAMSPSQQQIYAQANAASRKFFKKTPAQKQACVNDSSYAGYIASGEEITAGIADYSEIFTVTKDLQPSDPRVMEKWPCHGPCPWLDKRMKSAMTDYMADLGSSSGDKLLQLIELGLDVPQPGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFFANWKQSVAGMKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAVVKPLPGVYNANAASSQQEPPPPVVEPVDEPVVGIHYGTHFTNMALRNYPDRMTTKKLLEEGRYEMLSKEELRS
>Node286
MTDLQTFHLPTESITGTQSDIDLAHQMIQAWRTDGIFQVATNTIQDRKTQNAFEASRRFFRMPLEFKSQCISDLTYSGYIASGEEITAGESDYSEIFTICKDVPLDDVRVQAQWPCHGPVPWPDQNYQQSMKAFMDELGLIIGEKLLKLVALGLELDKDDINALTKLTKNGWHHMRVLRFPALSEKSTTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSNESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFNACVRPLFDDPSSSSSDDSSDESSDDYIHYGTHFTNMFMRCYPDRITTRRILDEDRLSVLALLKNAL
>Node281
MTELRTTFELPTDDSVSGTTVDRILGRQLVRTWETDGIFQVAATAEQDRITGEALAASRRFFARSLPEKAKFVSDLTYSGYIASSGEEVTAGEADYSEIFTVCPDIPLTDTRVTEGWPCHGPVPWPDEGYRTAMKAYLREVGEIIGEKLLKLVALGLGVIDRLTRLTRDGWHHMRVLRFPARSATTEERGIGAHTDYGLLVIAAQDDVGGLFIRPPVPGEPRNRNWLPEESTAGMYQDEEPWHHFVEPEPGVFTVFPGDILQFLTGGRLLSTPHKVRLADRERYTLAYFHEPSFGATVRPLGAGGGGGGGGEGERIHYGTHFTNMFTRCYPDRVTTRRIHEEGRLAVLDRLRARSDR
>Node284
MTELRTFRLPSETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHGLSLEEKSRLVSDLTYAGYIASGEEVTDGIADYSEIFTVCKDVPLDDSRVVGKWPCHGPVPWPDDSYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSEAVVDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVQHVFTVFPGDILQFLTGGRLLSTPHKVLLNNTRERFTMAYFHEPNFEAVVRPLDDDPPHGGGDDHGDDDGDGDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node290
LRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQVHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDQWPCHGPAPWPSPGYARAMKGYMTAVGELLGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLAADRERYTLAYFHEPAFNAVARPLDTDTTTTTADTATETTADPEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEADLR
>Node295
LETFVLPSEKVSGSVSDRAMAKAMIDAWRRDGILQIGMSQLQQRLYQSANAASRRFFSRPHAEKQACVDSRSYSGYIASGEEITDGIADYSEIFTVTKDLPHSDKRVANGWPCHGPCPWPDQDMKSVMSRYMADLGASSGEKLLQLIELGLDVPAGSLTRYTNDGWHHMRVLRFPARHRTNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQGERFFANWQKSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMMQYMTNNVLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNGGGGQQEPPPPPVVEPVEEPVVEGIHYGTHFTNMCLRNYPDRITTKRLVAEDRYQMLDSEELRD
>Node287
MTDLQTFHLPTESVDGTESDIQLAREMIQAWRSDGIFQVATDPAQDQKTQAAIEASRRFFRMPMEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQARWPCHGPAPWPDEEYQQSMKVFMDELGSVVGEKLLKLTALGLELDADDTDALTKLTADGWHHMRVLRFPARSEQTSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPNESAAGMYENEEPWTTFVKPVPSVLTVFPGDILQFLTNGFLLSTPHKVRLNTRERFALAYFHEPNFEARVRPLSGGPSSSSDSSEGYIHYGTHFTNMFMRSYPERITTRRILAEDRLSVLARLREAR
>Node276
VAQLETFTLPSDRVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSAAGMKEDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKKYDPDVVGGQQSPPPPPKKSPKESPKPKEGIHYGTHFTNMFLRNYPERITTKRMHAENRLAKLQQ
>Node301
MTNLQTFELPSEESVTGSQADIALGRAMIQAWRKDGIFQIATTPEQDQKTQQAMAASKRFFKMPMEEKSRCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDLPLDDQRVKAGGWPCHGPVPWPNEEYQQSMKAFMDELGDIIGEKLLKLTALGLGLERDINTLTDLTRDGWHHMRVLRFPAASSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFENEEPWTTFVKPVPSVFTVFPGDILQFMTDGYLLSTPHKVKLNTRERFALAYFHEPNFEAVVRPLDFFFGGQQSSSSDDSSDESSDERIHYGEHFTNMFMRCYPERITTQRIHKENRLAHLEEMKKREKRKRS
>Node310
QLKTFTLPSEKATGSPSDVQMGKAMINAWREDGILQVSMSPKQQDLFDKAFAASKRFFAMPPKQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVEAKWPCHGPCPWPDTDMKTPIQQYMDSLGNSSGETLLQMIEYGLSLDPKTLTSLTKDGWHHLRILRFPQNNNNTNGKGKDGGRGIGSHTDYGLLVIAAQDEVGGLFIRPPAEDEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDIMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVISPVAKALYYDGGQQPPPPPDDPPDEPPDEKIHYGTHFTNMFMRNYPDRITTERIIKEDRLKLLDD
>Node297
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKSQQRLYQAANAASRRFFAQPHAQKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDERVAQGWPCHGPCPWLDQDMKAVMNKYMNDLAVSSGEKLLQLIEMGLDIPTGSLTRYTNDGWHHMRVLRFPQNNRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHENERFFSNWEKSSAGLRENEAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGIYNNAAGGQQSPPPPIISPIESPGIHYGTHFTNMFLRNYPDRVTTARLREEGRYQLLGSEELRDDDDG
>Node300
NLQTFVLPSDKVTDSEENRKLGKAIVEAWQKDGILQIAMTPEQQALYKSANFASKRFFGKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMQSLGDVVGETLLQLTELGLNVPEGSLTNYTQDGWHHMRILRFPAINKTNGKGKDGGRGIGSHTDYGLLVIAAADDVGALMIRPPYQDEKLLANWETSAAGFKEDDERWVVFVPPAENVFTVFPGDMMQYMTNSALPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGKYDDAAGGQQEPPPPKKEPKEEPEKGIHYGKHFTDMFMRNYPQRITTQRLNDEGRYQLLEE
>Node298
LKTFTLPNQEVTGNEENRAIGKAMIEAWRKDGIFQIQMTPAQSDTLRQCFKTSKEFFSLPYEEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVQNKWPCHGPCPWPNESYKTKMNQLMDLLGESSGEKLLKLTALGLGLKEDDPNALTKLTEDGWHHMRILRFPQVDKTNGKGKTGGRGIGSHTDYGLLVIAGQDDVGGLFVRPKIEGESLLKNWQTSAAGFKENDDKWIIYVPPVENVLTVFPGDMMQFLTNDYLPSTPHKVGLNTRERYAFAYFHEPNFSSVLKPLPEYEKIHYGTHFTNMFMRNYPERITAKRMHAENRMDILEELE
>Node311
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFTQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKQDDGNALTRYTQDGWHHMRILRFPQTNKVNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNNTRERFAFAYFHEPNFSAVMKPLPGPPTFEPEAAGGQQEPPPPTPPTEEPTEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDD
>Node307
MTDLTTFHLPTETVTNTEAHRQLGREMIAAWQTDGIFQIALSTPQEQTTQEAFAESRRFFALPFETKSRHVSELTYSGYIASREEVTAGEADYSEIFTVCPDIPLDDARVREGWPCHGPVPWPGAAYRDRMKAFMGMLGTFFGERLLRLIALGLGLDLDDIETFTRLTRDGWHHMRVLRFPTAQSSESAARGIGAHTDYGMLVIAAQDDVGGLYIRPPIEGERRNRNWLASESTAGMYEHDEGWTTFVKPVPAVLTVFPGDFLQFLTGGHLLSTPHKVRLNTRERFAMAYFHEPNFDAWVRPLSAAPPAAAAAAAAAAEAAAAEPIHYGTHFTNMFMRCYPKRITTRRIEEEDLLSRLAALSEVAA
>Node304
MAELETFQLPSQSVSGSEADIALGLTMIRAWRRDGIFQVRMSPAQAEKAQRAFELSRRFFRQPLEAKARCVSDLTYSGYIASGEELTAGEADYSEVFTVCKDVPLTDARVQSRWPCHGPAPWPDEAYRQGMKAHTEELGSVVGERLLRLVALGLGLDRDIDALTALTRDGWHHMRVLRFPARSPTTTTRGIGAHTDYGMLVIAAQDDVGGLYVRPPVEGEKRPRNWLPGESSAGMYEHDEPWTTFVKPVPGVLTVFPGDILQFLTRGYLLSTPHKVVLNTRERFAMAYFHEPHFEACVRPLTAAPTRRRDDTRDETRDDEFIHYGTHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAW
>Node302
MTELQTFQLPGPTVEGGDADKALGQALIAAWQADGIFQIQATPEQEAATERALDASRAFFGRPLKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVVDKWPCHGPAPWPSEQYAAAMKDYMAVVGDIIGHRLLRLTALGLGLDDDMDHFTRLTDDGWHHMRVLRFPPADATSEERGIGSHTDYGLLVIAAQDDVGGLYIRPPVPGETRGRNWLPEESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTLAYFHEPSFQAVARPLPEPPPDDEPDDEDADDPEFIHYGTHFTNMFMRCYPERTTTARIEQEDRLAVLERLRKEAER
>Node299
LETFVLPSEKVSGSLSDRAMAKAMINAWRRDGILQIAMSPAQQRIYAQANAASRKFFKRTPAQKQACVNDSSYAGYIASGEEITAGIADYSEIFTVTKDLQPSDPRVMNKWPCHGPCPWPDKDMKSAMTDYMADLGSSSGEKLLQLIELGLDVPPGSLTKYTQDGWHHMRVLRFPARHRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPQQGESFFANWEQSVAGMKEDESGWVVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNVRERFAFAYFHEPNFRAVVKPLPGPVYNPNAAGGQQEPPPPPVVEPVDEPEVDGIHYGTHFTNMALRNYPDRITTKKLLEEGRYAMLDSEELRD
>Node305
MTDLQTFHLPTESITGTQSDIDLARQMIQAWRTDGIFQVATNTIQDRKTENAFEASRRFFRMPLEFKSQCISDLTYSGYIASGEEVTAGESDYSEIFTICKDVPLDDVRVQAQQWPCHGPVPWPDQNYQQSMKAFMDELGSIIGEKLLKLVALGLELDDINALTKLTKDGWHHMRVLRFPALSEKSTTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNNTRERFALAYFHEPNFNACVRPLDFFDDPPSSSSSSDDSSDESDDYIHYGTHFTNMFMRCYPDRITTRRIIDEDRLSVLALLKNALK
>Node303
MTELRTFRLPSETVTGSDDDVELGRQLIAAWQADGILQIATDAVQDRKTEEAMAASRRFFHDLSLEEKSRLVSDLTYAGYIASGEEVTDGVADYSEIFTVCKDVPLDDSRVVGKWPCHGPVPWPDESYRDAMREFMAMLGEAAGEKILKLTALGFGLDIDTFTRLTADGWHHMRVLRFPTRSEAVDDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVEPVPHVFTVFPGDILQFLTGGRLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDDPPHGGGDDHGDDHGDDDVIHYGTHFTNMFMRCYPDRVTTRRILDEDRLTVLSELRSSE
>Node308
LRTFRLPTAAVDGTDADRALGRALVAAWQSDGIFQIHATPAQQEATERALEASRAFCRRPLKEKAGYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKGYMTAVGEEVGHRLLRLTALGLGLDDPDHFTRLTADGWHHMRVLRFPPADAGSEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRHEHEEEPWTYVTPVPAVLTVFPGDIMQFMTGGALLSTPHKVRLAADRERYTLAYFHEPAFNAVARPLDTDTTTTTADTATETTADPEYIHYGTHFTRMFMRCYPERVTTTRIETEGRLKVLDRLREEADLR
>Node306
MTDLQTFHLPTESVDGTESDIQLAREMIQAWRTDGIFQVATDPAQDQKTQAAIEASRRFFRMPMEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDEEYQQSMKAFMDELGSIIGEKLLKLTALGLGLDRDDIDALTKLTRDGWHHMRVLRFPARSEQTSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESAAGMYENEEPWTTFVKPVPSVLTVFPGDILQFLTNGFLLSTPHKVRLNTRERFALAYFHEPNFEARVRPLSGGPSSSSDDSSDESSDEYIHYGTHFTNMFMRSYPERITTRRILAEDRLSVLARLREAR
>Node309
VAQLETFTLPSDRVTGSKGDRAMGKALVDAWRRDGILQVSMNDKQQAIFNEATAASKAFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDMAEPRVRAKWPCHGPAPWPDYEMKQPMTTYMDYLGESSGEKLLQLAELGLNVPAGSLTNLTADGWHHMRILRFPPTHNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYDGEHYYANWEKSSAGMKEDDEGWHHYVPPVPNVFTVFPGDIMQYITNSYLPSTPHKVGLNTRDRFAFAYFHEPSFQAVAKPLPGKYDPDVVGGQQSPPPPPKKSPKESPKPKEGIHYGTHFTNMFLRNYPERITTKRMHAENRLALLQQ
>Node315
MTNLQTFHLPTEESVTGTQSDIALGRAMIQAWRKDGIFQVATTPAQDQKTQQAMKASKRFFRMPMEEKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDLPLDDARVKAGGWPCHGPVPWPNEEYQQSMKAFMDELGSIIGEKLLKLTALGLGLERDDINTLTDLTRDGWHHMRVLRFPAASSSKSSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMFENEEPWTTFVKPVPRVFTVFPGDILQFMTNGYLLSTPHKVKLNTRERFALAYFHEPNFEAVVRPLDSFSFGGQQSSSSSDDSSDESSSDEYIHYGEHFTNMFMRCYPERITTRRILKENRLAHLEEMKKEKRKRS
>Node322
QLKTFTLPSEKVTGSPGDIQMGKALINAWREDGILQIAMDPKQQALFDKAFAASKRFFAMPPKQKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVKAKWPCHGPCPWPDTEMKTPIQQYMDSLGTSSGERLLQLIEYGLNLDQPKTLTSLTQDGWHHLRILRFPQTNNTNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKLLENWKKSAAGFREDDERWVVYVPPVPGVFTVFPGDMMQFMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPIPQDLYYDGGQQPPPPPDDPPDEPKIHYGTHFTNMFMRNYPDRITTERIIREDRMKLLDD
>Node312
LETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPHAQKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQDMKTVMNRYMNDLAVSSGEKLLQLIEMGLDIPNTGSLTRYTNDGWHHMRVLRFPQRDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQDERFFANWEKSSAGLRENDAGWVVYVPPVAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVRPLPGIYNNAAGGQQSPPPPIISPIGSPIIGIHYGTHFTNMFLRNYPDRVTTARLREEGRYQLLGSEELRDEDDG
>Node314
NLQTFVLPSEKVSGSEEDKKLGKAIVEAWQKDGILQIAMKPEQQALYKAANLASKRFFSKPYAQKAACVDSQTYSGYIASGEELTDGIADYSEIFTVTKDLELDEPRVVAKWPCHGRCPWPDYEMQNPMQRYMNSLGESSGETLLKLTELGLDVPEGSLTNYTQDGWHHMRILRFPAINKTNGKGKDGGRGIGSHTDYGLLVLAAADDVGALLIRPPYQDENFANWEKSAAGFKEDDEGWVFVPPAENVFTVFPGDMMQYLTNSALPSTPHKVGLNIRERFAFAYFHEPSFQAVIKPLPGPKYDDAAGGQEPPPPKKEPKEEPGVHYGKHFTDMFMRNYPQRITTQRLVDEGRYNLLEE
>Node313
LQTFTLPSQEVTGSQSDIALGKAMIEAWRKDGIFQIQMTPAQAATLRQAFKASKSFFSLPYEEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNESYKTTMKQLMSLLGESSGEKLLKLTALGLGLAEDDPNALTKLTEDGWHHMRVLRFPQVDKTNGKGKAAGRGIGSHTDYGLLVIAGQDDVGGLFVRPPIEGEKLLKNWEKSAAGFKENDDEKWTTYVPPVENVLTVFPGDMMQYLTNDYLPSTPHKVGLNTRERYAFAYFHEPNFSSVLKPLPEPYGGSSDDQQSSSSSDDSSDESSDEGIHYGTHFTNMFMRNYPERITAKRMKAENRMDILEELE
>Node323
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFGLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFSQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKTQDDGNALTRYTQDGWHHMRILRFPEANNVNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYANWKESAAGKHEDAADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNNTRERFAFAYFHEPNFSAVMKPLPGPAEPEAAGGQEPPPTPPTEEPETEGIHYGTHFTNMFMRNYPERITADRMRAEGRMALLDD
>Node318
MAELETFQLPSESVSGSEADIALALTMIRAWRRDGIFQVQMSPAQEQKAQRAFEASRRFFRQPLEAKARCVSDLTYSGYIASGEELTAGEADYSEVFTVCKDVPLTDARVQARWPCHGPAPWPDEAYRQAMKAHMEELGSVVGERLLRLVALGLGLDRDDIDALTRLTRDGWHHMRVLRFPARSPTTTTRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRPRNWLPGESSAGMYEHDEPWTTFVKPVPGVLTVFPGDILQFLTRGYLLSTPHKVRLNTRERFAMAYFHEPHFEACVRPLTAAPTGGGDDTGDETGDDEFIHYGTHFTNMFMRCYPDRVTTQRILDEDRLTTLAWLREEAVQRAW
>Node316
MTELQTFRLPSSAVEGGDADKALGRALIAAWQSDGIFQIQATPEQEAATERALDASRAFFSRPLKEKAGHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPEDDARVVDGWPCHGPAPWPSEQYAAAMKDYMGVVGDIIGHRLLRLTALGLGLDDDMDHFTRLTDDGWHHMRVLRFPPADATSEERGIGSHTDYGLLVIAAQDDVGGLYIRPPVPGEERGRNWLPEESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFITGGTLLSTPHKVRLADRERYTLAYFHEPSFQAVARPLPDPPPDDDPDDEDADDPEFIHYGTHFTNMFMRCYPERTTTARIEAEGRLAVLERLRKEAER
>Node319
MTDLQTFHLPTESITGTQSDIDLARQMIQAWRTDGIFQVATNTIQDRKTENAFEASRRFFRMPLEFKSQCISDLTYSGYIASGEEVTAGESDYSEIFTICKDVPLDDARVQAQQWPCHGPVPWPDQEYQQSMKAFMDELGSIIGEKLLKLVALGLELDDINALTKLTKDGWHHMRVLRFPALSQKSTTRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNNTRERFALAYFHEPNFNACVRPLDFFDDPPSSSSSSDDSSDESDDYIHYGTHFTNMFMRCYPDRITTRRIIDEDRLSVLAVLKNANK
>Node317
MTELRTFRLPSETVTGSDSDIALGRQLIAAWQADGILQIATDAVQDQKTEEAMAASRRFFHDLPLEEKSRLVSDLTYAGYIASGEEVTDGVADYSEIFTVCKDVPLDDSRVVDKWPCHGPVPWPDESYRQAMRTFMDMLGEAAGEKILKLTALGFGLDADIDTFTRLTADGWHHMRVLRFPTRSEATDDRGIGAHTDYGMLVLAAQDDVGGLYVRPPIEGEERPRNWLPDESAAGCFQDAEGWTTFVKPVPNVLTVFPGDILQFLTGGRLLSTPHKVLLNTRERFTMAYFHEPNFEAVVRPLDDDPAGGGDDAGDDDGDDDVIHYGTHFTNMFMRCYPDRVTTRRILAEDRLSVLAELRSAE
>Node320
LQTFRLPTAAAVTGTDADRALGRALVAAWQSDGIFQIHATPAQQAATERALAASRAFCRRPLKEKARYVSDLSYSGYVASGEEETAGERDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKDYMTVVGEIIGHRLLRLTALGLGLDADDPDHFTRLTADGWHHMRVLRFPPADATSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHEEPWTYVTPVPAVFTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLDTDPTTAADPAAEDAADPEYIHYGTHFTRMFMRCYPERVTTARIEAEGRLKVLDRLREEAEDR
>Node330
QLQTFTLPSEQVTGSPGDIAMGKALIDAWRRDGILQIAMDPKQQKLLDEAFAASKRFFAKPHKEKAACVDSQSYAGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAKWPCHGPCPWPDIEMKTPMQRYMDYLGTSSGERLLQLIEYGLNLPQPNSLTKYTQDGWHHMRILRFPATNNTNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKFEENWKKSAAGFREDDDRWVVYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLPEDLHHAAGGQQSPPPPDDSPDEPGIHYGTHFTNMFMRNYPDRITTKRMIRENRMSLLDD
>Node324
QLETFTLPSEKVSGSVSDKAIADAMIGAWRRDGILQIGMSKAQQRLYQAANAASRRFFSRPHAQKQACVDDMSYSGYIASGEEITDGIADYSEIFTVTKDLPHTDKRVAQGWPCHGPCPWPDQDMKTVMNRYMNDLAVSSGEKLLQLIEMGLDVPNTGSLTRYTNDGWHHMRVLRFPQRDRTNGKGKKKGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQDERFFANWEKSSAGLRENDAGWVVYVPPTAGTFTVFPGDMMQYMTNNFLQSTPHKVGLNVRERFAFAYFHEPNFRSVVKPLPGIYNNAAGGQQSPPPPIISPIGSPIIGIHYGTHFTNMFLRNYPDRVTTARLREEGRYQLLGSEELRDEDDG
>Node325
DLQTFTLPSQQVTGSQSDIALGKAMIEAWRKDGIFQIQMNPSQAATLRQAFKASKSFFSLPYEEKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNEAYKTTMKQYMSLLGESSGEKLLQLTALGLGLKLDDPNALTKLTRDGWHHMRVLRFPQVDKTNGKGKAAGRGIGSHTDYGLLVIAAQDDVGGLFVRPPIEGEKYKKNWEKSAAGFKENDDEKWTTYVPPVPNVLTVFPGDMMQYLTNSYLPSTPHKVGLNTRERYAFAYFHEPNFSAVLKPLPEPSGGSSDDQQSSSSSDDSSDESSSDEGIHYGTHFTNMFMRNYPERITAKRMRAENRMDILEELE
>Node327
MTDLQTFQLPEESVTGTESDIALARQMIRAWRRDGIFQVAMNPAQEQKTQRAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLTDARVQAQWPCHGPAPWPDEAYRQAMKAFMDELGSVVGEKLLRLTALGLGLDRDDIDALTRLTRDGWHHMRVLRFPARSSTTSSRGIGAHTDYGLLVIAAQDDVGGLYVRPPVEGEKRNRNWLPGESSAGMYEHEEPWTTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFAMAYFHEPNFEACVRPLSGGPSSSSDDSSDESGDDEFIHYGTHFTNMFMRCYPDRVTTRRILDEDRLSTLAWLREEAAW
>Node328
MTDLQTFHLPTESITGTQSDIDLAREMIQAWRTDGIFQVATNTIQDRKTESAFEASRRFFRMPMEFKSQCISDLTYSGYIASGEEVTAGESDYSEIFTICKDVPLDDARVQAQWPCHGPVPWPDQEYQQSMKAFMDELGSIIGEKLLKLTALGLGLDDDINALTKLTRDGWHHMRVLRFPALSQKSSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLSTESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFDDPPSSSSDDSSDESSDDEYIHYGTHFTNMFMRCYPDRITTRRIINEDRLSVLARLKNAR
>Node329
LQTFRLPTAAAVTGTDADRALGRALVAAWQSDGIFQIHATPAQQAATERALAASRAFCARPLKEKARYVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPADDPRVIDRWPCHGPAPWPSPGYARAMKDYMAVVGEIIGHRLLRLTALGLGLDADDPDHFTRLTADGWHHMRVLRFPPADATSSERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEERGRNWLPGESMAGRYEHEEPWTYVTPVPAVFTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPAFNAVARPLEPDPPPAADPAAEDAADPEYIHYGTHFTNMFMRCYPERVTTARIEAEGRLKVLDRLREEAEDR
>Node336
QLQTFTLPSEQVTGSPGDIAMGKALIDAWRRDGILQIAMDPKQQKLLDEAFAASKRFFAKPHKEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAKWPCHGPCPWPDAEMKDPMQRYMDYLGESSGEKLLQLIEYGLNVPQPGSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEDEKYAANWEKSAAGFREDDDGWVVYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLPGKADDAAGGQQSPPPPKKSPKEPGIHYGTHFTNMFMRNYPDRITTKRMIRENRMALLDD
>Node332
LETFVLPSEKVSGSVSDRAMAKAMIDAWRRDGILQIGMSQAQQRLYQSANAASRRFFSRPHAQKQACVDDQSYSGYIASGEEITDGIADYSEIFTVTKDLPHSDKRVANGWPCHGPCPWPDQDMKSVMSRYMADLGASSGEKLLQLIELGLDVPAGSLTRYTNDGWHHMRVLRFPARHRTNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFVRPPHQGERFFANWEKSAAGMKEDDSGWVVYVPPTPGVFTVFPGDMMQYMTNNFLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNAAGGQQEPPPPPVVEPVEEPVVEGIHYGTHFTNMCLRNYPDRITTKRLLEEGRYQMLDSEELRD
>Node333
NLQTFVLPSEKVTGSESDKKLGKALVAAWQQDGILQIAMNPEQQALYKAANAASKRFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLDLDDPRVLAKWPCHGRCPWPDAEMRDPMKRYMDSLGESSGEKLLQLIELGLNVPEGSLTNLTQDGWHHMRILRFPATNDTNGKGKDGGRGIGSHTDYGLLVLAAADDVGALFIRPPYQDESFANWEKSAAGFKEDDDEGWVFVPPAENVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLPGYDDAAGGQQEPQPPPRREPREPEGVHYGKHFTDMFMRNYPERITTQRLVDEGRYKLLEE
>Node337
VGHLQTFELPSESLSGSAADKALGKSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFSLDYKKKAACLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKDDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSFLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPTFEPEAAGGQQEPPPPPTTEPTEEPETEGIHYGTHFTNMFMRNYPERITAERMRAEGRMALLDD
>Node334
MTDLQTFHLPTESITGTQSDIDLAREMIQAWRTDGIFQVATNAIQDRKTESAFEASRRFFRMPMEFKSQCISDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDQEYQQSMKAFMDELGSIIGEKLLKLTALGLGLDDDINALTKLTRDGWHHMRVLRFPAASQKSAARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPTESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFDACVRPLFDDPPSSSSDDSSDESSDDEYIHYGTHFTNMFMRCYPDRITTRRIINEDRLSVLARLKNAR
>Node335
MTELQTFRLPTSAVTGTDADRALGRALIAAWQSDGIFQIQATPEQEAATERALAASRAFFARPLKEKARHVSDLTYSGYVASGEEETAGEKDGSEIFTVCPDIPADDARVVEGWPCHGPAPWPSEGYARAMKDYMSVVGEIIGHRLLRLTALGLGLDDDMDHFTRLTADGWHHMRVLRFPPADATSEERGIGAHTDYGLLVIAAQDDVGGLYIRPPVPGEERGRNWLPEESMAGRYENEEPWTTFVTPVPSVFTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPSFQAVARPLPDPPPDDDPDDEDADDPEYIHYGTHFTNMFMRCYPERVTTARIEAEGRLAVLERLREEAER
>Node342
QLQTFTLPSEQVTGSPGDRAMGKALIDAWRRDGILQIAMDPKQQKLLDEAFAASKRFFAKPHKEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAKWPCHGPCPWPDAEMKDPMQRYMDYLGESSGEKLLQLIEYGLNVPQPGSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSAAGMKEDDDGWVVYVPPVPGVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLPGKADDAAGGQQSPPPPKKSPKEPKEGIHYGTHFTNMFMRNYPDRITTKRMIAENRMALLEE
>Node338
LETFVLPSEEKVSGSVSDRAMAKAMINAWRRDGILQIAMSPAQQRLYQSANAASRRFFSRPHAQKQACVDDQSYSGYIASGEEITDGIADYSEIFTVTKDLPHSDPRVMNKWPCHGPCPWPDQDMKSAMTRYMADLGSSSGEKLLQLIELGLDVPAGSLTKYTNDGWHHMRVLRFPARHRTNGKGKAGGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQGEQFAANWEKSAAGMKEDDDSGWVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNNTRERFAFAYFHEPNFRSVVKPLPGVVYNPNAAGGQQEPPPPPVVEPVEPVVGIHYGTHFTNMCLRNYPDRITTKRLLEEGRYQMLDSEELRD
>Node339
LQTFVLPSEKVTGSASDRKMGKALVAAWQQDGILQIAMNPEQQALYKAANAASKRFFSKPYAQKAACVDSQSYSGYIASGEELTDGIADYSEIFTVTKDLDLDDPRVLAKWPCHGRCPWPDAEMRDPMKRYMDSLGESSGEKLLQLIELGLNVPEGSLTNLTQDGWHHMRILRFPATNDTNGKGKDGGRGIGSHTDYGLLVIAAADDVGALFIRPPYQDESFANWEKSAAGMKEDDDDGWVFVPPAENVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLPGYDDAAGGQQEPQPPPRREPREPKGIHYGKHFTNMFMRNYPERITTQRLVDEGRYKLLEE
>Node343
VGHLQTFELPSETLSGSPADKALGRSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFSMPYKEKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKQDDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGTFEPEAAGGQQEPPPPTTEPTEEPEPTEGIHYGTHFTNMFMRNYPERITAERMRAEGRMALLDD
>Node340
MTDLQTFHLPTESVTGTQSDIELAREMIQAWRTDGIFQVATDPVQDRKTEQAFEASRRFFRMPMEAKSQCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDEEYQQSMKAFMDELGSIIGEKLLKLTALGLGLDDDINALTKLTRDGWHHMRVLRFPAASSKTSARGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNTRERFALAYFHEPNFEACVRPLFEEPPSSSSSDDSSDESSDDEYIHYGTHFTNMFMRCYPERITTRRILKEDRLSVLARLKEAR
>Node341
MTELRTFRLPTDDAVTGTDADRALGREMIRAWQTDGIFQVAATPEQQAATQAALAASRRFFARPLAEKARHVSDLTYSGYIASSGEEVTAGEADYSEIFTVCPDIPADDARVTEGGWPCHGPVPWPDEGYRRAMKAYMSAVGEIIGHRLLRLTALGLGLDDMDAFTRLTDDGWHHMRVLRFPAASATTEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLPEESTAGMYENEEPWTTFVEPVPGVFTVFPGDIMQFMTGGALLSTPHKVRLADRERYTLAYFHEPSFQAVARPLAGPDGGPDGAGGDEDAGDPDYIHYGTHFTNMFMRCYPERVTTRRIHAEGRLAVLERLREKAEERLR
>Node344
LETFVLPSEEKVSGSASDRAMAKAMIEAWRRDGILQIAMSPQQQRIYQSANAASRRFFSRPHAQKQACVDDQSYSGYIASGEEITDGIADYSEIFTVTKDLPEDDPRVMNKWPCHGPCPWPDQDMKSAMTRYMDDLGSSSGEKLLQLIELGLHVPEGSLTRYTNDGWHHMRVLRFPARNNRTNGKGKAGGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQGEQYAANWEKSAAGMKEDDDSGWVYVPPTPGVFTVFPGDMMQYMTNSFLQSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNPNAAGGQQEQPPPAAPPAEPAGIHYGTHFTNMCLRNYPDRITTKRLLEEGRYKMLDD
>Node345
LQTFVLPSEKVTGSASDRAMGKALVDAWQRDGILQIAMNPEQQAIYKAAQAASKRFFSKPHAQKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLDDPRVLAKWPCHGPCPWPDAEMRDPMKRYMDYLGESSGEKLLQLIELGLNVPEGSLTNLTQDGWHHMRILRFPATNKTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQDEKYANWEKSAAGMKEDDDGWVYVPPVPNVFTVFPGDMMQYMTNSALPSTPHKVGLNTRERFAFAYFHEPSFQAVIKPLPGPYDDAAGGQQEPPPPKREPREPEGIHYGTHFTNMFMRNYPDRITTQRLVEEGRYKLLEE
>Node347
VGHLQTFELPSETLTGSPGDKALGRSMIDAWKRDGILQISMDPINRKLADAAFLCSKKFFSLPYKEKASCLDDQSFAGYIASGEEITDNIADYSEIFTVTKDLALSDPRVQQKWPCHGPCPWPFGQMKTVMQAYMDYLGESSGEKMLQLIAWGLGMKQDDGNALTRYTQDGWHHMRILRFPERNNVNGKGKDDGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGETYYANWKESAAGKHEDADGWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGTFEPEAAGGQQEPPPPTTEPTEEPEPTEGIHYGTHFTNMFMRNYPERITASRMRAEGRMALLDD
>Node346
MTELRTFRLPTDDSVTGTAADRALGREMVRAWQTDGIFQVAATPEQQAATQAALAASRRFFARPLAEKARYVSDLTYSGYIASSGEEVTAGEADYSEIFTVCPDIPADDARVTEGWPCHGPVPWPDEGYRRAMKAYMSAVGEIIGERLLRLVALGLGLDDMDALTRLTRDGWHHMRVLRFPARSATTEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLPEESTAGMYENEEPWTTFVEPVPGVFTVFPGDILQFLTGGALLSTPHKVRLADRERYTLAYFHEPSFQATVRPLAGAGGGDGAGGEDGGDPEPIHYGTHFTNMFMRCYPDRVTTRRIHAEGRLAVLERLREKAERLR
>Node348
LETFVLPSEKVSGSASDRAMGKAMIEAWRRDGILQIAMSPAQQRIYQAANAASRRFFSRPHAQKAACVDPQSYSGYIASGEEITDGIADYSEIFTVTKDLPEDDPRVMNKWPCHGPCPWPDQDMKSAMNRYMDDLGSSSGEKLLQLIELGLDVPEGSLTKYTNDGWHHMRILRFPARNRTNGKGKAAGRGIGSHTDYGLLVIAAQDEVGGLFVRPPHQGEKYAANWEKSAAGMKEDDDSGWVVYVPPVPGVFTVFPGDMMQYMTNSFLPSTPHKVGLNTRERFAFAYFHEPNFRSVVKPLPGPYNNAAGGQQEPPPPPVVPPAEEPEPVGIHYGTHFTNMCLRNYPDRITTKRLLEEGRYKMLDD
>Node350
VGNLQTFELPSETLTGSPGDKALGRSMIDAWRRDGILQISMDPINRKLADAAFLCSKKFFSLPYKEKASCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPLSDPRVQQKWPCHGPCPWPFAQMKTVMQAYMDYLGESSGEKMLQLIAWGLGLKQDDGNALTKYTQDGWHHMRILRFPERNNVNGKGKDGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGETYYANWKKSAAGMKEDDDKWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFDDAAGGQQEPPPPTTEPTEEPETGIHYGTHFTNMFMRNYPERITANRMRSENRMALLEE
>Node349
MTELRTFRLPTEESVTGTAADRALGREMVRAWQTDGIFQVAADPAQQAATQAAMAASRRFFARPLAEKARYVSDLTYSGYIASSGEEVTAGEADYSEIFTVCPDIPADDARVRAGWPCHGPVPWPDEGYRRAMKAYMSAVGEIIGERLLRLVALGLGLDDMDALTRLTRDGWHHMRVLRFPARSATTEERGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLPEESTAGMYENEEPWTFTFVEPVPGVFTVFPGDILQFLTGGALLSTPHKVRLAADRERYTLAYFHEPSFEATVRPLAGAGGGDGGGDEDGGDPEPIHYGTHFTNMFMRCYPDRVTTRRILAEDRLAVLARLREEARLR
>Node351
LETFVLPSEEKVTGSASDRAMGKAMIEAWRRDGILQIAMSPQQQQIYQAAQAASRRFFSRPHAQKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDDPRVMNKWPCHGPCPWPDADMKNAMTRYMDDLGSSSGEKLLQLIELGLHVPEGSLTKYTNDGWHHMRILRFPATNRTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPHQGEQYAANWEKSAAGMKEDDDSGWVYVPPVPGVFTVFPGDMMQYMTNSFLPSTPHKVGLNTRERFAFAYFHEPNFRAVVKPLPGAEYNPNAAGGQQEPPPPPVAEPAEEPAGIHYGTHFTNMFMRNYPDRITTKRLLEEGRYKLLDE
>Node353
VGNLQTFTLPSEQITGSPGDRALGRSMIDAWRRDGILQISMDPTNRKLADAAFAASKKFFSLPYKEKAKCVDDQSFAGYIASGEEITDNIADYSEIFTVTKDLPLSDPRVQQKWPCHGPCPWPNAQMKTVMQAYMDYLGESSGEKMLQLIAYGLGLAQDDGNALTKYTQDGWHHMRILRFPETNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWKKSAAGMKEDDDKWVVYVPPVPDVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFDDAAGGQQEPPPPTTEPTEEPETGIHYGTHFTNMFMRNYPERITANRMRSENRMALLEE
>Node352
MTELRTFQLPSESVTGSDADRALGREMVRAWQTDGIFQVAADPAQQAATQAAMAASRRFFALPLAEKARYVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDIPADDARVRAGWPCHGPVPWPDEAYRRAMKAFMSELGSIIGEKLLRLVALGLGLDADDMDALTRLTADGWHHMRVLRFPARSAATTARGIGAHTDYGLLVIAAQDDVGGLYVRPPVPGEKRNRNWLPEESTAGMYENEEPWTTFVEPVPGVFTVFPGDILQFLTGGALLSTPHKVRLADRERYTMAYFHEPNFQATVRPLAGAGGDGGGDDAGDPDEDGDDDEYIHYGTHFTNMFMRCYPDRVTTRRILAEDRLAVLARLREAR
>Node356
LETFTLPSEEKVTGSASDRAMGKALVDAWRRDGILQIAMNPEQQAIYKAAQAASKRFFAKPHNEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVLAKKWPCHGPCPWPDAEMRDPMKRYMDYLGESSGEKLLQLTELGLGVPEGSLTNYTQDGWHHMRILRFPPTNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYQGEKYYANWEKSAAGMKEDDDGWVYVPPVPNVFTVFPGDMMQYMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVVKPLPGPPKEYDDAGGQQEPPPPPKKEPKEEPGIHYGTHFTNMFMRNYPDRITTQRLLAEGRYKLLEE
>Node357
MTELRTFRLPSEETVTGSEADIALGREMVRAWQTDGIFQVAADPAQDQATQEAMAASRRFFRLPLEEKARHVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPADDARVRAGWPCHGPVPWPDEAYRQAMKAFMSALGSIIGEKLLRLIALGLGLDDIDALTRLTADGWHHMRVLRFPARSAQTAARGIGAHTDYGMLVIAAQDDVGGLYVRPPVPGEKRNRNWLPEESTAGMYENEEEPWTFVKPVPNVLTVFPGDILQFLTGGHLLSTPHKVRLNNTRERFTMAYFHEPNFEATVRPLGAGSDDPPAGGGGGGGAGGEGVIHYGTHFTNMFMRCYPDRITTRRILAEDRLAVLAARLREEARLR
>Node358
LETFTLPSEEKVTGSPGDRAMGKALVDAWRRDGILQIAMNPEQQAIYKAAQAASKRFFAKPHNEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLPLDEPRVLAKKWPCHGPCPWPDAEMRDPMKRYMDYLGESSGEKLLQLTEYGLGVPEGSLTNYTQDGWHHMRILRFPPTNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSSAGMKEDDDGWVYVPPVPNVFTVFPGDMMQYMTNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVVKPLPGPPKEYDDAGGQQEPPPPPKKEPKEEPGIHYGTHFTNMFMRNYPDRITTQRLLAEGRYKLLEE
>Node360
VGQLETFTLPSERVTGSPGDRAMGKALVDAWRRDGILQIAMNPKQQAIYDAAQAASKRFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLDEPRVKAKWPCHGPCPWPDAEMRDPMKRYMDYLGESSGEKLLQLTEYGLGVPQEGSLTNYTQDGWHHMRILRFPPTNKTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSSAGMKEDDDGWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVAKPLPGKYDDAAGGQQSPPPPKKSPKESPKKGIHYGTHFTNMFMRNYPDRITTQRLLAENRYSLLEE
>Node362
VGQLETFTLPSERVTGSPGDRAMGKALVDAWRRDGILQIAMNPKQQAIFDEATAASKRFFAKPHEEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAKWPCHGPCPWPDAEMKDPMKRYMDYLGESSGEKLLQLIEYGLNVPQPGSLTNYTQDGWHHMRILRFPPTNNTNGKGKEEGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSSAGMKEDDDGWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVAKPLPGKYDDAAGGQQSPPPPKKSPKESPKKGIHYGTHFTNMFMRNYPDRITTKRMLAENRMALLEE
>Node363
MTDLQTFHLPTESVTGTESDIQLAREMIRAWRTDGIFQVATDPAQDQKTQQAFEASRRFFRMPLEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDEAYRQAMKAFMDELGSIIGEKLLRLTALGLGLDDIDALTRLTRDGWHHMRVLRFPARSSQTTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFAMAYFHEPNFEACVRPLSGGPSSSSSDDSSDESYIHYGTHFTNMFMRCYPDRITTRRILAEDRLSVLARLREA
>Node364
VGQLQTFTLPSEQVTGSPGDRAMGKALIDAWRRDGILQIAMDPKQQKLLDEAFAASKRFFAKPHKEKAACVDSQSYSGYIASGEEITDGIADYSEIFTVTKDLDLSEPRVKAKWPCHGPCPWPDAEMKDPMQRYMDYLGESSGEKLLQLIEYGLNVPQPGSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSAAGMKEDDDGWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPSFQAVMKPLPGKADDAAGGQQSPPPPKKSPKEPKGIHYGTHFTNMFMRNYPDRITTKRMLAENRMALLEE
>Node365
MTDLQTFHLPTESVTGTESDIQLAREMIQAWRTDGIFQVATDPAQDQKTQQAFEASRRFFRMPMEAKARCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDEEYQQAMKAFMDELGSIIGEKLLKLTALGLGLDDIDALTKLTRDGWHHMRVLRFPARSSQTTSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFLTNGYLLSTPHKVRLNTRERFALAYFHEPNFEACVRPLSSGGPPSSPSSSDDSSDESDDYIHYGTHFTNMFMRCYPERITTRRILAEDRLSVLARLREARR
>Node366
VGQLQTFTLPSEQVTGSPGDRALGKAMIDAWRRDGILQIAMDPTQQKLADEAFAASKRFFAKPYKEKAKCVDDQSFSGYIASGEEITDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPNAEMKTTMQAYMDYLGESSGEKLLQLIAYGLNLPQPNSLTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSAAGMKEDDDGWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGKFDDAAGGQQSPPPPKKSPKESPSKGIHYGTHFTNMFMRNYPERITAKRMLAENRMDLLEE
>Node367
MTDLQTFHLPTESVTGTQSDIQLAREMIQAWRTDGIFQVATDPAQDQKTQQAFEASRRFFRMPMEAKSRCVSDLTYSGYIASGEEVTAGEADYSEIFTVCKDVPLDDARVQAQWPCHGPVPWPDEEYQQSMKAFMDELGSIIGEKLLKLTALGLGLDDINALTKLTRDGWHHMRVLRFPAASSKTSSRGIGAHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMYENEEPWTTFVKPVPSVLTVFPGDILQFMTNGYLLSTPHKVRLNNTRERFALAYFHEPNFEACVRPLSSGGPPSSSSSSDDSSDESDDYIHYGTHFTNMFMRCYPERITTRRILKEDRLSVLARLKEARLK
>Node368
VGNLQTFTLPSEQVTGSPGDRALGKAMIDAWRRDGILQISMDPTQRKLADEAFAASKRFFSLPYKEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPLSDPRVQAKWPCHGPCPWPNAQMKTTMQAYMDYLGESSGEKMLQLIALGLGLPQDDPNALTKYTQDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSAAGMKEDDDKWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVMKPLPGPFDDAAGGQQSPPPPKKSPKESPSKGIHYGTHFTNMFMRNYPERITAKRMRAENRMDLLEE
>Node370
DLQTFELPSQQVTGSPGDIALGKAMIDAWRRDGILQISMDPSQRKVADKAFAASKRFFSLPYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNAQMKTTMKAYMDYLGESSGEKLLQLIALGLGLPRDDPNALTKYTRDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYANWEKSAAGMYENDDDKWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVLKPLPGPFDSDSSGGQQSSSSSKKSSKESSSKEGIHYGTHFTNMFMRNYPERITAKRMRAENRMDVLEE
>Node371
MTDLQTFQLPTESVTGTQSDIQLGREMIQAWRRDGIFQIAMDPAQDAKTQQAFKASKRFFALPMEEKSRCVSDLTYSGYIASGEEITAGKADYSEIFTVCKDLPQDDPRVQAQWPCHGPVPWPNEEYQQAMKAFMDELGSIIGEKLLKLTALGLGLDDDINALTKLTRDGWHHMRVLRFPAASSGKTSSRGIGSHTDYGLLVIAAQDDVGGLYIRPPVEGEKRNRNWLPGESSAGMYENEEEPWTTFVKPVPSVFTVFPGDILQFLTNGYLLSTPHKVKLNTRERFALAYFHEPNFEAVIRPLSSGGQSSSSSDDSSDESDRIHYGTHFTNMFMRCYPERITTRRILKENRLAVLEEMKEELK
>Node372
DLQTFELPSQQVTGSPADIALGKAMIDAWRKDGILQIAMDPSQRKVAEKAFKASKRFFSLPYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNAAMKTTMKAYMDYLGSSSGEKLLQLIALGLGLPRDDPNALTKLTRDGWHHMRILRFPATNNTNGKGKEGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPYEGEKYYKNWEKSAAGMYENDDDKWVVYVPPVPNVFTVFPGDMMQYITNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVLKPLPGPFDSGSSGGQQSSSSSDDSSDESSSSDEGIHYGTHFTNMFMRNYPERITAKRMRAENRMDVLEELKE
>Node373
DLQTFELPSEQVTGSQSDIALGKEMIEAWRKDGILQIAMDPAQAATAQQAFKASKRFFALPYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNEAYQQAMKAYMDYLGSIIGEKLLQLTALGLGLQLDDPNALTKLTRDGWHHMRVLRFPATNATNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGEKYYKNWEKSAAGMYENDEKWTTYVPPVPNVFTVFPGDMMQFLTNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVIKPLPESGSGSSGGQQSSSSDDSSDESSDEGIHYGTHFTNMFMRNYPERITAKRMRKENRMDVLEELKE
>Node374
DLQTFELPSQQVTGSQSDIALGKAMIEAWRKDGILQIAMDPSQAATAQQAFKASKRFFSLPYEEKAKCVDDQSFAGYIASGEEITDGIADYSEIFTVTKDLPQSDPRVQAKWPCHGPCPWPNEAYKTTMKAYMDYLGSSSGEKLLQLTALGLGLKLDDPNALTKLTRDGWHHMRVLRFPATNKTNGKGKAGGRGIGSHTDYGLLVIAAQDDVGGLFIRPPIEGEKYYKNWEKSAAGMKENDEKWTTYVPPVPNVFTVFPGDMMQYLTNSYLPSTPHKVGLNTRERFAFAYFHEPNFSAVLKPLPESGSGSSGGQQSSSSSDDSSDESSDEGIHYGTHFTNMFMRNYPERITAKRMRAENRMDILEELKE
"""

## <mark>Input Final IQ Tree in newick format</mark>

- This is generated by exporting the newick tree after opening the "MAP.tree" from the Results folder of bp-analyze.

In [5]:
newick_tree = "(117_AKF86718, (116_WP_248542106, ((((((((128_TMH32900, (((((((((215_EOD48827, (214_KAH7065587, (216_KAF4302266, (223_KAF2851311, ((((((241_KAI4923174, ((242_XP_049196726, 244_XP_049254215) 429 , (243_XP_049228299, (245_KAI4918591, 246_KAI4941455) 431 ) 430 ) 428 ) 427 , (239_KAI4945826, 240_XP_046022954) 432 ) 426 , ((232_KNG49275, 233_XP_043175567) 434 , ((236_RYO28243, (237_KAH6843611, 238_XP_038783030) 437 ) 436 , (234_KAG9196060, 235_KAI5368897) 438 ) 435 ) 433 ) 425 , ((230_KAI1523220, 231_KAI2481640) 440 , (229_KAE8847895, (227_CAA9960346, 228_EFQ89844) 442 ) 441 ) 439 ) 424 , (224_RMZ70101, (225_USP78727, 226_XP_008020756) 444 ) 443 ) 423 , ((219_XP_007718836, (220_XP_014551369, (221_XP_007693478, 222_XP_007695071) 448 ) 447 ) 446 , (217_KAH7551596, 218_XP_014080001) 449 ) 445 ) 422 ) 421 ) 420 ) 419 ) 418 , ((209_KAF9634910, 210_XP_035369290) 451 , (211_XP_020126245, (212_KKY15282, 213_OMP85163) 453 ) 452 ) 450 ) 417 , (207_PSK59305, 208_TKX19899) 454 ) 416 , (247_KAF2835959, (((((((345_CAF9922273, (250_XP_049164345, (248_KAI1845470, 249_KAI1854398) 464 ) 463 ) 462 , ((((((((((((324_KAF4505756, 325_XP_044716689) 476 , (326_KJZ78591, 327_PHH93207) 477 ) 475 , (322_KND88345, 323_POR33302) 478 ) 474 , (316_PHH66231, 317_PHH72215) 479 ) 473 , ((315_RDA91345, (314_RCI10733, 313_RDA83884) 482 ) 481 , (310_PFH60078, (311_KAF4582743, 312_PHH77656) 484 ) 483 ) 480 ) 472 , ((320_ODA76862, 321_XP_040655762) 486 , (318_GJN75845, 319_GJN79908) 487 ) 485 ) 471 , ((299_OAA32211, (300_QPG95136, 301_XP_042995097) 490 ) 489 , ((302_TWU76274, 303_XP_018140447) 492 , (304_KAG8410572, (306_KID94921, (305_KID83261, (307_XP_014547936, (308_KAH0592203, 309_KJK79972) 497 ) 496 ) 495 ) 494 ) 493 ) 491 ) 488 ) 470 , ((334_XP_013958481, ((336_XP_024751012, (335_PTB78996, (337_KAH0494541, (338_ETS06778, 339_OTA08591) 504 ) 503 ) 502 ) 501 , (340_PNP54593, (341_OPB42620, (342_KKO98130, (343_KAF3069441, 344_KAH0525549) 508 ) 507 ) 506 ) 505 ) 500 ) 499 , (329_RFU77259, (328_KAH6610050, ((332_XP_013947392, 333_XP_018663437) 512 , (330_GFP54281, 331_UKZ86338) 513 ) 511 ) 510 ) 509 ) 498 ) 469 , (296_ATY65176, (297_OAA80456, 298_OAR04029) 515 ) 514 ) 468 , (251_KKA30863, 252_KKF94376) 516 ) 467 , (261_VUC22500, (262_CAH0048186, (265_CAH0016877, (264_CAG9936525, 263_CAG9985177) 520 ) 519 ) 518 ) 517 ) 466 , (294_KAF7559752, 295_KAH8173892) 521 ) 465 ) 461 , ((((352_KAH6975750, 351_XP_046094133) 525 , (353_KAH7113358, 354_KAH7160531) 526 ) 524 , (346_CAH0035662, ((349_RYP13485, 350_RYP78928) 529 , (347_KAH6669699, 348_KAH7350224) 530 ) 528 ) 527 ) 523 , ((((371_KAF4450525, (372_KAF5706270, (373_KAH7178219, (375_KAH7212114, (374_XP_046049950, (376_QKD49327, (377_KLO82560, 378_SCN93509) 540 ) 539 ) 538 ) 537 ) 536 ) 535 ) 534 , ((364_KAG7410234, (365_PCD21150, (366_KAI3572988, 367_XP_031038650) 544 ) 543 ) 542 , (368_KAH6991545, (369_KAH7236631, 370_KIL86278) 546 ) 545 ) 541 ) 533 , (361_UPL03496, (362_KAF4469781, 363_KAF5011898) 548 ) 547 ) 532 , (355_XP_042995244, ((359_XP_003000034, 360_XP_028492418) 551 , (356_CRK46671, (357_KAG7136030, 358_XP_009656748) 553 ) 552 ) 550 ) 549 ) 531 ) 522 ) 460 , (((269_KAI3400106, (((274_POS74919, 275_XP_044649230) 559 , (272_KAH8785419, 273_KAI7784882) 560 ) 558 , (270_KKY39969, 271_XP_043022746) 561 ) 557 ) 556 , (267_MBW0496253, 268_MBW0498692) 562 ) 555 , (266_KLU82627, (253_KAH7306032, (254_KAH8662548, 255_KAH8894156) 565 ) 564 ) 563 ) 554 ) 459 , ((258_TLD11728, (256_XP_029744816, 257_XP_030980121) 568 ) 567 , (259_KAI6304082, 260_QBZ65430) 569 ) 566 ) 458 , ((((289_XP_049125671, (290_GKT55711, (291_TKW52418, (292_TQN63987, 293_XP_018164639) 576 ) 575 ) 574 ) 573 , (287_KZL84523, 288_XP_008100791) 577 ) 572 , (280_TDZ53310, (281_XP_036579499, ((282_KAF6807323, 283_XP_038742924) 581 , (284_KAI8307980, (285_EQB53597, 286_KAH0427588) 583 ) 582 ) 580 ) 579 ) 578 ) 571 , (276_OLN91671, (277_XP_022481658, (278_EXF73961, 279_XP_035328441) 586 ) 585 ) 584 ) 570 ) 457 , (((385_CRG83534, (386_OXV10832, (387_XP_040650479, ((390_OQE35386, (391_OQE43453, ((397_KAF7521891, ((398_CRL20948, 399_XP_040811647) 598 , (395_KUM66907, 396_OQD69102) 599 ) 597 ) 596 , (392_KAG0154123, (393_KGO67238, 394_XP_016595579) 601 ) 600 ) 595 ) 594 ) 593 , (388_OQE30276, 389_XP_002562422) 602 ) 592 ) 591 ) 590 ) 589 , (381_XP_001270369, (382_PLN86625, (383_XP_024667451, 384_XP_024697818) 605 ) 604 ) 603 ) 588 , (379_KIM92818, 380_KKY23838) 606 ) 587 ) 456 ) 455 ) 415 , (202_XP_033538655, (203_KAF2730203, (204_PSN67428, (205_KAF9690909, 206_XP_038801042) 610 ) 609 ) 608 ) 607 ) 414 , (182_PVH92012, ((183_EME41613, 184_XP_047764044) 613 , (185_KAF2212808, (186_XP_044663983, (187_PPJ58082, 188_XP_023450082) 616 ) 615 ) 614 ) 612 ) 611 ) 413 , (189_KAH0536166, (((190_XP_001884879, 191_XP_001890626) 620 , (192_KAH0541234, (193_KAH0558849, 194_KIM35080) 622 ) 621 ) 619 , (195_KAH8752860, (196_PMD18135, (197_KAI1000636, (198_KAH8589804, (199_KAG4418218, (200_KAG4431558, 201_KAH9219826) 628 ) 627 ) 626 ) 625 ) 624 ) 623 ) 618 ) 617 ) 412 , (178_WP_111876543, (179_QBG97949, (180_AKN93304, 181_WP_024712742) 631 ) 630 ) 629 ) 411 , (((159_WP_181080955, 160_WP_253508315) 634 , (((174_WP_181286820, (175_WP_017528495, (176_WP_116665714, 177_WP_248161456) 639 ) 638 ) 637 , (172_WP_080427502, 173_WP_124232900) 640 ) 636 , (161_WP_019362686, ((168_WP_054596572, (169_WP_063343083, (170_WP_008068240, 171_WP_123496614) 645 ) 644 ) 643 , ((166_WP_057414127, 167_WP_057426137) 647 , (163_QYC97872, (164_User_Sequence, (162_PK2, 165_WP_054082735) 650 ) 649 ) 648 ) 646 ) 642 ) 641 ) 635 ) 633 , (157_OZG71672, 158_WP_020407952) 651 ) 632 ) 410 ) 409 , (138_WP_193874723, ((((149_RCJ18531, 150_WP_229496330) 656 , (151_WP_206263246, ((154_RAM52958, (155_WP_131118892, 156_WP_207390922) 660 ) 659 , (152_WP_063849526, 153_WP_071780333) 661 ) 658 ) 657 ) 655 , (144_MBW4490662, (145_VFK55271, (146_VFJ88583, (147_VFK18546, 148_VFK66213) 665 ) 664 ) 663 ) 662 ) 654 , ((141_WP_216670419, (142_WP_073633445, 143_WP_096562523) 668 ) 667 , (139_MBW4538205, 140_WP_191133867) 669 ) 666 ) 653 ) 652 ) 408 , ((132_WP_131746854, (133_WP_239333142, (134_WP_239309439, (135_WP_239376700, (136_WP_239346593, 137_WP_239394497) 675 ) 674 ) 673 ) 672 ) 671 , (131_WP_203005384, (130_WP_007510697, 129_WP_013423727) 677 ) 676 ) 670 ) 407 , ((((28_WP_091288634, (((50_CAF1459828, (((73_WP_152789356, ((90_WP_208776875, ((95_OPG07481, ((98_WP_245695288, 99_WP_260470499) 693 , (96_WP_007379180, (97_WP_141208136, ((103_WP_132836977, (100_WP_030327021, (101_WP_020135205, 102_WP_097259396) 699 ) 698 ) 697 , (104_WP_062029603, (105_WP_062237234, 106_WP_120050161) 701 ) 700 ) 696 ) 695 ) 694 ) 692 ) 691 , (91_WP_212670581, (92_WP_057617985, (93_WP_059210412, 94_WP_062048822) 704 ) 703 ) 702 ) 690 ) 689 , (((80_WP_086746405, (81_WP_152166644, (82_WP_086762515, 83_WP_216586771) 709 ) 708 ) 707 , (((76_WP_005474170, 77_WP_097251030) 712 , (78_WP_013006004, 79_WP_059078688) 713 ) 711 , (74_WP_037692315, 75_WP_046703800) 714 ) 710 ) 706 , (84_WP_053745397, ((85_WP_164312020, 86_WP_164413328) 717 , (87_WP_006381705, (88_WP_055523095, 89_WP_105968964) 719 ) 718 ) 716 ) 715 ) 705 ) 688 ) 687 , (66_WP_030204587, (67_WP_053789018, ((68_WP_215150000, 69_WP_257558168) 723 , (70_WP_254381577, (71_WP_243340464, 72_WP_250739343) 725 ) 724 ) 722 ) 721 ) 720 ) 686 , (55_WP_196425794, (((63_WP_176174048, (65_WP_086841583, (64_MBE8521782, (62_WP_206784437, (60_WP_013230840, 61_WP_103337982) 733 ) 732 ) 731 ) 730 ) 729 , (58_WP_003091748, 59_WP_103354041) 734 ) 728 , (56_WP_026468744, 57_WP_167441344) 735 ) 727 ) 726 ) 685 ) 684 , (51_WP_033210435, (52_WP_165984892, (53_WP_184884329, 54_WP_197010885) 738 ) 737 ) 736 ) 683 , ((49_WP_211770529, (48_WP_116173588, 47_WP_184858281) 741 ) 740 , (45_WP_123667040, 46_WP_153525272) 742 ) 739 ) 682 ) 681 , (29_WP_209639274, (30_SMD26909, 31_WP_037268971) 744 ) 743 ) 680 , (44_WP_149852286, (32_WP_146358578, (33_WP_030474037, ((((43_WP_166054208, (42_WP_086663953, 41_WP_245822033) 752 ) 751 , (38_WP_167977648, (39_WP_045317146, 40_WP_112276902) 754 ) 753 ) 750 , (36_WP_189259271, 37_WP_204767818) 755 ) 749 , (34_WP_109640897, 35_WP_112230130) 756 ) 748 ) 747 ) 746 ) 745 ) 679 , ((((9_MBU3055413, ((16_WP_118884900, (19_WP_197345905, (18_WP_028861955, (17_WP_071092598, ((22_WP_197386210, 23_WP_247311730) 767 , (20_WP_064051710, 21_WP_138930184) 768 ) 766 ) 765 ) 764 ) 763 ) 762 , ((10_WP_039570089, (13_WP_055324506, (14_WP_064046801, 15_WP_141514839) 772 ) 771 ) 770 , (11_QHB56733, 12_WP_003274237) 773 ) 769 ) 761 ) 760 , ((6_WP_050879374, (7_WP_219656666, 8_WP_221866169) 776 ) 775 , (1_WP_190877710, (2_NER04273, (3_NEO55874, (4_ARV63120, 5_NET46310) 780 ) 779 ) 778 ) 777 ) 774 ) 759 , (24_WP_157246697, 25_WP_184965276) 781 ) 758 , (26_WP_030440504, 27_WP_125810284) 782 ) 757 ) 678 ) 406 , ((107_NMO14212, 108_WP_253986668) 784 , (109_WP_164021163, (110_WP_163989172, 111_WP_206728402) 786 ) 785 ) 783 ) 405 , (112_WP_234064780, 113_WP_253971230) 787 ) 404 , (114_WP_015351455, 115_WP_206715722) 788 ) 403 , ((120_WP_253992635, (121_WP_163861341, 122_WP_176434236) 791 ) 790 , (123_WP_174461383, ((124_WP_141641058, 125_WP_174485597) 794 , (126_WP_174453320, 127_WP_174475597) 795 ) 793 ) 792 ) 789 ) 402 ) 401 , (118_WP_223745345, 119_WP_254036614) 796 ) 400 ;"

## Convert Newick Tree text statement into ete formate and initiate web server to explore the tree

In [6]:
t_basic = eteTree(newick_tree, parser=1)

In [7]:
t_basic.explore(keep_server=False)

Added tree tree-2 with id 0.


## Link MSA to tree using the PhyloTree module

In [24]:
t_msa = PhyloTree(newick_tree, alignment=fasta_txt, alg_format='fasta', parser=1)

## Determine leafs of desired nodes

In [101]:
''''
Takes a Target Node (given as a numeric number prepended by "Node" [e.g., "Node171"]) 
and return a .fasta file with the children leafs
''''
def get_childrenseqs(inp_Node): 
    Node_name = inp_Node.name # Get the Node Name
    os.chdir(wkdir)
    open(f"{Node_name}.fasta", "w", errors='ignore') # Create the fasta file
    filename = f"{Node_name}_Descendants.fasta"
   # print(f"{filename}")
    for leaf in inp_Node:
        header = leaf.name
        sequence = leaf.props['sequence']
        #print(f"{header}")
        with open(filename, 'a') as fasta_file:
            fasta_file.write(f'>{header}\n')
            fasta_file.write(f'{sequence}\n')

    return

## See these cells to understand how get_childrenseqs function works and was validated

In [26]:
Node171= t_msa['Node171']

In [25]:
Node384 = t_msa['Node384']
print(Node384)

 ╭╴CAF1459828
─┤ ╭─┬╴WP_219656666
 │ │ ╰─┬╴WP_221866169
 ╰─┤   ╰╴WP_050879374
   │   ╭─┬╴NER04273
   │ ╭─┤ ╰╴NEO55874
   ╰─┤ ╰─┬╴NET46310
     │   ╰╴ARV63120
     ╰╴WP_190877710


In [70]:
def read_fasta(filename):
    sequences = {}
    current_sequence = None

    with open(filename, 'r') as fasta_file:
        for line in fasta_file:
            line = line.strip()
            if line.startswith('>'):
                current_sequence = line[1:]
                sequences[current_sequence] = ''
            else:
                sequences[current_sequence] += line

    return sequences

In [83]:
filename = "Node171.fasta"
fasta_sequences = read_fasta(filename)

for header, sequence in fasta_sequences.items():
    print(f'Header: {header}')
    print(f'Sequence: {sequence}')

Header: XP_038801042
Sequence: This should be overwritten
Header: KAF9690909
Sequence: LKTFELPIFVSGSSKDLELGKAMIKAWQRDGILQIKMDDGNKRIAKEAFKSSKTFFSLPTSEKTKCVDDQSFAGYIASGEELTDGVADYSEIFTVTKDLPQSDPRVQAKWPCHGSCPWPTPSMQTTMQNYMDNLGSSGERILQLIALGLGLPNMNTLNELTRDGWHHLRILRFPKTGDENGKGLKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYADEQFQNWQKSTAGMAENDDRWVYVPPEDGVFTVFPGDMLQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLKPLGGSSENSVHYGTHFTNMFMRNYPDRITAKRMQREGRLGSLQNPELREY


In [87]:
filename = "Node171.fasta"
fasta_sequences = read_fasta(filename)

for header, sequence in fasta_sequences.items():
    print(f'Header: {header}')
    print(f'Sequence: {sequence}')

Header: XP_038801042
Sequence: LKTFKLPLVVRGSLDDIELGKAMIRTWQSDGILQIKMDEQNKRVAEEAFKSSKRFFSLPPSAKAKCVDDQSFAGYIASGEELTDGIADYSEIFTVTKDLPQSDPRVRAKWPCHGPCPWPNSAMQTTMQNYMDSLGSSGERMLQLIALGLGLPDMNTFTELTRDGWHHLRILRFPKTGDENGKGMKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYEDEKFQNWQKSTAGMAENDDRWVYVPPEEGVFTVFPGDMMQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLRPLGKSSEDSVHYGAHFTNMFMRNYPDRVTAKRMRAEDRMKLLESPELREY
Header: KAF9690909
Sequence: LKTFELPIFVSGSSKDLELGKAMIKAWQRDGILQIKMDDGNKRIAKEAFKSSKTFFSLPTSEKTKCVDDQSFAGYIASGEELTDGVADYSEIFTVTKDLPQSDPRVQAKWPCHGSCPWPTPSMQTTMQNYMDNLGSSGERILQLIALGLGLPNMNTLNELTRDGWHHLRILRFPKTGDENGKGLKGRGIGSHTDYGLLVLAAQDDVGGLFIRPPYADEQFQNWQKSTAGMAENDDRWVYVPPEDGVFTVFPGDMLQFITNGLLPSTPHKVGMNTRERFAFAYFHEPNFGAVLKPLGGSSENSVHYGTHFTNMFMRNYPDRITAKRMQREGRLGSLQNPELREY


In [57]:
Node253 = t_msa['Node253']
print(Node253)

         ╭─┬╴XP_016595579
       ╭─┤ ╰╴KGO67238
       │ ╰╴KAG0154123
     ╭─┤   ╭─┬╴XP_040811647
     │ │ ╭─┤ ╰─┬╴CRL20948
     │ │ │ │   ╰╴KAF7521891
   ╭─┤ ╰─┤ ╰─┬╴OQD69102
   │ │   │   ╰╴KUM66907
 ╭─┤ │   ╰╴OQE43453
 │ │ ╰╴OQE35386
─┤ ╰─┬╴OQE30276
 │   ╰╴XP_002562422
 ╰╴XP_040650479


In [59]:
Node13 = t_msa['Node13']
print(Node13)
for leaf in Node13:
    print(leaf.name, leaf.props['sequence'])

                 ╭─┬╴WP_096562523
                 │ ╰╴WP_073633445
                 │     ╭─┬╴RCJ18531
                 │     │ ╰╴WP_229496330
               ╭─┤   ╭─┤   ╭─┬╴RAM52958
               │ │   │ │ ╭─┤ ╰─┬╴WP_207390922
               │ │   │ │ │ │   ╰╴WP_131118892
               │ │ ╭─┤ ╰─┤ ╰─┬╴WP_063849526
               │ │ │ │   │   ╰╴WP_071780333
             ╭─┤ ╰─┤ │   ╰╴WP_206263246
             │ │   │ ╰╴MBW4490662
             │ │   │ ╭─┬╴VFJ88583
             │ │   ╰─┤ ╰─┬╴VFK66213
           ╭─┤ │     │   ╰╴VFK18546
           │ │ │     ╰╴VFK55271
           │ │ ╰╴WP_216670419
         ╭─┤ │ ╭─┬╴WP_191133867
         │ │ ╰─┤ ╰╴MBW4538205
         │ │   ╰╴WP_193874723
         │ │ ╭╴WP_131746854
         │ ╰─┤ ╭─┬╴WP_239333142
         │   │ │ ╰─┬╴WP_239309439
       ╭─┤   ╰─┤   ╰─┬╴WP_239376700
       │ │     │     ╰─┬╴WP_239394497
       │ │     │       ╰╴WP_239346593
       │ │     ╰─┬╴WP_203005384
       │ │       ╰─┬╴WP_007510697
       │ │         ╰╴WP_013423

In [61]:
Node10 = t_msa['Node10']
print(Node10)
for leaf in Node10:
    print(leaf.name, leaf.props['sequence'])

                       ╭─┬╴WP_096562523
                       │ ╰╴WP_073633445
                       │     ╭─┬╴RCJ18531
                       │     │ ╰╴WP_229496330
                     ╭─┤   ╭─┤   ╭─┬╴RAM52958
                     │ │   │ │ ╭─┤ ╰─┬╴WP_207390922
                     │ │   │ │ │ │   ╰╴WP_131118892
                     │ │ ╭─┤ ╰─┤ ╰─┬╴WP_063849526
                     │ │ │ │   │   ╰╴WP_071780333
                   ╭─┤ ╰─┤ │   ╰╴WP_206263246
                   │ │   │ ╰╴MBW4490662
                   │ │   │ ╭─┬╴VFJ88583
                   │ │   ╰─┤ ╰─┬╴VFK66213
                 ╭─┤ │     │   ╰╴VFK18546
                 │ │ │     ╰╴VFK55271
                 │ │ ╰╴WP_216670419
               ╭─┤ │ ╭─┬╴WP_191133867
               │ │ ╰─┤ ╰╴MBW4538205
               │ │   ╰╴WP_193874723
               │ │ ╭╴WP_131746854
               │ ╰─┤ ╭─┬╴WP_239333142
               │   │ │ ╰─┬╴WP_239309439
             ╭─┤   ╰─┤   ╰─┬╴WP_239376700
             │ │     │     ╰─┬╴WP_

## <mark> Actually Save .fasta files for all nodes or desired nodes </mark>

In [102]:
for node in t_msa.traverse:
    node_name = node.name
    if node_name.startswith('Node'):
        get_childrenseqs(node)


#Desired Nodes
get_childrenseqs(Node171)
get_childrenseqs(Node253)
get_childrenseqs(Node13)
get_childrenseqs(Node10)

# Sample from Bali-Phy output

## Define functions from original "sample from bali" script

### retrieve_ancestors func

In [1]:
def retrieve_ancestors(bali_phy_name,burnin,sampling,wkdir):
    ### Open descendants file and place all the sequence names in the labels list
    Desc_Labels = {}
    Outfiles = {}
    Node_Counts = {}
    Node_Counts_flawed = {}
    SampledTrees = []
    os.chdir(wkdir)
    print(f"\n\n{os.listdir('Descendants')}\n\n")
    # On Mac OS, the .DS_Store file in every folder would normally cause the following line to fail. I added the first if statement to fix that.
    for Desc in [x for x in os.listdir('Descendants') if x.split('.')[0].split('_')[-1] == 'Descendants']:
        with open('Descendants/{}'.format(Desc)) as infile:  
            Desc2 = Desc.split('.')[0]
            lines = infile.readlines()
            Desc_Labels[Desc2] = [x.split()[0].lstrip('>') for x in lines[0::2]]
            if not os.path.exists('Ancestor_Nodes'): # 3/5/2024 - this doesn't work for some reason
                os.mkdir('Ancestor_Nodes')
            Outfiles[Desc2] = open('Ancestor_Nodes/{}_Sampling_e{}.fasta'.format(Desc2,sampling),'w')
            Node_Counts[Desc2] = 0
            Node_Counts_flawed[Desc2] = 0
    ### Check for directories made by BAli-Phy
    dirs = [d for d in os.listdir() if d[0:len(bali_phy_name)] == bali_phy_name]
    for directory in dirs:
        print('Parsing {0}'.format(directory))
        with open('{0}/C1.trees'.format(directory)) as infile:
            Trees = infile.readlines()
        ### Initialize values
        get_sequence = False
        Searching = False
        iteration = 0
        Fastas = open('{0}/C1.P1.fastas'.format(directory))
        ### The following line I changed "tqdm" to "tqdm.tqdm" because of namespace resoloution errors.
        for i, line in enumerate(tqdm(Fastas.readlines())):
            if line[0:10] == 'iterations':
                iteration = int(line[12:-1])
                if iteration > burnin and iteration%sampling == 0:
                    tree = Tree()
                    tree = tree.get(data=Trees[iteration],schema='newick',tree_offset=0)
                    SampledTrees.append(Trees[iteration])
                    Node_Labels = {}
                    for Desc in Desc_Labels.keys():
                        node_label = tree.mrca(taxon_labels=Desc_Labels[Desc]).label
                        Node_Labels[Desc] = node_label
                        Node_Labels[node_label] = Desc
                        leaves = [x.taxon.label for x in tree.find_node_with_label(node_label).leaf_nodes()]
                        new_taxa = set(leaves) - set(Desc_Labels[Desc])
                        if len(new_taxa) > 0:
                            Node_Counts_flawed[Desc] += 1
                        Node_Counts[Desc] += 1
                    Searching = True
                else:
                    Searching = False
            elif Searching:
                
                if line.rstrip().lstrip('>') in list(Node_Labels.values()):
                    get_sequence = []
                    for Desc in Node_Labels.keys():
                        if Node_Labels[Desc] == line.rstrip().lstrip('>'):
                            get_sequence.append(Desc)
                elif line[0] == '>':
                    get_sequence = False
                elif get_sequence:
                    for i in range(len(get_sequence)):
                        Outfiles[get_sequence[i]].write('>{0}|Run_{1}|Iteration_{2}\n'.format(get_sequence[i],directory[len(bali_phy_name)+1:],iteration))
                        Outfiles[get_sequence[i]].write(line.rstrip()+'\n')
            Fastas.close()
    if not os.path.exists('SampledTrees'):
        os.mkdir('SampledTrees')
    with open('SampledTrees/e{}.tree'.format(sampling),'w') as outfile:
        outfile.write(''.join(SampledTrees))
    ### Close all files
    with open('Ancestor_Report_Sampling_e{}.csv'.format(sampling),'w') as outfile:
        outfile.write(','.join(['Ancestor','FlawedNodes','TotalNodes','PercentFlawed'])+'\n')
        for Desc in Outfiles.keys():
            Outfiles[Desc].close()
            perc = Node_Counts_flawed[Desc]/Node_Counts[Desc]
            line = '{},{},{},{:.2%}\n'.format(Desc,Node_Counts_flawed[Desc],Node_Counts[Desc],perc)
            outfile.write(line)

### retrieve_consensus

In [2]:
def retrieve_consensus(node_name,fasta):
    D = 'Ancestor_Consensus_Seqs/{}'.format(node_name)
    if not os.path.exists(D):
        os.mkdir(D)
    # I changed this line because I had an 'X' in my alignment which raised problems. I added an 'X' term to the list.
    AAs = ['G', 'A', 'V', 'L', 'I', 'W', 'F', 'Y', 'P',  'M', 'C', 'S', 'T', 'N', 'Q', 'D', 'E', 'H', 'K', 'R', 'x', '-']
    Seqs = []
    infile = open(fasta)  
    seq = ''
    for line in infile:
        if line[0] == '>':
            if seq != '':
                Seqs.append(seq)
            seq = ''
        else:
            seq = seq + line.rstrip()
    if seq != '':
        Seqs.append(seq)
    infile.close()
    consensus = []
    con_Seq = ''
    con_Perc = []
    bPhy_PPs = []
    BFacts = open(D+'/{0}_from_BAli-Phy_Colors.txt'.format(node_name),'w')
    for i in range(len(Seqs[0])):
        consensus.append({})
        for aa in AAs:
            consensus[i][aa] = 0
        for seq in Seqs:
            consensus[i][seq[i]] += 1
        for aa in AAs:
            perc = consensus[i][aa]/len(Seqs)
            consensus[i][aa] = perc
        con_AA = max(consensus[i].items(), key=operator.itemgetter(1))[0]
        con_Seq = con_Seq + con_AA
        con_Perc.append(consensus[i][con_AA])
        if aa != '-':
            BFacts.write(str(perc)+'\n')
    with open(D+'/{0}_Consensus.fasta'.format(node_name),'w') as outfile:
        outfile.write('>{0}_Sampled_Consensus\n{1}'.format(node_name,con_Seq))
    Seqs = {node_name:{'seq':con_Seq,'score':con_Perc}}
    with open(D+'/{0}_baliphy_PP.pickle'.format(node_name),'wb') as outfile:
        pickle.dump(Seqs[node_name],outfile)
    with open(D+'/{0}_baliphy_All_PPs.pickle'.format(node_name),'wb') as outfile:
        pickle.dump(consensus,outfile)
    return Seqs

### seq_heatmap

In [3]:
def seq_heatmap(Seqs,fname,n_col=50):
    import matplotlib.pyplot as plt
    ### Seqs is a dictionary where labels are the keys 
    ### and the values are two dictionaries: seq and score
    ### seq is used for the annotation
    ### score is used for the color value
    ### Seqs = {node_name:{'seqs':sequence,'score':scores}}
    ### Sequences must be aligned
    ### Sequence must be a string
    ### Score must be a list of same length of Sequence
    
    Sequences = {}
    Scores = {}
    for label in Seqs.keys():
        seqName = label
        seq_length = len(Seqs[label]['seq'])
        Sequences[label] = []
        Scores[label] = []
        for i in range(len(Seqs[label]['seq'])):
            Sequences[label].append(Seqs[label]['seq'][i])
            Scores[label].append(Seqs[label]['score'][i])
    remainder = seq_length%n_col
    for i in range(n_col-remainder):
        for label in Seqs.keys():
            Sequences[label].append(' ')
            Scores[label].append(1)
    ### Setup DataFrames
    df_Scores = pd.DataFrame(Scores)
    df_Scores = df_Scores.T
    df_Seqs = pd.DataFrame(Sequences)
    df_Seqs = df_Seqs.T
    
    
    ### Create cmap
    cmap = [(1, 0, 0), (1, 0, 0), (1, 0, 0),
         (1, 0, 0), (1, 0, 0), (1, 0, 0),
         (1, 0, 0), (1, 0, 0), (1, 0, 0),
         (1, 0, 0), (1, 0.2, 0), (1, 0.2, 0),
         (1, 0.4, 0), (1, 0.4, 0), (1, 0.6, 0),
         (1, 0.6, 0), (1, 0.8, 0), (1, 0.8, 0),
         (1, 0.9, 0), (1, 1, 0)]

#    fig, ax = plt.subplots(nrows=int(ceil(seq_length/n_col)+1),figsize=(10,int(ceil(seq_length/n_col)+1)))
    fig, ax = plt.subplots(nrows=int(ceil(seq_length/n_col)+1),figsize=(10,int(ceil(seq_length/n_col)+1)/3))
    for i in range(int(ceil(seq_length/n_col))):
        h1 = sns.heatmap(df_Scores[df_Scores.columns[i*n_col:(i+1)*n_col]], square=True,xticklabels=False,vmin=0,vmax=1,annot=df_Seqs[df_Seqs.columns[i*n_col:(i+1)*n_col]],fmt = '',ax=ax[i],cbar=False,cmap=cmap)
        ax[i].set_yticklabels(ax[i].get_yticklabels(),rotation=0)
    cmap_ticks = [0]
    for i in range(5):
        cmap_ticks.append(cmap_ticks[-1]+1/5)
    mappable = h1.get_children()[0]
    plt.colorbar(mappable,
                 cax = ax[-1],
                 ticks=cmap_ticks,
                 orientation = 'horizontal')
    title = '{0} Consensus at Each Position'.format(seqName)
    ax[0].set_title(title,fontsize=14)
    plt.savefig(fname,format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

### determine_nonconcensus_deviation

In [5]:
def determine_nonconcensus_deviation():
    AA_key = ['G', 'A', 'V', 'L', 'I', 'W', 'F', 'Y', 'P',  'M', 'C', 'S', 'T', 'N', 'Q', 'D', 'E', 'H', 'K', 'R', 'x', '-']
    Percent_Share_Cuttoff = 0.05
    
    for Ancestor_Alignment in [x for x in os.listdir('Ancestor_Nodes_Aligned') if x.split('.')[0].split('_')[-1] == 'Aligned']:
        #Sequences_list = []
        AA_Probability_Matrix = []
        Num_Sequences = 1
        # Each itteration of this loop is one file.
        Sequences_list = parse_multifasta(f"Ancestor_Nodes_Aligned/{Ancestor_Alignment}")
        # We now have the sequences parsed. Let's make a matrix.
        Num_Sequences = len(Sequences_list)
        AA_Probability_Matrix = calculate_data(AA_Probability_Matrix, Sequences_list, AA_key, Num_Sequences)
        #print(f'AA_Probability_Matrix: {AA_Probability_Matrix}')
        # Now let's write the output.
        write_output(AA_Probability_Matrix, Ancestor_Alignment, Num_Sequences, Percent_Share_Cuttoff, AA_key)
        #print(f'after write_output: AA_Probability_Matrix: {AA_Probability_Matrix}')

### parse_multifasta

In [6]:
def parse_multifasta (f_path):
    #print(f_path)
    Sequences_list = []
    with open(f_path) as handle:
        #Sequences_list = []
        for Seq_Rec_Itr in SeqIO.parse(handle, "fasta"):
            Sequences_list.append(str(Seq_Rec_Itr.seq))
    return Sequences_list

### calculate_data

In [7]:
def calculate_data(AA_Probability_Matrix, Sequences_list, AA_key,Num_Sequences):
    # For each position,
    for position in range(len(Sequences_list[0])):
        # Make a blank list
        position_aa_distribution = [0]*22
        # And tally each Amino Acid
        for i in range(Num_Sequences):
            AA = Sequences_list[i][position]
            (position_aa_distribution[AA_key.index(AA)])+=1
        # Then add the collumn to the matrix
        AA_Probability_Matrix.append(position_aa_distribution)
    return AA_Probability_Matrix

### write_output

In [8]:
def write_output (matrix, name, Num_Sequences, Percent_Share_Cuttoff, AA_key):
    space = "  "
    name = name[0:-26]
    name += "_Non-Concensus_Deviations.txt"

    if not os.path.isdir("Consensus Deviations"):
        os.mkdir("Consensus Deviations")
        
    with open(f"Consensus Deviations/{name}","w+") as fout:
        for position in range(len(matrix)):
            output = f"At position {space}{position+1}, "
            for i in range(len(matrix[position])):
                if ((matrix[position][i] / Num_Sequences) >= Percent_Share_Cuttoff):
                    output += f"{AA_key[i]} apears {round((matrix[position][i]/Num_Sequences)*100,2) }% of the time;  "
            output+="\n"
            fout.write(output)
            output = ""
            if (position==8):
                space=" "
            if (position==98):
                space=""


### align_ancestors func

In [9]:
def align_ancestors(mafft_exe):
    os.chdir(wkdir)
    if not os.path.exists('Ancestor_Nodes_Aligned'):
        os.mkdir('Ancestor_Nodes_Aligned')
    if not os.path.exists('Ancestor_Consensus_Seqs'):
        os.mkdir('Ancestor_Consensus_Seqs')
    for fasta in os.listdir('Ancestor_Nodes'):
        # Make sure that on Mac OS the .DS_Store file is removed, or this line will fail.
        print('Aligning and Analyzing {}'.format(fasta.split('.')[0]))
        in_file = 'Ancestor_Nodes/' + fasta
        outfile = 'Ancestor_Nodes_Aligned/' + fasta.split('.')[0] + '_Aligned.fasta'
        # mafft_cline = MafftCommandline(cmd='mafft', input=in_file)
        #stdout, stderr = mafft_cline()
        stdout = subprocess.check_output(f"/usr/local/anaconda3/envs/bali_ancestors/bin/mafft {in_file}", shell=True, text=True)
        with open(outfile, "w") as handle:
            handle.write(stdout)
        Seqs = retrieve_consensus(fasta.split('.')[0],outfile)
        fname = 'Ancestor_Consensus_Seqs/{0}/{0} Consensus from BAli-Phy.pdf'.format(fasta.split('.')[0])
        seq_heatmap(Seqs,fname)
        pass

## Full Implementation

## <mark> Sampling Parameters </mark>

In [ ]:
wkdir='/Users/nathanielpascual/OneDrive - Michigan State University/Research/Woldring_Lab_Share/AutoASR/Hausinger Collaboration/PK2/ASR_NEAN_fromBali'
burnin = 2000
# Sampling, i.e., tell the script whether you want to sample every single tree or every other tree (i.e., 2) or any other combo.
sampling = 1

#Give align_ancesotrs mafft_exe the location of the executable for mafft
#If mafft is in path just set mafft_exe="mafft"
mafft_exe='/usr/local/anaconda3/envs/bali_ancestors/bin/mafft'

## Run Sampling 

In [14]:
# The following two lines allow this to run on MacOS. If os.remove doesn't find these files, it will continue on
try:
    os.remove("Descendants/.DS_Store")
    os.remove("Ancestor_Nodes/.DS_Store")
except:
    pass
    
os.chdir(wkdir)

bali_phy_name = mode([x.split('-')[0] for x in os.listdir() if len(x.split('.')) == 1 and len(x.split('-'))>=2])

retrieve_ancestors(bali_phy_name,burnin,sampling, wkdir)
    
print("Executing MAFFT")
align_ancestors(mafft_exe)

print("Determining Non-consensus Deviations")
determine_nonconcensus_deviation()

print("Pipeline Done")



['Node171_Descendants.fasta', 'Node13_Descendants.fasta', 'Node10_Descendants.fasta', 'Node253_Descendants.fasta']


Parsing Final_Sequences_PK2-3


100%|██████████████████████████████████████████████████████████████████████████████████████| 902305/902305 [00:36<00:00, 24575.77it/s]


Parsing Final_Sequences_PK2-5


100%|██████████████████████████████████████████████████████████████████████████████████████| 879947/879947 [00:34<00:00, 25230.31it/s]


Parsing Final_Sequences_PK2-6


...

Aligning and Analyzing Node13_Descendants_Sampling_e1


nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 1001 / 1057
done.

Constructing a UPGMA tree (efffree=0) ... 
 1050 / 1057
done.

Progressive alignment 1/2... 
STEP  1001 / 1056 
done.

Making a distance matrix from msa.. 
 1000 / 1057
done.

Constructing a UPGMA tree (efffree=1) ... 
 1050 / 1057
done.

Progressive alignment 2/2... 
STEP  1001 / 1056 
done.

disttbfast (aa) Version 7.520
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leave

Aligning and Analyzing Node171_Descendants_Sampling_e1


nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 1001 / 1057
done.

Constructing a UPGMA tree (efffree=0) ... 
 1050 / 1057
done.

Progressive alignment 1/2... 
STEP  1001 / 1056 
done.

Making a distance matrix from msa.. 
 1000 / 1057
done.

Constructing a UPGMA tree (efffree=1) ... 
 1050 / 1057
done.

Progressive alignment 2/2... 
STEP  1001 / 1056 
done.

disttbfast (aa) Version 7.520
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leave

Determining Non-consensus Deviations
Pipeline Done
